<a href="https://colab.research.google.com/github/ErickJLA/Co-Met/blob/main/Co_Met_1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 30px; border-radius: 10px; color: white;'>
<h1 style='color: white; margin-top: 0;'>📊 Co-Met</h1>
<p style='font-size: 16px; color: #f0f0f0;'>A comprehensive Google Colab notebook for conducting publication-ready meta-analyses with advanced statistical methods.</p>
</div>

---

## 🎯 Quick Start Guide

### **Step-by-Step Workflow:**

1. **📚 Cell 1**: Import libraries & authenticate with Google
2. **📁 Cell 2**: Load your data from Google Sheets
3. **⚙️ Cell 3**: Configure analysis parameters
4. **🧹 Cell 4**: Apply configuration & clean data
5. **🔬 Cell 6**: Detect effect size type
6. **🧮 Cell 7**: Calculate effect sizes
7. **📊 Cell 8**: View overall meta-analysis results
8. **📈 Cells 9-19**: Advanced analyses (subgroups, regression, plots)

---

## 🎓 What is Meta-Analysis?

Meta-analysis is a statistical technique for **combining results from multiple studies** to estimate an **overall effect size**. It provides more precise estimates than individual studies and can identify patterns across research.

### **Supported Effect Sizes:**
- **lnRR**: Log Response Ratio (for ratio measures)
- **Hedges' g**: Standardized mean difference with small-sample correction
- **Cohen's d**: Standardized mean difference (uncorrected)
- **Log OR**: Log odds ratio

---

## 📋 Required Data Format

Your Google Sheet must have these columns:

| Column | Description | Example |
|--------|-------------|---------|
| `id` | Study identifier | "Smith2020" |
| `xe` | Experimental group mean | 25.3 |
| `sde` | Experimental group SD | 4.2 |
| `ne` | Experimental group sample size | 30 |
| `xc` | Control group mean | 22.1 |
| `sdc` | Control group SD | 3.8 |
| `nc` | Control group sample size | 28 |

**Optional:** Add categorical columns for **moderator analysis** (e.g., "species", "treatment_type", "year")

---

## 🔬 Advanced Features

### ✨ What Makes This Pipeline Special?

1. **Three-Level Models**: Accounts for multiple effect sizes per study
2. **Cluster-Robust Inference**: Handles dependency in data
3. **Multiple Heterogeneity Estimators**: DL, REML, ML, PM, SJ
4. **Publication Bias Assessment**: Funnel plots and statistical tests
5. **Meta-Regression**: Test continuous and categorical moderators
6. **Spline Analysis**: Model non-linear relationships
7. **Sensitivity Analysis**: Leave-one-out and cumulative methods

---

## ⚠️ Important Notes

- **Run cells in order**: Each cell depends on previous ones
- **Check prerequisites**: Some cells require specific prior cells to run
- **Google Sheets access**: Ensure your sheet is shared with your Colab email
- **Data quality**: Clean your data before uploading (remove blanks, check formatting)

---

## 📚 Statistical Methods Reference

This notebook implements methods from:

- **Borenstein et al. (2009)**: *Introduction to Meta-Analysis*
- **Viechtbauer (2010)**: *Conducting Meta-Analyses in R with the metafor Package*
- **Hedges & Olkin (1985)**: *Statistical Methods for Meta-Analysis*

For detailed methodology, see cell documentation throughout the notebook.

---

## 🐛 Troubleshooting

**"Authentication Failed"**
- Restart runtime and re-run Cell 1
- Check Google account permissions

**"Data Not Found"**
- Verify Google Sheet name spelling
- Ensure sheet is shared with Colab email
- Check that worksheet exists

**"Invalid Column Names"**
- Use Cell 3 accordion to map column names
- Ensure no duplicate mappings

---

## 📧 Support & Feedback

For issues or suggestions, please refer to the documentation or contact the maintainer.

---

<div style='background-color: #d4edda; border-left: 4px solid #28a745; padding: 15px; margin-top: 20px;'>
<strong>✅ Ready to start?</strong> Run <strong>Cell 1</strong> below to begin!
</div>

In [ ]:
#@title 📊 IMPORT LIBRARIES & AUTHENTICATE

# =============================================================================
# CELL 1: ENVIRONMENT SETUP
# Purpose: Import required libraries and authenticate Google Sheets access
# Dependencies: None
# Outputs: Authentication status, library versions, system info
# =============================================================================

import numpy as np
import pandas as pd
import gspread
from google.colab import auth
from google.auth import default
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from scipy.stats import norm, chi2
import matplotlib.pyplot as plt
import datetime
import sys
import warnings
from scipy.special import gamma

# Suppress unnecessary warnings for cleaner output
warnings.filterwarnings('ignore', category=FutureWarning)

# --- Configuration Constants ---
REQUIRED_COLUMNS = {
    'effect_data': ['xe', 'sde', 'ne', 'xc', 'sdc', 'nc'],
    'metadata': ['id']
}

SUPPORTED_EFFECT_SIZES = {
    'lnRR': 'Log Response Ratio',
    'hedges_g': "Hedges' g (corrected SMD)",
    'cohen_d': "Cohen's d (uncorrected SMD)",
    'log_OR': 'Log Odds Ratio'
}

# --- Authentication ---
print("=" * 70)
print("META-ANALYSIS PIPELINE - INITIALIZATION")
print("=" * 70)
print(f"Execution Time: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("-" * 70)

try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    auth_status = "✓ SUCCESS"
    auth_details = "Google Sheets API access granted"
except Exception as e:
    auth_status = "✗ FAILED"
    auth_details = str(e)
    print(f"\n❌ AUTHENTICATION ERROR: {e}")
    print("\nTroubleshooting:")
    print("  1. Ensure you're running in Google Colab")
    print("  2. Check your Google account permissions")
    print("  3. Try re-running the cell")
    raise Exception("Stopping execution due to authentication failure.")

# --- Library Version Check ---
print("\n📦 LIBRARY VERSIONS:")
print(f"  • NumPy:      {np.__version__}")
print(f"  • Pandas:     {pd.__version__}")
print(f"  • gspread:    {gspread.__version__}")
print(f"  • Matplotlib: {plt.matplotlib.__version__}")

# --- Configuration Summary ---
print("\n⚙️  CONFIGURATION:")
print(f"  • Required effect data columns: {', '.join(REQUIRED_COLUMNS['effect_data'])}")
print(f"  • Required metadata columns:    {', '.join(REQUIRED_COLUMNS['metadata'])}")
print(f"  • Supported effect sizes:       {len(SUPPORTED_EFFECT_SIZES)}")
for key, name in SUPPORTED_EFFECT_SIZES.items():
    print(f"      - {key}: {name}")

# --- Status Summary ---
print("\n" + "=" * 70)
print("INITIALIZATION STATUS")
print("=" * 70)
print(f"Authentication:  {auth_status}")
print(f"Details:         {auth_details}")
print(f"Ready:           {'YES ✓' if auth_status == '✓ SUCCESS' else 'NO ✗'}")
print("=" * 70)

# Store initialization metadata for later reference
INIT_METADATA = {
    'timestamp': datetime.datetime.now(),
    'auth_status': auth_status,
    'numpy_version': np.__version__,
    'pandas_version': pd.__version__,
    'supported_effects': list(SUPPORTED_EFFECT_SIZES.keys())
}

print("\n✅ Setup complete. Proceed to next cell to load data.\n")

# =============================================================================
# UTILITY FUNCTIONS - Extracted from original cells for reusability
# =============================================================================

# --- STATISTICAL FUNCTIONS ---

def calculate_tau_squared_DL(df, effect_col, var_col):
    """
    DerSimonian-Laird estimator for tau-squared

    Advantages:
    - Simple, fast
    - Non-iterative
    - Always converges

    Disadvantages:
    - Can underestimate tau² in small samples
    - Negative values truncated to 0
    - Less efficient than ML methods

    Parameters:
    -----------
    df : DataFrame
        Data with effect sizes and variances
    effect_col : str
        Name of effect size column
    var_col : str
        Name of variance column

    Returns:
    --------
    float : tau-squared estimate
    """
    k = len(df)
    if k < 2:
        return 0.0

    try:
        # Fixed-effects weights
        w = 1 / df[var_col]
        sum_w = w.sum()

        if sum_w <= 0:
            return 0.0

        # Fixed-effects pooled estimate
        pooled_effect = (w * df[effect_col]).sum() / sum_w

        # Q statistic
        Q = (w * (df[effect_col] - pooled_effect)**2).sum()
        df_Q = k - 1

        # C constant
        sum_w_sq = (w**2).sum()
        C = sum_w - (sum_w_sq / sum_w)

        # Tau-squared
        if C > 0 and Q > df_Q:
            tau_sq = (Q - df_Q) / C
        else:
            tau_sq = 0.0

        return max(0.0, tau_sq)

    except Exception as e:
        warnings.warn(f"Error in DL estimator: {e}")
        return 0.0



def calculate_tau_squared(df, effect_col, var_col, method='REML', **kwargs):
    """
    Unified function to calculate tau-squared using specified method

    Parameters:
    -----------
    df : DataFrame
        Data with effect sizes and variances
    effect_col : str
        Name of effect size column
    var_col : str
        Name of variance column
    method : str
        Estimation method: 'DL', 'REML', 'ML', 'PM', 'SJ'
        Default: 'REML' (recommended)
    **kwargs : dict
        Additional arguments passed to estimator

    Returns:
    --------
    float : tau-squared estimate
    dict : additional information (method used, convergence, etc.)
    """
    method = method.upper()

    estimators = {
        'DL': calculate_tau_squared_DL,
        'REML': calculate_tau_squared_REML,
        'ML': calculate_tau_squared_ML,
        'PM': calculate_tau_squared_PM,
        'SJ': calculate_tau_squared_SJ
    }

    if method not in estimators:
        warnings.warn(f"Unknown method '{method}', using REML")
        method = 'REML'

    try:
        tau_sq = estimators[method](df, effect_col, var_col, **kwargs)

        info = {
            'method': method,
            'tau_squared': tau_sq,
            'tau': np.sqrt(tau_sq),
            'success': True
        }

        return tau_sq, info

    except Exception as e:
        warnings.warn(f"Error with {method}, falling back to DL: {e}")
        tau_sq = calculate_tau_squared_DL(df, effect_col, var_col)

        info = {
            'method': 'DL',
            'tau_squared': tau_sq,
            'tau': np.sqrt(tau_sq),
            'success': False,
            'fallback': True,
            'error': str(e)
        }

        return tau_sq, info



def calculate_tau_squared_dl(df, effect_col, var_col):
    """
    Calculate Tau-squared. Uses Global Advanced Estimator (Cell 4.5) if available,
    otherwise falls back to DerSimonian-Laird (DL).
    """
    k = len(df)
    if k < 2: return 0.0

    # Try using the advanced REML estimator from Cell 4.5 first
    if 'calculate_tau_squared' in globals():
        tau_method = 'REML' # Prefer REML for consistency
        try:
            tau_sq, info = calculate_tau_squared(df, effect_col, var_col, method=tau_method)
            if info.get('success', True):
                return tau_sq
        except Exception:
            pass # Fall back to DL if REML fails (common in small cumulative steps)

    # Classic DL Method (Fallback)
    try:
        w_fixed = 1 / df[var_col]
        sum_w = w_fixed.sum()
        if sum_w <= 0: return 0.0
        pooled_effect = (w_fixed * df[effect_col]).sum() / sum_w
        Qt = (w_fixed * (df[effect_col] - pooled_effect)**2).sum()
        df_Q = k - 1
        sum_w_sq = (w_fixed**2).sum()
        C = sum_w - (sum_w_sq / sum_w)
        if C > 0 and Qt > df_Q:
            tau_squared = (Qt - df_Q) / C
        else:
            tau_squared = 0.0
        return max(0.0, tau_squared)
    except Exception:
        return 0.0


def calculate_re_pooled(df, tau_squared, effect_col, var_col, alpha=0.05):
    """Calculate Random-Effects pooled estimate with CI"""
    k = len(df)
    if k < 1: return np.nan, np.nan, np.nan, np.nan, np.nan
    try:
        w_re = 1 / (df[var_col] + tau_squared)
        sum_w_re = w_re.sum()
        if sum_w_re <= 0: return np.nan, np.nan, np.nan, np.nan, np.nan

        pooled_effect = (w_re * df[effect_col]).sum() / sum_w_re
        pooled_var = 1 / sum_w_re
        pooled_se = np.sqrt(pooled_var)

        z_crit = norm.ppf(1 - alpha / 2)
        ci_lower = pooled_effect - z_crit * pooled_se
        ci_upper = pooled_effect + z_crit * pooled_se

        # Calculate I-squared
        w_fixed = 1 / df[var_col]
        sum_w_fixed = w_fixed.sum()
        pooled_effect_fe = (w_fixed * df[effect_col]).sum() / sum_w_fixed
        Q = (w_fixed * (df[effect_col] - pooled_effect_fe)**2).sum()
        df_Q = k - 1
        I_sq = max(0, ((Q - df_Q) / Q) * 100) if Q > 0 else 0

        return pooled_effect, pooled_se, ci_lower, ci_upper, I_sq
    except Exception:
        return np.nan, np.nan, np.nan, np.nan, np.nan


def calculate_knapp_hartung_ci(yi, vi, tau_sq, pooled_effect, alpha=0.05):
    """
    Calculate Knapp-Hartung adjusted confidence interval
    """

    # Convert to numpy arrays
    yi = np.array(yi)
    vi = np.array(vi)

    # Random-effects weights
    wi_star = 1 / (vi + tau_sq)
    sum_wi_star = np.sum(wi_star)

    # Degrees of freedom
    k = len(yi)
    df = k - 1

    if df <= 0:
        # Can't use K-H with k=1
        return None

    # Calculate Q statistic (residual heterogeneity)
    Q = np.sum(wi_star * (yi - pooled_effect)**2)

    # Standard random-effects variance
    var_standard = 1 / sum_wi_star

    # Knapp-Hartung adjusted variance
    # SE_KH² = (Q / (k-1)) × (1 / Σw*)
    var_KH = (Q / df) * var_standard
    se_KH = np.sqrt(var_KH)

    # t-distribution critical value
    t_crit = t.ppf(1 - alpha/2, df)

    # Confidence interval
    ci_lower = pooled_effect - t_crit * se_KH
    ci_upper = pooled_effect + t_crit * se_KH

    # Test statistic and p-value
    t_stat = pooled_effect / se_KH
    p_value = 2 * (1 - t.cdf(abs(t_stat), df))

    return {
        'se_KH': se_KH,
        'var_KH': var_KH,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        't_stat': t_stat,
        't_crit': t_crit,
        'df': df,
        'p_value': p_value,
        'Q': Q
    }



def compare_tau_estimators(df, effect_col, var_col):
    """
    Compare all tau-squared estimators on the same dataset

    Useful for sensitivity analysis and understanding which method
    is most appropriate for your data.

    Parameters:
    -----------
    df : DataFrame
        Data with effect sizes and variances
    effect_col : str
        Name of effect size column
    var_col : str
        Name of variance column

    Returns:
    --------
    DataFrame : Comparison of all methods
    """
    methods = ['DL', 'REML', 'ML', 'PM', 'SJ']
    results = []

    for method in methods:
        try:
            tau_sq, info = calculate_tau_squared(df, effect_col, var_col, method=method)

            results.append({
                'Method': method,
                'τ²': tau_sq,
                'τ': np.sqrt(tau_sq),
                'Success': info['success']
            })
        except Exception as e:
            results.append({
                'Method': method,
                'τ²': np.nan,
                'τ': np.nan,
                'Success': False
            })

    comparison_df = pd.DataFrame(results)

    return comparison_df



def calculate_hedges_g_python(df):
    """Calculate Hedges' g using EXACT Gamma correction."""
    df = df.copy()

    # Pooled SD
    n_e, n_c = df['ne'], df['nc']
    sd_e, sd_c = df['sde'], df['sdc']
    mean_e, mean_c = df['xe'], df['xc']

    df_d = n_e + n_c - 2
    sd_pooled = np.sqrt(((n_e - 1)*sd_e**2 + (n_c - 1)*sd_c**2) / df_d)

    # Cohen's d
    d = (mean_e - mean_c) / sd_pooled

    # Hedges' correction (J) - EXACT FORMULA to match metafor
    # J = exp(lgamma(m/2) - log(sqrt(m/2)) - lgamma((m-1)/2))
    m = df_d
    J = gamma(m / 2) / (np.sqrt(m / 2) * gamma((m - 1) / 2))

    g = d * J

    # Variance of g (Exact)
    vg = ((n_e + n_c) / (n_e * n_c) + (g**2 / (2 * (n_e + n_c)))) * J**2

    return g, vg


# --- THREE-LEVEL MODEL FUNCTIONS ---

def _get_three_level_estimates(params, y_all, v_all, N_total, M_studies):
    """
    Core function to calculate estimates using Sherman-Morrison inversion.
    Matches R's metafor implementation logic.
    """
    try:
        tau_sq, sigma_sq = params
        # Safety check for negatives
        if tau_sq < 0: tau_sq = 1e-10
        if sigma_sq < 0: sigma_sq = 1e-10

        sum_log_det_Vi = 0.0
        sum_S = 0.0       # 1' * V_i⁻¹ * 1
        sum_Sy = 0.0      # 1' * V_i⁻¹ * y_i
        sum_ySy = 0.0     # y_i' * V_i⁻¹ * y_i

        for i in range(M_studies):
            y_i = y_all[i]
            v_i = v_all[i]

            # V_i = A + τ²J, where A = diag(v_ij + σ²)
            A_diag = v_i + sigma_sq
            inv_A_diag = 1.0 / A_diag

            # Components for Sherman-Morrison
            sum_inv_A = np.sum(inv_A_diag)
            denom = 1 + tau_sq * sum_inv_A

            # Log Determinant
            log_det_A = np.sum(np.log(A_diag))
            log_det_Vi = log_det_A + np.log(denom)
            sum_log_det_Vi += log_det_Vi

            # Inversion: V⁻¹y
            inv_A_y = inv_A_diag * y_i
            sum_inv_A_y = np.sum(inv_A_y)
            w_y = inv_A_y - (tau_sq * inv_A_diag * sum_inv_A_y) / denom

            # Inversion: V⁻¹1
            w_1 = inv_A_diag - (tau_sq * inv_A_diag * sum_inv_A) / denom

            # Summing up
            sum_S += np.sum(w_1)
            sum_Sy += np.sum(w_y) # Note: sum(w_y) is effectively 1' * V^-1 * y
            sum_ySy += np.dot(y_i, w_y)

        if sum_S <= 1e-10:
            return {'log_lik_reml': np.inf}

        # Pooled Effect (μ)
        mu_hat = sum_Sy / sum_S
        var_mu = 1.0 / sum_S
        se_mu = np.sqrt(var_mu)

        # Residual Sum of Squares
        # (y - Xb)' V^-1 (y - Xb) = y'V^-1y - 2b X'V^-1y + b' X'V^-1X b
        residual_ss = sum_ySy - 2.0 * mu_hat * sum_Sy + mu_hat**2 * sum_S

        # REML Log-Likelihood
        log_lik_reml = -0.5 * (sum_log_det_Vi + np.log(sum_S) + residual_ss)

        # ML Log-Likelihood (for AIC/BIC)
        log_lik_ml = -0.5 * (N_total * np.log(2.0 * np.pi) + sum_log_det_Vi + residual_ss)

        return {
            'mu': mu_hat, 'se_mu': se_mu, 'var_mu': var_mu,
            'log_lik_reml': log_lik_reml, 'log_lik_ml': log_lik_ml,
            'tau_sq': tau_sq, 'sigma_sq': sigma_sq
        }

    except (FloatingPointError, ValueError, np.linalg.LinAlgError):
        return {'log_lik_reml': np.inf}


def _negative_log_likelihood_reml(params, y_all, v_all, N_total, M_studies):
    """Wrapper for optimizer."""
    estimates = _get_three_level_estimates(params, y_all, v_all, N_total, M_studies)
    return -estimates['log_lik_reml']


def _get_three_level_estimates_loo(params, y_all, v_all, N_total, M_studies):
    """Core calculation for 3-level estimates (Silent Version)."""
    try:
        tau_sq, sigma_sq = params
        if tau_sq < 0: tau_sq = 1e-10
        if sigma_sq < 0: sigma_sq = 1e-10

        sum_log_det_Vi = 0.0
        sum_S = 0.0
        sum_Sy = 0.0
        sum_ySy = 0.0

        for i in range(M_studies):
            y_i = y_all[i]
            v_i = v_all[i]

            A_diag = v_i + sigma_sq
            inv_A_diag = 1.0 / A_diag
            sum_inv_A = np.sum(inv_A_diag)
            denom = 1 + tau_sq * sum_inv_A

            log_det_A = np.sum(np.log(A_diag))
            sum_log_det_Vi += log_det_A + np.log(denom)

            inv_A_y = inv_A_diag * y_i
            sum_inv_A_y = np.sum(inv_A_y)

            w_y = inv_A_y - (tau_sq * inv_A_diag * sum_inv_A_y) / denom
            w_1 = inv_A_diag - (tau_sq * inv_A_diag * sum_inv_A) / denom

            sum_S += np.sum(w_1)
            sum_Sy += np.sum(w_y)
            sum_ySy += np.dot(y_i, w_y)

        if sum_S <= 1e-10: return {'log_lik_reml': np.inf}

        mu_hat = sum_Sy / sum_S
        var_mu = 1.0 / sum_S
        se_mu = np.sqrt(var_mu)
        residual_ss = sum_ySy - 2.0 * mu_hat * sum_Sy + mu_hat**2 * sum_S

        log_lik_reml = -0.5 * (sum_log_det_Vi + np.log(sum_S) + residual_ss)
        if np.isnan(log_lik_reml): return {'log_lik_reml': np.inf}

        return {'mu': mu_hat, 'se_mu': se_mu, 'log_lik_reml': log_lik_reml,
                'tau_sq': tau_sq, 'sigma_sq': sigma_sq}

    except (FloatingPointError, ValueError, np.linalg.LinAlgError):
        return {'log_lik_reml': np.inf}


def _neg_log_lik_reml_loo(params, y_all, v_all, N_total, M_studies):
    est = _get_three_level_estimates_loo(params, y_all, v_all, N_total, M_studies)
    return -est['log_lik_reml']


def _negative_log_likelihood_reml_loo(params, y_all, v_all, N_total, M_studies):
    """Wrapper for optimizer."""
    estimates = _get_three_level_estimates_loo(params, y_all, v_all, N_total, M_studies)
    return -estimates['log_lik_reml']


def _neg_log_lik_reml(params, y, v, groups):
    tau2, sigma2 = params
    # Bounds are handled by optimizer, but safe-guard here for math domain errors
    if tau2 < 0: tau2 = 1e-10
    if sigma2 < 0: sigma2 = 1e-10

    unique_groups = np.unique(groups)

    log_lik = 0
    sum_S = 0
    sum_Sy = 0
    sum_ySy = 0

    for grp in unique_groups:
        mask = (groups == grp)
        y_i = y[mask]
        v_i = v[mask]

        # V_i = D + sigma2*I + tau2*J
        # A = D + sigma2*I (Diagonal matrix)
        A_diag = v_i + sigma2
        inv_A_diag = 1.0 / A_diag

        # Woodbury/Sherman-Morrison components
        # (A + uv^T)^-1 = A^-1 - (A^-1 u v^T A^-1) / (1 + v^T A^-1 u)
        # Here u = v = tau * 1

        sum_inv_A = np.sum(inv_A_diag)
        denom = 1 + tau2 * sum_inv_A

        # Log Determinant of V_i
        # det(A + uv^T) = det(A) * (1 + v^T A^-1 u)
        log_det_A = np.sum(np.log(A_diag))
        log_det_Vi = log_det_A + np.log(denom)
        log_lik += log_det_Vi

        # Inversion Operations
        inv_A_y = inv_A_diag * y_i
        # w_y = V_i^-1 * y_i
        w_y = inv_A_y - (tau2 * inv_A_diag * np.sum(inv_A_y)) / denom

        # w_1 = V_i^-1 * 1
        w_1 = inv_A_diag - (tau2 * inv_A_diag * sum_inv_A) / denom

        sum_S += np.sum(w_1)      # 1^T V^-1 1
        sum_Sy += np.sum(w_y)     # 1^T V^-1 y
        sum_ySy += np.dot(y_i, w_y) # y^T V^-1 y

    # REML Profile Likelihood Calculation
    mu = sum_Sy / sum_S
    resid = sum_ySy - 2*mu*sum_Sy + mu**2 * sum_S

    # Full REML Log Likelihood
    total_log_lik = -0.5 * (log_lik + np.log(sum_S) + resid)

    return -total_log_lik


def run_python_3level(yi, vi, study_ids):
    # 1. First Pass: L-BFGS-B (Global search)
    best_res = None
    best_fun = np.inf

    # Multiple start points to avoid local minima
    start_points = [[0.01, 0.01], [0.5, 0.1], [0.1, 0.5], [0.001, 0.001]]

    for start in start_points:
        res = minimize(_neg_log_lik_reml, x0=start, args=(yi, vi, study_ids),
                       bounds=[(1e-8, None), (1e-8, None)],
                       method='L-BFGS-B',
                       options={'ftol': 1e-12, 'gtol': 1e-12}) # High precision
        if res.success and res.fun < best_fun:
            best_fun = res.fun
            best_res = res

    if not best_res: return None

    # 2. Second Pass: Nelder-Mead (Polishing)
    # Sometimes gradient methods get stuck slightly off in flat valleys
    final_res = minimize(_neg_log_lik_reml, x0=best_res.x, args=(yi, vi, study_ids),
                         method='Nelder-Mead',
                         bounds=[(1e-8, None), (1e-8, None)],
                         options={'xatol': 1e-12, 'fatol': 1e-12})

    tau2, sigma2 = final_res.x
    return tau2, sigma2


# --- REGRESSION FUNCTIONS ---

def _get_three_level_regression_estimates_v2(params, y_all, v_all, X_all, N_total, M_studies, p_params):
    """Calculates betas (slopes) and likelihood for a given tau2/sigma2."""
    try:
        tau_sq, sigma_sq = params
        if tau_sq < 0: tau_sq = 1e-10
        if sigma_sq < 0: sigma_sq = 1e-10

        sum_log_det_Vi = 0.0
        sum_XWX = np.zeros((p_params, p_params))
        sum_XWy = np.zeros(p_params)
        sum_yWy = 0.0

        for i in range(M_studies):
            y_i = y_all[i]
            v_i = v_all[i]
            X_i = X_all[i] # Predictors for study i

            # V_i = D + sigma2*I + tau2*J
            # Inversion using Sherman-Morrison
            A_diag = v_i + sigma_sq
            inv_A_diag = 1.0 / A_diag

            sum_inv_A = np.sum(inv_A_diag)
            denom = 1 + tau_sq * sum_inv_A

            # Log Determinant
            log_det_A = np.sum(np.log(A_diag))
            sum_log_det_Vi += log_det_A + np.log(denom)

            # Fast Matrix Multiplication for X' V^-1 X and X' V^-1 y
            # V^-1 = A^-1 - (tau2 * A^-1 J A^-1) / denom

            # 1. Precompute A^-1 * X and A^-1 * y
            # (Since A is diagonal, this is just element-wise mult)
            inv_A_X = inv_A_diag[:, None] * X_i
            inv_A_y = inv_A_diag * y_i

            # 2. Compute column sums (equivalent to 1' A^-1 X)
            sum_inv_A_X = np.sum(inv_A_X, axis=0)
            sum_inv_A_y = np.sum(inv_A_y)

            # 3. Compute W * X and W * y
            # W X = inv_A_X - (tau2 * inv_A_1 * sum_inv_A_X) / denom
            # Note: inv_A_1 is just inv_A_diag

            # We don't need full W_X, just X' W X
            # X' W X = X' (A^-1 X) - X' (correction)
            #        = (X' A^-1 X) - (tau2 / denom) * (X' A^-1 1) * (1' A^-1 X)

            xt_invA_x = X_i.T @ inv_A_X
            correction_term = (tau_sq / denom) * np.outer(sum_inv_A_X, sum_inv_A_X)
            sum_XWX += xt_invA_x - correction_term

            # Same for y
            xt_invA_y = X_i.T @ inv_A_y
            correction_y = (tau_sq / denom) * sum_inv_A_X * sum_inv_A_y
            sum_XWy += xt_invA_y - correction_y

            # Same for y'Wy
            yt_invA_y = np.dot(y_i, inv_A_y)
            correction_yy = (tau_sq / denom) * (sum_inv_A_y**2)
            sum_yWy += yt_invA_y - correction_yy

        # Solve for Betas: (X' W X) beta = X' W y
        # Add small jitter for stability if singular
        try:
            betas = np.linalg.solve(sum_XWX, sum_XWy)
            var_betas = np.linalg.inv(sum_XWX)
        except np.linalg.LinAlgError:
            return {'log_lik_reml': np.inf}

        # Calculate Residual Sum of Squares for REML
        # RSS = y'Wy - b' X'Wy
        residual_ss = sum_yWy - np.dot(betas, sum_XWy)

        # REML Log Likelihood
        # L = -0.5 * [ log|V| + log|X'V^-1X| + (y-Xb)'V^-1(y-Xb) ]
        # We use sign, logdet = np.linalg.slogdet(sum_XWX) for stability
        sign, log_det_XWX = np.linalg.slogdet(sum_XWX)

        log_lik_reml = -0.5 * (sum_log_det_Vi + log_det_XWX + residual_ss)

        return {
            'betas': betas,
            'se_betas': np.sqrt(np.diag(var_betas)),
            'var_betas_robust': var_betas, # Saving standard var-cov for now
            'log_lik_reml': log_lik_reml,
            'tau_sq': tau_sq,
            'sigma_sq': sigma_sq
        }

    except (FloatingPointError, ValueError, np.linalg.LinAlgError):
        return {'log_lik_reml': np.inf}


def _neg_log_lik_reml_reg(params, y_all, v_all, X_all, N_total, M_studies, p_params):
    est = _get_three_level_regression_estimates_v2(params, y_all, v_all, X_all, N_total, M_studies, p_params)
    return -est['log_lik_reml']


def _negative_log_likelihood_reml_reg_v2(params, y_all, v_all, X_all, N_total, M_studies, p_params):
    """Wrapper for optimizer."""
    estimates = _get_three_level_regression_estimates_v2(params, y_all, v_all, X_all, N_total, M_studies, p_params)
    return -estimates['log_lik_reml']


# --- PLOTTING FUNCTIONS ---

def plot_trim_fill(data, effect_col, se_col, results, es_label):
    """Simple Forest Plot for Trim/Fill (Preview)"""
    k0 = results['k0']
    orig_est = results['pooled_original']
    fill_est = results['pooled_filled']

    fig, ax = plt.subplots(figsize=(10, 6))

    # Plot Original Studies
    ax.scatter(data[effect_col], data[se_col], c='black', alpha=0.6, label='Observed Studies')

    # Plot Filled Studies
    if k0 > 0:
        se_filled = np.sqrt(results['vi_filled'])
        ax.scatter(results['yi_filled'], se_filled, c='white', edgecolors='red', marker='o', label='Imputed Studies')

    # Plot Center Lines
    ax.axvline(orig_est, color='black', linestyle='--', label=f'Original: {orig_est:.3f}')
    ax.axvline(fill_est, color='red', linestyle='-', label=f'Adjusted: {fill_est:.3f}')

    y_max = data[se_col].max() * 1.1
    ax.set_ylim(y_max, 0)
    ax.set_xlabel(es_label)
    ax.set_ylabel("Standard Error")
    ax.set_title(f"Trim-and-Fill Funnel Plot (Missing: {results['side']})")
    ax.legend()
    plt.show()

# functions

#@title 🔧 ADVANCED HETEROGENEITY ESTIMATORS

# =============================================================================
# CELL 4.5: ADVANCED TAU-SQUARED ESTIMATORS
# Purpose: Provides multiple methods for estimating between-study variance
# Dependencies: None (standalone functions)
# Used by: Cell 6 (Overall Analysis), Cell 8 (Subgroup Analysis)
# =============================================================================

import numpy as np
import pandas as pd
from scipy.optimize import minimize_scalar, minimize
from scipy.stats import chi2
import warnings

print("="*70)
print("HETEROGENEITY ESTIMATORS MODULE")
print("="*70)

# --- 1. DERSIMONIAN-LAIRD (Your current method) ---

# --- 2. RESTRICTED MAXIMUM LIKELIHOOD (REML) ---

def calculate_tau_squared_REML(df, effect_col, var_col, max_iter=100, tol=1e-8):
    """
    REML estimator for tau-squared (RECOMMENDED - Gold Standard)

    Advantages:
    - Unbiased for tau²
    - Accounts for uncertainty in estimating mu
    - Better performance in small samples
    - Generally preferred in literature

    Disadvantages:
    - Iterative (slightly slower)
    - Can fail to converge in extreme cases

    Reference:
    Viechtbauer, W. (2005). Bias and efficiency of meta-analytic variance
    estimators in the random-effects model. Journal of Educational and
    Behavioral Statistics, 30(3), 261-293.

    Parameters:
    -----------
    df : DataFrame
        Data with effect sizes and variances
    effect_col : str
        Name of effect size column
    var_col : str
        Name of variance column
    max_iter : int
        Maximum iterations for optimization
    tol : float
        Convergence tolerance

    Returns:
    --------
    float : tau-squared estimate
    """
    k = len(df)
    if k < 2:
        return 0.0

    try:
        # Extract data
        yi = df[effect_col].values
        vi = df[var_col].values

        # Remove any infinite or negative variances
        valid_mask = np.isfinite(vi) & (vi > 0)
        if not valid_mask.all():
            warnings.warn(f"Removed {(~valid_mask).sum()} observations with invalid variances")
            yi = yi[valid_mask]
            vi = vi[valid_mask]
            k = len(yi)

        if k < 2:
            return 0.0

        # REML objective function (negative log-likelihood)
        def reml_objective(tau2):
            # Ensure tau2 is non-negative
            tau2 = max(0, tau2)

            # Weights
            wi = 1 / (vi + tau2)
            sum_wi = wi.sum()

            if sum_wi <= 0:
                return 1e10

            # Pooled estimate
            mu = (wi * yi).sum() / sum_wi

            # Q statistic
            Q = (wi * (yi - mu)**2).sum()

            # REML log-likelihood (negative for minimization)
            # L = -0.5 * [sum(log(vi + tau2)) + log(sum(wi)) + Q]
            log_lik = -0.5 * (
                np.sum(np.log(vi + tau2)) +
                np.log(sum_wi) +
                Q
            )

            return -log_lik  # Return negative for minimization

        # Get reasonable bounds for tau2
        # Lower bound: 0
        # Upper bound: Use variance of effect sizes as upper limit
        var_yi = np.var(yi, ddof=1) if k > 2 else 1.0
        upper_bound = max(10 * var_yi, 100)

        # Optimize
        result = minimize_scalar(
            reml_objective,
            bounds=(0, upper_bound),
            method='bounded',
            options={'maxiter': max_iter, 'xatol': tol}
        )

        if result.success:
            tau_sq = result.x
        else:
            warnings.warn("REML optimization did not converge, using DL fallback")
            tau_sq = calculate_tau_squared_DL(df, effect_col, var_col)

        return max(0.0, tau_sq)

    except Exception as e:
        warnings.warn(f"Error in REML estimator: {e}, using DL fallback")
        return calculate_tau_squared_DL(df, effect_col, var_col)


# --- 3. MAXIMUM LIKELIHOOD (ML) ---

def calculate_tau_squared_ML(df, effect_col, var_col, max_iter=100, tol=1e-8):
    """
    Maximum Likelihood estimator for tau-squared

    Advantages:
    - Efficient asymptotically
    - Produces valid estimates

    Disadvantages:
    - Biased downward (underestimates tau²)
    - Less preferred than REML
    - REML is generally recommended instead

    Parameters:
    -----------
    df : DataFrame
        Data with effect sizes and variances
    effect_col : str
        Name of effect size column
    var_col : str
        Name of variance column
    max_iter : int
        Maximum iterations
    tol : float
        Convergence tolerance

    Returns:
    --------
    float : tau-squared estimate
    """
    k = len(df)
    if k < 2:
        return 0.0

    try:
        yi = df[effect_col].values
        vi = df[var_col].values

        valid_mask = np.isfinite(vi) & (vi > 0)
        if not valid_mask.all():
            yi = yi[valid_mask]
            vi = vi[valid_mask]
            k = len(yi)

        if k < 2:
            return 0.0

        # ML objective function
        def ml_objective(tau2):
            tau2 = max(0, tau2)
            wi = 1 / (vi + tau2)
            sum_wi = wi.sum()

            if sum_wi <= 0:
                return 1e10

            mu = (wi * yi).sum() / sum_wi
            Q = (wi * (yi - mu)**2).sum()

            # ML log-likelihood (without the constant term)
            log_lik = -0.5 * (np.sum(np.log(vi + tau2)) + Q)

            return -log_lik

        var_yi = np.var(yi, ddof=1) if k > 2 else 1.0
        upper_bound = max(10 * var_yi, 100)

        result = minimize_scalar(
            ml_objective,
            bounds=(0, upper_bound),
            method='bounded',
            options={'maxiter': max_iter, 'xatol': tol}
        )

        if result.success:
            tau_sq = result.x
        else:
            warnings.warn("ML optimization did not converge, using DL fallback")
            tau_sq = calculate_tau_squared_DL(df, effect_col, var_col)

        return max(0.0, tau_sq)

    except Exception as e:
        warnings.warn(f"Error in ML estimator: {e}, using DL fallback")
        return calculate_tau_squared_DL(df, effect_col, var_col)


# --- 4. PAULE-MANDEL (PM) ---

def calculate_tau_squared_PM(df, effect_col, var_col, max_iter=100, tol=1e-8):
    """
    Paule-Mandel estimator for tau-squared

    Advantages:
    - Exact solution to Q = k-1 equation
    - Non-iterative in principle
    - Good performance

    Disadvantages:
    - Can be unstable with few studies
    - Requires iterative solution in practice

    Reference:
    Paule, R. C., & Mandel, J. (1982). Consensus values and weighting factors.
    Journal of Research of the National Bureau of Standards, 87(5), 377-385.

    Parameters:
    -----------
    df : DataFrame
        Data with effect sizes and variances
    effect_col : str
        Name of effect size column
    var_col : str
        Name of variance column
    max_iter : int
        Maximum iterations
    tol : float
        Convergence tolerance

    Returns:
    --------
    float : tau-squared estimate
    """
    k = len(df)
    if k < 2:
        return 0.0

    try:
        yi = df[effect_col].values
        vi = df[var_col].values

        valid_mask = np.isfinite(vi) & (vi > 0)
        if not valid_mask.all():
            yi = yi[valid_mask]
            vi = vi[valid_mask]
            k = len(yi)

        if k < 2:
            return 0.0

        df_Q = k - 1

        # PM objective: Find tau2 such that Q(tau2) = k - 1
        def pm_objective(tau2):
            tau2 = max(0, tau2)
            wi = 1 / (vi + tau2)
            sum_wi = wi.sum()

            if sum_wi <= 0:
                return 1e10

            mu = (wi * yi).sum() / sum_wi
            Q = (wi * (yi - mu)**2).sum()

            # We want Q = k - 1
            return (Q - df_Q)**2

        var_yi = np.var(yi, ddof=1) if k > 2 else 1.0
        upper_bound = max(10 * var_yi, 100)

        result = minimize_scalar(
            pm_objective,
            bounds=(0, upper_bound),
            method='bounded',
            options={'maxiter': max_iter, 'xatol': tol}
        )

        if result.success and result.fun < 1:  # Good convergence
            tau_sq = result.x
        else:
            # If PM fails, use DL
            tau_sq = calculate_tau_squared_DL(df, effect_col, var_col)

        return max(0.0, tau_sq)

    except Exception as e:
        warnings.warn(f"Error in PM estimator: {e}, using DL fallback")
        return calculate_tau_squared_DL(df, effect_col, var_col)


# --- 5. SIDIK-JONKMAN (SJ) ---

def calculate_tau_squared_SJ(df, effect_col, var_col):
    """
    Sidik-Jonkman estimator for tau-squared

    Advantages:
    - Simple, non-iterative
    - Good performance with few studies
    - Conservative (tends to produce larger estimates)

    Disadvantages:
    - Can be overly conservative
    - Less commonly used

    Reference:
    Sidik, K., & Jonkman, J. N. (2005). Simple heterogeneity variance
    estimation for meta-analysis. Journal of the Royal Statistical Society,
    Series C, 54(2), 367-384.

    Parameters:
    -----------
    df : DataFrame
        Data with effect sizes and variances
    effect_col : str
        Name of effect size column
    var_col : str
        Name of variance column

    Returns:
    --------
    float : tau-squared estimate
    """
    k = len(df)
    if k < 3:  # Need at least 3 studies for SJ
        return calculate_tau_squared_DL(df, effect_col, var_col)

    try:
        yi = df[effect_col].values
        vi = df[var_col].values

        valid_mask = np.isfinite(vi) & (vi > 0)
        if not valid_mask.all():
            yi = yi[valid_mask]
            vi = vi[valid_mask]
            k = len(yi)

        if k < 3:
            return calculate_tau_squared_DL(df, effect_col, var_col)

        # Weights for typical average
        wi = 1 / vi
        sum_wi = wi.sum()

        # Typical average (weighted mean)
        y_bar = (wi * yi).sum() / sum_wi

        # SJ estimator
        numerator = ((yi - y_bar)**2 / vi).sum()
        denominator = k - 1

        tau_sq = (numerator / denominator) - (k / sum_wi)

        return max(0.0, tau_sq)

    except Exception as e:
        warnings.warn(f"Error in SJ estimator: {e}, using DL fallback")
        return calculate_tau_squared_DL(df, effect_col, var_col)


# --- 6. UNIFIED ESTIMATOR FUNCTION ---

# --- 7. COMPARISON FUNCTION ---

# --- 8. DISPLAY MODULE INFO ---
print("\n✅ Heterogeneity estimators loaded successfully")


META-ANALYSIS PIPELINE - INITIALIZATION
Execution Time: 2025-11-21 22:31:40
----------------------------------------------------------------------

📦 LIBRARY VERSIONS:
  • NumPy:      2.0.2
  • Pandas:     2.2.2
  • gspread:    6.2.1
  • Matplotlib: 3.10.0

⚙️  CONFIGURATION:
  • Required effect data columns: xe, sde, ne, xc, sdc, nc
  • Required metadata columns:    id
  • Supported effect sizes:       4
      - lnRR: Log Response Ratio
      - hedges_g: Hedges' g (corrected SMD)
      - cohen_d: Cohen's d (uncorrected SMD)
      - log_OR: Log Odds Ratio

INITIALIZATION STATUS
Authentication:  ✓ SUCCESS
Details:         Google Sheets API access granted
Ready:           YES ✓

✅ Setup complete. Proceed to next cell to load data.

HETEROGENEITY ESTIMATORS MODULE

✅ Heterogeneity estimators loaded successfully


In [ ]:
#@title 📁 Step 1: LOAD DATA

# =============================================================================
# CELL 2: LOAD DATA FROM GOOGLE SHEETS
# Purpose: Authenticate and load the raw DataFrame from a selected worksheet.
# Dependencies: Cell 1 (authentication and libraries)
# Outputs: Global 'raw_data_from_sheet' DataFrame
# =============================================================================

# --- 1. Authenticate (Silently) ---
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
except Exception as e:
    print(f"✗ Authentication failed: {e}")
    raise

# --- 2. Widget Definitions ---

# Step 1: Select Google Sheet
sheetName_widget = widgets.Text(
    value='tesis',
    description='1. GSheet Name:',
    layout=widgets.Layout(width='500px'),
    style={'description_width': '120px'}
)
load_sheets_button = widgets.Button(description="Fetch Worksheets", button_style='primary')
sheet_loader_output = widgets.Output()

# Step 2: Select Worksheet
worksheet_select_widget = widgets.Dropdown(
    options=[],
    description='2. Select Sheet:',
    layout=widgets.Layout(width='500px'),
    style={'description_width': '120px'},
    disabled=True
)
load_data_button = widgets.Button(description="Load Data from Sheet", button_style='success', disabled=True)
data_loader_output = widgets.Output()

# --- 3. Widget Handlers ---

def on_load_sheets_clicked(b):
    """Event handler for 'Fetch Worksheets' button."""
    with sheet_loader_output:
        clear_output(wait=True)
        sheet_name = sheetName_widget.value
        if not sheet_name:
            print("✗ Please enter a Google Sheet name.")
            return

        print(f"Opening '{sheet_name}'...")
        try:
            global spreadsheet
            spreadsheet = gc.open(sheet_name)
            worksheets = spreadsheet.worksheets()
            worksheet_names = [ws.title for ws in worksheets]

            worksheet_select_widget.options = worksheet_names
            worksheet_select_widget.disabled = False
            load_data_button.disabled = False
            print(f"✓ Success! Found {len(worksheet_names)} worksheets. Please select one below.")

        except Exception as e:
            print(f"✗ ERROR opening Google Sheet: {e}")
            print("  Troubleshooting:")
            print("  1. Is the name spelled correctly?")
            print("  2. Have you shared the sheet with your Google Colab email?")
            worksheet_select_widget.options = []
            worksheet_select_widget.disabled = True
            load_data_button.disabled = True

def on_load_data_clicked(b):
    """Event handler for 'Load Data from Sheet' button."""
    with data_loader_output:
        clear_output(wait=True)
        worksheet_name = worksheet_select_widget.value
        if not worksheet_name:
            print("✗ Please select a worksheet.")
            return

        print(f"Loading data from '{worksheet_name}'...")
        try:
            worksheet = spreadsheet.worksheet(worksheet_name)
            rows = worksheet.get_all_values()

            if not rows or len(rows) < 2:
                raise ValueError("Worksheet has no data or no header row.")

            # Create DataFrame
            column_names = rows[0]
            data_records = rows[1:]

            # Store in a global variable for the next cell
            global raw_data_from_sheet
            raw_data_from_sheet = pd.DataFrame.from_records(data_records, columns=column_names)

            print(f"✓ Data loaded successfully!")
            print(f"  • {raw_data_from_sheet.shape[0]} rows × {raw_data_from_sheet.shape[1]} columns found.")
            print("\n" + "="*70)
            print("✅ PLEASE PROCEED TO THE NEXT CELL TO CONFIGURE YOUR DATA")
            print("="*70)

        except Exception as e:
            print(f"✗ ERROR reading worksheet: {e}")

# --- 4. Attach Handlers ---
load_sheets_button.on_click(on_load_sheets_clicked)
load_data_button.on_click(on_load_data_clicked)

# --- 5. Display UI ---
box1 = widgets.VBox([
    widgets.HTML("<h3 style='color: #2E86AB;'>Step 1: Load Google Sheet</h3>"),
    sheetName_widget,
    load_sheets_button,
    sheet_loader_output
])

box2 = widgets.VBox([
    widgets.HTML("<h3 style='color: #2E86AB;'>Step 2: Select Worksheet & Load Data</h3>"),
    worksheet_select_widget,
    load_data_button,
    data_loader_output
])

display(box1, box2)

In [ ]:
#@title ⚙️ Step 2: CONFIGURE ANALYSIS

# =============================================================================
# CELL 3: CONFIGURE ANALYSIS FILTERS
# Purpose: Set up all filters and mappings for the analysis.
# Dependencies: Cell 2 (global 'raw_data_from_sheet')
# Outputs: 'ANALYSIS_CONFIG' dictionary with user's choices.
# =============================================================================

import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import pandas as pd
import numpy as np
import traceback

# --- 1. PRE-RUN: Check for Data and Find Moderators ---
try:
    if 'raw_data_from_sheet' not in globals():
        raise NameError("raw_data_from_sheet")

    # --- 1a. Helper function for auto-guessing columns ---
    def guess_column(options, matches, default=None):
        """Finds the best match from a list of options."""
        options_lower = [str(o).lower() for o in options]
        for match in matches:
            if match in options_lower:
                return options[options_lower.index(match)]
        return default if default else options[0] if options else None

    # --- 1b. Load data and find all columns ---
    all_column_names = list(raw_data_from_sheet.columns)
    if not all_column_names:
        raise ValueError("Data loaded from sheet has no columns.")

    # --- 1c. Auto-guess core columns to build a temporary_raw_data ---
    temp_col_map = {
        guess_column(all_column_names, ['id', 'study', 'study_id', 'paper']): 'id',
        guess_column(all_column_names, ['xe', 'mean_e', 'mean_exp', 'x_e']): 'xe',
        guess_column(all_column_names, ['sde', 'sd_e', 'sd_exp']): 'sde',
        guess_column(all_column_names, ['ne', 'n_e', 'n_exp']): 'ne',
        guess_column(all_column_names, ['xc', 'mean_c', 'mean_ctrl', 'x_c']): 'xc',
        guess_column(all_column_names, ['sdc', 'sd_c', 'sd_ctrl']): 'sdc',
        guess_column(all_column_names, ['nc', 'n_c', 'n_ctrl']): 'nc'
    }

    # Invert map for renaming, but handle None if a column wasn't found
    temp_col_map_inv = {v: k for k, v in temp_col_map.items() if k is not None}

    # Find other non-core columns
    other_cols = [col for col in all_column_names if col not in temp_col_map_inv.values()]

    # Create temporary cleaned data
    temp_raw_data = raw_data_from_sheet[list(temp_col_map_inv.values()) + other_cols].copy()
    temp_raw_data.rename(columns=temp_col_map_inv, inplace=True)

    # --- 1d. Run minimal cleaning just to find moderators ---
    for col in ['id']: # Only need ID for this step
        if col not in temp_raw_data.columns:
            temp_raw_data[col] = pd.Series(dtype='object')
    temp_raw_data['id'] = temp_raw_data['id'].astype(str).str.strip()

    # Find moderators
    excluded_cols = ['id', 'xe', 'sde', 'ne', 'xc', 'sdc', 'nc']
    available_moderators = [col for col in temp_raw_data.columns
                            if col not in excluded_cols
                            and temp_raw_data[col].dtype == 'object']

except NameError:
    display(HTML("<div style='background-color: #fff3cd; border: 1px solid #ffeeba; padding: 15px; border-radius: 5px; color: #856404;'>"
                 "<b>❌ ERROR: No data found.</b> Please run Cell 2 (LOAD DATA) successfully before running this cell."
                 "</div>"))
    raise
except Exception as e:
    display(HTML(f"<div style='background-color: #f8d7da; border: 1px solid #f5c6cb; padding: 15px; border-radius: 5px; color: #721c24;'>"
                 f"<b>❌ An error occurred during pre-load:</b> {e}<br>"
                 f"Please check your sheet and column names."
                 f"</div>"))
    raise

# --- 2. Widget Definitions ---

# --- Box 1: Column Mapping (Hidden in Accordion) ---
id_col_widget = widgets.Dropdown(description='Study ID (id):', options=all_column_names,
                                 value=temp_col_map_inv.get('id'),
                                 layout=widgets.Layout(width='500px'), style={'description_width': '150px'})
xe_col_widget = widgets.Dropdown(description='Exp. Mean (xe):', options=all_column_names,
                                 value=temp_col_map_inv.get('xe'),
                                 layout=widgets.Layout(width='500px'), style={'description_width': '150px'})
sde_col_widget = widgets.Dropdown(description='Exp. SD (sde):', options=all_column_names,
                                  value=temp_col_map_inv.get('sde'),
                                  layout=widgets.Layout(width='500px'), style={'description_width': '150px'})
ne_col_widget = widgets.Dropdown(description='Exp. N (ne):', options=all_column_names,
                                 value=temp_col_map_inv.get('ne'),
                                 layout=widgets.Layout(width='500px'), style={'description_width': '150px'})
xc_col_widget = widgets.Dropdown(description='Ctrl. Mean (xc):', options=all_column_names,
                                 value=temp_col_map_inv.get('xc'),
                                 layout=widgets.Layout(width='500px'), style={'description_width': '150px'})
sdc_col_widget = widgets.Dropdown(description='Ctrl. SD (sdc):', options=all_column_names,
                                  value=temp_col_map_inv.get('sdc'),
                                  layout=widgets.Layout(width='500px'), style={'description_width': '150px'})
nc_col_widget = widgets.Dropdown(description='Ctrl. N (nc):', options=all_column_names,
                                 value=temp_col_map_inv.get('nc'),
                                 layout=widgets.Layout(width='500px'), style={'description_width': '150px'})

column_mapping_box = widgets.VBox([
    widgets.HTML("Map your sheet's columns to the names the pipeline requires. The system has auto-guessed, but please verify."),
    id_col_widget,
    xe_col_widget, sde_col_widget, ne_col_widget,
    xc_col_widget, sdc_col_widget, nc_col_widget
])
column_accordion = widgets.Accordion(children=[column_mapping_box])
column_accordion.set_title(0, 'Step 2a (Optional): Verify Column Names')
column_accordion.selected_index = None # Start closed

# --- Box 2: Analysis Configuration ---
prefilter_col_widget = widgets.Dropdown(description='Filter by:', options=['None'] + available_moderators, value='None',
                                        style={'description_width': '120px'}, layout=widgets.Layout(width='500px'))
prefilter_values_widget = widgets.VBox()
filterCol1_widget = widgets.Dropdown(description='Factor 1:', options=available_moderators if available_moderators else ['None'],
                                     value=available_moderators[0] if available_moderators else 'None',
                                     style={'description_width': '120px'}, layout=widgets.Layout(width='500px'))
filterCol2_widget = widgets.Dropdown(description='Factor 2:', options=['None'] + available_moderators, value='None',
                                     style={'description_width': '120px'}, layout=widgets.Layout(width='500px'))
minPapers_widget = widgets.IntSlider(value=2, min=1, max=10, step=1, description='Min Papers:',
                                     style={'description_width': '120px'}, layout=widgets.Layout(width='500px'))
minObservations_widget = widgets.IntSlider(value=2, min=1, max=20, step=1, description='Min Observations:',
                                           style={'description_width': '120px'}, layout=widgets.Layout(width='500px'))

# --- Box 3: Final Button ---
save_config_button = widgets.Button(
    description='▶ Save Configuration',
    button_style='success',
    layout=widgets.Layout(width='500px', height='50px'),
    style={'font_weight': 'bold', 'font_size': '14px'}
)
output_area = widgets.Output()

# --- 4. Widget Handlers ---

def update_prefilter_checkboxes(change):
    """Update checkboxes when column selection changes"""
    selected_col = change['new']
    if selected_col == 'None':
        prefilter_values_widget.children = []
        return

    try:
        # Use the *uncleaned* temp_raw_data for a quick preview
        unique_values = sorted(temp_raw_data[selected_col].dropna().unique())
        checkboxes = [
            widgets.Checkbox(
                value=True,
                description=f"{val} (n={len(temp_raw_data[temp_raw_data[selected_col] == val])})",
                style={'description_width': 'initial'},
                layout=widgets.Layout(width='500px')
            ) for val in unique_values
        ]
        prefilter_values_widget.children = [
            widgets.HTML("<p style='margin: 10px 0; font-weight: bold;'>Select values to KEEP:</p>")
        ] + checkboxes
    except Exception as e:
        prefilter_values_widget.children = [widgets.HTML(f"<p style='color: red;'>Error updating list: {e}</p>")]

prefilter_col_widget.observe(update_prefilter_checkboxes, names='value')

@save_config_button.on_click
def on_save_config_clicked(b):
    """Main function: JUST save the config."""
    with output_area:
        clear_output(wait=True)
        """
        print("="*70)
        print("CONFIGURING ANALYSIS")
        print("="*70)
        """
        try:
            # --- 1. Get Column Mappings ---
            global col_map
            col_map = {
                id_col_widget.value: 'id',
                xe_col_widget.value: 'xe',
                sde_col_widget.value: 'sde',
                ne_col_widget.value: 'ne',
                xc_col_widget.value: 'xc',
                sdc_col_widget.value: 'sdc',
                nc_col_widget.value: 'nc'
            }

            # Check for duplicate mappings
            mapped_keys = [k for k in col_map.keys() if k is not None]
            if len(set(mapped_keys)) != len(mapped_keys):
                raise ValueError("Duplicate columns mapped. Please assign one sheet column to one role.")

            # --- 2. Get Pre-filter selections ---
            prefilter_col = prefilter_col_widget.value
            selected_values = []
            if prefilter_col != 'None':
                selected_values = [
                    cb.description.split(' (n=')[0]
                    for cb in prefilter_values_widget.children[1:] # Skip HTML title
                    if hasattr(cb, 'value') and cb.value
                ]

            # --- 3. Save Configuration to Global ANALYSIS_CONFIG ---
            global ANALYSIS_CONFIG
            ANALYSIS_CONFIG = {
                'col_map': col_map,
                'prefilter_col': prefilter_col,
                'prefilter_values_kept': selected_values if prefilter_col != 'None' else 'All',
                'filterCol1': filterCol1_widget.value,
                'filterCol2': filterCol2_widget.value,
                'minPapers': minPapers_widget.value,
                'minObservations': minObservations_widget.value,
            }

            # --- 4. Print Final Summary ---

            print("\n" + "="*70)
            print("✅ CONFIGURATION SAVED RUN THE NEXT CELL TO APPLY")
            print("="*70)
            """
            print("\n📋 Analysis Configuration Summary:")
            print("-" * 70)
            print(f"  1️⃣  COLUMN MAPPING:")
            print(f"      • Study ID: '{id_col_widget.value}'")
            print(f"      • Exp. Mean: '{xe_col_widget.value}'")
            print(f"      • Ctrl. Mean: '{xc_col_widget.value}'")
            print(f"  2️⃣  SUBGROUP ANALYSIS:")
            print(f"      • Primary factor:   {ANALYSIS_CONFIG['filterCol1']}")
            print(f"      • Secondary factor: {ANALYSIS_CONFIG['filterCol2']}")
            print(f"  3️⃣  QUALITY THRESHOLDS:")
            print(f"      • Min Papers:       {ANALYSIS_CONFIG['minPapers']}")
            print(f"      • Min Observations: {ANALYSIS_CONFIG['minObservations']}")
            print("\n" + "="*70)
            print("▶️  Run the next cell to clean data and apply this configuration.")
            print("="*70)
            """


        except Exception as e:
            print(f"\n❌ AN ERROR OCCURRED:\n")
            print(f"  Type: {type(e).__name__}")
            print(f"  Message: {e}")
            print("\n  Traceback:")
            traceback.print_exc(file=sys.stdout)

# --- 5. Assemble & Display Final UI ---
box1 = column_accordion
box2 = widgets.VBox([
    widgets.HTML("<h3 style='color: #2E86AB;'>Step 2b: Configure Analysis Filters</h3>"),
    widgets.HTML("<h4 style='color: #444; margin-bottom: 5px;'>📌 Pre-Filter (Optional)</h4>"),
    prefilter_col_widget,
    prefilter_values_widget,
    widgets.HTML("<hr style='margin: 10px 0; border: none; border-top: 1px solid #eee;'>"),
    widgets.HTML("<h4 style='color: #444; margin-bottom: 5px;'>📊 Subgroup Analysis</h4>"),
    filterCol1_widget,
    filterCol2_widget,
    widgets.HTML("<hr style='margin: 10px 0; border: none; border-top: 1px solid #eee;'>"),
    widgets.HTML("<h4 style='color: #444; margin-bottom: 5px;'>⚙️ Quality Filters</h4>"),
    minPapers_widget,
    minObservations_widget
])
box3 = widgets.VBox([
    widgets.HTML("<hr style='margin: 20px 0; border: none; border-top: 2px solid #ddd;'>"),
    widgets.HTML("<h3 style='color: #2E86AB;'>Step 2c: Save Configuration</h3>"),
    save_config_button,
    output_area
])

display(box1, box2, box3)

Accordion(children=(VBox(children=(HTML(value="Map your sheet's columns to the names the pipeline requires. Th…

In [ ]:
#@title ⚙️ Step 3: APPLY CONFIGURATION & PREPARE DATA

# =============================================================================
# CELL 4: CLEAN DATA & APPLY CONFIGURATION
# Purpose: Run cleaning and filtering based on choices from Cell 3.
# Dependencies: Cell 2 (global 'raw_data_from_sheet'), Cell 3 (global 'ANALYSIS_CONFIG')
# Outputs: Global 'raw_data' (cleaned), 'data_filtered', 'LOAD_METADATA'
# =============================================================================

import pandas as pd
import numpy as np
import traceback
"""
print("="*70)
print("APPLYING CONFIGURATION & PREPARING DATA")
print("="*70)
"""
try:
    # --- 1. Check for inputs ---
    if 'raw_data_from_sheet' not in globals():
        raise NameError("Data not loaded. Please re-run Cell 2.")
    if 'ANALYSIS_CONFIG' not in globals():
        raise NameError("Configuration not set. Please run Cell 3 and click 'Save Configuration'.")

    #print("STEP 1: Loading configuration from Cell 3...")
    col_map = ANALYSIS_CONFIG['col_map']

    # --- 2. Rename & Clean Data ---
    #print("STEP 2: Cleaning and converting data...")
    global raw_data

    mapped_cols = col_map.keys()
    other_cols = [col for col in raw_data_from_sheet.columns if col not in mapped_cols]

    raw_data = raw_data_from_sheet[list(mapped_cols) + other_cols].copy()
    raw_data.rename(columns=col_map, inplace=True)

    original_rows = len(raw_data)
    cleaning_log = []

    # Convert numeric columns
    numeric_columns = ['xe', 'sde', 'ne', 'xc', 'sdc', 'nc']
    for col in numeric_columns:
        if col not in raw_data.columns:
             raise ValueError(f"Mapped column '{col}' not found after loading.")
        raw_data[col] = raw_data[col].astype(str).str.strip().replace('', np.nan)
        raw_data[col] = pd.to_numeric(raw_data[col], errors='coerce')

    # Ensure ID is string
    raw_data['id'] = raw_data['id'].astype(str).str.strip()

    # Drop rows with missing essential values
    essential_cols = ['xe', 'ne', 'xc', 'nc']
    missing_essential = raw_data[essential_cols].isna().any(axis=1).sum()
    raw_data.dropna(subset=essential_cols, inplace=True)
    if missing_essential > 0:
        cleaning_log.append(f"Dropped {missing_essential} rows (missing xe/ne/xc/nc)")

    # Ensure N >= 1
    invalid_n_count = 0
    for col in ['ne', 'nc']:
        raw_data[col] = raw_data[col].fillna(0).astype(int)
        invalid_n = (raw_data[col] < 1).sum()
        if invalid_n > 0:
            raw_data = raw_data[raw_data[col] >= 1]
            invalid_n_count += invalid_n
    if invalid_n_count > 0:
        cleaning_log.append(f"Dropped {invalid_n_count} rows (n < 1)")

    final_rows = len(raw_data)
    print(f"  ✓ Clean dataset ready: {final_rows} rows remaining ({original_rows - final_rows} total removed)")

    # --- 3. Identify Moderators ---
    #print("STEP 3: Identifying moderators...")
    excluded_cols = ['id', 'xe', 'sde', 'ne', 'xc', 'sdc', 'nc']
    global available_moderators
    available_moderators = [col for col in raw_data.columns
                            if col not in excluded_cols
                            and raw_data[col].dtype == 'object']

    print(f"  ✓ Found {len(available_moderators)} potential moderators.")

    # --- 4. Apply Pre-filter (if selected) ---
    #print("STEP 4: Applying pre-filter...")
    global data_filtered
    data_filtered = raw_data.copy()

    prefilter_col = ANALYSIS_CONFIG['prefilter_col']
    selected_values = ANALYSIS_CONFIG['prefilter_values_kept']

    if prefilter_col != 'None':
        data_filtered = data_filtered[data_filtered[prefilter_col].isin(selected_values)]
        print(f"  ✓ Pre-filter applied. {len(data_filtered)} rows remain.")
    else:
        print("  ✓ No pre-filter applied.")

    # --- 5. Save Metadata ---
    global LOAD_METADATA
    LOAD_METADATA = {
        'timestamp': datetime.datetime.now(),
        'original_rows': original_rows,
        'final_rows_cleaned': final_rows,
        'final_rows_filtered': len(data_filtered),
        'cleaning_log': cleaning_log,
        'available_moderators': available_moderators,
        'column_map': col_map
    }

    # Update ANALYSIS_CONFIG with final counts
    ANALYSIS_CONFIG['n_observations_pre_filter'] = final_rows
    ANALYSIS_CONFIG['n_observations_post_filter'] = len(data_filtered)
    ANALYSIS_CONFIG['n_papers_post_filter'] = data_filtered['id'].nunique()

    # --- 6. Print Final Summary ---
    """
    print("\n" + "="*70)
    print("✅ DATA READY FOR ANALYSIS")
    print("="*70)
    """
    print("\n📋 Final Data Summary:")
    print("-" * 70)
    print(f"  • Rows available for analysis: {len(data_filtered)}")
    print(f"  • Unique studies: {data_filtered['id'].nunique()}")
    print(f"  • Subgroup Factor 1: {ANALYSIS_CONFIG['filterCol1']}")
    print(f"  • Subgroup Factor 2: {ANALYSIS_CONFIG['filterCol2']}")
    print("\n" + "="*70)
    print("▶️  Run the next cell to proceed.")
    print("="*70)

except Exception as e:
    print(f"\n❌ AN ERROR OCCURRED:\n")
    print(f"  Type: {type(e).__name__}")
    print(f"  Message: {e}")
    print("\n  Traceback:")
    traceback.print_exc(file=sys.stdout)

  ✓ Clean dataset ready: 429 rows remaining (0 total removed)
  ✓ Found 12 potential moderators.
  ✓ No pre-filter applied.

📋 Final Data Summary:
----------------------------------------------------------------------
  • Rows available for analysis: 429
  • Unique studies: 84
  • Subgroup Factor 1: Crop
  • Subgroup Factor 2: None

▶️  Run the next cell to proceed.


In [ ]:
#@title 🔬 DETECT & SELECT EFFECT SIZE TYPE

# =============================================================================
# CELL 4: EFFECT SIZE TYPE DETECTION AND SELECTION
# Purpose: Analyze data characteristics and recommend appropriate effect size
# Fixes: Added educational context to Tabs 2 & 3 for new users.
# =============================================================================

import numpy as np
import pandas as pd
import datetime
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# --- 1. STABILITY FIX: USE RAW DATA IF AVAILABLE ---
target_df = raw_data if 'raw_data' in globals() else data_filtered

# --- 2. ANALYZE DATA ---
xe_stats = target_df['xe'].describe()
xc_stats = target_df['xc'].describe()

# Standard Deviations
has_sde = 'sde' in target_df.columns and target_df['sde'].notna().any()
has_sdc = 'sdc' in target_df.columns and target_df['sdc'].notna().any()
sd_availability = target_df[['sde', 'sdc']].notna().all(axis=1).sum() if has_sde and has_sdc else 0
sd_pct = (sd_availability / len(target_df)) * 100 if len(target_df) > 0 else 0

# 1. Normalization Check
control_near_one = ((target_df['xc'] >= 0.95) & (target_df['xc'] <= 1.05)).sum()
control_exactly_one = (target_df['xc'] == 1.0).sum()
pct_control_near_one = (control_near_one / len(target_df)) * 100
pct_control_exactly_one = (control_exactly_one / len(target_df)) * 100

# 2. Negative Values
n_negative_xe = (target_df['xe'] < 0).sum()
n_negative_xc = (target_df['xc'] < 0).sum()
has_negative = n_negative_xe > 0 or n_negative_xc > 0

# 3. Zero Values
n_zero_xe = (target_df['xe'] == 0).sum()
n_zero_xc = (target_df['xc'] == 0).sum()
has_zero = n_zero_xe > 0 or n_zero_xc > 0

# 4. Scale Heterogeneity
xe_range = xe_stats['max'] - xe_stats['min']
xc_range = xc_stats['max'] - xc_stats['min']
scale_ratio = max(xe_range, xc_range) / (min(xe_range, xc_range) + 0.0001)

# --- 3. RECOMMENDATION LOGIC ---
score_lnRR = 0
score_hedges_g = 0
reasons = []

# Rule 1: Negatives (The "Hard" Constraint)
if has_negative:
    score_hedges_g += 10
    reasons.append(('Negative Values', '+++', 'Hedges g', 'Ratio metrics (lnRR) mathematically fail with negative numbers.'))
else:
    score_lnRR += 2
    reasons.append(('All Positive', '+', 'lnRR', 'Data is compatible with ratio-based metrics.'))

# Rule 2: Normalization
if pct_control_exactly_one > 50:
    score_lnRR += 5
    reasons.append(('Fold-Change Data', '+++', 'lnRR', 'Controls set to 1.0 implies data is already a ratio.'))
elif pct_control_near_one > 30:
    score_lnRR += 3
    reasons.append(('Normalized Data', '++', 'lnRR', 'Controls clustered around 1.0 suggests ratio data.'))
elif 0.8 < xc_stats['mean'] < 1.2:
    score_lnRR += 1
    reasons.append(('Unity Baseline', '+', 'lnRR', 'Control group mean is close to 1.0.'))

# Rule 3: Heterogeneity
if scale_ratio > 100:
    score_lnRR += 3
    reasons.append(('High Scale Variance', '+++', 'lnRR', 'Studies measure vastly different scales. Ratios handle this best.'))
elif scale_ratio > 10:
    score_lnRR += 2
    reasons.append(('Moderate Scale Variance', '++', 'lnRR', 'Ratios normalize scale differences effectively.'))
else:
    score_hedges_g += 1
    reasons.append(('Consistent Scales', '+', 'Hedges g', 'Scales are similar across studies; standardized differences work well.'))

# Rule 4: Zeros
if has_zero:
    score_hedges_g += 2
    reasons.append(('Zero Values', '++', 'Hedges g', 'log(0) is undefined. lnRR requires adding arbitrary constants.'))

# Rule 5: SD Availability
if sd_pct > 80:
    score_hedges_g += 1
    reasons.append(('Good SD Data', '+', 'Hedges g', 'Hedges g requires SDs. Your data has good coverage.'))
elif sd_pct < 20:
    reasons.append(('Missing SDs', '⚠', 'Neither', 'Hedges g requires imputation. Response Ratios might be safer if SDs are rare.'))

# Winner
score_diff = abs(score_lnRR - score_hedges_g)
if score_lnRR > score_hedges_g:
    recommended_type = 'lnRR'
    confidence = "High" if score_diff >= 5 else "Moderate" if score_diff >= 3 else "Low"
elif score_hedges_g > score_lnRR:
    recommended_type = 'hedges_g'
    confidence = "High" if score_diff >= 5 else "Moderate" if score_diff >= 3 else "Low"
else:
    recommended_type = 'hedges_g'
    confidence = "Low"

# --- 4. SETUP UI ---
tab_main = widgets.Output()
tab_patterns = widgets.Output()
tab_logic = widgets.Output()

tabs = widgets.Tab(children=[tab_main, tab_patterns, tab_logic])
tabs.set_title(0, '💡 Recommendation')
tabs.set_title(1, '📊 Data Patterns')
tabs.set_title(2, '🧠 Decision Logic')

# --- TAB 2: DATA PATTERNS (Educational) ---
with tab_patterns:
    display(HTML(f"""
    <div style='padding:10px; font-size:14px; line-height:1.6;'>
        <h4 style='margin-top:0; color:#2E86AB;'>🔍 Diagnostic Checks</h4>
        <p>We analyzed <b>{len(target_df)} observations</b> to determine the statistical properties of your dataset.
        Here is what we found:</p>

        <hr>

        <b>1️⃣ Control Group Normalization</b><br>
        Values near 1.0 often indicate "Fold-Change" data (e.g., gene expression normalized to a control).<br>
        • <b>Result:</b> {pct_control_exactly_one:.1f}% of controls are exactly 1.0.<br>
        • <b>Implication:</b> {'Strong evidence for Ratio data.' if pct_control_exactly_one > 50 else 'No strong evidence of pre-normalization.'}
        <br><br>

        <b>2️⃣ Negative Values</b><br>
        Log-based metrics (like lnRR) <i>cannot</i> mathematically handle negative numbers.<br>
        • <b>Result:</b> Found {n_negative_xe + n_negative_xc} negative values.<br>
        • <b>Implication:</b> {'❌ MUST use Standardized Difference (Hedges g).' if has_negative else '✓ Compatible with Ratio metrics.'}
        <br><br>

        <b>3️⃣ Zero Values</b><br>
        Log of zero is undefined. Zeros require adding a "small constant" to work with lnRR.<br>
        • <b>Result:</b> Found {n_zero_xe + n_zero_xc} zero values.<br>
        • <b>Implication:</b> {'⚠️ lnRR will require adjustment.' if has_zero else '✓ Clean data.'}
        <br><br>

        <b>4️⃣ Scale Heterogeneity</b><br>
        Do studies measure things on the same scale (e.g., all in grams) or different scales (grams vs. tons)?<br>
        • <b>Result:</b> Largest value is {scale_ratio:.1f}× larger than the smallest range.<br>
        • <b>Implication:</b> {'High variation favors Ratios (lnRR).' if scale_ratio > 10 else 'Low variation allows Standardized Differences.'}
        <br><br>

        <b>5️⃣ Data Completeness</b><br>
        Standardized differences (Hedges' g) require Standard Deviations (SD) to calculate.<br>
        • <b>Result:</b> {sd_pct:.1f}% of rows have valid SDs.<br>
    </div>
    """))

# --- TAB 3: LOGIC (Educational) ---
with tab_logic:
    # Create HTML table rows
    rows_html = ""
    for r in reasons:
        rows_html += f"<tr><td><b>{r[0]}</b></td><td>{r[1]}</td><td>{r[2]}</td><td>{r[3]}</td></tr>"

    display(HTML(f"""
    <div style='padding:10px; font-size:14px;'>
        <h4 style='margin-top:0; color:#2E86AB;'>🧠 How the Algorithm Decides</h4>
        <p>We use a weighted scoring system to recommend the most statistically appropriate effect size.
        Some factors (like negative values) are "hard constraints," while others are preferences.</p>

        <table style='width:100%; border-collapse:collapse; margin-top:10px;'>
            <tr style='background-color:#f0f0f0; text-align:left; border-bottom:2px solid #ddd;'>
                <th style='padding:8px;'>Diagnostic Factor</th>
                <th style='padding:8px;'>Weight</th>
                <th style='padding:8px;'>Favors</th>
                <th style='padding:8px;'>Educational Note</th>
            </tr>
            {rows_html}
        </table>

        <div style='margin-top:20px; padding:10px; background-color:#eef; border-radius:5px;'>
            <b>Final Score:</b><br>
            📊 <b>log Response Ratio (lnRR):</b> {score_lnRR} points<br>
            📊 <b>Hedges' g (SMD):</b> {score_hedges_g} points
        </div>
    </div>
    """))

# --- TAB 1: MAIN (Selection) ---
with tab_main:
    # 1. Recommendation Box
    if recommended_type == 'lnRR':
        html_rec = f"""
        <div style='background-color: #d4edda; border-left: 5px solid #28a745; padding: 15px; margin-bottom: 20px;'>
            <h3 style='color: #155724; margin-top: 0;'>💡 Recommendation: log Response Ratio (lnRR)</h3>
            <p style='color: #155724; margin-bottom: 0;'><b>Why?</b> Your data appears to be <b>ratio-based</b> (e.g., fold-changes, growth rates).
            lnRR is the natural metric for this data type because it handles scale differences and has a direct biological interpretation (% change).</p>
        </div>"""
    else:
        html_rec = f"""
        <div style='background-color: #d1ecf1; border-left: 5px solid #17a2b8; padding: 15px; margin-bottom: 20px;'>
            <h3 style='color: #0c5460; margin-top: 0;'>💡 Recommendation: Hedges' g (SMD)</h3>
            <p style='color: #0c5460; margin-bottom: 0;'><b>Why?</b> Your data appears to be <b>absolute measurements</b> on potentially different scales.
            Hedges' g is ideal here because it standardizes effects into "SD units," making them comparable even if units differ.</p>
        </div>"""

    display(HTML(html_rec))

    # 2. Selection Widget
    effect_size_widget = widgets.RadioButtons(
        options=[
            ('log Response Ratio (lnRR) - for ratio/fold-change data', 'lnRR'),
            ("Hedges' g - for standardized mean differences (corrected)", 'hedges_g'),
            ("Cohen's d - for standardized mean differences (uncorrected)", 'cohen_d'),
            ('log Odds Ratio (logOR) - for binary outcomes', 'log_or')
        ],
        value=recommended_type,
        description='Select Type:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='600px')
    )

    # 3. Info Panel
    info_output = widgets.Output()
    info_panels = {
        'lnRR': "<div style='padding:10px; background:#fff; border:1px solid #ddd; color:#555; font-size:13px;'><b>🎓 About lnRR:</b> Calculates the log-ratio of means (ln(Xe/Xc)). Essential for data where 'doubling' is the same magnitude of effect as 'halving'. Commonly used in ecology for biomass, abundance, and size.</div>",
        'hedges_g': "<div style='padding:10px; background:#fff; border:1px solid #ddd; color:#555; font-size:13px;'><b>🎓 About Hedges' g:</b> A variation of Cohen's d that includes a correction factor (J) for small sample sizes. It prevents overestimation of effects in small studies, making it the gold standard for SMD in meta-analysis.</div>",
        'cohen_d': "<div style='padding:10px; background:#fff; border:1px solid #ddd; color:#555; font-size:13px;'><b>🎓 About Cohen's d:</b> The classic standardized mean difference. It is slightly biased (too high) when sample sizes are small (N < 20). Hedges' g is usually preferred.</div>",
        'log_or': "<div style='padding:10px; background:#fff; border:1px solid #ddd; color:#555; font-size:13px;'><b>🎓 About logOR:</b> The log-odds ratio. Strictly for binary 'Yes/No' or 'Event/Non-event' data. Do not use for continuous measurements like weight or length.</div>"
    }

    def update_info(change):
        with info_output:
            clear_output()
            display(HTML(info_panels[change['new']]))

    effect_size_widget.observe(update_info, names='value')
    with info_output: display(HTML(info_panels[recommended_type]))

    # 4. Confirm Button
    proceed_button = widgets.Button(
        description='✓ Confirm Selection',
        button_style='success',
        layout=widgets.Layout(width='300px', height='40px'),
        style={'font_weight': 'bold'}
    )
    proceed_output = widgets.Output()

    def on_proceed(b):
        with proceed_output:
            clear_output()
            sel = effect_size_widget.value
            print(f"✓ Confirmed Selection: {sel}")

            # --- CONFIGURATION ---
            es_configs = {
                'lnRR': {
                    'effect_col': 'lnRR', 'var_col': 'var_lnRR', 'se_col': 'SE_lnRR',
                    'ci_lower_col': 'CI_lower_lnRR', 'ci_upper_col': 'CI_upper_lnRR',
                    'effect_label': 'log Response Ratio', 'effect_label_short': 'lnRR',
                    'has_fold_change': True, 'null_value': 0, 'scale': 'log', 'allows_negative': False
                },
                'hedges_g': {
                    'effect_col': 'hedges_g', 'var_col': 'Vg', 'se_col': 'SE_g',
                    'ci_lower_col': 'CI_lower_g', 'ci_upper_col': 'CI_upper_g',
                    'effect_label': "Hedges' g", 'effect_label_short': 'g',
                    'has_fold_change': False, 'null_value': 0, 'scale': 'standardized', 'allows_negative': True
                },
                'cohen_d': {
                    'effect_col': 'cohen_d', 'var_col': 'Vd', 'se_col': 'SE_d',
                    'ci_lower_col': 'CI_lower_d', 'ci_upper_col': 'CI_upper_d',
                    'effect_label': "Cohen's d", 'effect_label_short': 'd',
                    'has_fold_change': False, 'null_value': 0, 'scale': 'standardized', 'allows_negative': True
                },
                'log_or': {
                    'effect_col': 'log_OR', 'var_col': 'var_log_OR', 'se_col': 'SE_log_OR',
                    'ci_lower_col': 'CI_lower_log_OR', 'ci_upper_col': 'CI_upper_log_OR',
                    'effect_label': 'log Odds Ratio', 'effect_label_short': 'logOR',
                    'has_fold_change': True, 'null_value': 0, 'scale': 'log', 'allows_negative': False
                }
            }

            ANALYSIS_CONFIG['effect_size_type'] = sel
            ANALYSIS_CONFIG['es_config'] = es_configs[sel]

            # Pre-set global vars
            ANALYSIS_CONFIG['effect_col'] = es_configs[sel]['effect_col']
            ANALYSIS_CONFIG['var_col'] = es_configs[sel]['var_col']
            ANALYSIS_CONFIG['se_col'] = es_configs[sel]['se_col']

            print(f"✓ Configuration saved. Please run the next cell to calculate values.")

    proceed_button.on_click(on_proceed)

    display(widgets.VBox([
        effect_size_widget,
        info_output,
        widgets.HTML("<br>"),
        proceed_button,
        proceed_output
    ]))

# --- DISPLAY ---
display(tabs)

In [ ]:
#@title 🧮 CALCULATE EFFECT SIZES (V1)

# =============================================================================
# CELL 5: EFFECT SIZE CALCULATION
# Purpose: Calculate effect sizes, variances, and weights for meta-analysis
# Dependencies: Cell 4 (ANALYSIS_CONFIG, data_filtered)
# Outputs: data_filtered with effect sizes, EFFECT_SIZE_METADATA
# =============================================================================

print("\n" + "="*70)
print("EFFECT SIZE CALCULATION")
print("="*70)
print(f"Timestamp: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# --- STEP 1: LOAD CONFIGURATION ---
print("\n" + "="*70)
print("STEP 1: LOADING CONFIGURATION")
print("="*70)

try:
    effect_size_type = ANALYSIS_CONFIG['effect_size_type']
    es_config = ANALYSIS_CONFIG['es_config']
    print(f"✓ Configuration loaded successfully")
    print(f"  Effect size type: {es_config['effect_label']} ({es_config['effect_label_short']})")
    print(f"  Scale: {es_config['scale']}")
    print(f"  Allows negatives: {es_config['allows_negative']}")
    print(f"  Null value: {es_config['null_value']}")
except KeyError as e:
    print(f"❌ ERROR: Configuration not found - {e}")
    print("\nTroubleshooting:")
    print("  1. Ensure Cell 4 (effect size selection) was run successfully")
    print("  2. Check that you clicked 'Confirm Selection' button")
    print("  3. Verify ANALYSIS_CONFIG exists with 'effect_size_type' key")
    raise

# Store initial dataset size
initial_obs = len(data_filtered)
initial_papers = data_filtered['id'].nunique()

print(f"\n📊 Input Dataset:")
print(f"  Observations: {initial_obs}")
print(f"  Papers: {initial_papers}")

# --- STEP 2: VERIFY REQUIRED DATA COLUMNS ---
print("\n" + "="*70)
print("STEP 2: DATA VALIDATION")
print("="*70)

required_for_calculation = ['xe', 'sde', 'ne', 'xc', 'sdc', 'nc']
missing_cols = [col for col in required_for_calculation if col not in data_filtered.columns]

if missing_cols:
    print(f"❌ ERROR: Missing required columns: {missing_cols}")
    raise ValueError(f"Missing required columns: {missing_cols}")

print(f"✓ All required columns present")

# Check data availability
data_availability = {}
for col in required_for_calculation:
    n_valid = data_filtered[col].notna().sum()
    pct_valid = (n_valid / len(data_filtered)) * 100
    data_availability[col] = {'valid': n_valid, 'pct': pct_valid}
    print(f"  • {col}: {n_valid}/{len(data_filtered)} valid ({pct_valid:.1f}%)")

# --- STEP 3: HANDLE ZERO/MISSING STANDARD DEVIATIONS ---
print("\n" + "="*70)
print("STEP 3: STANDARD DEVIATION IMPUTATION")
print("="*70)

print("🔧 Processing standard deviations...")

# Track imputation statistics
imputation_log = {
    'method': 'median_cv',
    'sde_zeros': 0,
    'sdc_zeros': 0,
    'sde_missing': 0,
    'sdc_missing': 0,
    'sde_imputed': 0,
    'sdc_imputed': 0
}

# Count initial issues
imputation_log['sde_zeros'] = (data_filtered['sde'] == 0).sum()
imputation_log['sdc_zeros'] = (data_filtered['sdc'] == 0).sum()
imputation_log['sde_missing'] = data_filtered['sde'].isna().sum()
imputation_log['sdc_missing'] = data_filtered['sdc'].isna().sum()

print(f"\n📋 Initial SD Status:")
print(f"  Experimental (sde):")
print(f"    • Zero values:    {imputation_log['sde_zeros']}")
print(f"    • Missing values: {imputation_log['sde_missing']}")
print(f"    • Total issues:   {imputation_log['sde_zeros'] + imputation_log['sde_missing']}")
print(f"  Control (sdc):")
print(f"    • Zero values:    {imputation_log['sdc_zeros']}")
print(f"    • Missing values: {imputation_log['sdc_missing']}")
print(f"    • Total issues:   {imputation_log['sdc_zeros'] + imputation_log['sdc_missing']}")

# Replace zeros with NaN for proper imputation
data_filtered['sde'] = data_filtered['sde'].replace(0, np.nan)
data_filtered['sdc'] = data_filtered['sdc'].replace(0, np.nan)

# Calculate Coefficient of Variation (CV = SD/Mean) for imputation
print(f"\n🔬 Calculating Coefficient of Variation (CV)...")

data_filtered['cv_e'] = np.nan
data_filtered['cv_c'] = np.nan

# Calculate CV only for valid entries (non-missing SD, positive mean)
valid_cv_e = (data_filtered['sde'] > 0) & (data_filtered['xe'] > 0)
valid_cv_c = (data_filtered['sdc'] > 0) & (data_filtered['xc'] > 0)

data_filtered.loc[valid_cv_e, 'cv_e'] = data_filtered.loc[valid_cv_e, 'sde'] / data_filtered.loc[valid_cv_e, 'xe']
data_filtered.loc[valid_cv_c, 'cv_c'] = data_filtered.loc[valid_cv_c, 'sdc'] / data_filtered.loc[valid_cv_c, 'xc']

# Use MEDIAN CV for robustness (less sensitive to outliers than mean)
median_cv_e = data_filtered['cv_e'].median()
median_cv_c = data_filtered['cv_c'].median()
mean_cv_e = data_filtered['cv_e'].mean()
mean_cv_c = data_filtered['cv_c'].mean()

print(f"\n  CV Statistics (Experimental):")
print(f"    • Valid CVs:   {valid_cv_e.sum()}/{len(data_filtered)} ({(valid_cv_e.sum()/len(data_filtered))*100:.1f}%)")
print(f"    • Median CV:   {median_cv_e:.4f}")
print(f"    • Mean CV:     {mean_cv_e:.4f}")
print(f"    • Min CV:      {data_filtered['cv_e'].min():.4f}")
print(f"    • Max CV:      {data_filtered['cv_e'].max():.4f}")

print(f"\n  CV Statistics (Control):")
print(f"    • Valid CVs:   {valid_cv_c.sum()}/{len(data_filtered)} ({(valid_cv_c.sum()/len(data_filtered))*100:.1f}%)")
print(f"    • Median CV:   {median_cv_c:.4f}")
print(f"    • Mean CV:     {mean_cv_c:.4f}")
print(f"    • Min CV:      {data_filtered['cv_c'].min():.4f}")
print(f"    • Max CV:      {data_filtered['cv_c'].max():.4f}")

# Store CV statistics
imputation_log['median_cv_e'] = median_cv_e
imputation_log['median_cv_c'] = median_cv_c
imputation_log['mean_cv_e'] = mean_cv_e
imputation_log['mean_cv_c'] = mean_cv_c
imputation_log['n_valid_cv_e'] = valid_cv_e.sum()
imputation_log['n_valid_cv_c'] = valid_cv_c.sum()

# Create imputed SD columns
print(f"\n🔧 Applying imputation...")

data_filtered['sde_imputed'] = data_filtered['sde'].copy()
data_filtered['sdc_imputed'] = data_filtered['sdc'].copy()

# Track which rows were imputed
data_filtered['sde_was_imputed'] = False
data_filtered['sdc_was_imputed'] = False

# Impute experimental group
impute_e = (data_filtered['sde_imputed'].isna()) & (data_filtered['xe'] > 0)
n_imputed_e = impute_e.sum()

if n_imputed_e > 0 and pd.notna(median_cv_e):
    data_filtered.loc[impute_e, 'sde_imputed'] = median_cv_e * data_filtered.loc[impute_e, 'xe']
    data_filtered.loc[impute_e, 'sde_was_imputed'] = True
    imputation_log['sde_imputed'] = n_imputed_e
    print(f"  ✓ Imputed {n_imputed_e} experimental SDs using median CV method")
    print(f"    Formula: SD_imputed = {median_cv_e:.4f} × mean")
elif n_imputed_e > 0:
    print(f"  ⚠️  Warning: {n_imputed_e} experimental SDs need imputation but CV unavailable")

# Impute control group
impute_c = (data_filtered['sdc_imputed'].isna()) & (data_filtered['xc'] > 0)
n_imputed_c = impute_c.sum()

if n_imputed_c > 0 and pd.notna(median_cv_c):
    data_filtered.loc[impute_c, 'sdc_imputed'] = median_cv_c * data_filtered.loc[impute_c, 'xc']
    data_filtered.loc[impute_c, 'sdc_was_imputed'] = True
    imputation_log['sdc_imputed'] = n_imputed_c
    print(f"  ✓ Imputed {n_imputed_c} control SDs using median CV method")
    print(f"    Formula: SD_imputed = {median_cv_c:.4f} × mean")
elif n_imputed_c > 0:
    print(f"  ⚠️  Warning: {n_imputed_c} control SDs need imputation but CV unavailable")

# Final check for remaining issues
remaining_issues_e = (data_filtered['sde_imputed'].isna()) | (data_filtered['sde_imputed'] <= 0)
remaining_issues_c = (data_filtered['sdc_imputed'].isna()) | (data_filtered['sdc_imputed'] <= 0)
remaining_issues = remaining_issues_e | remaining_issues_c

if remaining_issues.any():
    n_issues = remaining_issues.sum()
    print(f"\n  ⚠️  WARNING: {n_issues} observations still have invalid SDs after imputation")
    print(f"    These observations will be removed from analysis")

    # Show details
    print(f"\n    Breakdown:")
    print(f"      • Experimental SD issues: {remaining_issues_e.sum()}")
    print(f"      • Control SD issues:      {remaining_issues_c.sum()}")

    # Remove problematic rows
    data_filtered = data_filtered[~remaining_issues].copy()
    imputation_log['removed_after_imputation'] = n_issues
else:
    print(f"\n  ✓ All observations have valid SDs (original or imputed)")
    imputation_log['removed_after_imputation'] = 0

# Summary of imputation
total_imputed = n_imputed_e + n_imputed_c
total_original_issues = (imputation_log['sde_zeros'] + imputation_log['sde_missing'] +
                         imputation_log['sdc_zeros'] + imputation_log['sdc_missing'])

print(f"\n📊 Imputation Summary:")
print(f"  Total SD issues found:     {total_original_issues}")
print(f"  Total SDs imputed:         {total_imputed}")
print(f"  Observations removed:      {imputation_log['removed_after_imputation']}")
print(f"  Observations remaining:    {len(data_filtered)}")
print(f"  Imputation success rate:   {(total_imputed/(total_original_issues + 0.0001))*100:.1f}%")

# --- STEP 4: HANDLE ZERO/NEGATIVE VALUES (FOR RATIO MEASURES) ---
if effect_size_type in ['lnRR', 'log_or']:
    print("\n" + "="*70)
    print("STEP 4: ZERO/NEGATIVE VALUE HANDLING (RATIO MEASURES)")
    print("="*70)

    print(f"\n🔍 Checking for incompatible values...")

    # Check for zero values
    zero_xe = (data_filtered['xe'] == 0).sum()
    zero_xc = (data_filtered['xc'] == 0).sum()

    # Check for negative values
    neg_xe = (data_filtered['xe'] < 0).sum()
    neg_xc = (data_filtered['xc'] < 0).sum()

    print(f"\n  Zero values:")
    print(f"    • Treatment (xe): {zero_xe}")
    print(f"    • Control (xc):   {zero_xc}")
    print(f"    • Total:          {zero_xe + zero_xc}")

    print(f"\n  Negative values:")
    print(f"    • Treatment (xe): {neg_xe}")
    print(f"    • Control (xc):   {neg_xc}")
    print(f"    • Total:          {neg_xe + neg_xc}")

    # Handle negative values (must be removed)
    if neg_xe > 0 or neg_xc > 0:
        print(f"\n  ❌ Removing {neg_xe + neg_xc} observations with negative values")
        print(f"     (log ratio requires all positive values)")
        negative_mask = (data_filtered['xe'] < 0) | (data_filtered['xc'] < 0)
        data_filtered = data_filtered[~negative_mask].copy()

    # Handle zero values (add small constant)
    if zero_xe > 0 or zero_xc > 0:
        ZERO_CONSTANT = 0.001
        print(f"\n  🔧 Handling {zero_xe + zero_xc} zero values:")
        print(f"     Adding small constant: {ZERO_CONSTANT}")

        data_filtered.loc[data_filtered['xe'] == 0, 'xe'] = ZERO_CONSTANT
        data_filtered.loc[data_filtered['xc'] == 0, 'xc'] = ZERO_CONSTANT

        print(f"     ✓ Zero values adjusted to avoid log(0)")

    if neg_xe + neg_xc + zero_xe + zero_xc == 0:
        print(f"\n  ✓ All values positive and non-zero")

    print(f"\n  Observations remaining: {len(data_filtered)}")
    # --- STEP 5: CALCULATE EFFECT SIZE BASED ON TYPE ---
print("\n" + "="*70)
print("STEP 5: EFFECT SIZE CALCULATION")
print("="*70)

calculation_log = {
    'type': effect_size_type,
    'timestamp': datetime.datetime.now(),
    'n_observations': len(data_filtered)
}

print(f"\n🧮 Calculating {es_config['effect_label']}...")
print(f"   Method: {effect_size_type}")
print(f"   Observations: {len(data_filtered)}")

if effect_size_type == 'lnRR':
    # ========================================
    # LOG RESPONSE RATIO (lnRR)
    # ========================================

    print(f"\n📐 Formula: lnRR = ln(x̄ₑ / x̄ₜ)")
    print(f"   Variance: Var(lnRR) = SD²ₑ/(nₑ·x̄²ₑ) + SD²ₜ/(nₜ·x̄²ₜ)")

    # Calculate lnRR
    data_filtered['lnRR'] = np.log(data_filtered['xe'] / data_filtered['xc'])

    # Calculate variance using delta method
    data_filtered['var_lnRR'] = (
        (data_filtered['sde_imputed']**2 / (data_filtered['ne'] * data_filtered['xe']**2)) +
        (data_filtered['sdc_imputed']**2 / (data_filtered['nc'] * data_filtered['xc']**2))
    )

    # Calculate standard error
    data_filtered['SE_lnRR'] = np.sqrt(data_filtered['var_lnRR'])

    # Calculate 95% confidence intervals
    data_filtered['CI_lower_lnRR'] = data_filtered['lnRR'] - 1.96 * data_filtered['SE_lnRR']
    data_filtered['CI_upper_lnRR'] = data_filtered['lnRR'] + 1.96 * data_filtered['SE_lnRR']

    # Convert to Response Ratio (RR) for interpretation
    data_filtered['Response_Ratio'] = np.exp(data_filtered['lnRR'])
    data_filtered['RR_CI_lower'] = np.exp(data_filtered['CI_lower_lnRR'])
    data_filtered['RR_CI_upper'] = np.exp(data_filtered['CI_upper_lnRR'])

    # Calculate fold-change (with sign for direction)
    # Positive lnRR = upregulation (e.g., 2-fold increase = 2×)
    # Negative lnRR = downregulation (e.g., 2-fold decrease = -2×)
    data_filtered['fold_change'] = data_filtered.apply(
        lambda row: row['Response_Ratio'] if row['lnRR'] >= 0 else -1/row['Response_Ratio'],
        axis=1
    )

    # Calculate percent change
    data_filtered['Percent_Change'] = (data_filtered['Response_Ratio'] - 1) * 100

    # Set primary effect size column names
    effect_col = 'lnRR'
    var_col = 'var_lnRR'
    se_col = 'SE_lnRR'

    calculation_log['columns_created'] = [
        'lnRR', 'var_lnRR', 'SE_lnRR', 'CI_lower_lnRR', 'CI_upper_lnRR',
        'Response_Ratio', 'RR_CI_lower', 'RR_CI_upper', 'fold_change', 'Percent_Change'
    ]

    print(f"\n  ✓ lnRR calculated for {len(data_filtered)} observations")
    print(f"\n  📊 Columns created:")
    print(f"     • lnRR: Log response ratio (effect size)")
    print(f"     • var_lnRR: Variance of lnRR")
    print(f"     • SE_lnRR: Standard error of lnRR")
    print(f"     • CI_lower/upper_lnRR: 95% confidence intervals")
    print(f"     • Response_Ratio: RR = exp(lnRR)")
    print(f"     • fold_change: Directional fold-change")
    print(f"     • Percent_Change: % change from control")

elif effect_size_type == 'hedges_g':
    # ========================================
    # HEDGES' G (STANDARDIZED MEAN DIFFERENCE)
    # ========================================

    print(f"\n📐 Formula: g = [(x̄ₑ - x̄ₜ) / SDₚₒₒₗₑ𝒹] × J")
    print(f"   J = 1 - 3/(4·df - 1)  [small-sample correction]")
    print(f"   Variance: Vg = [(nₑ+nₜ)/(nₑ·nₜ) + g²/(2(nₑ+nₜ))] × J²")

    # Degrees of freedom
    data_filtered['df'] = data_filtered['ne'] + data_filtered['nc'] - 2

    print(f"\n  🔢 Calculating pooled standard deviation...")

    # Pooled Standard Deviation
    data_filtered['sp_squared'] = (
        ((data_filtered['ne'] - 1) * data_filtered['sde_imputed']**2 +
         (data_filtered['nc'] - 1) * data_filtered['sdc_imputed']**2) /
        data_filtered['df']
    )
    data_filtered['sp'] = np.sqrt(data_filtered['sp_squared'])

    print(f"     • Mean pooled SD: {data_filtered['sp'].mean():.4f}")
    print(f"     • Median pooled SD: {data_filtered['sp'].median():.4f}")

    # Cohen's d (uncorrected)
    data_filtered['cohen_d'] = (data_filtered['xe'] - data_filtered['xc']) / data_filtered['sp']

    print(f"\n  🔢 Applying Hedges' correction for small samples...")

    # Hedges' g correction factor (J)
    # Using approximation: J ≈ 1 - 3/(4*df - 1)
    #data_filtered['hedges_j'] = 1 - (3 / (4 * data_filtered['df'] - 1))
    m_df = data_filtered['df']
    data_filtered['hedges_j'] = gamma(m_df / 2) / (np.sqrt(m_df / 2) * gamma((m_df - 1) / 2))

    print(f"     • Mean J factor: {data_filtered['hedges_j'].mean():.6f}")

    print(f"     • Min J factor: {data_filtered['hedges_j'].min():.6f}")
    print(f"     • Max J factor: {data_filtered['hedges_j'].max():.6f}")

    # Hedges' g
    data_filtered['hedges_g'] = data_filtered['cohen_d'] * data_filtered['hedges_j']

    # Variance of Hedges' g
    data_filtered['Vg'] = (
        ((data_filtered['ne'] + data_filtered['nc']) / (data_filtered['ne'] * data_filtered['nc']) +
         (data_filtered['hedges_g']**2) / (2 * (data_filtered['ne'] + data_filtered['nc']))) *
        (data_filtered['hedges_j']**2)
    )

    # Standard error
    data_filtered['SE_g'] = np.sqrt(data_filtered['Vg'])

    # Calculate 95% confidence intervals
    data_filtered['CI_lower_g'] = data_filtered['hedges_g'] - 1.96 * data_filtered['SE_g']
    data_filtered['CI_upper_g'] = data_filtered['hedges_g'] + 1.96 * data_filtered['SE_g']

    # Set primary effect size column names
    effect_col = 'hedges_g'
    var_col = 'Vg'
    se_col = 'SE_g'

    calculation_log['columns_created'] = [
        'hedges_g', 'Vg', 'SE_g', 'CI_lower_g', 'CI_upper_g',
        'cohen_d', 'hedges_j', 'sp', 'sp_squared', 'df'
    ]

    print(f"\n  ✓ Hedges' g calculated for {len(data_filtered)} observations")
    print(f"\n  📊 Columns created:")
    print(f"     • hedges_g: Hedges' g (effect size with correction)")
    print(f"     • cohen_d: Cohen's d (uncorrected)")
    print(f"     • Vg: Variance of Hedges' g")
    print(f"     • SE_g: Standard error of Hedges' g")
    print(f"     • CI_lower/upper_g: 95% confidence intervals")
    print(f"     • sp: Pooled standard deviation")
    print(f"     • hedges_j: Small-sample correction factor")

    # Effect size magnitude classification
    small = ((data_filtered['hedges_g'].abs() >= 0.2) & (data_filtered['hedges_g'].abs() < 0.5)).sum()
    medium = ((data_filtered['hedges_g'].abs() >= 0.5) & (data_filtered['hedges_g'].abs() < 0.8)).sum()
    large = (data_filtered['hedges_g'].abs() >= 0.8).sum()
    negligible = (data_filtered['hedges_g'].abs() < 0.2).sum()

    print(f"\n  📏 Effect Size Magnitude (Cohen's benchmarks):")
    print(f"     • Negligible (|g| < 0.2):   {negligible} ({negligible/len(data_filtered)*100:.1f}%)")
    print(f"     • Small (0.2 ≤ |g| < 0.5):  {small} ({small/len(data_filtered)*100:.1f}%)")
    print(f"     • Medium (0.5 ≤ |g| < 0.8): {medium} ({medium/len(data_filtered)*100:.1f}%)")
    print(f"     • Large (|g| ≥ 0.8):        {large} ({large/len(data_filtered)*100:.1f}%)")

elif effect_size_type == 'cohen_d':
    # ========================================
    # COHEN'S D (NO SMALL-SAMPLE CORRECTION)
    # ========================================

    print(f"\n📐 Formula: d = (x̄ₑ - x̄ₜ) / SDₚₒₒₗₑ𝒹")
    print(f"   Variance: Vd = (nₑ+nₜ)/(nₑ·nₜ) + d²/(2(nₑ+nₜ))")
    print(f"   Note: No small-sample correction applied")

    # Degrees of freedom
    data_filtered['df'] = data_filtered['ne'] + data_filtered['nc'] - 2

    print(f"\n  🔢 Calculating pooled standard deviation...")

    # Pooled Standard Deviation
    data_filtered['sp_squared'] = (
        ((data_filtered['ne'] - 1) * data_filtered['sde_imputed']**2 +
         (data_filtered['nc'] - 1) * data_filtered['sdc_imputed']**2) /
        data_filtered['df']
    )
    data_filtered['sp'] = np.sqrt(data_filtered['sp_squared'])

    print(f"     • Mean pooled SD: {data_filtered['sp'].mean():.4f}")
    print(f"     • Median pooled SD: {data_filtered['sp'].median():.4f}")

    # Cohen's d
    data_filtered['cohen_d'] = (data_filtered['xe'] - data_filtered['xc']) / data_filtered['sp']

    # Variance of Cohen's d
    data_filtered['Vd'] = (
        (data_filtered['ne'] + data_filtered['nc']) / (data_filtered['ne'] * data_filtered['nc']) +
        (data_filtered['cohen_d']**2) / (2 * (data_filtered['ne'] + data_filtered['nc']))
    )

    # Standard error
    data_filtered['SE_d'] = np.sqrt(data_filtered['Vd'])

    # Calculate 95% confidence intervals
    data_filtered['CI_lower_d'] = data_filtered['cohen_d'] - 1.96 * data_filtered['SE_d']
    data_filtered['CI_upper_d'] = data_filtered['cohen_d'] + 1.96 * data_filtered['SE_d']

    # Set primary effect size column names
    effect_col = 'cohen_d'
    var_col = 'Vd'
    se_col = 'SE_d'

    calculation_log['columns_created'] = [
        'cohen_d', 'Vd', 'SE_d', 'CI_lower_d', 'CI_upper_d',
        'sp', 'sp_squared', 'df'
    ]

    print(f"\n  ✓ Cohen's d calculated for {len(data_filtered)} observations")
    print(f"\n  📊 Columns created:")
    print(f"     • cohen_d: Cohen's d (effect size)")
    print(f"     • Vd: Variance of Cohen's d")
    print(f"     • SE_d: Standard error of Cohen's d")
    print(f"     • CI_lower/upper_d: 95% confidence intervals")
    print(f"     • sp: Pooled standard deviation")

    # Effect size magnitude classification
    small = ((data_filtered['cohen_d'].abs() >= 0.2) & (data_filtered['cohen_d'].abs() < 0.5)).sum()
    medium = ((data_filtered['cohen_d'].abs() >= 0.5) & (data_filtered['cohen_d'].abs() < 0.8)).sum()
    large = (data_filtered['cohen_d'].abs() >= 0.8).sum()
    negligible = (data_filtered['cohen_d'].abs() < 0.2).sum()

    print(f"\n  📏 Effect Size Magnitude (Cohen's benchmarks):")
    print(f"     • Negligible (|d| < 0.2):   {negligible} ({negligible/len(data_filtered)*100:.1f}%)")
    print(f"     • Small (0.2 ≤ |d| < 0.5):  {small} ({small/len(data_filtered)*100:.1f}%)")
    print(f"     • Medium (0.5 ≤ |d| < 0.8): {medium} ({medium/len(data_filtered)*100:.1f}%)")
    print(f"     • Large (|d| ≥ 0.8):        {large} ({large/len(data_filtered)*100:.1f}%)")

    # Sample size warning
    small_samples = (data_filtered['df'] < 20).sum()
    if small_samples > 0:
        print(f"\n  ⚠️  Warning: {small_samples} observations have small samples (df < 20)")
        print(f"     Consider using Hedges' g instead for small-sample correction")

elif effect_size_type == 'log_or':
    # ========================================
    # LOG ODDS RATIO
    # ========================================

    print(f"\n⚠️  Note: log Odds Ratio implementation")
    print(f"   Current implementation treats xe/xc as odds or proportions")
    print(f"   For 2×2 contingency tables, ensure proper data format")

    print(f"\n📐 Formula: logOR = ln(xe / xc)")
    print(f"   Variance: Var(logOR) ≈ SD²ₑ/(nₑ·xe²) + SD²ₜ/(nₜ·xc²)")

    # Check for values in valid range
    invalid_values = ((data_filtered['xe'] < 0) | (data_filtered['xc'] < 0) |
                      (data_filtered['xe'] == 0) | (data_filtered['xc'] == 0))

    if invalid_values.any():
        print(f"\n  ⚠️  WARNING: {invalid_values.sum()} observations have invalid values")
        print(f"     Removing observations with xe ≤ 0 or xc ≤ 0")
        data_filtered = data_filtered[~invalid_values].copy()

    # Calculate log OR
    data_filtered['log_OR'] = np.log(data_filtered['xe'] / data_filtered['xc'])

    # Calculate variance (simplified - assumes xe, xc are odds/proportions)
    data_filtered['var_log_OR'] = (
        (data_filtered['sde_imputed']**2 / (data_filtered['ne'] * data_filtered['xe']**2)) +
        (data_filtered['sdc_imputed']**2 / (data_filtered['nc'] * data_filtered['xc']**2))
    )

    # Standard error
    data_filtered['SE_log_OR'] = np.sqrt(data_filtered['var_log_OR'])

    # Calculate 95% confidence intervals
    data_filtered['CI_lower_log_OR'] = data_filtered['log_OR'] - 1.96 * data_filtered['SE_log_OR']
    data_filtered['CI_upper_log_OR'] = data_filtered['log_OR'] + 1.96 * data_filtered['SE_log_OR']

    # Convert to Odds Ratio
    data_filtered['Odds_Ratio'] = np.exp(data_filtered['log_OR'])
    data_filtered['OR_CI_lower'] = np.exp(data_filtered['CI_lower_log_OR'])
    data_filtered['OR_CI_upper'] = np.exp(data_filtered['CI_upper_log_OR'])

    # Set primary effect size column names
    effect_col = 'log_OR'
    var_col = 'var_log_OR'
    se_col = 'SE_log_OR'

    calculation_log['columns_created'] = [
        'log_OR', 'var_log_OR', 'SE_log_OR', 'CI_lower_log_OR', 'CI_upper_log_OR',
        'Odds_Ratio', 'OR_CI_lower', 'OR_CI_upper'
    ]

    print(f"\n  ✓ log Odds Ratio calculated for {len(data_filtered)} observations")
    print(f"\n  📊 Columns created:")
    print(f"     • log_OR: Log odds ratio (effect size)")
    print(f"     • var_log_OR: Variance of log OR")
    print(f"     • SE_log_OR: Standard error of log OR")
    print(f"     • CI_lower/upper_log_OR: 95% confidence intervals")
    print(f"     • Odds_Ratio: OR = exp(logOR)")
    print(f"\n  ⚠️  Please verify results are appropriate for your data structure")

else:
    raise ValueError(f"Unknown effect size type: {effect_size_type}")

calculation_log['effect_col'] = effect_col
calculation_log['var_col'] = var_col
calculation_log['se_col'] = se_col

# --- STEP 6: CALCULATE FIXED-EFFECTS WEIGHTS ---
print("\n" + "="*70)
print("STEP 6: CALCULATING WEIGHTS")
print("="*70)

print(f"\n⚖️  Calculating inverse-variance weights...")
print(f"   Formula: w = 1 / Var({es_config['effect_label_short']})")

data_filtered['w_fixed'] = 1 / data_filtered[var_col]

# Handle infinite weights
inf_weights = np.isinf(data_filtered['w_fixed']).sum()
if inf_weights > 0:
    print(f"\n  ⚠️  Warning: {inf_weights} infinite weights detected (variance = 0)")
    print(f"     Replacing with NaN for removal")
    data_filtered['w_fixed'] = data_filtered['w_fixed'].replace([np.inf, -np.inf], np.nan)

# Weight statistics
print(f"\n  📊 Weight Statistics:")
print(f"     • Mean weight:   {data_filtered['w_fixed'].mean():.2f}")
print(f"     • Median weight: {data_filtered['w_fixed'].median():.2f}")
print(f"     • Min weight:    {data_filtered['w_fixed'].min():.2f}")
print(f"     • Max weight:    {data_filtered['w_fixed'].max():.2f}")
print(f"     • Std weight:    {data_filtered['w_fixed'].std():.2f}")

# Check weight distribution
weight_ratio = data_filtered['w_fixed'].max() / (data_filtered['w_fixed'].min() + 0.0001)
print(f"\n  📏 Weight ratio (max/min): {weight_ratio:.2f}")

if weight_ratio > 1000:
    print(f"     ⚠️  Very large weight range - one study may dominate")
elif weight_ratio > 100:
    print(f"     ⚠️  Large weight range - check for influential studies")
else:
    print(f"     ✓ Reasonable weight range")

print(f"\n  ✓ Fixed-effects weights calculated")

# --- STEP 7: CLEAN DATA ---
print("\n" + "="*70)
print("STEP 7: FINAL DATA CLEANING")
print("="*70)

print(f"\n🧹 Removing observations with missing critical values...")

# Define critical columns
critical_cols = [effect_col, var_col, se_col, 'w_fixed']
initial_n = len(data_filtered)

# Check for missing values
missing_summary = {}
for col in critical_cols:
    n_missing = data_filtered[col].isna().sum()
    missing_summary[col] = n_missing
    if n_missing > 0:
        print(f"  • {col}: {n_missing} missing")

# Remove rows with NaN in critical columns
data_filtered = data_filtered.dropna(subset=critical_cols).copy()
final_n = len(data_filtered)
removed = initial_n - final_n

if removed > 0:
    print(f"\n  ⚠️  Removed {removed} observations with missing critical values")
    print(f"     ({(removed/initial_n)*100:.1f}% of dataset)")
else:
    print(f"\n  ✓ No missing values in critical columns")

print(f"\n  📊 Final dataset: {final_n} observations")

calculation_log['final_n'] = final_n
calculation_log['removed_in_cleaning'] = removed

# Continue to Part 3...
# --- STEP 8: EFFECT SIZE SUMMARY STATISTICS ---
print("\n" + "="*70)
print("STEP 8: EFFECT SIZE SUMMARY STATISTICS")
print("="*70)

# Calculate comprehensive statistics
effect_stats = {
    'count': data_filtered[effect_col].count(),
    'mean': data_filtered[effect_col].mean(),
    'median': data_filtered[effect_col].median(),
    'std': data_filtered[effect_col].std(),
    'min': data_filtered[effect_col].min(),
    'max': data_filtered[effect_col].max(),
    'q25': data_filtered[effect_col].quantile(0.25),
    'q75': data_filtered[effect_col].quantile(0.75),
    'iqr': data_filtered[effect_col].quantile(0.75) - data_filtered[effect_col].quantile(0.25)
}

var_stats = {
    'mean': data_filtered[var_col].mean(),
    'median': data_filtered[var_col].median(),
    'std': data_filtered[var_col].std(),
    'min': data_filtered[var_col].min(),
    'max': data_filtered[var_col].max()
}

se_stats = {
    'mean': data_filtered[se_col].mean(),
    'median': data_filtered[se_col].median(),
    'std': data_filtered[se_col].std(),
    'min': data_filtered[se_col].min(),
    'max': data_filtered[se_col].max()
}

print(f"\n📊 {es_config['effect_label']} ({es_config['effect_label_short']}):")
print(f"  {'Statistic':<15} {'Value':>12}")
print(f"  {'-'*15} {'-'*12}")
print(f"  {'Count':<15} {effect_stats['count']:>12}")
print(f"  {'Mean':<15} {effect_stats['mean']:>12.4f}")
print(f"  {'Median':<15} {effect_stats['median']:>12.4f}")
print(f"  {'Std Dev':<15} {effect_stats['std']:>12.4f}")
print(f"  {'Min':<15} {effect_stats['min']:>12.4f}")
print(f"  {'Q1 (25%)':<15} {effect_stats['q25']:>12.4f}")
print(f"  {'Q3 (75%)':<15} {effect_stats['q75']:>12.4f}")
print(f"  {'Max':<15} {effect_stats['max']:>12.4f}")
print(f"  {'IQR':<15} {effect_stats['iqr']:>12.4f}")

print(f"\n📊 Variance ({var_col}):")
print(f"  {'Statistic':<15} {'Value':>12}")
print(f"  {'-'*15} {'-'*12}")
print(f"  {'Mean':<15} {var_stats['mean']:>12.6f}")
print(f"  {'Median':<15} {var_stats['median']:>12.6f}")
print(f"  {'Std Dev':<15} {var_stats['std']:>12.6f}")
print(f"  {'Min':<15} {var_stats['min']:>12.6f}")
print(f"  {'Max':<15} {var_stats['max']:>12.6f}")

print(f"\n📊 Standard Error ({se_col}):")
print(f"  {'Statistic':<15} {'Value':>12}")
print(f"  {'-'*15} {'-'*12}")
print(f"  {'Mean':<15} {se_stats['mean']:>12.4f}")
print(f"  {'Median':<15} {se_stats['median']:>12.4f}")
print(f"  {'Std Dev':<15} {se_stats['std']:>12.4f}")
print(f"  {'Min':<15} {se_stats['min']:>12.4f}")
print(f"  {'Max':<15} {se_stats['max']:>12.4f}")

# Store statistics
calculation_log['effect_stats'] = effect_stats
calculation_log['var_stats'] = var_stats
calculation_log['se_stats'] = se_stats

# --- STEP 9: DIRECTION AND MAGNITUDE ANALYSIS ---
print("\n" + "="*70)
print("STEP 9: EFFECT DIRECTION & MAGNITUDE ANALYSIS")
print("="*70)

# Analysis depends on effect size type
if effect_size_type == 'lnRR':
    # Direction analysis for log response ratio
    print(f"\n📈 Effect Direction Analysis:")

    # Define thresholds
    upregulation_threshold = 0.05  # ~5% increase
    downregulation_threshold = -0.05  # ~5% decrease

    n_upregulation = (data_filtered[effect_col] > upregulation_threshold).sum()
    n_downregulation = (data_filtered[effect_col] < downregulation_threshold).sum()
    n_no_change = len(data_filtered) - n_upregulation - n_downregulation

    print(f"\n  Based on lnRR threshold = ±{abs(upregulation_threshold):.2f}:")
    print(f"  {'Direction':<25} {'Count':>8} {'Percentage':>12}")
    print(f"  {'-'*25} {'-'*8} {'-'*12}")
    print(f"  {'Upregulated (lnRR > 0.05)':<25} {n_upregulation:>8} {(n_upregulation/len(data_filtered)*100):>11.1f}%")
    print(f"  {'No change (|lnRR| ≤ 0.05)':<25} {n_no_change:>8} {(n_no_change/len(data_filtered)*100):>11.1f}%")
    print(f"  {'Downregulated (lnRR < -0.05)':<25} {n_downregulation:>8} {(n_downregulation/len(data_filtered)*100):>11.1f}%")

    # Fold-change magnitude categories
    print(f"\n📏 Fold-Change Magnitude:")

    fc_2x_up = (data_filtered['Response_Ratio'] >= 2.0).sum()
    fc_2x_down = (data_filtered['Response_Ratio'] <= 0.5).sum()
    fc_3x_up = (data_filtered['Response_Ratio'] >= 3.0).sum()
    fc_3x_down = (data_filtered['Response_Ratio'] <= 0.33).sum()
    fc_5x_up = (data_filtered['Response_Ratio'] >= 5.0).sum()
    fc_5x_down = (data_filtered['Response_Ratio'] <= 0.2).sum()

    print(f"  {'Category':<30} {'Count':>8} {'Percentage':>12}")
    print(f"  {'-'*30} {'-'*8} {'-'*12}")
    print(f"  {'≥5× increase (RR ≥ 5.0)':<30} {fc_5x_up:>8} {(fc_5x_up/len(data_filtered)*100):>11.1f}%")
    print(f"  {'≥3× increase (RR ≥ 3.0)':<30} {fc_3x_up:>8} {(fc_3x_up/len(data_filtered)*100):>11.1f}%")
    print(f"  {'≥2× increase (RR ≥ 2.0)':<30} {fc_2x_up:>8} {(fc_2x_up/len(data_filtered)*100):>11.1f}%")
    print(f"  {'≥2× decrease (RR ≤ 0.5)':<30} {fc_2x_down:>8} {(fc_2x_down/len(data_filtered)*100):>11.1f}%")
    print(f"  {'≥3× decrease (RR ≤ 0.33)':<30} {fc_3x_down:>8} {(fc_3x_down/len(data_filtered)*100):>11.1f}%")
    print(f"  {'≥5× decrease (RR ≤ 0.2)':<30} {fc_5x_down:>8} {(fc_5x_down/len(data_filtered)*100):>11.1f}%")

    # Percent change summary
    print(f"\n📊 Percent Change from Control:")
    print(f"  Mean: {data_filtered['Percent_Change'].mean():+.1f}%")
    print(f"  Median: {data_filtered['Percent_Change'].median():+.1f}%")
    print(f"  Range: [{data_filtered['Percent_Change'].min():+.1f}%, {data_filtered['Percent_Change'].max():+.1f}%]")

    calculation_log['direction_analysis'] = {
        'upregulated': n_upregulation,
        'downregulated': n_downregulation,
        'no_change': n_no_change,
        'fc_2x_up': fc_2x_up,
        'fc_2x_down': fc_2x_down,
        'fc_3x_up': fc_3x_up,
        'fc_3x_down': fc_3x_down
    }

elif effect_size_type in ['hedges_g', 'cohen_d']:
    # Direction and magnitude for standardized mean differences
    print(f"\n📈 Effect Direction:")

    n_positive = (data_filtered[effect_col] > 0).sum()
    n_negative = (data_filtered[effect_col] < 0).sum()
    n_zero = (data_filtered[effect_col] == 0).sum()

    print(f"  {'Direction':<25} {'Count':>8} {'Percentage':>12}")
    print(f"  {'-'*25} {'-'*8} {'-'*12}")
    print(f"  {'Positive effect (g > 0)':<25} {n_positive:>8} {(n_positive/len(data_filtered)*100):>11.1f}%")
    print(f"  {'No effect (g = 0)':<25} {n_zero:>8} {(n_zero/len(data_filtered)*100):>11.1f}%")
    print(f"  {'Negative effect (g < 0)':<25} {n_negative:>8} {(n_negative/len(data_filtered)*100):>11.1f}%")

    # Already calculated in step 5, but show again for clarity
    negligible = (data_filtered[effect_col].abs() < 0.2).sum()
    small = ((data_filtered[effect_col].abs() >= 0.2) & (data_filtered[effect_col].abs() < 0.5)).sum()
    medium = ((data_filtered[effect_col].abs() >= 0.5) & (data_filtered[effect_col].abs() < 0.8)).sum()
    large = (data_filtered[effect_col].abs() >= 0.8).sum()

    print(f"\n📏 Effect Magnitude (Cohen's benchmarks):")
    print(f"  {'Category':<30} {'Count':>8} {'Percentage':>12}")
    print(f"  {'-'*30} {'-'*8} {'-'*12}")
    print(f"  {'Negligible (|g| < 0.2)':<30} {negligible:>8} {(negligible/len(data_filtered)*100):>11.1f}%")
    print(f"  {'Small (0.2 ≤ |g| < 0.5)':<30} {small:>8} {(small/len(data_filtered)*100):>11.1f}%")
    print(f"  {'Medium (0.5 ≤ |g| < 0.8)':<30} {medium:>8} {(medium/len(data_filtered)*100):>11.1f}%")
    print(f"  {'Large (|g| ≥ 0.8)':<30} {large:>8} {(large/len(data_filtered)*100):>11.1f}%")

    calculation_log['direction_analysis'] = {
        'positive': n_positive,
        'negative': n_negative,
        'negligible': negligible,
        'small': small,
        'medium': medium,
        'large': large
    }

elif effect_size_type == 'log_or':
    # Direction for odds ratios
    print(f"\n📈 Effect Direction:")

    n_positive = (data_filtered[effect_col] > 0).sum()
    n_negative = (data_filtered[effect_col] < 0).sum()
    n_null = (data_filtered[effect_col] == 0).sum()

    print(f"  {'Direction':<30} {'Count':>8} {'Percentage':>12}")
    print(f"  {'-'*30} {'-'*8} {'-'*12}")
    print(f"  {'Positive association (OR > 1)':<30} {n_positive:>8} {(n_positive/len(data_filtered)*100):>11.1f}%")
    print(f"  {'No association (OR = 1)':<30} {n_null:>8} {(n_null/len(data_filtered)*100):>11.1f}%")
    print(f"  {'Negative association (OR < 1)':<30} {n_negative:>8} {(n_negative/len(data_filtered)*100):>11.1f}%")

    print(f"\n📊 Odds Ratio Summary:")
    print(f"  Mean OR: {data_filtered['Odds_Ratio'].mean():.3f}")
    print(f"  Median OR: {data_filtered['Odds_Ratio'].median():.3f}")
    print(f"  Range: [{data_filtered['Odds_Ratio'].min():.3f}, {data_filtered['Odds_Ratio'].max():.3f}]")

# --- STEP 10: IDENTIFY EXTREME VALUES ---
print("\n" + "="*70)
print("STEP 10: EXTREME VALUE DETECTION")
print("="*70)

print(f"\n🔍 Identifying outliers and extreme effect sizes...")

# Define thresholds based on effect size type
if effect_size_type == 'lnRR':
    threshold = 3.0  # ~20-fold change
    extreme_label = "RR > 20× or RR < 0.05×"
    interpretation = "More than 20-fold change"
elif effect_size_type in ['hedges_g', 'cohen_d']:
    threshold = 2.0  # Very large standardized effect
    extreme_label = "|g| > 2.0"
    interpretation = "Very large effect (exceeds typical benchmarks)"
elif effect_size_type == 'log_or':
    threshold = 3.0  # OR > 20
    extreme_label = "OR > 20 or OR < 0.05"
    interpretation = "Odds ratio > 20× or < 0.05×"

extreme_effects = data_filtered[np.abs(data_filtered[effect_col]) > threshold].copy()

print(f"\n  Threshold: {extreme_label}")
print(f"  Interpretation: {interpretation}")

if len(extreme_effects) > 0:
    print(f"\n  ⚠️  Found {len(extreme_effects)} extreme effects ({len(extreme_effects)/len(data_filtered)*100:.1f}% of dataset):")
    print(f"\n  {'Paper ID':<15} {es_config['effect_label_short']:>10} {'SE':>10} {'Treatment':>12} {'Control':>12}")
    print(f"  {'-'*15} {'-'*10} {'-'*10} {'-'*12} {'-'*12}")

    # Show extreme effects
    for idx, row in extreme_effects.head(20).iterrows():
        paper_id = str(row['id'])[:15]
        effect = row[effect_col]
        se = row[se_col]
        xe = row['xe']
        xc = row['xc']
        print(f"  {paper_id:<15} {effect:>10.4f} {se:>10.4f} {xe:>12.4f} {xc:>12.4f}")

    if len(extreme_effects) > 20:
        print(f"  ... and {len(extreme_effects) - 20} more")

    print(f"\n  💡 Recommendations:")
    print(f"     1. Review these observations for data entry errors")
    print(f"     2. Check original papers for these effect sizes")
    print(f"     3. Consider sensitivity analysis excluding these values")
    print(f"     4. Examine if they represent true biological phenomena")

    calculation_log['extreme_effects'] = {
        'count': len(extreme_effects),
        'threshold': threshold,
        'paper_ids': extreme_effects['id'].tolist()
    }
else:
    print(f"\n  ✓ No extreme values detected")
    print(f"    All effect sizes within expected range")

    calculation_log['extreme_effects'] = {
        'count': 0,
        'threshold': threshold
    }

# Additional outlier detection using IQR method
print(f"\n📊 Outlier Detection (IQR Method):")
q1 = data_filtered[effect_col].quantile(0.25)
q3 = data_filtered[effect_col].quantile(0.75)
iqr = q3 - q1
lower_fence = q1 - 1.5 * iqr
upper_fence = q3 + 1.5 * iqr

outliers_iqr = data_filtered[(data_filtered[effect_col] < lower_fence) |
                              (data_filtered[effect_col] > upper_fence)]

print(f"  Q1 (25th percentile): {q1:.4f}")
print(f"  Q3 (75th percentile): {q3:.4f}")
print(f"  IQR: {iqr:.4f}")
print(f"  Lower fence: {lower_fence:.4f}")
print(f"  Upper fence: {upper_fence:.4f}")
print(f"\n  Outliers detected: {len(outliers_iqr)} ({len(outliers_iqr)/len(data_filtered)*100:.1f}%)")

if len(outliers_iqr) > 0:
    print(f"    • Below lower fence: {(data_filtered[effect_col] < lower_fence).sum()}")
    print(f"    • Above upper fence: {(data_filtered[effect_col] > upper_fence).sum()}")

calculation_log['outliers_iqr'] = {
    'count': len(outliers_iqr),
    'lower_fence': lower_fence,
    'upper_fence': upper_fence,
    'paper_ids': outliers_iqr['id'].tolist()
}

# --- STEP 11: CONFIDENCE INTERVAL COVERAGE ---
print("\n" + "="*70)
print("STEP 11: CONFIDENCE INTERVAL ANALYSIS")
print("="*70)

ci_lower_col = es_config['ci_lower_col']
ci_upper_col = es_config['ci_upper_col']

# Check CI coverage of null hypothesis
null_value = es_config['null_value']
ci_includes_null = ((data_filtered[ci_lower_col] <= null_value) &
                    (data_filtered[ci_upper_col] >= null_value)).sum()
ci_excludes_null = len(data_filtered) - ci_includes_null

print(f"\n📊 95% Confidence Interval Coverage:")
print(f"  Null hypothesis value: {null_value}")
print(f"\n  {'Category':<35} {'Count':>8} {'Percentage':>12}")
print(f"  {'-'*35} {'-'*8} {'-'*12}")
print(f"  {'CI includes null (not significant)':<35} {ci_includes_null:>8} {(ci_includes_null/len(data_filtered)*100):>11.1f}%")
print(f"  {'CI excludes null (significant)':<35} {ci_excludes_null:>8} {(ci_excludes_null/len(data_filtered)*100):>11.1f}%")

# Average CI width
data_filtered['ci_width'] = data_filtered[ci_upper_col] - data_filtered[ci_lower_col]
mean_ci_width = data_filtered['ci_width'].mean()
median_ci_width = data_filtered['ci_width'].median()

print(f"\n📏 Confidence Interval Width:")
print(f"  Mean CI width:   {mean_ci_width:.4f}")
print(f"  Median CI width: {median_ci_width:.4f}")
print(f"  Min CI width:    {data_filtered['ci_width'].min():.4f}")
print(f"  Max CI width:    {data_filtered['ci_width'].max():.4f}")

# Precision categories
narrow_ci = (data_filtered['ci_width'] < median_ci_width * 0.5).sum()
moderate_ci = ((data_filtered['ci_width'] >= median_ci_width * 0.5) &
               (data_filtered['ci_width'] <= median_ci_width * 2)).sum()
wide_ci = (data_filtered['ci_width'] > median_ci_width * 2).sum()

print(f"\n📊 Precision Distribution:")
print(f"  {'Category':<30} {'Count':>8} {'Percentage':>12}")
print(f"  {'-'*30} {'-'*8} {'-'*12}")
print(f"  {'High precision (narrow CI)':<30} {narrow_ci:>8} {(narrow_ci/len(data_filtered)*100):>11.1f}%")
print(f"  {'Moderate precision':<30} {moderate_ci:>8} {(moderate_ci/len(data_filtered)*100):>11.1f}%")
print(f"  {'Low precision (wide CI)':<30} {wide_ci:>8} {(wide_ci/len(data_filtered)*100):>11.1f}%")

calculation_log['ci_analysis'] = {
    'ci_includes_null': ci_includes_null,
    'ci_excludes_null': ci_excludes_null,
    'mean_ci_width': mean_ci_width,
    'median_ci_width': median_ci_width
}

# --- STEP 12: UPDATE CONFIGURATION ---
print("\n" + "="*70)
print("STEP 12: UPDATING CONFIGURATION")
print("="*70)

ANALYSIS_CONFIG['effect_col'] = effect_col
ANALYSIS_CONFIG['var_col'] = var_col
ANALYSIS_CONFIG['se_col'] = se_col
ANALYSIS_CONFIG['ci_lower_col'] = ci_lower_col
ANALYSIS_CONFIG['ci_upper_col'] = ci_upper_col
ANALYSIS_CONFIG['final_n'] = len(data_filtered)
ANALYSIS_CONFIG['calculation_timestamp'] = datetime.datetime.now()

print(f"\n✓ Configuration updated with effect size information:")
print(f"  • Effect column:    {effect_col}")
print(f"  • Variance column:  {var_col}")
print(f"  • SE column:        {se_col}")
print(f"  • CI columns:       {ci_lower_col}, {ci_upper_col}")
print(f"  • Final n:          {len(data_filtered)}")

# Store comprehensive metadata
EFFECT_SIZE_METADATA = {
    'timestamp': datetime.datetime.now(),
    'effect_size_type': effect_size_type,
    'n_initial': initial_obs,
    'n_final': len(data_filtered),
    'n_removed': initial_obs - len(data_filtered),
    'papers_initial': initial_papers,
    'papers_final': data_filtered['id'].nunique(),
    'imputation_log': imputation_log,
    'calculation_log': calculation_log,
    'effect_stats': effect_stats,
    'var_stats': var_stats,
    'se_stats': se_stats,
    'columns_created': calculation_log['columns_created']
}

print(f"\n✓ Metadata saved to EFFECT_SIZE_METADATA")

# --- STEP 13: DATA PREVIEW ---
print("\n" + "="*70)
print("STEP 13: DATA PREVIEW")
print("="*70)

print(f"\n📋 Preview of Calculated Data (first 10 observations):\n")

# Select columns for preview
preview_cols = ['id', 'xe', 'xc', 'ne', 'nc', effect_col, se_col]

# Add CI columns
preview_cols.extend([ci_lower_col, ci_upper_col])

# Add fold-change if available
if es_config['has_fold_change']:
    if 'fold_change' in data_filtered.columns:
        preview_cols.append('fold_change')
    if 'Response_Ratio' in data_filtered.columns:
        preview_cols.append('Response_Ratio')
    elif 'Odds_Ratio' in data_filtered.columns:
        preview_cols.append('Odds_Ratio')

# Add weight
preview_cols.append('w_fixed')

# Display preview
preview_df = data_filtered[preview_cols].head(10).copy()

# Format numeric columns
for col in preview_df.select_dtypes(include=[np.number]).columns:
    if col in ['ne', 'nc']:
        preview_df[col] = preview_df[col].astype(int)
    elif col == 'w_fixed':
        preview_df[col] = preview_df[col].apply(lambda x: f'{x:.2f}')
    else:
        preview_df[col] = preview_df[col].apply(lambda x: f'{x:.4f}')

print(preview_df.to_string(index=False))

if len(data_filtered) > 10:
    print(f"\n... and {len(data_filtered) - 10} more observations")

# --- FINAL STATUS ---
print("\n" + "="*70)
print("✅ EFFECT SIZE CALCULATION COMPLETE")
print("="*70)

print(f"\n📊 Final Dataset Summary:")
print(f"  • Observations:           {len(data_filtered)}")
print(f"  • Unique papers:          {data_filtered['id'].nunique()}")
print(f"  • Effect size type:       {es_config['effect_label']} ({es_config['effect_label_short']})")
print(f"  • Mean effect size:       {effect_stats['mean']:.4f}")
print(f"  • Median effect size:     {effect_stats['median']:.4f}")
print(f"  • Effect size range:      [{effect_stats['min']:.4f}, {effect_stats['max']:.4f}]")

if es_config['has_fold_change']:
    if 'Response_Ratio' in data_filtered.columns:
        print(f"  • Mean response ratio:    {data_filtered['Response_Ratio'].mean():.3f}")
        print(f"  • Median fold-change:     {data_filtered['fold_change'].median():.2f}×")

print(f"\n📁 Columns Available:")
print(f"  Primary: {effect_col}, {var_col}, {se_col}")
print(f"  CI: {ci_lower_col}, {ci_upper_col}")
print(f"  Weight: w_fixed")
if es_config['has_fold_change']:
    print(f"  Interpretation: {', '.join([c for c in data_filtered.columns if 'fold' in c.lower() or 'ratio' in c.lower() or 'percent' in c.lower()])}")

print(f"\n⚠️  Quality Notes:")
if imputation_log['sde_imputed'] + imputation_log['sdc_imputed'] > 0:
    print(f"  • {imputation_log['sde_imputed'] + imputation_log['sdc_imputed']} SDs were imputed using median CV")
if calculation_log.get('extreme_effects', {}).get('count', 0) > 0:
    print(f"  • {calculation_log['extreme_effects']['count']} extreme effect sizes detected")
if outliers_iqr is not None and len(outliers_iqr) > 0:
    print(f"  • {len(outliers_iqr)} outliers detected using IQR method")

print(f"\n▶️  Next Steps:")
print(f"  1. Review the summary statistics and data quality notes")
print(f"  2. Run the next cell to perform meta-analysis and calculate pooled estimates")
print(f"  3. Consider the extreme values and outliers flagged above")

print("\n" + "="*70)

In [ ]:
#@title 🧮 CALCULATE EFFECT SIZES (V2)

# =============================================================================
# CELL 5: EFFECT SIZE CALCULATION
# Purpose: Calculate effect sizes, variances, and weights for meta-analysis
# Fix: Corrected KeyError by using dynamic column names for filtering.
# =============================================================================

import numpy as np
import pandas as pd
import datetime
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from scipy.special import gamma

# --- 1. SETUP TABS ---
tab_summary = widgets.Output()
tab_diag = widgets.Output()
tab_stats = widgets.Output()
tab_interp = widgets.Output()

tabs = widgets.Tab(children=[tab_summary, tab_diag, tab_stats, tab_interp])
tabs.set_title(0, '📊 Summary')
tabs.set_title(1, '📉 Diagnostics')
tabs.set_title(2, '📏 Detailed Stats')
tabs.set_title(3, '🧠 Interpretation')

# --- 2. CALCULATION ENGINE ---
def run_calculation():
    # Logs for different tabs
    log_diag = []
    log_summary = []

    # --- CONFIG ---
    try:
        if 'ANALYSIS_CONFIG' not in globals():
             print("❌ ERROR: ANALYSIS_CONFIG not found. Run Cell 4 first.")
             return
        effect_size_type = ANALYSIS_CONFIG['effect_size_type']
        es_config = ANALYSIS_CONFIG['es_config']
        log_summary.append(f"Configuration: {es_config['effect_label']} ({es_config['effect_label_short']})")
    except KeyError:
        print("❌ ERROR: Configuration keys missing. Run Cell 4 first.")
        return

    # --- DATA LOADING ---
    if 'data_filtered' not in globals():
        print("❌ ERROR: Data not found. Run Cell 3 first.")
        return

    df = data_filtered.copy()
    initial_obs = len(df)
    initial_papers = df['id'].nunique()

    # --- VALIDATION ---
    req_cols = ['xe', 'sde', 'ne', 'xc', 'sdc', 'nc']
    missing = [c for c in req_cols if c not in df.columns]
    if missing:
        print(f"❌ ERROR: Missing columns: {missing}")
        return

    # --- IMPUTATION (SD) ---
    log_diag.append("<b>1. Standard Deviation Imputation</b>")

    # Handle zeros
    if 'sde' in df.columns: df['sde'] = df['sde'].replace(0, np.nan)
    if 'sdc' in df.columns: df['sdc'] = df['sdc'].replace(0, np.nan)

    # Calculate CV
    valid_e = (df['sde'] > 0) & (df['xe'] > 0)
    valid_c = (df['sdc'] > 0) & (df['xc'] > 0)

    # Calculate median CV for imputation (use default 0.1 if no valid data)
    cv_e = (df.loc[valid_e, 'sde'] / df.loc[valid_e, 'xe']).median() if valid_e.any() else 0.1
    cv_c = (df.loc[valid_c, 'sdc'] / df.loc[valid_c, 'xc']).median() if valid_c.any() else 0.1

    df['sde_imputed'] = df['sde'].fillna(df['xe'] * cv_e)
    df['sdc_imputed'] = df['sdc'].fillna(df['xc'] * cv_c)

    n_imp_e = df['sde'].isna().sum()
    n_imp_c = df['sdc'].isna().sum()

    log_diag.append(f"• Imputed {n_imp_e} Exp SDs (using CV={cv_e:.4f})")
    log_diag.append(f"• Imputed {n_imp_c} Ctl SDs (using CV={cv_c:.4f})")

    # --- CLEANING (Negative/Zero) ---
    log_diag.append("<br><b>2. Data Cleaning</b>")

    if effect_size_type in ['lnRR', 'log_or']:
        # Remove negatives
        neg_mask = (df['xe'] < 0) | (df['xc'] < 0)
        n_neg = neg_mask.sum()
        if n_neg > 0:
            df = df[~neg_mask]
            log_diag.append(f"• Removed {n_neg} rows with negative values (invalid for {effect_size_type})")

        # Handle zeros
        zero_mask = (df['xe'] == 0) | (df['xc'] == 0)
        n_zero = zero_mask.sum()
        if n_zero > 0:
            df.loc[zero_mask, ['xe', 'xc']] += 0.001
            log_diag.append(f"• Adjusted {n_zero} rows with zero values (added 0.001)")

    # --- CALCULATION ---
    # Initialize dynamic column names to avoid KeyErrors later
    effect_col = es_config['effect_col']
    var_col = es_config['var_col']
    se_col = es_config['se_col']

    if effect_size_type == 'lnRR':
        df[effect_col] = np.log(df['xe'] / df['xc'])
        df[var_col] = (df['sde_imputed']**2 / (df['ne']*df['xe']**2)) + (df['sdc_imputed']**2 / (df['nc']*df['xc']**2))
        df[se_col] = np.sqrt(df[var_col])

        # Fold Change
        df['Response_Ratio'] = np.exp(df[effect_col])
        df['fold_change'] = df.apply(lambda r: r['Response_Ratio'] if r[effect_col]>=0 else -1/r['Response_Ratio'], axis=1)
        df['Percent_Change'] = (df['Response_Ratio'] - 1) * 100

    elif effect_size_type == 'hedges_g':
        df['df'] = df['ne'] + df['nc'] - 2
        df['sp'] = np.sqrt(((df['ne']-1)*df['sde_imputed']**2 + (df['nc']-1)*df['sdc_imputed']**2) / df['df'])
        df['d'] = (df['xe'] - df['xc']) / df['sp']

        # Exact Gamma correction
        m = df['df']
        df['J'] = gamma(m/2) / (np.sqrt(m/2) * gamma((m-1)/2))

        df[effect_col] = df['d'] * df['J']
        df[var_col] = ((df['ne']+df['nc']) / (df['ne']*df['nc']) + (df[effect_col]**2)/(2*(df['ne']+df['nc']))) * df['J']**2
        df[se_col] = np.sqrt(df[var_col])

    elif effect_size_type == 'cohen_d':
        df['df'] = df['ne'] + df['nc'] - 2
        df['sp'] = np.sqrt(((df['ne']-1)*df['sde_imputed']**2 + (df['nc']-1)*df['sdc_imputed']**2) / df['df'])
        df[effect_col] = (df['xe'] - df['xc']) / df['sp']
        df[var_col] = (df['ne']+df['nc']) / (df['ne']*df['nc']) + (df[effect_col]**2)/(2*(df['ne']+df['nc']))
        df[se_col] = np.sqrt(df[var_col])

    elif effect_size_type == 'log_or':
        # Simplified logOR
        df[effect_col] = np.log(df['xe'] / df['xc'])
        df[var_col] = (1/df['xe'] + 1/df['ne'] + 1/df['xc'] + 1/df['nc'])
        df[se_col] = np.sqrt(df[var_col])

    # --- CI & WEIGHTS ---
    # Use dynamic column names from es_config
    ci_lower_col = es_config.get('ci_lower_col', f"CI_lower_{es_config['effect_label_short']}")
    ci_upper_col = es_config.get('ci_upper_col', f"CI_upper_{es_config['effect_label_short']}")

    df[ci_lower_col] = df[effect_col] - 1.96 * df[se_col]
    df[ci_upper_col] = df[effect_col] + 1.96 * df[se_col]
    df['w_fixed'] = 1 / df[var_col]

    # --- FINAL CLEANING ---
    # Use the DYNAMIC variable names, not hardcoded strings
    df = df.dropna(subset=[effect_col, var_col]).copy()
    df = df[df[var_col] > 0].copy()

    # --- UPDATE CONFIG ---
    ANALYSIS_CONFIG['analysis_data'] = df
    # Ensure these match what was used
    ANALYSIS_CONFIG['effect_col'] = effect_col
    ANALYSIS_CONFIG['var_col'] = var_col
    ANALYSIS_CONFIG['se_col'] = se_col
    ANALYSIS_CONFIG['ci_lower_col'] = ci_lower_col
    ANALYSIS_CONFIG['ci_upper_col'] = ci_upper_col

    # --- POPULATE TABS ---

    # 1. SUMMARY TAB
    with tab_summary:
        clear_output()
        final_n = len(df)
        final_papers = df['id'].nunique()

        html_sum = f"""
        <div style='display:flex; gap:20px; margin-bottom:20px;'>
            <div style='background:#e8f5e9; padding:15px; border-radius:8px; flex:1; text-align:center;'>
                <h2 style='margin:0; color:#2e7d32;'>{final_n}</h2>
                <p style='margin:0; color:#1b5e20;'>Observations</p>
            </div>
            <div style='background:#e3f2fd; padding:15px; border-radius:8px; flex:1; text-align:center;'>
                <h2 style='margin:0; color:#1565c0;'>{final_papers}</h2>
                <p style='margin:0; color:#0d47a1;'>Studies</p>
            </div>
            <div style='background:#fff3e0; padding:15px; border-radius:8px; flex:1; text-align:center;'>
                <h2 style='margin:0; color:#e65100;'>{initial_obs - final_n}</h2>
                <p style='margin:0; color:#bf360c;'>Removed</p>
            </div>
        </div>

        <div style='padding:10px; border-left:4px solid #2E86AB; background:#f8f9fa;'>
            <b>✅ Status:</b> Calculation complete using <b>{es_config['effect_label']}</b>.<br>
            Data is ready for Meta-Analysis.
        </div>
        """
        display(HTML(html_sum))

        # Stats Summary
        if not df.empty:
            desc = df[effect_col].describe()
            print(f"\n📊 {es_config['effect_label']} Statistics:")
            print(f"   Mean:   {desc['mean']:.4f}")
            print(f"   Median: {desc['50%']:.4f}")
            print(f"   Min:    {desc['min']:.4f}")
            print(f"   Max:    {desc['max']:.4f}")
            print(f"   StdDev: {desc['std']:.4f}")
        else:
            print("⚠️ No valid data remaining.")

    # 2. DIAGNOSTICS TAB
    with tab_diag:
        clear_output()
        display(HTML("<b>🔍 Processing Log:</b>"))
        for line in log_diag:
            display(HTML(line))

        if not df.empty:
            # Outliers
            q1, q3 = df[effect_col].quantile([0.25, 0.75])
            iqr = q3 - q1
            outliers = df[(df[effect_col] < q1 - 1.5*iqr) | (df[effect_col] > q3 + 1.5*iqr)]

            display(HTML("<br><b>⚠️ Outlier Check (IQR Method):</b>"))
            if len(outliers) > 0:
                print(f"   Found {len(outliers)} potential outliers.")
                print(f"   Range: {outliers[effect_col].min():.2f} to {outliers[effect_col].max():.2f}")
            else:
                print("   No statistical outliers detected.")

    # 3. DETAILED STATS TAB
    with tab_stats:
        clear_output()
        if not df.empty:
            # Create a nice summary table
            stats_df = pd.DataFrame({
                'Effect Size': df[effect_col].describe(),
                'Variance': df[var_col].describe(),
                'Standard Error': df[se_col].describe(),
                'Weight (Fixed)': df['w_fixed'].describe()
            })
            display(stats_df.round(4))

            print("\n📋 Preview (First 5 rows):")
            cols_show = ['id', 'xe', 'xc', 'ne', 'nc', effect_col, se_col]
            display(df[cols_show].head())

    # 4. INTERPRETATION TAB
    with tab_interp:
        clear_output()
        if not df.empty:
            # Direction
            n_pos = (df[effect_col] > 0).sum()
            n_neg = (df[effect_col] < 0).sum()

            # Magnitude (Cohen's benchmarks for g/d)
            if effect_size_type in ['hedges_g', 'cohen_d']:
                mag_small = ((df[effect_col].abs() >= 0.2) & (df[effect_col].abs() < 0.5)).sum()
                mag_med = ((df[effect_col].abs() >= 0.5) & (df[effect_col].abs() < 0.8)).sum()
                mag_large = (df[effect_col].abs() >= 0.8).sum()

                mag_html = f"""
                <br><b>📏 Magnitude (Cohen's Benchmarks):</b>
                <ul>
                    <li><b>Small (0.2-0.5):</b> {mag_small} ({mag_small/len(df):.1%})</li>
                    <li><b>Medium (0.5-0.8):</b> {mag_med} ({mag_med/len(df):.1%})</li>
                    <li><b>Large (>0.8):</b> {mag_large} ({mag_large/len(df):.1%})</li>
                </ul>
                """
            else:
                mag_html = ""

            html_interp = f"""
            <div style='font-size:14px;'>
                <h4>📈 Effect Direction</h4>
                <ul>
                    <li><b>Positive Effect:</b> {n_pos} studies ({n_pos/len(df):.1%})<br>
                    <i>(Treatment > Control)</i></li>
                    <li><b>Negative Effect:</b> {n_neg} studies ({n_neg/len(df):.1%})<br>
                    <i>(Treatment < Control)</i></li>
                </ul>

                {mag_html}

                <br><b>🎯 Precision Check</b>
                <ul>
                    <li><b>Mean CI Width:</b> {(df[ANALYSIS_CONFIG['ci_upper_col']] - df[ANALYSIS_CONFIG['ci_lower_col']]).mean():.4f}</li>
                    <li><b>Zero Variance Studies:</b> {(df[var_col] == 0).sum()} (Removed)</li>
                </ul>
            </div>
            """
            display(HTML(html_interp))

# --- 3. RUN ---
run_calculation()
display(tabs)

In [ ]:
#@title 📊 Step 2: Overall Meta-Analysis (V1)

# =============================================================================
# MERGED CELL: OVERALL META-ANALYSIS (Standard 2-Level + Three-Level REML)
#
# This cell combines:
#   - Cell 6 (Original): Standard 2-level meta-analysis with heterogeneity
#   - Cell 6.5 (Original): Three-level REML model for dependent effect sizes
#
# Purpose: Both analyses run to ensure backward compatibility.
#          The 3-level model is the PRIMARY recommended result.
#          Standard 2-level results preserved for comparison.
#
# Outputs:
#   - ANALYSIS_CONFIG['overall_results']: Standard 2-level results
#   - ANALYSIS_CONFIG['three_level_results']: Three-level REML results
#   - ANALYSIS_CONFIG['analysis_data']: Filtered data for downstream cells
# =============================================================================


# =============================================================================
# CELL 6: OVERALL META-ANALYSIS
# Purpose: Calculate pooled effect sizes and assess heterogeneity
# Dependencies: Cell 5 (data_filtered with effect sizes, ANALYSIS_CONFIG)
# Outputs: Overall pooled estimates (fixed & random effects), heterogeneity stats
# =============================================================================

import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import datetime
from scipy.stats import norm, chi2, t

# Ensure ANALYSIS_CONFIG is initialized if it doesn't exist (for fresh runs)
if 'ANALYSIS_CONFIG' not in globals():
    ANALYSIS_CONFIG = {'tau_method': 'REML', 'use_knapp_hartung': True, 'overall_results': {}}

# --- ADD THIS AT THE START OF CELL 6 (before main analysis) ---

print("\n" + "="*70)
print("TAU-SQUARED ESTIMATOR SELECTION")

# =============================================================================
# KNAPP-HARTUNG CORRECTION FUNCTION
# =============================================================================

print("="*70)

# -----------------------------------------------------------------------------
# WIDGET DEFINITION AND PERSISTENCE LOGIC
# -----------------------------------------------------------------------------

# Tau-Squared Method Widget
if 'calculate_tau_squared' in globals():
    print("✅ Advanced estimators available")
    method_options = [
        ('REML (Recommended)', 'REML'),
        ('DerSimonian-Laird (Classic)', 'DL'),
        ('Maximum Likelihood', 'ML'),
        ('Paule-Mandel', 'PM'),
        ('Sidik-Jonkman', 'SJ')
    ]
    method_default = ANALYSIS_CONFIG.get('tau_method', 'REML')
    method_help = widgets.HTML(
        "<div style='background-color: #e8f4f8; padding: 10px; margin: 10px 0; border-radius: 5px;'>"
        "<b>💡 Method Guide:</b><br>"
        "• <b>REML:</b> ⭐ Best choice for most analyses. Unbiased and accurate.<br>"
        "• <b>DL:</b> Fast but can underestimate τ² with few studies.<br>"
        "• <b>ML:</b> Efficient but biased downward.<br>"
        "• <b>PM:</b> Exact Q = k-1 solution.<br>"
        "• <b>SJ:</b> Conservative, good for k < 10."
        "</div>"
    )
else:
    print("⚠️  Using DerSimonian-Laird method only")
    print("  Run Cell 4.5 to enable REML and other methods")
    method_options = [('DerSimonian-Laird', 'DL')]
    method_default = 'DL'
    method_help = widgets.HTML(
        "<div style='background-color: #fff3cd; padding: 10px; margin: 10px 0; border-radius: 5px;'>"
        "⚠️ Run <b>Cell 4.5 (Heterogeneity Estimators)</b> to access REML and other methods."
        "</div>"
    )

tau_method_widget = widgets.Dropdown(
    options=method_options,
    value=method_default, # Initialize from config/default
    description='τ² Method:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='400px')
)

# Persistence Handler for τ² Method
def on_method_change(change):
    ANALYSIS_CONFIG['tau_method'] = change['new']
tau_method_widget.observe(on_method_change, names='value')


# Knapp-Hartung Correction Widget
kh_default = ANALYSIS_CONFIG.get('use_knapp_hartung', True)
use_kh_widget = widgets.Checkbox(
    value=kh_default, # Initialize from config/default
    description='Use Knapp-Hartung correction for confidence intervals (Recommended for k<20)',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

kh_help = widgets.HTML(
    "<div style='background-color: #e7f3ff; padding: 10px; margin: 10px 0; border-radius: 5px;'>"
    "<b>ℹ️ Knapp-Hartung Correction:</b><br>"
    "• Uses t-distribution instead of normal (better for small k)<br>"
    "• Adjusts SE based on observed variability (Q statistic)<br>"
    "• <b>Recommended</b>, especially for k < 20 studies<br>"
    "• Produces more conservative (wider) confidence intervals<br>"
    "• Reduces false positive rate (better Type I error control)"
    "</div>"
)

# Persistence Handler for Knapp-Hartung
def on_kh_change(change):
    ANALYSIS_CONFIG['use_knapp_hartung'] = change['new']
use_kh_widget.observe(on_kh_change, names='value')

# Ensure the config reflects the current value for the current run
ANALYSIS_CONFIG['tau_method'] = tau_method_widget.value
ANALYSIS_CONFIG['use_knapp_hartung'] = use_kh_widget.value


# Display re-run reminder
rerun_message = widgets.HTML(
    "<div style='background-color: #fffbf0; padding: 8px; margin: 10px 0; border-left: 3px solid #ff9800; border-radius: 3px;'>"
    "⚠️ <b>Important:</b> After changing the method, you must re-run this cell to apply the new estimator."
    "</div>"
)
# Rerun message will be displayed at the end

print("\n" + "="*70)

print("\n" + "="*70)
print("OVERALL META-ANALYSIS")
print("="*70)
print(f"Timestamp: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# --- STEP 1: LOAD CONFIGURATION ---
print("\n" + "="*70)
print("STEP 1: LOADING CONFIGURATION")
print("="*70)

try:
    effect_col = ANALYSIS_CONFIG['effect_col']
    var_col = ANALYSIS_CONFIG['var_col']
    se_col = ANALYSIS_CONFIG['se_col']
    es_config = ANALYSIS_CONFIG['es_config']
    effect_type = ANALYSIS_CONFIG['effect_size_type']

    print(f"✓ Configuration loaded successfully")
    print(f"  Effect size: {es_config['effect_label']} ({es_config['effect_label_short']})")
    print(f"  Effect column: {effect_col}")
    print(f"  Variance column: {var_col}")
    print(f"  SE column: {se_col}")
except KeyError as e:
    print(f"❌ ERROR: Configuration not found - {e}")
    print("\nTroubleshooting:")
    print("  1. Ensure Cell 5 (effect size calculation) was run successfully")
    print("  2. Check that ANALYSIS_CONFIG dictionary exists")
    print("  3. Verify effect sizes were calculated properly")
    raise

# --- STEP 2: PREPARE ANALYSIS DATA ---
print("\n" + "="*70)
print("STEP 2: DATA PREPARATION")
print("="*70)

print(f"\n🔍 Preparing data for meta-analysis...")

# Store initial counts
initial_count = len(data_filtered)
initial_papers = data_filtered['id'].nunique()

print(f"\n  Initial dataset:")
print(f"    • Observations: {initial_count}")
print(f"    • Unique papers: {initial_papers}")

# Use only valid data points (non-missing effect size, variance, and weight)
analysis_data = data_filtered.dropna(subset=[effect_col, var_col, 'w_fixed']).copy()

# Ensure variance is positive
positive_var = analysis_data[var_col] > 0
n_non_positive = (~positive_var).sum()

if n_non_positive > 0:
    print(f"\n  ⚠️  Removing {n_non_positive} observations with non-positive variance")
    analysis_data = analysis_data[positive_var].copy()

# Final counts
k = len(analysis_data)
k_papers = analysis_data['id'].nunique()

if k < 1:
    print(f"\n❌ ERROR: No valid studies available for meta-analysis")
    print(f"  Possible causes:")
    print(f"    • All variances are zero or negative")
    print(f"    • Missing effect size data")
    print(f"    • All weights are invalid")
    raise ValueError("No valid studies available for meta-analysis after filtering.")

ANALYSIS_CONFIG['analysis_data'] = analysis_data # Save filtered data for downstream cells

print(f"\n  ✓ Final analysis dataset:")
print(f"    • Observations (k): {k}")
print(f"    • Unique papers: {k_papers}")
print(f"    • Removed: {initial_count - k} observations")

# Calculate average observations per paper
avg_obs_per_paper = k / k_papers if k_papers > 0 else 0
print(f"    • Avg obs per paper: {avg_obs_per_paper:.2f}")

# --- STEP 3: HANDLE SINGLE STUDY CASE ---
if k == 1:
    print("\n" + "="*70)
    print("⚠️  SINGLE STUDY ANALYSIS")
    print("="*70)

    print(f"\n⚠️  WARNING: Only one observation available (k=1)")
    print(f"  Meta-analysis requires multiple studies")
    print(f"  Reporting single study results:")

    single_study = analysis_data.iloc[0]

    print(f"\n📋 Single Study Details:")
    print(f"  Study ID: {single_study.get('id', 'N/A')}")
    print(f"  {es_config['effect_label_short']}: {single_study[effect_col]:.4f}")
    print(f"  Variance: {single_study[var_col]:.6f}")
    print(f"  SE: {single_study[se_col]:.4f}")
    print(f"  Treatment mean: {single_study['xe']:.4f}")
    print(f"  Control mean: {single_study['xc']:.4f}")
    print(f"  Sample size (treatment): {int(single_study['ne'])}")
    print(f"  Sample size (control): {int(single_study['nc'])}")

    if es_config['has_fold_change']:
        if 'fold_change' in single_study:
            print(f"  Fold-change: {single_study['fold_change']:.2f}×")
        if 'Response_Ratio' in single_study:
            print(f"  Response Ratio: {single_study['Response_Ratio']:.3f}")

    # Calculate confidence interval
    z_crit = norm.ppf(0.975)  # 1.96
    ci_lower = single_study[effect_col] - z_crit * single_study[se_col]
    ci_upper = single_study[effect_col] + z_crit * single_study[se_col]

    print(f"\n  95% CI: [{ci_lower:.4f}, {ci_upper:.4f}]")

    # Set variables to NaN for consistency
    pooled_effect_fixed = single_study[effect_col]
    pooled_var_fixed = single_study[var_col]
    pooled_SE_fixed = single_study[se_col]
    ci_lower_fixed = ci_lower
    ci_upper_fixed = ci_upper
    p_value_fixed = np.nan

    Qt = np.nan
    p_heterogeneity = np.nan
    I_squared = np.nan
    tau_squared_DL = np.nan

    pooled_effect_random = pooled_effect_fixed
    pooled_var_random = pooled_var_fixed
    pooled_SE_random = pooled_SE_fixed
    ci_lower_random = ci_lower
    ci_upper_random = ci_upper
    p_value_random = np.nan

    pi_lower_random = np.nan
    pi_upper_random = np.nan

    pooled_SE_random_KH = np.nan
    ci_lower_random_KH = np.nan
    ci_upper_random_KH = np.nan
    p_value_random_KH = np.nan
    kh_results = None

    print(f"\n" + "="*70)
    print(f"⚠️  META-ANALYSIS NOT POSSIBLE WITH ONE STUDY")
    print(f"="*70)
    print(f"\nRecommendations:")
    print(f"  1. Report single study results with appropriate caution")
    print(f"  2. Cannot assess heterogeneity or publication bias")
    print(f"  3. Consider collecting more studies before drawing conclusions")

else:
    # --- STEP 4: FIXED-EFFECTS MODEL ---
    print("\n" + "="*70)
    print("STEP 3: FIXED-EFFECTS MODEL")
    print("="*70)

    print(f"\n📐 Model Assumption:")
    print(f"  All studies share a common true effect size")
    print(f"  Differences between studies are due to sampling error only")

    print(f"\n🔢 Calculating inverse-variance weighted mean...")

    # Significance level
    alpha = 0.05
    z_crit = norm.ppf(1 - alpha / 2)  # ~1.96 for 95% CI

    # Calculate sum of weights
    sum_w_fixed = analysis_data['w_fixed'].sum()

    if sum_w_fixed <= 0:
        print(f"❌ ERROR: Sum of fixed-effects weights is non-positive")
        raise ValueError("Sum of fixed-effects weights is non-positive. Check variance values.")

    print(f"  Sum of weights: {sum_w_fixed:.2f}")

    # Pooled effect size (weighted mean)
    pooled_effect_fixed = (analysis_data['w_fixed'] * analysis_data[effect_col]).sum() / sum_w_fixed

    # Variance of pooled effect
    pooled_var_fixed = 1 / sum_w_fixed
    pooled_SE_fixed = np.sqrt(pooled_var_fixed)

    # 95% Confidence Interval
    ci_lower_fixed = pooled_effect_fixed - z_crit * pooled_SE_fixed
    ci_upper_fixed = pooled_effect_fixed + z_crit * pooled_SE_fixed

    # Test significance (H0: effect = 0)
    z_stat_fixed = pooled_effect_fixed / pooled_SE_fixed
    p_value_fixed = 2 * (1 - norm.cdf(abs(z_stat_fixed)))

    # Display results
    print(f"\n📊 Fixed-Effects Results:")
    print(f"  {'Statistic':<25} {'Value':>15}")
    print(f"  {'-'*25} {'-'*15}")
    print(f"  {'Pooled ' + es_config['effect_label_short']:<25} {pooled_effect_fixed:>15.4f}")
    print(f"  {'Standard Error':<25} {pooled_SE_fixed:>15.4f}")
    print(f"  {'Variance':<25} {pooled_var_fixed:>15.6f}")
    print(f"  {'95% CI Lower':<25} {ci_lower_fixed:>15.4f}")
    print(f"  {'95% CI Upper':<25} {ci_upper_fixed:>15.4f}")
    print(f"  {'Z-statistic':<25} {z_stat_fixed:>15.4f}")
    print(f"  {'P-value':<25} {p_value_fixed:>15.4g}")

    # Interpretation for ratio-based measures
    if es_config['has_fold_change']:
        print(f"\n📈 Biological Interpretation:")

        if effect_type == 'lnRR':
            pooled_RR_fixed = np.exp(pooled_effect_fixed)
            pooled_fold_fixed = pooled_RR_fixed if pooled_effect_fixed >= 0 else -1/pooled_RR_fixed
            pooled_pct_fixed = (pooled_RR_fixed - 1) * 100
            ci_lower_RR = np.exp(ci_lower_fixed)
            ci_upper_RR = np.exp(ci_upper_fixed)

            print(f"  {'Metric':<30} {'Value':>15}")
            print(f"  {'-'*30} {'-'*15}")
            print(f"  {'Response Ratio (RR)':<30} {pooled_RR_fixed:>15.3f}")
            print(f"  {'Fold-change':<30} {pooled_fold_fixed:>+14.2f}×")
            print(f"  {'Percent change':<30} {pooled_pct_fixed:>+14.1f}%")
            print(f"  {'95% CI (RR scale)':<30} [{ci_lower_RR:.3f}, {ci_upper_RR:.3f}]")

            # Direction interpretation
            if pooled_effect_fixed > 0.05:
                direction = "INCREASE (upregulation)"
            elif pooled_effect_fixed < -0.05:
                direction = "DECREASE (downregulation)"
            else:
                direction = "NO CHANGE"
            print(f"\n  Overall direction: {direction}")

        elif effect_type == 'log_or':
            pooled_OR_fixed = np.exp(pooled_effect_fixed)
            ci_lower_OR = np.exp(ci_lower_fixed)
            ci_upper_OR = np.exp(ci_upper_fixed)

            print(f"  {'Metric':<30} {'Value':>15}")
            print(f"  {'-'*30} {'-'*15}")
            print(f"  {'Odds Ratio (OR)':<30} {pooled_OR_fixed:>15.3f}")
            print(f"  {'95% CI (OR scale)':<30} [{ci_lower_OR:.3f}, {ci_upper_OR:.3f}]")

            if pooled_OR_fixed > 1:
                direction = "Positive association"
            elif pooled_OR_fixed < 1:
                direction = "Negative association"
            else:
                direction = "No association"
            print(f"\n  Interpretation: {direction}")

    # Significance interpretation
    print(f"\n📌 Statistical Significance:")
    if p_value_fixed < 0.001:
        sig_text = "HIGHLY SIGNIFICANT (p < 0.001)"
        sig_symbol = "***"
    elif p_value_fixed < 0.01:
        sig_text = "VERY SIGNIFICANT (p < 0.01)"
        sig_symbol = "**"
    elif p_value_fixed < 0.05:
        sig_text = "SIGNIFICANT (p < 0.05)"
        sig_symbol = "*"
    else:
        sig_text = "NOT SIGNIFICANT (p ≥ 0.05)"
        sig_symbol = "ns"

    print(f"  The overall effect is {sig_text} {sig_symbol}")

    # --- STEP 5: HETEROGENEITY ASSESSMENT ---
    print("\n" + "="*70)
    print("STEP 4: HETEROGENEITY ASSESSMENT")
    print("="*70)

    print(f"\n📊 Testing for variability across studies...")

    # Cochran's Q statistic
    Qt = (analysis_data['w_fixed'] * (analysis_data[effect_col] - pooled_effect_fixed)**2).sum()
    df_Q = k - 1

    print(f"\n🔬 Cochran's Q Test:")
    print(f"  Q statistic: {Qt:.4f}")
    print(f"  Degrees of freedom: {df_Q}")
    print(f"  Expected value under H₀: {df_Q}")

    # P-value for Q test (H0: homogeneous effects)
    if df_Q > 0:
        p_heterogeneity = 1 - chi2.cdf(Qt, df_Q)
        print(f"  P-value (χ² test): {p_heterogeneity:.4g}")

        if p_heterogeneity < 0.001:
            q_interp = "Highly significant heterogeneity (p < 0.001)"
        elif p_heterogeneity < 0.01:
            q_interp = "Very significant heterogeneity (p < 0.01)"
        elif p_heterogeneity < 0.10:
            q_interp = "Significant heterogeneity (p < 0.10)"
        else:
            q_interp = "No significant heterogeneity (p ≥ 0.10)"

        print(f"  Interpretation: {q_interp}")
    else:
        p_heterogeneity = np.nan
        print(f"  P-value: N/A (only one study)")

    # I-squared (proportion of variance due to heterogeneity)
    print(f"\n📏 I² (I-squared) Statistic:")

    if Qt > df_Q:
        I_squared = ((Qt - df_Q) / Qt) * 100
    else:
        I_squared = 0

    print(f"  I² = {I_squared:.2f}%")
    print(f"  Interpretation: {I_squared:.2f}% of total variation is due to heterogeneity")

    # Interpretation of I² with color coding
    if I_squared < 25:
        i2_interp = "Low heterogeneity (might not be important)"
        i2_color = "🟢"
        i2_recommendation = "Fixed or random effects both acceptable"
    elif I_squared < 50:
        i2_interp = "Moderate heterogeneity"
        i2_color = "🟡"
        i2_recommendation = "Consider random effects model"
    elif I_squared < 75:
        i2_interp = "Substantial heterogeneity"
        i2_color = "🟠"
        i2_recommendation = "Use random effects model; explore sources"
    else:
        i2_interp = "Considerable heterogeneity"
        i2_color = "🔴"
        i2_recommendation = "Use random effects model; investigate thoroughly"

    print(f"  {i2_color} {i2_interp}")
    print(f"  → {i2_recommendation}")

    # Tau-squared (between-study variance) - using selected method
    print(f"\n🔬 Between-Study Variance (Tau²):")

    # FIX: Read the currently selected method from the persistent config
    selected_method = ANALYSIS_CONFIG['tau_method']
    print(f"  Reading selected τ² method: {selected_method}")

    # Calculate tau-squared using selected method
    if 'calculate_tau_squared' in globals() and selected_method != 'DL':
        # Use advanced estimators from Cell 4.5
        print(f"  Using {selected_method} estimator...")
        tau_squared_DL, tau_info = calculate_tau_squared(
            analysis_data,
            effect_col,
            var_col,
            method=selected_method
        )
        tau_squared_DL = float(tau_squared_DL)
        tau_DL = np.sqrt(tau_squared_DL)
        method_used = selected_method

        # Also calculate DL for comparison
        sum_w_fixed_sq = (analysis_data['w_fixed']**2).sum()
        C = sum_w_fixed - (sum_w_fixed_sq / sum_w_fixed)
        if C > 0 and Qt > df_Q:
            tau_squared_DL_comparison = (Qt - df_Q) / C
        else:
            tau_squared_DL_comparison = 0
    else:
        # Fallback to DerSimonian-Laird (inline calculation)
        sum_w_fixed_sq = (analysis_data['w_fixed']**2).sum()
        C = sum_w_fixed - (sum_w_fixed_sq / sum_w_fixed)

        print(f"  Using DL estimator (default or selected)...")
        print(f"  C constant: {C:.4f}")

        if C > 0 and Qt > df_Q:
            tau_squared_DL = (Qt - df_Q) / C
        else:
            tau_squared_DL = 0

        tau_DL = np.sqrt(tau_squared_DL)
        method_used = 'DL'
        tau_squared_DL_comparison = None

    print(f"  Tau² (variance): {tau_squared_DL:.6f}")
    print(f"  Tau (SD): {tau_DL:.4f}")
    print(f"  Method: {method_used}")

    if tau_squared_DL > 0:
        print(f"  Interpretation: Average between-study variation = {tau_DL:.4f} {es_config['effect_label_short']} units")
    else:
        print(f"  Interpretation: No detectable between-study variation")

    # Display method status and comparison
    if 'compare_tau_estimators' in globals() and method_used != 'DL' and k >= 5:
        # Enhanced comparison using all tau estimators
        print(f"\n" + "="*70)
        print("📊 TAU-SQUARED ESTIMATOR COMPARISON")
        print("="*70)
        print(f"\nComparing all available tau-squared estimation methods:")
        print(f"(Sample size: k = {k} studies)\n")

        # Get all estimator results
        comparison_df = compare_tau_estimators(analysis_data, effect_col, var_col)

        # Convert DataFrame to dict for easier access
        comparison_results = dict(zip(comparison_df['Method'], comparison_df['τ²']))

        # Display formatted table
        print(f"{'Method':<15} {'τ²':>12} {'τ':>12} {'% Diff from REML':>18}   ")
        print(f"{'-'*15} {'-'*12} {'-'*12} {'-'*18}   ")

        # Get REML value for comparison
        reml_tau_sq = float(comparison_results['REML'])

        # Display each method
        for method_name, tau_sq in comparison_results.items():
            tau = np.sqrt(float(tau_sq))

            # Calculate % difference from REML
            if reml_tau_sq > 0:
                pct_diff = ((float(tau_sq) - reml_tau_sq) / reml_tau_sq) * 100
            else:
                pct_diff = 0

            # Add indicator for the method that was actually used
            indicator = " ←" if method_name == method_used else ""

            print(f"{method_name:<15} {float(tau_sq):>12.6f} {tau:>12.4f} {pct_diff:>17.1f}%{indicator:>3}")

        print()

        # Calculate REML vs DL difference for interpretation
        dl_tau_sq = float(comparison_results['DL'])
        if reml_tau_sq > 0:
            reml_dl_diff = abs((reml_tau_sq - dl_tau_sq) / reml_tau_sq) * 100
        else:
            reml_dl_diff = 0

        # Provide interpretation
        print(f"📋 Interpretation:")
        print(f"  REML vs DL difference: {reml_dl_diff:.1f}%")

        if reml_dl_diff > 20:
            print(f"  ⚠️  Large difference - method choice is important")
            print(f"  → REML provides more accurate estimate for this dataset")
        elif reml_dl_diff > 10:
            print(f"  ℹ️  Moderate difference - REML recommended")
            print(f"  → Consider using REML for more reliable heterogeneity estimates")
        else:
            print(f"  ✓ Small difference - methods agree")
            print(f"  → All methods provide similar tau-squared estimates")

        print(f"\n💡 Note: The method marked with ← was used in this analysis")

    elif tau_squared_DL_comparison is not None and method_used != 'DL':
        # Fallback to simple comparison for k < 5
        # Calculate difference
        diff_abs = abs(tau_squared_DL - tau_squared_DL_comparison)
        if tau_squared_DL_comparison > 0:
            diff_pct = (diff_abs / tau_squared_DL_comparison) * 100
        else:
            diff_pct = 0

        # Display comparison
        print(f"\n📊 Method Comparison:")
        print(f"  {method_used} τ²: {tau_squared_DL:.6f}")
        print(f"  DL τ²:   {tau_squared_DL_comparison:.6f}")
        print(f"  Difference: {diff_abs:.6f} ({diff_pct:.1f}%)")

        if diff_pct > 10:
            print(f"  ⚠️  WARNING: Difference >10% - method choice may substantially affect results")
        elif diff_pct > 5:
            print(f"  ⚡ Moderate difference - {method_used} provides more accurate estimate")
        else:
            print(f"  ✓ Methods agree closely")

        print(f"\n💡 Note: Full comparison available with k ≥ 5 studies (current: k = {k})")


    # Overall heterogeneity summary
    print(f"\n📋 Heterogeneity Summary:")
    print(f"  {'Statistic':<20} {'Value':>15} {'Interpretation':<30}")
    print(f"  {'-'*20} {'-'*15} {'-'*30}")
    print(f"  {'Q':<20} {Qt:>15.2f} {'Test statistic':<30}")
    print(f"  {'P-value':<20} {p_heterogeneity:>15.4g} {q_interp.split('(')[0].strip():<30}")
    print(f"  {'I²':<20} {I_squared:>14.1f}% {i2_interp.split('(')[0].strip():<30}")
    print(f"  {'Tau²':<20} {tau_squared_DL:>15.4f} {'Between-study variance':<30}")
    print(f"  {'Tau':<20} {tau_DL:>15.4f} {'Between-study SD':<30}")

    # Continue to Part 2...
    # --- STEP 6: RANDOM-EFFECTS MODEL ---
    print("\n" + "="*70)
    print("STEP 5: RANDOM-EFFECTS MODEL")
    print("="*70)

    print(f"\n📐 Model Assumption:")
    print(f"  Studies estimate different but related true effects")
    print(f"  Accounts for both within-study and between-study variation")
    print(f"  More conservative when heterogeneity is present")

    print(f"\n🔢 Calculating random-effects weights...")
    print(f"  Formula: w_random = 1 / (variance + τ²)")

    # Calculate random-effects weights
    analysis_data['w_random'] = 1 / (analysis_data[var_col] + tau_squared_DL)
    sum_w_random = analysis_data['w_random'].sum()

    # Variables for K-H use and results storage
    tau_squared = tau_squared_DL
    pooled_SE_random_KH = np.nan
    ci_lower_random_KH = np.nan
    ci_upper_random_KH = np.nan
    p_value_random_KH = np.nan
    kh_results = None

    if sum_w_random <= 0:
        print(f"\n❌ WARNING: Sum of random-effects weights is non-positive")
        print(f"  This should not occur with valid data")

        pooled_effect_random = np.nan
        pooled_var_random = np.nan
        pooled_SE_random = np.nan
        ci_lower_random = np.nan
        ci_upper_random = np.nan
        z_stat_random = np.nan
        p_value_random = np.nan
        pi_lower_random = np.nan
        pi_upper_random = np.nan
    else:
        print(f"  Sum of random-effects weights: {sum_w_random:.2f}")
        print(f"  Sum of fixed-effects weights:  {sum_w_fixed:.2f}")

        # Ratio comparison
        weight_ratio = sum_w_random / sum_w_fixed
        print(f"  Weight ratio (RE/FE): {weight_ratio:.3f}")

        if weight_ratio < 0.5:
            print(f"  → Random effects gives much less weight to studies (high heterogeneity)")
        elif weight_ratio < 0.8:
            print(f"  → Random effects moderately reduces weights")
        else:
            print(f"  → Random effects similar to fixed effects (low heterogeneity)")

        # Pooled effect size
        pooled_effect_random = (analysis_data['w_random'] * analysis_data[effect_col]).sum() / sum_w_random

        # Variance of pooled effect
        pooled_var_random = 1 / sum_w_random
        pooled_SE_random = np.sqrt(pooled_var_random)

        # 95% CI (Standard Z-test)
        ci_lower_random = pooled_effect_random - z_crit * pooled_SE_random
        ci_upper_random = pooled_effect_random + z_crit * pooled_SE_random

        # Test significance (Standard Z-test)
        z_stat_random = pooled_effect_random / pooled_SE_random
        p_value_random = 2 * (1 - norm.cdf(abs(z_stat_random)))

        # =============================================================================
        # APPLY KNAPP-HARTUNG CORRECTION (if enabled)
        # =============================================================================
        kh_enabled = ANALYSIS_CONFIG.get('use_knapp_hartung', False)

        if k > 1 and kh_enabled:
            print("\n" + "="*70)
            print("KNAPP-HARTUNG ADJUSTMENT")
            print("="*70)

            kh_results = calculate_knapp_hartung_ci(
                yi=analysis_data[effect_col].values,
                vi=analysis_data[var_col].values,
                tau_sq=tau_squared,  # Uses the selected estimator (tau_squared_DL)
                pooled_effect=pooled_effect_random,
                alpha=0.05
            )

            if kh_results is not None:
                print(f"\n📐 Applying Knapp-Hartung correction to random-effects CI:")
                print(f"  • Degrees of freedom: {kh_results['df']}")
                print(f"  • t critical value: {kh_results['t_crit']:.3f} (vs. 1.96 for normal)")
                print(f"  • Q statistic: {kh_results['Q']:.3f}")

                # Compare standard vs K-H
                print(f"\n📊 Comparison of Methods:")
                print(f"  {'Method':<22} {'SE':<10} {'95% CI Lower':<13} {'95% CI Upper':<13} {'P-value':<10}")
                print(f"  {'-'*70}")
                print(f"  {'Standard (Z-test)':<22} {pooled_SE_random:<10.4f} {ci_lower_random:<13.4f} {ci_upper_random:<13.4f} {p_value_random:<10.4g}")
                print(f"  {'Knapp-Hartung (t)':<22} {kh_results['se_KH']:<10.4f} {kh_results['ci_lower']:<13.4f} {kh_results['ci_upper']:<13.4f} {kh_results['p_value']:<10.4g}")

                # Calculate CI width difference
                ci_width_standard = ci_upper_random - ci_lower_random
                ci_width_kh = kh_results['ci_upper'] - kh_results['ci_lower']
                width_increase = ((ci_width_kh - ci_width_standard) / ci_width_standard) * 100

                print(f"\n  • K-H CI is {abs(width_increase):.1f}% {'wider' if width_increase > 0 else 'narrower'} than standard CI")

                # Check if conclusion changes
                standard_sig = p_value_random < 0.05
                kh_sig = kh_results['p_value'] < 0.05

                if standard_sig != kh_sig:
                    print(f"\n  ⚠️  IMPORTANT: Statistical significance CHANGES with K-H correction!")
                    print(f"    Standard: p = {p_value_random:.4g} ({'significant' if standard_sig else 'non-significant'})")
                    print(f"    K-H:      p = {kh_results['p_value']:.4g} ({'significant' if kh_sig else 'non-significant'})")
                else:
                    print(f"  ✓ Conclusion does not change (both {'significant' if kh_sig else 'non-significant'})")

                # Recommendation based on k
                print(f"\n💡 RECOMMENDATION:")
                if k < 20:
                    print(f"  With k = {k} studies, the Knapp-Hartung method is RECOMMENDED.")
                    print(f"  Report the K-H confidence interval as your primary result.")
                else:
                    print(f"  With k = {k} studies, both methods give similar results.")
                    print(f"  K-H is more conservative and may be preferred.")

                # Store K-H results for later use
                pooled_SE_random_KH = kh_results['se_KH']
                ci_lower_random_KH = kh_results['ci_lower']
                ci_upper_random_KH = kh_results['ci_upper']
                p_value_random_KH = kh_results['p_value']

                # Save K-H specific results to config
                if 'overall_results' not in ANALYSIS_CONFIG: ANALYSIS_CONFIG['overall_results'] = {}
                ANALYSIS_CONFIG['overall_results']['knapp_hartung'] = {
                    'used': True, 'se': pooled_SE_random_KH, 'ci_lower': ci_lower_random_KH,
                    'ci_upper': ci_upper_random_KH, 'p_value': p_value_random_KH,
                    'comparison': {'standard_se': pooled_SE_random, 'width_increase_percent': width_increase, 'significance_changed': standard_sig != kh_sig}
                }
                print(f"\n  ✓ Results saved to ANALYSIS_CONFIG['overall_results']['knapp_hartung']")
            else:
                print("  ⚠️  Knapp-Hartung calculation failed (e.g., numerical issue)")
                if 'overall_results' not in ANALYSIS_CONFIG: ANALYSIS_CONFIG['overall_results'] = {}
                ANALYSIS_CONFIG['overall_results']['knapp_hartung'] = {'used': False, 'reason': 'calc_error'}

        elif k <= 1:
            print(f"\n  ℹ️  Knapp-Hartung correction not applicable (k={k})")
            if 'overall_results' not in ANALYSIS_CONFIG: ANALYSIS_CONFIG['overall_results'] = {}
            ANALYSIS_CONFIG['overall_results']['knapp_hartung'] = {'used': False, 'reason': 'k<=1'}
        else:
            print(f"\n  ℹ️  Knapp-Hartung correction not applied (user disabled)")
            if 'overall_results' not in ANALYSIS_CONFIG: ANALYSIS_CONFIG['overall_results'] = {}
            ANALYSIS_CONFIG['overall_results']['knapp_hartung'] = {'used': False, 'reason': 'user_disabled'}


        # Display results
        print(f"\n📊 Random-Effects Results:")
        print(f"  {'Statistic':<25} {'Value':>15}")
        print(f"  {'-'*25} {'-'*15}")
        print(f"  {'Pooled ' + es_config['effect_label_short']:<25} {pooled_effect_random:>15.4f}")
        print(f"  {'Standard Error (Z-test)':<25} {pooled_SE_random:>15.4f}")
        print(f"  {'Variance':<25} {pooled_var_random:>15.6f}")
        print(f"  {'95% CI Lower (Z-test)':<25} {ci_lower_random:>15.4f}")
        print(f"  {'95% CI Upper (Z-test)':<25} {ci_upper_random:>15.4f}")
        print(f"  {'Z-statistic':<25} {z_stat_random:>15.4f}")
        print(f"  {'P-value (Z-test)':<25} {p_value_random:>15.4g}")

        # Check K-H status and display K-H results if available
        if kh_results is not None:
            print(f"  {'SE (K-H t-test)':<25} {pooled_SE_random_KH:>15.4f}")
            print(f"  {'95% CI Lower (K-H t-test)':<25} {ci_lower_random_KH:>15.4f}")
            print(f"  {'95% CI Upper (K-H t-test)':<25} {ci_upper_random_KH:>15.4f}")
            print(f"  {'P-value (K-H t-test)':<25} {p_value_random_KH:>15.4g}")

            re_p_value_disp = p_value_random_KH
            re_ci_lower_disp = ci_lower_random_KH
            re_ci_upper_disp = ci_upper_random_KH
            re_se_disp = pooled_SE_random_KH
        else:
            re_p_value_disp = p_value_random
            re_ci_lower_disp = ci_lower_random
            re_ci_upper_disp = ci_upper_random
            re_se_disp = pooled_SE_random


        # Interpretation for ratio-based measures
        if es_config['has_fold_change']:
            print(f"\n📈 Biological Interpretation:")

            if effect_type == 'lnRR':
                pooled_RR_random = np.exp(pooled_effect_random)
                pooled_fold_random = pooled_RR_random if pooled_effect_random >= 0 else -1/pooled_RR_random
                pooled_pct_random = (pooled_RR_random - 1) * 100
                ci_lower_RR_random = np.exp(ci_lower_random)
                ci_upper_RR_random = np.exp(ci_upper_random)

                print(f"  {'Metric':<30} {'Value':>15}")
                print(f"  {'-'*30} {'-'*15}")
                print(f"  {'Response Ratio (RR)':<30} {pooled_RR_random:>15.3f}")
                print(f"  {'Fold-change':<30} {pooled_fold_random:>+14.2f}×")
                print(f"  {'Percent change':<30} {pooled_pct_random:>+14.1f}%")
                print(f"  {'95% CI (RR scale)':<30} [{ci_lower_RR_random:.3f}, {ci_upper_RR_random:.3f}]")

                # Direction interpretation
                if pooled_effect_random > 0.05:
                    direction = "INCREASE (upregulation)"
                elif pooled_effect_random < -0.05:
                    direction = "DECREASE (downregulation)"
                else:
                    direction = "NO CHANGE"
                print(f"\n  Overall direction: {direction}")

            elif effect_type == 'log_or':
                pooled_OR_random = np.exp(pooled_effect_random)
                ci_lower_OR_random = np.exp(ci_lower_random)
                ci_upper_OR_random = np.exp(ci_upper_random)

                print(f"  {'Metric':<30} {'Value':>15}")
                print(f"  {'-'*30} {'-'*15}")
                print(f"  {'Odds Ratio (OR)':<30} {pooled_OR_random:>15.3f}")
                print(f"  {'95% CI (OR scale)':<30} [{ci_lower_OR_random:.3f}, {ci_upper_OR_random:.3f}]")

                if pooled_OR_random > 1:
                    direction = "Positive association"
                elif pooled_OR_random < 1:
                    direction = "Negative association"
                else:
                    direction = "No association"
                print(f"\n  Interpretation: {direction}")

        # Significance interpretation
        print(f"\n📌 Statistical Significance:")
        if re_p_value_disp < 0.001:
            sig_text_re = "HIGHLY SIGNIFICANT (p < 0.001)"
            sig_symbol_re = "***"
        elif re_p_value_disp < 0.01:
            sig_text_re = "VERY SIGNIFICANT (p < 0.01)"
            sig_symbol_re = "**"
        elif re_p_value_disp < 0.05:
            sig_text_re = "SIGNIFICANT (p < 0.05)"
            sig_symbol_re = "*"
        else:
            sig_text_re = "NOT SIGNIFICANT (p ≥ 0.05)"
            sig_symbol_re = "ns"

        re_model_type = " (K-H)" if kh_results is not None else " (Z-test)"
        print(f"  The overall effect{re_model_type} is {sig_text_re} {sig_symbol_re}")

        # --- STEP 7: 95% PREDICTION INTERVAL ---
        print("\n" + "="*70)
        print("STEP 6: 95% PREDICTION INTERVAL")
        print("="*70)

        print(f"\n📊 Prediction Interval (PI):")
        print(f"  Estimates where the true effect in a NEW study is expected to fall")
        print(f"  Wider than CI because it accounts for between-study heterogeneity")
        print(f"  More clinically relevant than CI for assessing effect consistency")

        if k > 2:
            # Degrees of freedom for t-distribution
            df_pi = k - 2
            t_crit = t.ppf(1 - alpha / 2, df=df_pi)

            # Standard error for prediction
            # SE_prediction = sqrt(τ² + SE²_pooled)
            se_prediction = np.sqrt(tau_squared_DL + pooled_var_random)

            # Calculate prediction interval
            pi_lower_random = pooled_effect_random - t_crit * se_prediction
            pi_upper_random = pooled_effect_random + t_crit * se_prediction

            print(f"\n  📏 Calculation Details:")
            print(f"    Pooled effect: {pooled_effect_random:.4f}")
            print(f"    Tau² (between-study var): {tau_squared_DL:.6f}")
            print(f"    SE² (pooled estimate): {pooled_var_random:.6f}")
            print(f"    SE (prediction): {se_prediction:.4f}")
            print(f"    t-critical value (df={df_pi}): {t_crit:.3f}")
            print(f"    Margin of error: ±{t_crit * se_prediction:.4f}")

            print(f"\n  📊 Results:")
            print(f"    95% Prediction Interval: [{pi_lower_random:.4f}, {pi_upper_random:.4f}]")

            # Compare PI width to CI width
            ci_width = re_ci_upper_disp - re_ci_lower_disp
            pi_width = pi_upper_random - pi_lower_random
            width_ratio = pi_width / ci_width if ci_width > 0 else np.inf

            print(f"\n  📐 Interval Comparison:")
            print(f"    CI width ({re_model_type.strip()}): {ci_width:.4f}")
            print(f"    PI width: {pi_width:.4f}")
            print(f"    Ratio (PI/CI): {width_ratio:.2f}×")

            if width_ratio > 3:
                print(f"    → PI much wider than CI (substantial heterogeneity)")
            elif width_ratio > 1.5:
                print(f"    → PI moderately wider than CI (moderate heterogeneity)")
            else:
                print(f"    → PI similar to CI (low heterogeneity)")

            # Interpretation for ratio measures
            if es_config['has_fold_change'] and effect_type == 'lnRR':
                pi_lower_RR = np.exp(pi_lower_random)
                pi_upper_RR = np.exp(pi_upper_random)

                print(f"\n  📈 Prediction Interval (RR scale):")
                print(f"    95% PI: [{pi_lower_RR:.3f}, {pi_upper_RR:.3f}]")

            # Check if PI includes null
            null_value = es_config['null_value']
            pi_includes_null = (pi_lower_random <= null_value <= pi_upper_random)

            print(f"\n  💡 Interpretation:")
            if pi_includes_null:
                print(f"    ⚠️  PI includes null effect ({null_value})")
                print(f"    → A future study could plausibly find no effect")
                print(f"    → Effect direction may not be consistent across all contexts")
            else:
                print(f"    ✓ PI excludes null effect ({null_value})")
                print(f"    → Future studies expected to show consistent effect direction")
                print(f"    → High confidence in effect direction")

            print(f"\n  📝 Note: In 95% of similar future studies, the true effect")
            print(f"    is predicted to lie between {pi_lower_random:.4f} and {pi_upper_random:.4f}")

        else:
            print(f"\n  ⚠️  Skipped: Not enough studies for prediction interval")
            print(f"    Requires at least 3 studies (k ≥ 3)")
            print(f"    Current k = {k}")

            pi_lower_random = np.nan
            pi_upper_random = np.nan

# --- STEP 8: MODEL COMPARISON ---
print("\n" + "="*70)
print("STEP 7: MODEL COMPARISON")
print("="*70)

if k > 1:
    print(f"\n📊 Side-by-Side Comparison:")
    print(f"\n  {'Model':<20} {'Effect':>12} {'SE':>10} {'95% CI':>28} {'P-value':>10}")
    print(f"  {'-'*82}")

    # Fixed-effects
    fe_ci_str = f"[{ci_lower_fixed:>7.4f}, {ci_upper_fixed:>7.4f}]"
    print(f"  {'Fixed-Effects':<20} {pooled_effect_fixed:>12.4f} {pooled_SE_fixed:>10.4f} {fe_ci_str:>28} {p_value_fixed:>10.4g}")

    # Random-effects
    if pd.notna(pooled_effect_random):
        re_ci_str = f"[{re_ci_lower_disp:>7.4f}, {re_ci_upper_disp:>7.4f}]"
        print(f"  {'Random-Effects' + re_model_type:<20} {pooled_effect_random:>12.4f} {re_se_disp:>10.4f} {re_ci_str:>28} {re_p_value_disp:>10.4g}")

        # Prediction interval
        if pd.notna(pi_lower_random):
            pi_str = f"[{pi_lower_random:>7.4f}, {pi_upper_random:>7.4f}]"
            print(f"  {'95% Pred. Interval':<20} {'':<12} {'':<10} {pi_str:>28} {'':<10}")

    # Calculate and display differences
    if pd.notna(pooled_effect_random):
        effect_diff = pooled_effect_random - pooled_effect_fixed
        effect_diff_pct = (effect_diff / abs(pooled_effect_fixed)) * 100 if pooled_effect_fixed != 0 else np.inf
        se_diff = re_se_disp - pooled_SE_fixed
        se_ratio = re_se_disp / pooled_SE_fixed if pooled_SE_fixed > 0 else np.inf

        print(f"\n  📏 Model Differences (RE{re_model_type} vs FE):")
        print(f"    Effect difference (RE - FE): {effect_diff:+.4f} ({effect_diff_pct:+.1f}%)")
        print(f"    SE difference (RE - FE): {se_diff:+.4f}")
        print(f"    SE ratio (RE / FE): {se_ratio:.2f}×")

        # Interpretation
        print(f"\n  💡 Interpretation:")
        if abs(effect_diff) < 0.05:
            print(f"    ✓ Models agree very closely")
            print(f"      → Low heterogeneity, either model acceptable")
        elif abs(effect_diff) < 0.15:
            print(f"    ⚠️  Models show small differences")
            print(f"      → Some heterogeneity present, random-effects preferred")
        elif abs(effect_diff) < 0.3:
            print(f"    ⚠️  Models show moderate differences")
            print(f"      → Moderate heterogeneity, use random-effects")
        else:
            print(f"    🔴 Models show substantial differences")
            print(f"      → High heterogeneity, must use random-effects")
            print(f"      → Investigate sources of heterogeneity")

        if se_ratio > 1.5:
            print(f"\n    ⚠️  Random-effects SE is {se_ratio:.1f}× larger than fixed-effects")
            print(f"      → Random-effects provides more conservative estimates")

        # Check agreement on significance
        fe_sig = p_value_fixed < 0.05
        re_sig_comparison = re_p_value_disp < 0.05

        if fe_sig == re_sig_comparison:
            print(f"\n    ✓ Both models agree on statistical significance")
        else:
            print(f"\n    ⚠️  Models disagree on statistical significance!")
            if fe_sig and not re_sig_comparison:
                print(f"      → Fixed-effects significant, random-effects{re_model_type} not")
                print(f"      → Use random-effects (more conservative)")
            else:
                print(f"      → Random-effects{re_model_type} significant, fixed-effects not")
                print(f"      → Unlikely scenario, verify data")

# --- STEP 9: RECOMMENDATIONS ---
print("\n" + "="*70)
print("STEP 8: INTERPRETATION & RECOMMENDATIONS")
print("="*70)

# The logic here relies on re_p_value_disp being set, which it is in step 6 if k > 1

if k == 1:
    print(f"\n🔴 SINGLE STUDY LIMITATION")
    print(f"\n  Current Status:")
    # ... (single study recommendation remains the same) ...

elif I_squared > 50 or (pd.notna(p_heterogeneity) and p_heterogeneity < 0.10):
    print(f"\n🔴 HIGH HETEROGENEITY DETECTED")
    print(f"\n  Heterogeneity Metrics:")
    # ... (high heterogeneity recommendation remains the same) ...

else:
    print(f"\n🟢 LOW-TO-MODERATE HETEROGENEITY")
    print(f"\n  Heterogeneity Metrics:")
    # ... (low/moderate heterogeneity recommendation remains the same) ...

# Effect size type specific recommendations
# ... (rest of step 9 remains the same) ...

# --- STEP 10: SAVE RESULTS ---
print("\n" + "="*70)
print("STEP 9: SAVING RESULTS")
print("="*70)

ANALYSIS_CONFIG['overall_results'] = {
    'timestamp': datetime.datetime.now(),
    'k': k,
    'k_papers': k_papers,

    # Fixed-effects
    'pooled_effect_fixed': pooled_effect_fixed,
    'pooled_var_fixed': pooled_var_fixed,
    'pooled_SE_fixed': pooled_SE_fixed if k > 1 else np.nan,
    'ci_lower_fixed': ci_lower_fixed if k > 1 else np.nan,
    'ci_upper_fixed': ci_upper_fixed if k > 1 else np.nan,
    'p_value_fixed': p_value_fixed if k > 1 else np.nan,

    # Heterogeneity
    'Qt': Qt,
    'I_squared': I_squared,
    'tau_squared': tau_squared_DL,

    # Random-effects (Standard Z-test results for historical consistency)
    'pooled_effect_random': pooled_effect_random if k > 1 else pooled_effect_fixed,
    'pooled_SE_random_Z': pooled_SE_random if k > 1 and pd.notna(pooled_effect_random) else np.nan,
    'ci_lower_random_Z': ci_lower_random if k > 1 and pd.notna(pooled_effect_random) else np.nan,
    'ci_upper_random_Z': ci_upper_random if k > 1 and pd.notna(pooled_effect_random) else np.nan,
    'p_value_random_Z': p_value_random if k > 1 and pd.notna(pooled_effect_random) else np.nan,

    # Primary Reported Random-effects (K-H if run, Z-test otherwise)
    'pooled_SE_random_reported': re_se_disp,
    'ci_lower_random_reported': re_ci_lower_disp,
    'ci_upper_random_reported': re_ci_upper_disp,
    'p_value_random_reported': re_p_value_disp,

    # Knapp-Hartung specific results are already saved above if executed.

    # Prediction interval
    'pi_lower_random': pi_lower_random,
    'pi_upper_random': pi_upper_random,

    # Interpretation
    'recommended_model': 'random-effects' if k > 1 and (I_squared > 25 or p_heterogeneity < 0.10) else 'either',
    'heterogeneity_level': i2_color if k > 1 else 'N/A'
}

# Add fold-changes if applicable
if es_config['has_fold_change'] and k > 1:
    if effect_type == 'lnRR':
        ANALYSIS_CONFIG['overall_results']['pooled_fold_fixed'] = pooled_fold_fixed
        ANALYSIS_CONFIG['overall_results']['pooled_fold_random'] = pooled_fold_random
        ANALYSIS_CONFIG['overall_results']['pooled_RR_fixed'] = pooled_RR_fixed
        ANALYSIS_CONFIG['overall_results']['pooled_RR_random'] = pooled_RR_random
        ANALYSIS_CONFIG['overall_results']['pooled_pct_change_random'] = pooled_pct_random
    elif effect_type == 'log_or':
        ANALYSIS_CONFIG['overall_results']['pooled_OR_fixed'] = pooled_OR_fixed
        ANALYSIS_CONFIG['overall_results']['pooled_OR_random'] = pooled_OR_random

print(f"\n✓ Results saved to ANALYSIS_CONFIG['overall_results']")

# Create summary metadata
OVERALL_META_METADATA = {
    'timestamp': datetime.datetime.now(),
    'n_studies': k,
    'n_papers': k_papers,
    'model_recommended': ANALYSIS_CONFIG['overall_results']['recommended_model'],
    'heterogeneity': {
        'I_squared': I_squared,
        'level': i2_interp if k > 1 else 'N/A',
        'p_value': p_heterogeneity,
        'tau_squared': tau_squared_DL
    },
    'primary_result': {
        'effect': pooled_effect_random if k > 1 else pooled_effect_fixed,
        'ci_lower': re_ci_lower_disp if k > 1 else ci_lower_fixed,
        'ci_upper': re_ci_upper_disp if k > 1 else ci_upper_fixed,
        'p_value': re_p_value_disp if k > 1 else p_value_fixed,
        'significant': (re_p_value_disp < 0.05) if k > 1 and pd.notna(re_p_value_disp) else False
    }
}

print(f"\n✓ Metadata saved to OVERALL_META_METADATA")

# --- FINAL STATUS ---
print("\n" + "="*70)
print("✅ OVERALL META-ANALYSIS COMPLETE")
print("="*70)

if k > 1:
    # Use reported variables for summary
    print(f"\n📊 Key Findings Summary:")
    print(f"  • Studies analyzed: {k} observations from {k_papers} papers")
    print(f"  • Pooled effect ({ANALYSIS_CONFIG['overall_results']['recommended_model']}): {pooled_effect_random:.4f}")
    print(f"  • 95% CI{re_model_type}: [{re_ci_lower_disp:.4f}, {re_ci_upper_disp:.4f}]")
    if pd.notna(pi_lower_random):
        print(f"  • 95% PI: [{pi_lower_random:.4f}, {pi_upper_random:.4f}]")
    print(f"  • Statistical significance: {sig_text_re}")
    print(f"  • Heterogeneity (I²): {I_squared:.1f}% - {i2_interp}")

    if es_config['has_fold_change'] and effect_type == 'lnRR':
        print(f"\n📈 Biological Interpretation:")
        print(f"  • Pooled fold-change: {pooled_fold_random:+.2f}×")
        print(f"  • Response ratio: {pooled_RR_random:.3f}")
        print(f"  • Percent change: {pooled_pct_random:+.1f}%")

    # ... (Conclusion interpretation remains the same) ...
else:
    # ... (Single Study Summary remains the same) ...
    pass

print(f"\n▶️  Next Steps:")
print(f"  1. Review the overall pooled estimates above")
print(f"  2. Run SUBGROUP ANALYSIS to explore heterogeneity (next cell)")
print(f"  3. Create FOREST PLOTS for visualization")
print(f"  4. Assess PUBLICATION BIAS with funnel plots")
print(f"  5. Conduct SENSITIVITY ANALYSES (leave-one-out)")
if kh_results is not None:
    print(f"  6. Note: Overall Random-Effects CI/P-value uses Knapp-Hartung correction.")

if I_squared > 50:
    print(f"\n💡 Priority Recommendations:")
    print(f"  • High heterogeneity detected - subgroup analysis is essential")
    print(f"  • Consider meta-regression if moderators are available")
    print(f"  • Check for outliers and influential studies")



# =============================================================================
# DISPLAY CONFIGURATION WIDGET AT END
# =============================================================================
# Widget is displayed here so it's visible and not buried by analysis output

print("\n" + "="*70)
print("⚙️  META-ANALYSIS CONFIGURATION")
print("="*70)
print()
print("You can modify the heterogeneity estimator and CI method below:")
print()

# Display method selection widget
config_box = widgets.VBox([
    method_help,
    tau_method_widget,
    kh_help,
    use_kh_widget,
    rerun_message
], layout=widgets.Layout(
    border='2px solid #2E86AB',
    padding='15px',
    margin='10px 0'
))

display(config_box)

# Add helpful completion message
display(widgets.HTML(
    "<div style='background-color: #d4edda; border-left: 4px solid #28a745; padding: 12px; margin: 15px 0; border-radius: 4px;'>"
    "✅ <b>Analysis Complete!</b><br><br>"
    "• Review the results above<br>"
    "• Modify the estimator/correction in the widget above if needed<br>"
    "• Re-run this cell to recalculate with a different method<br>"
    "• Proceed to the next cell for advanced analyses"
    "</div>"
))

print("\n" + "="*70)
# =============================================================================
# THREE-LEVEL META-ANALYSIS (PRIMARY RESULT)
# =============================================================================
# The following section runs a three-level REML model to account for
# within-study dependency of effect sizes. This is the RECOMMENDED approach
# when studies contribute multiple effect sizes.
# Standard I² calculated for comparison but 3-level model is primary.
# =============================================================================

# --- 1. CORE HELPER FUNCTIONS (THREE-LEVEL REML) ---

# --- 1. CORE HELPER FUNCTIONS (HIGH PRECISION) ---

def _run_three_level_reml(analysis_data, effect_col, var_col):
    """
    Main optimization function for Three-Level Meta-Analysis.
    Uses Two-Pass optimization (Global L-BFGS-B -> Local Nelder-Mead)
    with high precision tolerances to match R's metafor.
    """
    print("  Preparing data for optimization...")

    # --- 1. Data Preparation ---
    grouped = analysis_data.groupby('id')
    y_all = [group[effect_col].values for _, group in grouped]
    v_all = [group[var_col].values for _, group in grouped]
    N_total = len(analysis_data)
    M_studies = len(y_all)

    # --- 2. First Pass: Global Search (L-BFGS-B) ---
    start_points = [[0.01, 0.01], [0.5, 0.1], [0.1, 0.5], [0.001, 0.001]]
    best_result = None
    best_lik = np.inf

    print(f"  Optimizing (Pass 1: Global Search)...")

    for start in start_points:
        res = minimize(
            _negative_log_likelihood_reml,
            x0=start,
            args=(y_all, v_all, N_total, M_studies),
            method='L-BFGS-B',
            # Use tight tolerances here
            bounds=[(1e-8, None), (1e-8, None)],
            options={'ftol': 1e-12, 'gtol': 1e-12}
        )
        if res.success and res.fun < best_lik:
            best_lik = res.fun
            best_result = res

    if not best_result or not best_result.success:
        print(f"  ❌ OPTIMIZATION FAILED")
        return None, None, None

    # --- 3. Second Pass: Polishing (Nelder-Mead) ---
    # Sometimes gradient methods get stuck slightly off in flat valleys.
    # We polish the result using a direct search method.
    print(f"  Optimizing (Pass 2: High Precision Polishing)...")

    final_res = minimize(
        _negative_log_likelihood_reml,
        x0=best_result.x,
        args=(y_all, v_all, N_total, M_studies),
        method='Nelder-Mead',
        bounds=[(1e-8, None), (1e-8, None)],
        options={'xatol': 1e-12, 'fatol': 1e-12}
    )

    # Use the better of the two results
    if final_res.fun < best_lik:
        best_result = final_res

    print(f"  ✓ Optimization successful (LogLik: {-best_result.fun:.4f})")

    # --- 4. Calculate Final Estimates ---
    tau_sq_est, sigma_sq_est = best_result.x
    final_estimates = _get_three_level_estimates(
        [tau_sq_est, sigma_sq_est], y_all, v_all, N_total, M_studies
    )

    # --- 5. Standard Errors for Variances (via Hessian) ---
    # (This part remains from your original code or can be approximated)
    # For now, we focus on returning the accurate point estimates
    final_estimates.update({
        'se_tau_sq': np.nan, 'ci_lower_tau_sq': np.nan, 'ci_upper_tau_sq': np.nan,
        'se_sigma_sq': np.nan, 'ci_lower_sigma_sq': np.nan, 'ci_upper_sigma_sq': np.nan
    })

    return final_estimates, (y_all, v_all, N_total, M_studies), best_result

    # --- 5. Singular Fit Check ---
    # This warns users if the model collapses (e.g., variance ~ 0)
    if tau_sq_est < 0.001 or sigma_sq_est < 0.001:
        print("\n⚠️  WARNING: SINGULAR FIT DETECTED")
        print("   One or more variance components are estimated as near-zero.")
        print("   This implies that the multi-level structure might be too complex for your data.")
        print("   (e.g., differences between studies are negligible compared to sampling error).")

    # --- 6. AI-Readable & Human-Readable Summary ---
    mu = final_estimates['mu']
    se_mu = final_estimates['se_mu']
    ci_lower = mu - 1.96 * se_mu
    ci_upper = mu + 1.96 * se_mu
    z_score = mu / se_mu
    p_value = 2 * (1 - norm.cdf(abs(z_score)))

    # Calculate Intraclass Correlation Coefficients (ICC)
    total_var = tau_sq_est + sigma_sq_est
    icc_l3 = (tau_sq_est / total_var * 100) if total_var > 0 else 0
    icc_l2 = (sigma_sq_est / total_var * 100) if total_var > 0 else 0

    print("\n" + "="*30 + " MODEL SUMMARY " + "="*30)
    print(f"Model: Three-Level REML (k={N_total}, m={M_studies})")
    print(f"Pooled Effect: {mu:.4f} (SE: {se_mu:.4f})")
    print(f"95% CI: [{ci_lower:.4f}, {ci_upper:.4f}]")
    print(f"Z-score: {z_score:.3f} (p={p_value:.4f})")
    print("-" * 75)
    print(f"Variance L3 (Tau^2):   {tau_sq_est:.4f} (Between-Study)")
    print(f"Variance L2 (Sigma^2): {sigma_sq_est:.4f} (Within-Study)")
    print(f"ICC L3: {icc_l3:.1f}% | ICC L2: {icc_l2:.1f}%")
    print("="*75)

    return final_estimates, (y_all, v_all, N_total, M_studies), best_result


# --- 2. WIDGET DEFINITIONS ---

run_button = widgets.Button(
    description='▶ Run Three-Level Analysis',
    button_style='success',
    layout=widgets.Layout(width='450px', height='50px'),
    style={'font_weight': 'bold'}
)

analysis_output = widgets.Output()

# --- 3. MAIN BUTTON HANDLER ---

@run_button.on_click
def run_analysis(b):
    with analysis_output:
        clear_output(wait=True)

        print("="*70)
        print("RUNNING THREE-LEVEL META-ANALYSIS")
        print("="*70)
        print(f"Timestamp: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

        try:
            # --- 1. Load Config and Data ---
            print("STEP 1: LOADING CONFIGURATION")
            print("---------------------------------")

            if 'ANALYSIS_CONFIG' not in globals():
                raise NameError("ANALYSIS_CONFIG not found. Run previous cells first.")

            # Check if 'analysis_data' was saved in Cell 6, otherwise use 'data_filtered'
            if 'analysis_data' in ANALYSIS_CONFIG:
                analysis_data = ANALYSIS_CONFIG['analysis_data']
            elif 'data_filtered' in globals():
                # We need to ensure 'data_filtered' is clean before use
                analysis_data = data_filtered.dropna(subset=[ANALYSIS_CONFIG['effect_col'], ANALYSIS_CONFIG['var_col'], 'w_fixed']).copy()
                analysis_data = analysis_data[analysis_data[ANALYSIS_CONFIG['var_col']] > 0].copy()
            else:
                raise ValueError("Cannot find 'analysis_data' or 'data_filtered'")

            effect_col = ANALYSIS_CONFIG['effect_col']
            var_col = ANALYSIS_CONFIG['var_col']
            es_config = ANALYSIS_CONFIG['es_config']
            overall_results = ANALYSIS_CONFIG['overall_results'] # This is the dictionary from Cell 6

            print(f"  ✓ Effect: {es_config['effect_label']} ({effect_col})")
            print(f"  ✓ Variance: {var_col}")

            # --- 2. Auto-Detection Check ---
            print("\nSTEP 2: CHECKING DATA STRUCTURE")
            print("---------------------------------")

            k_obs = len(analysis_data)
            k_studies = analysis_data['id'].nunique()
            avg_obs = k_obs / k_studies

            print(f"  • Total observations (k_obs): {k_obs}")
            print(f"  • Total studies (k_studies):  {k_studies}")
            print(f"  • Avg. observations/study:  {avg_obs:.2f}")

            if k_obs == k_studies:
                print("\n✅ AUTO-DETECTION: NOT REQUIRED")
                print("  Each study contributes only one effect size.")
                print("  The standard meta-analysis (Cell 6) is appropriate.")
                print("  Three-level model is not necessary.")
                ANALYSIS_CONFIG['three_level_results'] = {'status': 'not_required'}
                return

            print("\n  ✓ Dependent effect sizes detected. Proceeding with three-level model.")

            # --- 3. Run REML Optimization ---
            print("\nSTEP 3: RUNNING THREE-LEVEL REML ESTIMATION")
            print("---------------------------------")

            estimates, data_lists, optimizer_result = _run_three_level_reml(analysis_data, effect_col, var_col)

            if estimates is None:
                raise RuntimeError("REML optimization failed to converge.")

            # Unpack data_lists to get N_total and M_studies
            y_all, v_all, N_total, M_studies = data_lists

            # --- 4. Calculate Final Results ---
            print("\nSTEP 4: CALCULATING FINAL ESTIMATES")
            print("---------------------------------")

            mu = estimates['mu']
            se_mu = estimates['se_mu']
            var_mu = estimates['var_mu']

            ci_lower = mu - 1.96 * se_mu
            ci_upper = mu + 1.96 * se_mu
            p_value = 2 * (1 - norm.cdf(abs(mu / se_mu)))

            tau_sq = estimates['tau_sq']
            sigma_sq = estimates['sigma_sq']

            ci_lower_tau_sq = estimates['ci_lower_tau_sq']
            ci_upper_tau_sq = estimates['ci_upper_tau_sq']
            ci_lower_sigma_sq = estimates['ci_lower_sigma_sq']
            ci_upper_sigma_sq = estimates['ci_upper_sigma_sq']

            # --- 5. Calculate Diagnostics ---
            print("\nSTEP 5: CALCULATING DIAGNOSTICS")
            print("---------------------------------")

            # ICC
            total_var = tau_sq + sigma_sq
            if total_var == 0:
                ICC_level2, ICC_level3 = 0.0, 0.0
            else:
                ICC_level2 = (sigma_sq / total_var) * 100 # Within-study
                ICC_level3 = (tau_sq / total_var) * 100   # Between-study

            # AIC/BIC (k=3 params: mu, tau_sq, sigma_sq)
            k_params = 3
            log_lik_ml = estimates['log_lik_ml']
            AIC = (2 * k_params) - (2 * log_lik_ml)
            BIC = (k_params * np.log(N_total)) - (2 * log_lik_ml)

            print("  ✓ Diagnostics calculated")

            # --- 6. Display Results ---
            print("\n" + "="*70)
            print("THREE-LEVEL MODEL: POOLED EFFECT")
            print("="*70)

            print(f"\n  {'Metric':<20} {'Estimate':>15} {'95% CI Lower':>15} {'95% CI Upper':>15}")
            print(f"  {'-'*20} {'-'*15} {'-'*15} {'-'*15}")
            print(f"  {es_config['effect_label']:<20} {mu:>15.4f} {ci_lower:>15.4f} {ci_upper:>15.4f}")

            if es_config['has_fold_change']:
                RR = np.exp(mu)
                RR_CI_lower = np.exp(ci_lower)
                RR_CI_upper = np.exp(ci_upper)
                print(f"  {'Response Ratio (RR)':<20} {RR:>15.4f} {RR_CI_lower:>15.4f} {RR_CI_upper:>15.4f}")

            print(f"\n  Z-value: {mu/se_mu:.4f}  |  P-value: {p_value:.4g}")

            print("\n" + "="*70)
            print("THREE-LEVEL MODEL: VARIANCE COMPONENTS")
            print("="*70)

            print(f"\n  {'Component':<25} {'Estimate (Var)':>15} {'95% CI Lower':>15} {'95% CI Upper':>15}")
            print(f"  {'-'*25} {'-'*15} {'-'*15} {'-'*15}")
            print(f"  Level 3: Between-Study (τ²): {tau_sq:>15.4f} {ci_lower_tau_sq:>15.4f} {ci_upper_tau_sq:>15.4f}")
            print(f"  Level 2: Within-Study (σ²):  {sigma_sq:>15.4f} {ci_lower_sigma_sq:>15.4f} {ci_upper_sigma_sq:>15.4f}")

            print(f"\n  Intraclass Correlation (ICC):")
            print(f"  • {ICC_level3:6.1f}% of variance is between studies (Level 3)")
            print(f"  • {ICC_level2:6.1f}% of variance is within studies (Level 2)")

            print(f"\n  Model Fit:")
            print(f"  • Log-Likelihood (REML): {estimates['log_lik_reml']:.3f}")
            print(f"  • AIC: {AIC:.3f} | BIC: {BIC:.3f}")

            # --- 7. Comparison Table (FIXED) ---
            print("\n" + "="*70)
            print("COMPARISON: STANDARD VS. THREE-LEVEL MODEL")
            print("="*70)

            # --- FIX: Retrieve the primary reported results from Cell 6 ---
            # These keys contain the standard Z-test results OR the K-H corrected results.

            std_effect = overall_results['pooled_effect_random']
            # Use the reported CI and SE keys which are guaranteed to be present and hold the final RE result
            std_ci_lower = overall_results.get('ci_lower_random_reported', overall_results['ci_lower_random_Z'])
            std_ci_upper = overall_results.get('ci_upper_random_reported', overall_results['ci_upper_random_Z'])
            std_se = overall_results.get('pooled_SE_random_reported', overall_results['pooled_SE_random_Z'])

            std_ci_lower = overall_results['ci_lower_random_reported']
            std_ci_upper = overall_results['ci_upper_random_reported']
            std_se = overall_results['pooled_SE_random_reported']

            # Determine which type of CI was used in Cell 6 for label clarity
            if overall_results.get('knapp_hartung', {}).get('used', False):
                 std_model_label = 'Standard (K-H)'
            else:
                 std_model_label = 'Standard (Z-test)'

            print(f"\n  {'Model':<25} {'Effect':>12} {'Std. Error':>12} {'95% CI Width':>12} {'95% CI':<25}")
            print(f"  {'-'*25} {'-'*12} {'-'*12} {'-'*12} {'-'*25}")

            print(f"  {std_model_label:<25} {std_effect:>12.4f} {std_se:>12.4f} "
                  f"{(std_ci_upper - std_ci_lower):>12.4f} "
                  f"[{std_ci_lower:.4f}, {std_ci_upper:.4f}]")

            print(f"  {'Three-Level (REML)':<25} {mu:>12.4f} {se_mu:>12.4f} "
                  f"{(ci_upper - ci_lower):>12.4f} "
                  f"[{ci_lower:.4f}, {ci_upper:.4f}]")

            print("\n  💡 Interpretation:")
            if se_mu > std_se:
                se_diff = (se_mu - std_se) / std_se * 100
                print(f"  ✓ Three-level model SE is {se_diff:.1f}% larger (more conservative).")
                print("  ✓ This correctly accounts for data dependency.")
            else:
                print("  ⚠️  Three-level model SE is not larger. Check model assumptions.")
            # --- END FIX ---

            # --- 8. Save Results ---
            print("\nSTEP 6: SAVING RESULTS")
            print("---------------------------------")

            results_dict = {
                'timestamp': datetime.datetime.now(),
                'status': 'completed',
                'k_obs': k_obs,
                'k_studies': k_studies,
                'pooled_effect': mu,
                'se': se_mu,
                'var': var_mu,
                'ci_lower': ci_lower,
                'ci_upper': ci_upper,
                'p_value': p_value,
                'tau_squared': tau_sq,
                'se_tau_sq': estimates.get('se_tau_sq'),
                'ci_lower_tau_sq': estimates.get('ci_lower_tau_sq'),
                'ci_upper_tau_sq': estimates.get('ci_upper_tau_sq'),
                'sigma_squared': sigma_sq,
                'se_sigma_sq': estimates.get('se_sigma_sq'),
                'ci_lower_sigma_sq': estimates.get('ci_lower_sigma_sq'),
                'ci_upper_sigma_sq': estimates.get('ci_upper_sigma_sq'),
                'ICC_level2_pct': ICC_level2,
                'ICC_level3_pct': ICC_level3,
                'log_lik_reml': estimates['log_lik_reml'],
                'log_lik_ml': estimates['log_lik_ml'],
                'AIC': AIC,
                'BIC': BIC,
                'optimizer_result': optimizer_result
            }
            ANALYSIS_CONFIG['three_level_results'] = results_dict
            print("  ✓ Results saved to ANALYSIS_CONFIG['three_level_results']")


        except Exception as e:
            print(f"\n❌ AN ERROR OCCURRED:\n")
            print(f"  Type: {type(e).__name__}")
            print(f"  Message: {e}")
            print("\n  Traceback:")
            traceback.print_exc(file=sys.stdout)
            print("\n" + "="*70)
            print("ANALYSIS FAILED. See error message above.")
            print("Please check your data and configuration.")
            print("="*70)

# --- 5. DISPLAY WIDGETS ---

# Check if required data is present before displaying
try:
    if 'ANALYSIS_CONFIG' not in globals() or 'overall_results' not in ANALYSIS_CONFIG:
        print("="*70)
        print("⚠️  PREREQUISITE NOT MET")
        print("="*70)
        print("Please run Cell 6 (Overall Meta-Analysis) before running this cell.")
    else:
        # Check for auto-detection
        # If 'analysis_data' was saved in config use that, otherwise rely on global 'data_filtered'
        if 'analysis_data' in ANALYSIS_CONFIG:
            data_check = ANALYSIS_CONFIG['analysis_data']
        elif 'data_filtered' in globals():
            data_check = data_filtered.dropna(subset=[ANALYSIS_CONFIG['effect_col'], ANALYSIS_CONFIG['var_col']]).copy()
        else:
             raise ValueError("Data (data_filtered/analysis_data) not found.")

        k_obs_check = len(data_check)
        k_studies_check = data_check['id'].nunique()

        if k_obs_check == k_studies_check:
            print("="*70)
            print("✅ THREE-LEVEL ANALYSIS NOT REQUIRED")
            print("="*70)
            print("  Your dataset has only one effect size per study.")
            print("  The standard meta-analysis from Cell 6 is sufficient and correct.")
        else:
            print("="*70)
            print("THREE-LEVEL ANALYSIS INTERFACE READY")
            print("="*70)
            print(f"  ✓ Dependent effect sizes detected ({k_obs_check} effects from {k_studies_check} studies).")
            print("  ✓ This model is recommended to account for data dependency.")
            print("  Click the button below to run the analysis.")

            display(widgets.VBox([
                widgets.HTML("<hr style='margin: 15px 0;'>"),
                run_button,
                analysis_output
            ]))

except Exception as e:
    print(f"❌ An error occurred during initialization: {e}")
    print("Please ensure the notebook has been run in order.")

In [ ]:
#@title 📊 Step 2: Overall Meta-Analysis (V2)

# =============================================================================
# CELL: OVERALL META-ANALYSIS (DASHBOARD)
# Purpose: Calculate pooled effects (Standard & 3-Level) and display as a dashboard.
# Fix: Defined missing 'ci_lower_fixed' variables to prevent NameError.
# =============================================================================

import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import datetime
from scipy.stats import norm, chi2, t
from scipy.optimize import minimize

# --- 1. LAYOUT & WIDGETS ---
tab_main = widgets.Output()
tab_hetero = widgets.Output()
tab_compare = widgets.Output()
tab_settings = widgets.Output()

tabs = widgets.Tab(children=[tab_main, tab_hetero, tab_compare, tab_settings])
tabs.set_title(0, '📊 Primary Result')
tabs.set_title(1, '📉 Heterogeneity')
tabs.set_title(2, '⚖️ Model Comparison')
tabs.set_title(3, '⚙️ Settings')

# Settings Widgets
method_options = ['REML', 'DL', 'ML', 'PM', 'SJ'] if 'calculate_tau_squared' in globals() else ['DL']
tau_method_widget = widgets.Dropdown(options=method_options, value='REML' if 'REML' in method_options else 'DL', description='τ² Method:')
use_kh_widget = widgets.Checkbox(value=True, description='Knapp-Hartung Correction')

# --- 2. ENGINE: 3-LEVEL OPTIMIZER ---
def _run_three_level_reml(df, effect_col, var_col):
    """Core 3-Level Optimizer."""
    grouped = df.groupby('id')
    y_all = [g[effect_col].values for _, g in grouped]
    v_all = [g[var_col].values for _, g in grouped]
    N, M = len(df), len(y_all)

    def nll(params):
        tau2, sigma2 = params
        if tau2 < 0 or sigma2 < 0: return np.inf
        ll = 0; sum_S = 0; sum_Sy = 0; sum_ySy = 0
        for i in range(M):
            y, v = y_all[i], v_all[i]
            A_inv = 1.0 / (v + sigma2)
            sum_A_inv = np.sum(A_inv)
            denom = 1 + tau2 * sum_A_inv

            ll += np.sum(np.log(v + sigma2)) + np.log(denom)

            w_y = A_inv * y - (tau2 * A_inv * np.sum(A_inv * y)) / denom
            w_1 = A_inv - (tau2 * A_inv * sum_A_inv) / denom

            sum_S += np.sum(w_1)
            sum_Sy += np.sum(w_y)
            sum_ySy += np.dot(y, w_y)

        mu = sum_Sy / sum_S
        resid = sum_ySy - 2*mu*sum_Sy + mu**2 * sum_S
        return 0.5 * (ll + np.log(sum_S) + resid)

    # Optimize
    res = minimize(nll, [0.01, 0.01], bounds=[(1e-8, None)]*2, method='L-BFGS-B', options={'ftol':1e-9})
    if not res.success: res = minimize(nll, [0.1, 0.1], bounds=[(1e-8, None)]*2, method='Nelder-Mead')

    tau2, sigma2 = res.x

    # Re-calc stats at optimum
    sum_S = 0; sum_Sy = 0
    for i in range(M):
        y, v = y_all[i], v_all[i]
        A_inv = 1.0 / (v + sigma2)
        denom = 1 + tau2 * np.sum(A_inv)
        w_y = A_inv * y - (tau2 * A_inv * np.sum(A_inv * y)) / denom
        w_1 = A_inv - (tau2 * A_inv * np.sum(A_inv)) / denom
        sum_S += np.sum(w_1)
        sum_Sy += np.sum(w_y)

    mu = sum_Sy / sum_S
    se = np.sqrt(1.0 / sum_S)

    total_var = tau2 + sigma2
    icc_l3 = (tau2 / total_var * 100) if total_var > 0 else 0
    icc_l2 = (sigma2 / total_var * 100) if total_var > 0 else 0

    return {'mu': mu, 'se': se, 'tau2': tau2, 'sigma2': sigma2, 'icc_l3': icc_l3, 'icc_l2': icc_l2, 'n': N, 'm': M}

# --- 3. MAIN ANALYSIS ---
def run_analysis(change=None):
    # Clear Tabs
    for tab in [tab_main, tab_hetero, tab_compare, tab_settings]: tab.clear_output()

    # --- CONFIG CHECK ---
    if 'ANALYSIS_CONFIG' not in globals():
        print("❌ ERROR: Config not found. Run Step 1 first.")
        return

    eff_col = ANALYSIS_CONFIG.get('effect_col')
    var_col = ANALYSIS_CONFIG.get('var_col')
    es_config = ANALYSIS_CONFIG.get('es_config', {})

    # Load Data
    if 'analysis_data' in ANALYSIS_CONFIG:
        df_source = ANALYSIS_CONFIG['analysis_data']
    elif 'data_filtered' in globals():
        df_source = data_filtered
    else:
        print("❌ ERROR: No data found. Run Step 1 first.")
        return

    if eff_col not in df_source.columns or var_col not in df_source.columns:
        print(f"❌ ERROR: Columns '{eff_col}' or '{var_col}' missing.")
        return

    # Clean Data
    df = df_source.dropna(subset=[eff_col, var_col]).copy()
    df = df[df[var_col] > 0]

    if len(df) == 0:
        print("❌ ERROR: No valid data points remaining.")
        return

    # --- A. STANDARD ANALYSIS ---
    w_fe = 1 / df[var_col]
    mu_fe = np.average(df[eff_col], weights=w_fe)
    se_fe = np.sqrt(1 / np.sum(w_fe))

    # --- FIX: Calculate Fixed-Effect CIs ---
    ci_lower_fixed = mu_fe - 1.96 * se_fe
    ci_upper_fixed = mu_fe + 1.96 * se_fe

    # Heterogeneity
    Q = np.sum(w_fe * (df[eff_col] - mu_fe)**2)
    df_Q = len(df) - 1
    p_Q = 1 - chi2.cdf(Q, df_Q)
    I2 = max(0, (Q - df_Q) / Q * 100) if Q > 0 else 0

    # Random Effect
    method = tau_method_widget.value
    if 'calculate_tau_squared' in globals() and method != 'DL':
        tau2_re, _ = calculate_tau_squared(df, eff_col, var_col, method=method)
    else:
        C = np.sum(w_fe) - np.sum(w_fe**2)/np.sum(w_fe)
        tau2_re = max(0, (Q - df_Q) / C) if C > 0 else 0

    w_re = 1 / (df[var_col] + tau2_re)
    mu_re = np.average(df[eff_col], weights=w_re)
    se_re = np.sqrt(1 / np.sum(w_re))

    # Knapp-Hartung
    use_kh = use_kh_widget.value
    if use_kh and len(df) > 1:
        q_re = np.sum(w_re * (df[eff_col] - mu_re)**2)
        se_re = se_re * np.sqrt(max(1, q_re / df_Q))
        dist = t(df_Q)
    else:
        dist = norm

    ci_lo_re = mu_re - dist.ppf(0.975) * se_re
    ci_hi_re = mu_re + dist.ppf(0.975) * se_re
    p_re = 2 * (1 - dist.cdf(abs(mu_re / se_re)))

    # --- B. 3-LEVEL ANALYSIS ---
    k_obs, k_studies = len(df), df['id'].nunique()
    res_3l = None
    if k_obs > k_studies:
        try:
            res_3l = _run_three_level_reml(df, eff_col, var_col)
            mu_3l, se_3l = res_3l['mu'], res_3l['se']
            ci_lo_3l, ci_hi_3l = mu_3l - 1.96*se_3l, mu_3l + 1.96*se_3l
            p_3l = 2*(1-norm.cdf(abs(mu_3l/se_3l)))
        except: pass

    # --- C. SAVE RESULTS ---
    ANALYSIS_CONFIG['overall_results'] = {
        'pooled_effect_fixed': mu_fe, 'pooled_SE_fixed': se_fe,
        'ci_lower_fixed': ci_lower_fixed, 'ci_upper_fixed': ci_upper_fixed,
        'pooled_effect_random': mu_re, 'pooled_SE_random_reported': se_re,
        'ci_lower_random_reported': ci_lo_re, 'ci_upper_random_reported': ci_hi_re,
        'p_value_random_reported': p_re,
        'Qt': Q, 'I_squared': I2, 'tau_squared': tau2_re, 'k': k_obs, 'k_papers': k_studies,
        'knapp_hartung': {'used': use_kh}
    }
    if res_3l:
        ANALYSIS_CONFIG['three_level_results'] = {
            'status': 'completed', 'pooled_effect': mu_3l, 'se': se_3l,
            'ci_lower': ci_lo_3l, 'ci_upper': ci_hi_3l,
            'tau_squared': res_3l['tau2'], 'sigma_squared': res_3l['sigma2']
        }

    # --- D. RENDER TABS ---
    with tab_main:
        if res_3l:
            mu_p, ci_lo_p, ci_hi_p, p_p = mu_3l, ci_lo_3l, ci_hi_3l, p_3l
            model_label, note = "3-Level REML (Robust)", "Adjusted for dependency."
        else:
            mu_p, ci_lo_p, ci_hi_p, p_p = mu_re, ci_lo_re, ci_hi_re, p_re
            model_label, note = "Random-Effects", "Standard model."

        color = "#28a745" if p_p < 0.05 else "#6c757d"
        sig_str = "Significant" if p_p < 0.05 else "Not Significant"

        display(HTML(f"""
        <div style="padding:15px; border-left:5px solid {color}; background:#f9f9f9;">
            <h3 style="margin:0; color:{color};">{sig_str} Effect</h3>
            <p style="font-size:16px; margin:5px 0;">
                <b>{es_config.get('effect_label', 'Effect')}:</b> {mu_p:.4f}
                <span style="color:#666;">[95% CI: {ci_lo_p:.4f}, {ci_hi_p:.4f}]</span>
            </p>
            <p style="font-size:12px; color:#666; margin:0;">
                <b>Model:</b> {model_label} | <b>p-value:</b> {p_p:.4g} | {note}
            </p>
        </div>
        """))

        if es_config.get('has_fold_change'):
            rr = np.exp(mu_p)
            direction = "Increase" if mu_p > 0 else "Decrease"
            display(HTML(f"""<div style="margin-top:10px; padding:10px; border:1px solid #eee;">
            <b>🧬 Interpretation:</b> {abs(rr-1)*100:.1f}% {direction} relative to control.</div>"""))

    with tab_hetero:
        i2_color = "green" if I2 < 25 else "orange" if I2 < 75 else "red"
        display(HTML(f"""
        <div style="display:flex; gap:20px;">
            <div style="flex:1; padding:10px; border:1px solid #ddd; text-align:center;">
                <div style="font-size:24px; font-weight:bold; color:{i2_color};">{I2:.1f}%</div>
                <div style="font-size:12px; color:#666;">I² (Total Heterogeneity)</div>
            </div>
            <div style="flex:1; padding:10px; border:1px solid #ddd; text-align:center;">
                <div style="font-size:24px; font-weight:bold;">{tau2_re:.4f}</div>
                <div style="font-size:12px; color:#666;">τ² (Between-Study Variance)</div>
            </div>
        </div>
        """))
        if res_3l:
            display(HTML(f"<br><b>🔍 3-Level Variance:</b><br>Between-Study: {res_3l['icc_l3']:.1f}%<br>Within-Study: {res_3l['icc_l2']:.1f}%"))

    with tab_compare:
        rows = [["Fixed-Effect", mu_fe, f"[{ci_lower_fixed:.3f}, {ci_upper_fixed:.3f}]"],
                ["Random-Effects", mu_re, f"[{ci_lo_re:.3f}, {ci_hi_re:.3f}]"]]
        if res_3l: rows.append(["3-Level REML", mu_3l, f"[{ci_lo_3l:.3f}, {ci_hi_3l:.3f}]"])

        html = "<table style='width:100%;'><tr><th style='text-align:left'>Model</th><th style='text-align:right'>Effect</th><th style='text-align:right'>95% CI</th></tr>"
        for r in rows: html += f"<tr><td>{r[0]}</td><td style='text-align:right; font-weight:bold'>{r[1]:.4f}</td><td style='text-align:right'>{r[2]}</td></tr>"
        html += "</table>"
        display(HTML(html))

    with tab_settings:
        display(widgets.VBox([tau_method_widget, use_kh_widget, widgets.HTML(f"<hr><b>Stats:</b> k={k_obs}, Studies={k_studies}")]))

# --- 4. RUN ---
tau_method_widget.observe(run_analysis, names='value')
use_kh_widget.observe(run_analysis, names='value')
run_analysis()
display(tabs)

In [ ]:
#@title ⚙️ Step 3a: Subgroup Analysis - Configuration

# =============================================================================
# SUBGROUP ANALYSIS CONFIGURATION
#
# Purpose: Configure moderator variables for subgroup analysis
#
# Instructions:
#   1. Select your moderator variable from the dropdown
#   2. Adjust minimum thresholds if needed
#   3. Click 'Run Subgroup Analysis' to save configuration
#   4. Review the subgroup distribution and statistics
#   5. Proceed to the next cell to run the analysis
#
# Output: ANALYSIS_CONFIG['subgroup_config']
# =============================================================================


# =============================================================================
# CELL 7: SUBGROUP ANALYSIS CONFIGURATION
# Purpose: Configure moderator variables and settings for subgroup analysis
# Dependencies: Cell 6 (overall_results, analysis_data)
# Outputs: ANALYSIS_CONFIG['subgroup_config'], interactive widgets
# =============================================================================

print("\n" + "="*70)
print("SUBGROUP ANALYSIS CONFIGURATION")
print("="*70)
print(f"Timestamp: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# --- STEP 1: CHECK PREREQUISITES ---
print("\n" + "="*70)
print("STEP 1: VERIFYING PREREQUISITES")
print("="*70)

try:
    effect_col = ANALYSIS_CONFIG['effect_col']
    var_col = ANALYSIS_CONFIG['var_col']
    se_col = ANALYSIS_CONFIG['se_col']
    es_config = ANALYSIS_CONFIG['es_config']
    overall_results = ANALYSIS_CONFIG['overall_results']
    Qt_overall = overall_results['Qt']
    I_squared_overall = overall_results['I_squared']

    print(f"✓ Overall analysis results loaded successfully")
    print(f"  Effect size: {es_config['effect_label']} ({es_config['effect_label_short']})")
    print(f"  Effect column: {effect_col}")
    print(f"  Overall Q statistic: {Qt_overall:.4f}")
    print(f"  Overall I²: {I_squared_overall:.2f}%")

except KeyError as e:
    print(f"❌ ERROR: Overall analysis results not found - {e}")
    print("\nTroubleshooting:")
    print("  1. Ensure Cell 6 (overall meta-analysis) was run successfully")
    print("  2. Check that ANALYSIS_CONFIG['overall_results'] exists")
    print("  3. Verify that analysis_data DataFrame is available")
    raise

# Check if analysis_data exists (handle both old and new cell formats)
if 'analysis_data' not in globals():
    # Check if it's stored in ANALYSIS_CONFIG instead (V2 format)
    if 'analysis_data' in ANALYSIS_CONFIG:
        print(f"✓ Loading analysis_data from ANALYSIS_CONFIG")
        analysis_data = ANALYSIS_CONFIG['analysis_data']
    else:
        print(f"\n❌ ERROR: analysis_data not found")
        print(f"   Please ensure Step 2 (Overall Meta-Analysis) was executed successfully")
        raise NameError("analysis_data not defined")

# Dataset information
k_total = len(analysis_data)
k_papers = analysis_data['id'].nunique()

print(f"\n📊 Dataset Summary:")
print(f"  • Total observations: {k_total}")
print(f"  • Unique papers: {k_papers}")
print(f"  • Avg obs per paper: {k_total/k_papers:.2f}")

# Check if subgroup analysis is appropriate
if k_total < 10:
    print(f"\n⚠️  WARNING: Limited data for subgroup analysis")
    print(f"   With only {k_total} observations, subgroup analyses may be underpowered")
    print(f"   Results should be interpreted with caution")
elif k_total < 20:
    print(f"\n⚠️  CAUTION: Moderate data for subgroup analysis")
    print(f"   With {k_total} observations, some subgroup combinations may have few studies")
else:
    print(f"\n✓ Adequate data for subgroup analysis ({k_total} observations)")

# --- STEP 2: IDENTIFY AVAILABLE MODERATOR COLUMNS ---
print("\n" + "="*70)
print("STEP 2: IDENTIFYING MODERATOR VARIABLES")
print("="*70)

print(f"\n🔍 Scanning dataset for potential moderator variables...")

# Exclude technical columns
excluded_cols = [
    'xe', 'sde', 'ne', 'xc', 'sdc', 'nc', 'id',
    'sde_imputed', 'sdc_imputed', 'cv_e', 'cv_c',
    'sde_was_imputed', 'sdc_was_imputed',
    effect_col, var_col, se_col, 'w_fixed', 'w_random',
    'ci_width'
]

# Add effect-size-specific columns to exclude
if es_config['has_fold_change']:
    if 'Response_Ratio' in analysis_data.columns:
        excluded_cols.extend(['Response_Ratio', 'RR_CI_lower', 'RR_CI_upper',
                             'fold_change', 'Percent_Change'])
    if 'Odds_Ratio' in analysis_data.columns:
        excluded_cols.extend(['Odds_Ratio', 'OR_CI_lower', 'OR_CI_upper'])

if 'hedges_g' in effect_col or 'cohen_d' in effect_col:
    excluded_cols.extend(['df', 'sp', 'sp_squared', 'cohen_d', 'hedges_j'])

# Add CI columns
ci_cols = [c for c in analysis_data.columns if 'CI_' in c or 'ci_' in c]
excluded_cols.extend(ci_cols)

# Get categorical columns (potential moderators)
available_moderators = [
    col for col in analysis_data.columns
    if col not in excluded_cols
    and analysis_data[col].dtype == 'object'
    and analysis_data[col].notna().sum() > 0  # Has some non-missing values
]

print(f"\n📋 Available Moderator Variables: {len(available_moderators)}")

if not available_moderators:
    print(f"\n❌ ERROR: No categorical moderator columns found in dataset")
    print(f"\nAvailable columns in dataset:")
    for col in analysis_data.columns:
        dtype = analysis_data[col].dtype
        n_unique = analysis_data[col].nunique()
        print(f"  • {col}: {dtype} ({n_unique} unique values)")

    print(f"\n💡 Troubleshooting:")
    print(f"  1. Ensure your dataset contains categorical variables for grouping")
    print(f"  2. Check that moderator columns are not all numeric")
    print(f"  3. Verify column names match expected moderator variables")
    raise ValueError("No moderators available for subgroup analysis")

# Analyze moderator characteristics
moderator_info = []
for col in available_moderators:
    n_categories = analysis_data[col].nunique()
    n_missing = analysis_data[col].isna().sum()
    pct_missing = (n_missing / len(analysis_data)) * 100
    categories = sorted(analysis_data[col].dropna().unique())

    # Calculate distribution statistics
    value_counts = analysis_data[col].value_counts()
    min_count = value_counts.min()
    max_count = value_counts.max()

    moderator_info.append({
        'variable': col,
        'n_categories': n_categories,
        'n_missing': n_missing,
        'pct_missing': pct_missing,
        'categories': categories,
        'min_count': min_count,
        'max_count': max_count,
        'value_counts': value_counts
    })

# Display moderator information
print(f"\n{'Variable':<25} {'Categories':>12} {'Missing':>10} {'Range':>15}")
print(f"{'-'*25} {'-'*12} {'-'*10} {'-'*15}")

for info in moderator_info:
    print(f"{info['variable']:<25} {info['n_categories']:>12} "
          f"{info['n_missing']:>10} {info['min_count']:>6}-{info['max_count']:<6}")

print(f"\n📊 Detailed Moderator Information:")
for info in moderator_info:
    print(f"\n  🔹 {info['variable']}")
    print(f"     Categories: {info['n_categories']}")
    print(f"     Missing: {info['n_missing']} ({info['pct_missing']:.1f}%)")
    print(f"     Values: {', '.join(str(c) for c in info['categories'][:5])}"
          f"{' ...' if len(info['categories']) > 5 else ''}")

    # Show distribution
    print(f"     Distribution:")
    for category, count in info['value_counts'].items():
        papers = analysis_data[analysis_data[info['variable']] == category]['id'].nunique()
        pct = (count / len(analysis_data)) * 100
        print(f"       • {category}: {count} obs ({pct:.1f}%), {papers} papers")

    # Warning for imbalanced categories
    if info['min_count'] < 3:
        print(f"     ⚠️  Warning: Some categories have very few observations")

# --- STEP 3: CREATE ANALYSIS TYPE SELECTION ---
print("\n" + "="*70)
print("STEP 3: CREATING INTERACTIVE CONFIGURATION")
print("="*70)

print(f"\n🎨 Building interactive widgets...")

# Analysis type selection
analysis_type_widget = widgets.RadioButtons(
    options=[
        ('Single-Factor Subgroup Analysis', 'single'),
        ('Two-Factor Subgroup Analysis (Interaction)', 'two_way')
    ],
    value='single',
    description='Analysis Type:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='600px')
)

# Info panel for analysis types
analysis_type_info = {
    'single': f"""
    <div style='background-color: #e7f3ff; padding: 15px; border-radius: 8px; margin-top: 10px; border-left: 4px solid #0066cc;'>
        <h4 style='margin-top: 0; color: #0066cc;'>📊 Single-Factor Subgroup Analysis</h4>

        <p><b>Purpose:</b> Test if effect size varies across levels of <b>ONE</b> moderator variable</p>

        <p><b>Example Question:</b></p>
        <ul>
            <li>Does the treatment effect differ between Blood Feeders vs. Herbivores?</li>
            <li>Is the effect larger for JH Addition vs. JH Inhibition?</li>
        </ul>

        <p><b>Statistical Output:</b></p>
        <ul>
            <li>Pooled effect size for each subgroup (with 95% CI)</li>
            <li>Test for differences between subgroups (Q<sub>between</sub> test)</li>
            <li>Heterogeneity within each subgroup (Q<sub>within</sub>, I²)</li>
            <li>Proportion of heterogeneity explained by moderator (R²)</li>
        </ul>

        <p><b>Best For:</b></p>
        <ul>
            <li>Exploring one main source of variation</li>
            <li>When you have a primary moderator hypothesis</li>
            <li>Datasets with 10+ observations per subgroup</li>
        </ul>

        <p><b>Current Dataset:</b> {k_total} total observations</p>
    </div>
    """,
    'two_way': f"""
    <div style='background-color: #fff3cd; padding: 15px; border-radius: 8px; margin-top: 10px; border-left: 4px solid #ff9800;'>
        <h4 style='margin-top: 0; color: #856404;'>📊 Two-Factor Subgroup Analysis (Interaction)</h4>

        <p><b>Purpose:</b> Test if effect size varies across combinations of <b>TWO</b> moderator variables</p>

        <p><b>Example Question:</b></p>
        <ul>
            <li>Is the effect of treatment type (JH Addition vs. Inhibition) different for Blood Feeders vs. Herbivores?</li>
            <li>Does the combination of diet type and treatment method influence effect size?</li>
        </ul>

        <p><b>Statistical Output:</b></p>
        <ul>
            <li>Pooled effect for each combination (e.g., Blood Feeders × JH Addition)</li>
            <li>Test for overall differences across all combinations</li>
            <li>Main effect of each factor</li>
            <li>Interaction test (does Factor 1 effect depend on Factor 2?)</li>
        </ul>

        <p><b>Best For:</b></p>
        <ul>
            <li>Testing interaction effects between two variables</li>
            <li>When effect of one moderator may depend on another</li>
            <li>Datasets with sufficient observations per combination</li>
        </ul>

        <p><b>⚠️ Requirements:</b></p>
        <ul>
            <li>Minimum 3-5 studies per combination cell</li>
            <li>Ideally 20+ total observations</li>
            <li>Balanced or near-balanced design preferred</li>
        </ul>

        <p><b>Current Dataset:</b> {k_total} total observations → check distribution carefully!</p>
    </div>
    """
}

analysis_type_output = widgets.Output()

def update_analysis_type_info(change):
    """Update info panel when analysis type changes"""
    with analysis_type_output:
        clear_output()
        display(HTML(analysis_type_info[change['new']]))

        # Update visibility of second moderator selector
        if change['new'] == 'single':
            moderator2_container.layout.visibility = 'hidden'
            moderator2_container.layout.display = 'none'
        else:
            moderator2_container.layout.visibility = 'visible'
            moderator2_container.layout.display = 'block'

analysis_type_widget.observe(update_analysis_type_info, names='value')

# Initialize with default
with analysis_type_output:
    display(HTML(analysis_type_info['single']))

# --- STEP 4: CREATE MODERATOR SELECTION WIDGETS ---
print(f"  ✓ Analysis type selector created")

moderator1_label = widgets.HTML(
    "<h4 style='color: #2E86AB; margin-bottom: 5px;'>🔍 Select Moderator Variable(s)</h4>"
    "<p style='margin-top: 0; color: #666;'><i>Choose categorical variables to explore sources of heterogeneity</i></p>"
)

moderator1_widget = widgets.Dropdown(
    options=available_moderators,
    value=available_moderators[0],
    description='Moderator 1:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='600px')
)

# Moderator 2 (only for two-way analysis)
moderator2_widget = widgets.Dropdown(
    options=['None'] + available_moderators,
    value='None',
    description='Moderator 2:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='600px')
)

moderator2_container = widgets.VBox([moderator2_widget])
moderator2_container.layout.visibility = 'hidden'
moderator2_container.layout.display = 'none'

print(f"  ✓ Moderator selectors created")

# Preview of selected moderator(s)
preview_output = widgets.Output()

def update_moderator_preview(change=None):
    """Show preview of selected moderator(s)"""
    with preview_output:
        clear_output()

        mod1 = moderator1_widget.value
        mod2 = moderator2_widget.value if analysis_type_widget.value == 'two_way' else None

        print("\n" + "="*70)
        print("MODERATOR SELECTION PREVIEW")
        print("="*70)

        # Moderator 1 info
        print(f"\n📊 Moderator 1: {mod1}")
        mod1_counts = analysis_data[mod1].value_counts().sort_index()

        print(f"\n  Distribution:")
        print(f"  {'Category':<30} {'Observations':>15} {'Papers':>10} {'Percent':>10}")
        print(f"  {'-'*30} {'-'*15} {'-'*10} {'-'*10}")

        for category, count in mod1_counts.items():
            papers = analysis_data[analysis_data[mod1] == category]['id'].nunique()
            pct = (count / len(analysis_data)) * 100
            print(f"  {str(category):<30} {count:>15} {papers:>10} {pct:>9.1f}%")

        print(f"  {'-'*30} {'-'*15} {'-'*10} {'-'*10}")
        print(f"  {'TOTAL':<30} {len(analysis_data):>15} {analysis_data['id'].nunique():>10} {'100.0':>9}%")

        # Check for adequate sample sizes
        min_group = mod1_counts.min()
        if min_group < 5:
            print(f"\n  ⚠️  WARNING: Smallest group has only {min_group} observations")
            print(f"     Consider raising minimum thresholds or combining categories")
        else:
            print(f"\n  ✓ All groups have ≥ 5 observations")

        # Moderator 2 info (if two-way)
        if mod2 and mod2 != 'None':
            print(f"\n{'─'*70}")
            print(f"📊 Moderator 2: {mod2}")
            mod2_counts = analysis_data[mod2].value_counts().sort_index()

            print(f"\n  Distribution:")
            print(f"  {'Category':<30} {'Observations':>15} {'Papers':>10} {'Percent':>10}")
            print(f"  {'-'*30} {'-'*15} {'-'*10} {'-'*10}")

            for category, count in mod2_counts.items():
                papers = analysis_data[analysis_data[mod2] == category]['id'].nunique()
                pct = (count / len(analysis_data)) * 100
                print(f"  {str(category):<30} {count:>15} {papers:>10} {pct:>9.1f}%")

            # Show combination matrix
            print(f"\n{'─'*70}")
            print(f"📊 Combination Matrix: {mod1} × {mod2}")
            print(f"\n  Number of observations in each combination:\n")

            crosstab = pd.crosstab(
                analysis_data[mod1],
                analysis_data[mod2],
                margins=True,
                margins_name='Total'
            )
            print(crosstab.to_string())

            # Detailed cell analysis
            print(f"\n  📋 Cell-by-Cell Analysis:")
            for cat1 in mod1_counts.index:
                for cat2 in mod2_counts.index:
                    cell_data = analysis_data[(analysis_data[mod1] == cat1) & (analysis_data[mod2] == cat2)]
                    n_obs = len(cell_data)
                    n_papers = cell_data['id'].nunique()

                    if n_obs > 0:
                        status = "✓" if n_obs >= 5 else "⚠️"
                        print(f"    {status} {cat1} × {cat2}: {n_obs} obs, {n_papers} papers")

            # Warnings for small cells
            min_cell = crosstab.iloc[:-1, :-1].min().min()
            if min_cell == 0:
                print(f"\n  🔴 ERROR: Some combinations have ZERO observations!")
                print(f"     Two-way analysis not possible with empty cells")
                print(f"     Recommendation: Use single-factor analysis")
            elif min_cell < 3:
                print(f"\n  ⚠️  WARNING: Some combinations have very few observations (min = {min_cell})")
                print(f"     Recommendations:")
                print(f"       1. Increase minimum thresholds")
                print(f"       2. Consider combining categories")
                print(f"       3. Use single-factor analysis instead")
            elif min_cell < 5:
                print(f"\n  ⚠️  CAUTION: Some combinations have limited observations (min = {min_cell})")
                print(f"     Results may be unstable for small groups")
            else:
                print(f"\n  ✓ All combinations have ≥ 5 observations")

# Attach observers
moderator1_widget.observe(update_moderator_preview, names='value')
moderator2_widget.observe(update_moderator_preview, names='value')
analysis_type_widget.observe(lambda change: update_moderator_preview(), names='value')

print(f"  ✓ Preview function configured")

# Initialize preview
update_moderator_preview()

# Continue to Part 2...
# --- STEP 5: MINIMUM THRESHOLDS ---
print("\n" + "="*70)
print("STEP 4: QUALITY THRESHOLD CONFIGURATION")
print("="*70)

thresholds_label = widgets.HTML(
    "<h4 style='color: #2E86AB; margin-bottom: 5px;'>⚙️ Quality Thresholds</h4>"
    "<p style='margin-top: 0; color: #666;'><i>Subgroups not meeting these criteria will be excluded from analysis</i></p>"
)

thresholds_desc = widgets.HTML("""
    <div style='background-color: #f8f9fa; padding: 12px; border-radius: 5px; margin-bottom: 10px;'>
        <p style='margin: 0;'><b>Purpose:</b> Ensure each subgroup has sufficient data for reliable estimation</p>
        <ul style='margin: 5px 0;'>
            <li><b>Min Papers:</b> Accounts for multiple observations from same study</li>
            <li><b>Min Observations:</b> Total data points needed for stable estimates</li>
        </ul>
        <p style='margin: 0;'><b>Recommendation:</b> Higher thresholds = more reliable but fewer subgroups</p>
    </div>
""")

min_papers_subgroup = widgets.IntSlider(
    value=3,
    min=1,
    max=10,
    step=1,
    description='Min Papers/Group:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='550px')
)

min_obs_subgroup = widgets.IntSlider(
    value=5,
    min=2,
    max=20,
    step=1,
    description='Min Observations/Group:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='550px')
)

# Dynamic threshold feedback
threshold_feedback = widgets.Output()

def update_threshold_feedback(change=None):
    """Show impact of current thresholds"""
    with threshold_feedback:
        clear_output()

        min_papers = min_papers_subgroup.value
        min_obs = min_obs_subgroup.value
        mod1 = moderator1_widget.value

        print("\n📊 Impact Analysis:")
        print(f"  Current thresholds: ≥{min_papers} papers AND ≥{min_obs} observations")
        print(f"\n  Checking subgroups in '{mod1}'...")

        # Check which subgroups meet criteria
        groups_meeting_criteria = []
        groups_failing_criteria = []

        for category in analysis_data[mod1].dropna().unique():
            group_data = analysis_data[analysis_data[mod1] == category]
            n_papers = group_data['id'].nunique()
            n_obs = len(group_data)

            if n_papers >= min_papers and n_obs >= min_obs:
                groups_meeting_criteria.append((category, n_obs, n_papers))
            else:
                reason = []
                if n_papers < min_papers:
                    reason.append(f"papers: {n_papers}<{min_papers}")
                if n_obs < min_obs:
                    reason.append(f"obs: {n_obs}<{min_obs}")
                groups_failing_criteria.append((category, n_obs, n_papers, ", ".join(reason)))

        print(f"\n  ✓ Groups meeting criteria: {len(groups_meeting_criteria)}")
        for cat, obs, papers in groups_meeting_criteria:
            print(f"    • {cat}: {obs} obs, {papers} papers")

        if groups_failing_criteria:
            print(f"\n  ✗ Groups excluded: {len(groups_failing_criteria)}")
            for cat, obs, papers, reason in groups_failing_criteria:
                print(f"    • {cat}: {obs} obs, {papers} papers (excluded: {reason})")

        # Overall assessment
        if len(groups_meeting_criteria) < 2:
            print(f"\n  🔴 ERROR: Need at least 2 groups for subgroup analysis!")
            print(f"     Current thresholds too strict - please lower them")
        elif len(groups_meeting_criteria) == 2:
            print(f"\n  ⚠️  WARNING: Only 2 groups available")
            print(f"     Analysis will be limited to comparing these two groups")
        else:
            print(f"\n  ✓ {len(groups_meeting_criteria)} groups available for analysis")

        # Calculate total retained data
        total_retained_obs = sum(obs for _, obs, _ in groups_meeting_criteria)
        retention_rate = (total_retained_obs / len(analysis_data)) * 100

        print(f"\n  📈 Data Retention:")
        print(f"     Observations retained: {total_retained_obs}/{len(analysis_data)} ({retention_rate:.1f}%)")

        if retention_rate < 50:
            print(f"     ⚠️  Less than 50% of data retained - consider lowering thresholds")
        elif retention_rate < 75:
            print(f"     ⚠️  Moderate data loss - verify this is acceptable")
        else:
            print(f"     ✓ Good data retention")

# Attach observers to thresholds
min_papers_subgroup.observe(update_threshold_feedback, names='value')
min_obs_subgroup.observe(update_threshold_feedback, names='value')
moderator1_widget.observe(update_threshold_feedback, names='value')

print(f"  ✓ Threshold widgets created")

# Initialize threshold feedback
update_threshold_feedback()

# --- STEP 6: RUN ANALYSIS BUTTON ---
print("\n" + "="*70)
print("STEP 5: CREATING RUN BUTTON")
print("="*70)

run_button = widgets.Button(
    description='▶ Run Subgroup Analysis',
    button_style='success',
    layout=widgets.Layout(width='450px', height='50px'),
    style={'font_weight': 'bold', 'font_size': '14px'}
)

run_output = widgets.Output()

def on_run_button_clicked(b):
    """Save configuration and prepare for analysis"""
    with run_output:
        clear_output()

        print("\n" + "="*70)
        print("VALIDATING CONFIGURATION")
        print("="*70)

        # Get selections
        analysis_type = analysis_type_widget.value
        moderator1 = moderator1_widget.value
        moderator2 = moderator2_widget.value if analysis_type == 'two_way' and moderator2_widget.value != 'None' else None
        min_papers = min_papers_subgroup.value
        min_obs = min_obs_subgroup.value

        # --- Validation Checks ---
        validation_errors = []
        validation_warnings = []

        # Check 1: Two-way analysis requires moderator 2
        if analysis_type == 'two_way' and not moderator2:
            validation_errors.append("Two-way analysis requires selecting Moderator 2")

        # Check 2: Moderators cannot be the same
        if moderator1 == moderator2:
            validation_errors.append("Moderator 1 and Moderator 2 cannot be the same variable")

        # Check 3: At least 2 groups must meet criteria
        groups_meeting_criteria = 0
        valid_groups_list = []

        if analysis_type == 'single':
            for category in analysis_data[moderator1].dropna().unique():
                group_data = analysis_data[analysis_data[moderator1] == category]
                n_papers = group_data['id'].nunique()
                n_obs = len(group_data)
                if n_papers >= min_papers and n_obs >= min_obs:
                    groups_meeting_criteria += 1
                    valid_groups_list.append(category)
        else:
            # Two-way analysis - check each combination
            for cat1 in analysis_data[moderator1].dropna().unique():
                for cat2 in analysis_data[moderator2].dropna().unique():
                    cell_data = analysis_data[(analysis_data[moderator1] == cat1) &
                                             (analysis_data[moderator2] == cat2)]
                    n_papers = cell_data['id'].nunique()
                    n_obs = len(cell_data)
                    if n_papers >= min_papers and n_obs >= min_obs:
                        groups_meeting_criteria += 1
                        valid_groups_list.append((cat1, cat2))

        if groups_meeting_criteria < 2:
            validation_errors.append(f"Only {groups_meeting_criteria} group(s) meet criteria. Need at least 2 groups for subgroup analysis. Lower thresholds or choose different moderator.")

        # Check 4: For two-way, check for empty cells (WARNING, not ERROR)
        if analysis_type == 'two_way' and moderator2:
            crosstab = pd.crosstab(analysis_data[moderator1], analysis_data[moderator2])
            n_empty_cells = (crosstab == 0).sum().sum()
            total_cells = crosstab.shape[0] * crosstab.shape[1]

            if n_empty_cells > 0:
                validation_warnings.append(
                    f"{n_empty_cells}/{total_cells} combinations have zero observations. "
                    f"These empty cells will be automatically excluded from analysis. "
                    f"Proceeding with {groups_meeting_criteria} valid combinations."
                )

            # Check for very small cells
            min_cell = crosstab[crosstab > 0].min().min() if (crosstab > 0).any().any() else 0
            if min_cell > 0 and min_cell < 3:
                validation_warnings.append(
                    f"Some combinations have very few observations (minimum = {min_cell}). "
                    f"Results for these groups may be unstable."
                )

        # Check 5: Sufficient overall sample size
        if len(analysis_data) < 10:
            validation_warnings.append(
                f"Limited total sample size ({len(analysis_data)} observations). "
                f"Subgroup analysis may be underpowered."
            )

        # Display validation results
        if validation_errors:
            print("\n❌ VALIDATION FAILED")
            print("\nErrors that must be fixed:")
            for i, error in enumerate(validation_errors, 1):
                print(f"  {i}. {error}")
            print("\n⚠️  Please fix the errors above and try again")
            return

        if validation_warnings:
            print("\n⚠️  VALIDATION WARNINGS")
            print("\nWarnings (analysis will proceed, but be cautious):")
            for i, warning in enumerate(validation_warnings, 1):
                print(f"  {i}. {warning}")
            print("\n✓ Analysis can proceed - empty cells will be automatically excluded")

        # --- Configuration Summary ---
        print("\n" + "="*70)
        print("✓ VALIDATION PASSED - CONFIGURATION SAVED")
        print("="*70)

        print(f"\n📋 Subgroup Analysis Configuration:")
        print(f"  {'Parameter':<30} {'Value':<40}")
        print(f"  {'-'*30} {'-'*40}")
        print(f"  {'Analysis Type':<30} {analysis_type:<40}")
        print(f"  {'Primary Moderator':<30} {moderator1:<40}")

        if moderator2:
            print(f"  {'Secondary Moderator':<30} {moderator2:<40}")

        print(f"  {'Min Papers per Group':<30} {min_papers:<40}")
        print(f"  {'Min Observations per Group':<30} {min_obs:<40}")
        print(f"  {'Valid Groups/Combinations':<30} {groups_meeting_criteria:<40}")

        # Calculate expected data retention
        if analysis_type == 'single':
            retained_data = analysis_data[analysis_data[moderator1].isin(valid_groups_list)].copy()
        else:
            retained_data = analysis_data[
                analysis_data.apply(
                    lambda row: (row[moderator1], row[moderator2]) in valid_groups_list,
                    axis=1
                )
            ].copy()

        retention_pct = (len(retained_data) / len(analysis_data)) * 100
        print(f"  {'Data Retained':<30} {len(retained_data)}/{len(analysis_data)} ({retention_pct:.1f}%)")

        # Show which groups will be included
        if analysis_type == 'two_way' and n_empty_cells > 0:
            print(f"\n📊 Valid Combinations to be Analyzed:")
            for i, (cat1, cat2) in enumerate(valid_groups_list, 1):
                cell_data = analysis_data[(analysis_data[moderator1] == cat1) &
                                         (analysis_data[moderator2] == cat2)]
                print(f"  {i}. {cat1} × {cat2}: k={len(cell_data)}, papers={cell_data['id'].nunique()}")

        # Save to config
        ANALYSIS_CONFIG['subgroup_config'] = {
            'timestamp': datetime.datetime.now(),
            'analysis_type': analysis_type,
            'moderator1': moderator1,
            'moderator2': moderator2,
            'min_papers': min_papers,
            'min_obs': min_obs,
            'expected_groups': groups_meeting_criteria,
            'valid_groups_list': valid_groups_list,  # NEW: Store valid groups
            'data_retained': len(retained_data),
            'retention_pct': retention_pct,
            'has_empty_cells': n_empty_cells > 0 if analysis_type == 'two_way' else False,
            'n_empty_cells': n_empty_cells if analysis_type == 'two_way' else 0
        }

        # Save moderator information
        ANALYSIS_CONFIG['subgroup_config']['moderator1_info'] = {
            'name': moderator1,
            'n_categories': analysis_data[moderator1].nunique(),
            'categories': sorted(analysis_data[moderator1].dropna().unique().tolist())
        }

        if moderator2:
            ANALYSIS_CONFIG['subgroup_config']['moderator2_info'] = {
                'name': moderator2,
                'n_categories': analysis_data[moderator2].nunique(),
                'categories': sorted(analysis_data[moderator2].dropna().unique().tolist())
            }

        print(f"\n" + "="*70)
        print("✓ CONFIGURATION SAVED SUCCESSFULLY")
        print("="*70)

        print(f"\n📊 Configuration saved to: ANALYSIS_CONFIG['subgroup_config']")

        print(f"\n▶️  Next Steps:")
        print(f"  1. Review the configuration summary above")
        if validation_warnings:
            print(f"  2. Note the warnings - empty combinations will be excluded automatically")
            print(f"  3. Run the next cell to perform subgroup analysis")
        else:
            print(f"  2. Run the next cell to perform subgroup analysis")
        print(f"  4. Results will include:")
        if analysis_type == 'single':
            print(f"     • Pooled effects for each subgroup")
            print(f"     • Test for between-group differences (Q-test)")
            print(f"     • Within-group heterogeneity (I²)")
            print(f"     • Proportion of heterogeneity explained (R²)")
        else:
            print(f"     • Pooled effects for {groups_meeting_criteria} valid combinations")
            print(f"     • Main effects and interaction tests")
            print(f"     • Heterogeneity decomposition")
            if n_empty_cells > 0:
                print(f"     • Note: {n_empty_cells} empty combinations automatically excluded")

        print("\n" + "="*70)

run_button.on_click(on_run_button_clicked)

print(f"  ✓ Run button configured with validation")

# --- STEP 7: ASSEMBLE WIDGET LAYOUT ---
print("\n" + "="*70)
print("STEP 6: ASSEMBLING WIDGET INTERFACE")
print("="*70)

widget_layout = widgets.VBox([
    widgets.HTML("<hr style='margin: 20px 0; border: none; border-top: 2px solid #ddd;'>"),

    # Analysis Type Section
    widgets.HTML("<h3 style='color: #2E86AB;'>1️⃣ Select Analysis Type</h3>"),
    analysis_type_widget,
    analysis_type_output,

    widgets.HTML("<hr style='margin: 20px 0; border: none; border-top: 2px solid #ddd;'>"),

    # Moderator Selection Section
    widgets.HTML("<h3 style='color: #2E86AB;'>2️⃣ Select Moderator Variable(s)</h3>"),
    moderator1_label,
    moderator1_widget,
    moderator2_container,
    preview_output,

    widgets.HTML("<hr style='margin: 20px 0; border: none; border-top: 2px solid #ddd;'>"),

    # Threshold Section
    widgets.HTML("<h3 style='color: #2E86AB;'>3️⃣ Set Quality Thresholds</h3>"),
    thresholds_label,
    thresholds_desc,
    min_papers_subgroup,
    min_obs_subgroup,
    threshold_feedback,

    widgets.HTML("<hr style='margin: 20px 0; border: none; border-top: 2px solid #ddd;'>"),

    # Run Button Section
    widgets.HTML("<h3 style='color: #2E86AB;'>4️⃣ Run Analysis</h3>"),
    widgets.HTML("<p style='color: #666;'><i>Review your configuration above, then click the button to proceed</i></p>"),
    run_button,
    run_output
])

print(f"  ✓ Widget layout assembled")

# Display widgets
display(widget_layout)

print(f"\n✓ Interactive interface displayed")

# --- FINAL STATUS ---
print("\n" + "="*70)
print("✅ SUBGROUP ANALYSIS CONFIGURATION READY")
print("="*70)

print(f"\n📊 Configuration Summary:")
print(f"  • Available moderators: {len(available_moderators)}")
print(f"  • Total observations: {k_total}")
print(f"  • Unique papers: {k_papers}")
print(f"  • Overall heterogeneity (I²): {I_squared_overall:.2f}%")

if I_squared_overall > 50:
    print(f"\n  🔴 High heterogeneity detected - subgroup analysis highly recommended")
    print(f"     Explore which moderators explain the variation between studies")
elif I_squared_overall > 25:
    print(f"\n  🟡 Moderate heterogeneity - subgroup analysis may be informative")
else:
    print(f"\n  🟢 Low heterogeneity - subgroup analysis exploratory")

print(f"\n👆 INSTRUCTIONS:")
print(f"  1. Select analysis type (single-factor or two-factor)")
print(f"  2. Choose moderator variable(s) from the dropdown(s)")
print(f"  3. Review the distribution preview")
print(f"  4. Adjust quality thresholds if needed")
print(f"  5. Click '▶ Run Subgroup Analysis' button")
print(f"  6. After validation, proceed to next cell for results")

print(f"\n💡 Tips:")
print(f"  • Start with single-factor analysis to identify main moderators")
print(f"  • Use two-factor analysis to test interactions")
print(f"  • Higher thresholds = more reliable but fewer groups")
print(f"  • Check distribution preview for balance and sample sizes")

print("\n" + "="*70)

# Store configuration metadata
SUBGROUP_CONFIG_METADATA = {
    'timestamp': datetime.datetime.now(),
    'available_moderators': available_moderators,
    'moderator_info': moderator_info,
    'total_observations': k_total,
    'total_papers': k_papers,
    'overall_heterogeneity_I2': I_squared_overall,
    'interface_created': True
}

print(f"\n📊 Metadata saved to SUBGROUP_CONFIG_METADATA")

In [ ]:
#@title ⚙️ Step 3a: Subgroup Analysis - Configuration (V2)

# =============================================================================
# SUBGROUP ANALYSIS CONFIGURATION (DASHBOARD VERSION)
# Purpose: Configure moderator variables with organized tabbed interface
# Dependencies: Step 2 (overall_results, analysis_data)
# Outputs: ANALYSIS_CONFIG['subgroup_config']
# =============================================================================

import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import datetime

# --- 1. CREATE TAB LAYOUT ---
tab_config = widgets.Output()
tab_moderators = widgets.Output()
tab_thresholds = widgets.Output()
tab_details = widgets.Output()

tabs = widgets.Tab(children=[tab_config, tab_moderators, tab_thresholds, tab_details])
tabs.set_title(0, '📋 Configuration')
tabs.set_title(1, '📊 Moderator Preview')
tabs.set_title(2, '⚙️ Thresholds')
tabs.set_title(3, '📝 Details')

# --- 2. WIDGETS ---
analysis_type_widget = widgets.RadioButtons(
    options=[('Single-Factor Analysis', 'single'), ('Two-Factor Analysis (Interaction)', 'two_way')],
    value='single', description='', layout=widgets.Layout(width='auto')
)

moderator1_widget = None
moderator2_widget = None

min_papers_widget = widgets.IntSlider(
    value=3, min=1, max=10, step=1, description='Min Papers:',
    style={'description_width': '120px'}, layout=widgets.Layout(width='400px')
)

min_obs_widget = widgets.IntSlider(
    value=5, min=2, max=20, step=1, description='Min Observations:',
    style={'description_width': '120px'}, layout=widgets.Layout(width='400px')
)

run_button = widgets.Button(
    description='💾 Save Configuration & Proceed',
    button_style='success',
    layout=widgets.Layout(width='400px', height='50px'),
    style={'font_weight': 'bold'},
    tooltip='Click to save configuration for use in the next cell'
)

run_button_output = widgets.Output()
status_output = widgets.Output()

# --- 3. INITIALIZATION ---
def initialize_configuration():
    global moderator1_widget, moderator2_widget

    with tab_details:
        clear_output()
        print("="*70)
        print("INITIALIZATION & VALIDATION")
        print("="*70)
        print(f"Timestamp: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

        try:
            effect_col = ANALYSIS_CONFIG['effect_col']
            var_col = ANALYSIS_CONFIG['var_col']
            es_config = ANALYSIS_CONFIG['es_config']
            overall_results = ANALYSIS_CONFIG['overall_results']

            print("✓ Prerequisites Check:")
            print(f"  • Effect: {es_config['effect_label']} ({es_config['effect_label_short']})")
            print(f"  • Q: {overall_results['Qt']:.4f}, I²: {overall_results['I_squared']:.2f}%")
        except KeyError as e:
            print(f"❌ ERROR: {e}")
            print("  Please run Step 2 first")
            raise

        if 'analysis_data' not in globals():
            # Check if it's stored in ANALYSIS_CONFIG instead (V2 format)
            if 'analysis_data' in ANALYSIS_CONFIG:
                print("✓ Loading analysis_data from ANALYSIS_CONFIG")
                globals()['analysis_data'] = ANALYSIS_CONFIG['analysis_data']
            else:
                print("❌ ERROR: analysis_data not found")
                print("   Please ensure Step 2 (Overall Meta-Analysis) was executed")
                raise NameError("analysis_data not defined")

        k_total, k_papers = len(analysis_data), analysis_data['id'].nunique()
        print(f"\n✓ Dataset: {k_total} obs, {k_papers} papers, {k_total/k_papers:.2f} avg")

        if k_total < 10:
            print(f"⚠️  WARNING: Limited data ({k_total} obs)")
        elif k_total < 20:
            print(f"⚠️  CAUTION: Moderate data ({k_total} obs)")

        excluded = ['xe', 'sde', 'ne', 'xc', 'sdc', 'nc', 'id', 'sde_imputed', 'sdc_imputed',
                   'cv_e', 'cv_c', 'sde_was_imputed', 'sdc_was_imputed',
                   effect_col, var_col, ANALYSIS_CONFIG.get('se_col', ''), 'w_fixed', 'w_random', 'ci_width']

        if es_config.get('has_fold_change'):
            excluded.extend(['Response_Ratio', 'RR_CI_lower', 'RR_CI_upper', 'fold_change',
                           'Percent_Change', 'Odds_Ratio', 'OR_CI_lower', 'OR_CI_upper'])

        if 'hedges_g' in effect_col or 'cohen_d' in effect_col:
            excluded.extend(['df', 'sp', 'sp_squared', 'cohen_d', 'hedges_j'])

        excluded.extend([c for c in analysis_data.columns if 'CI_' in c or 'ci_' in c])

        available_moderators = [
            col for col in analysis_data.columns
            if col not in excluded and analysis_data[col].dtype == 'object'
            and analysis_data[col].notna().sum() > 0
        ]

        print(f"\n✓ Found {len(available_moderators)} moderators:")
        for mod in available_moderators:
            print(f"  • {mod}: {analysis_data[mod].nunique()} categories")

        if not available_moderators:
            print("❌ ERROR: No moderators found")
            raise ValueError("No moderators available")

        moderator1_widget = widgets.Dropdown(
            options=available_moderators, value=available_moderators[0],
            description='Moderator 1:', style={'description_width': '100px'},
            layout=widgets.Layout(width='450px')
        )

        moderator2_widget = widgets.Dropdown(
            options=['None'] + available_moderators, value='None',
            description='Moderator 2:', style={'description_width': '100px'},
            layout=widgets.Layout(width='450px')
        )

        analysis_type_widget.observe(update_all_tabs, names='value')
        moderator1_widget.observe(update_all_tabs, names='value')
        moderator2_widget.observe(update_all_tabs, names='value')
        min_papers_widget.observe(update_thresholds_tab, names='value')
        min_obs_widget.observe(update_thresholds_tab, names='value')
        run_button.on_click(save_configuration)

        print("\n✓ Initialized successfully")
        return available_moderators

# --- 4. TAB UPDATES ---
def update_config_tab(change=None):
    # Clear any previous save messages
    with status_output:
        clear_output()

    with tab_config:
        clear_output()
        analysis_type = analysis_type_widget.value

        help_html = """<div style='background:#e7f3ff; padding:12px; border-radius:6px; border-left:4px solid #0066cc; margin-bottom:15px;'>
            <b>📊 Single-Factor Subgroup Analysis</b><br>
            <span style='font-size:13px; color:#555;'>Test if effect varies across ONE moderator<br>
            <b>Best for:</b> Primary hypotheses, 10+ obs/group</span></div>""" if analysis_type == 'single' else """
            <div style='background:#fff3cd; padding:12px; border-radius:6px; border-left:4px solid #ff9800; margin-bottom:15px;'>
            <b>📊 Two-Factor Analysis (Interaction)</b><br>
            <span style='font-size:13px; color:#555;'>Test combinations of TWO moderators<br>
            <b>Requires:</b> 3-5 studies/combo, 20+ total obs</span></div>"""

        display(HTML("<h3 style='margin-top:0;'>Configure Subgroup Analysis</h3>"))
        display(HTML(help_html))
        display(HTML("<h4>1. Select Analysis Type</h4>"))
        display(analysis_type_widget)
        display(HTML("<h4 style='margin-top:20px;'>2. Select Moderator(s)</h4>"))
        display(moderator1_widget)

        if analysis_type == 'two_way':
            display(moderator2_widget)

        display(HTML("<h4 style='margin-top:20px;'>3. Set Quality Thresholds</h4>"))
        display(HTML("<p style='color:#666; font-size:13px;'>Adjust in <b>⚙️ Thresholds</b> tab</p>"))
        display(widgets.HBox([
            widgets.HTML(f"<div style='padding:8px; background:#f0f0f0; border-radius:4px; margin-right:10px;'>"
                        f"<b>Min Papers:</b> {min_papers_widget.value}</div>"),
            widgets.HTML(f"<div style='padding:8px; background:#f0f0f0; border-radius:4px;'>"
                        f"<b>Min Obs:</b> {min_obs_widget.value}</div>")
        ]))

        display(HTML("<h4 style='margin-top:20px;'>4. Save Configuration</h4>"))
        display(HTML("<p style='color:#666; font-size:13px; margin-top:0;'>"
                    "Click the button below to validate and save your configuration.<br>"
                    "The configuration will be stored in <code>ANALYSIS_CONFIG['subgroup_config']</code> "
                    "for use in the next cell.</p>"))
        with run_button_output:
          run_button_output.clear_output()
          display(run_button)

        display(run_button)
        display(status_output)

def update_moderators_tab(change=None):
    with tab_moderators:
        clear_output()

        if moderator1_widget is None:
            print("Initializing...")
            return

        mod1 = moderator1_widget.value
        analysis_type = analysis_type_widget.value
        mod2 = moderator2_widget.value if analysis_type == 'two_way' and moderator2_widget.value != 'None' else None

        display(HTML("<h3 style='margin-top:0;'>Moderator Variable Preview</h3>"))
        display(HTML(f"<h4>📊 {mod1}</h4>"))

        mod1_counts = analysis_data[mod1].value_counts().sort_index()

        table_html = """<table style='width:100%; border-collapse:collapse;'>
            <tr style='background:#f0f0f0; border-bottom:2px solid #ddd;'>
                <th style='text-align:left; padding:8px;'>Category</th>
                <th style='text-align:right; padding:8px;'>Observations</th>
                <th style='text-align:right; padding:8px;'>Papers</th>
                <th style='text-align:right; padding:8px;'>Percent</th></tr>"""

        for category, count in mod1_counts.items():
            papers = analysis_data[analysis_data[mod1] == category]['id'].nunique()
            pct = (count / len(analysis_data)) * 100
            row_color = '#fff' if count >= 5 else '#fff3cd'
            table_html += f"""<tr style='background:{row_color}; border-bottom:1px solid #eee;'>
                <td style='padding:6px;'>{category}</td>
                <td style='text-align:right; padding:6px;'><b>{count}</b></td>
                <td style='text-align:right; padding:6px;'>{papers}</td>
                <td style='text-align:right; padding:6px;'>{pct:.1f}%</td></tr>"""

        table_html += "</table>"
        display(HTML(table_html))

        min_group = mod1_counts.min()
        if min_group < 5:
            display(HTML(f"<div style='background:#fff3cd; padding:10px; margin-top:10px; border-radius:4px;'>"
                        f"⚠️ Smallest group: {min_group} obs - consider adjusting thresholds</div>"))
        else:
            display(HTML("<div style='background:#d4edda; padding:10px; margin-top:10px; border-radius:4px;'>"
                        "✓ All groups have ≥5 observations</div>"))

        if mod2:
            display(HTML(f"<h4 style='margin-top:25px;'>📊 {mod2}</h4>"))
            mod2_counts = analysis_data[mod2].value_counts().sort_index()

            table2_html = """<table style='width:100%; border-collapse:collapse;'>
                <tr style='background:#f0f0f0; border-bottom:2px solid #ddd;'>
                    <th style='text-align:left; padding:8px;'>Category</th>
                    <th style='text-align:right; padding:8px;'>Observations</th>
                    <th style='text-align:right; padding:8px;'>Papers</th>
                    <th style='text-align:right; padding:8px;'>Percent</th></tr>"""

            for category, count in mod2_counts.items():
                papers = analysis_data[analysis_data[mod2] == category]['id'].nunique()
                pct = (count / len(analysis_data)) * 100
                table2_html += f"""<tr style='border-bottom:1px solid #eee;'>
                    <td style='padding:6px;'>{category}</td>
                    <td style='text-align:right; padding:6px;'><b>{count}</b></td>
                    <td style='text-align:right; padding:6px;'>{papers}</td>
                    <td style='text-align:right; padding:6px;'>{pct:.1f}%</td></tr>"""

            table2_html += "</table>"
            display(HTML(table2_html))

            display(HTML(f"<h4 style='margin-top:25px;'>🔀 Combination Matrix: {mod1} × {mod2}</h4>"))
            crosstab = pd.crosstab(analysis_data[mod1], analysis_data[mod2], margins=True, margins_name='Total')
            display(crosstab.style.background_gradient(cmap='Blues', subset=pd.IndexSlice[crosstab.index[:-1], crosstab.columns[:-1]]))

            n_empty = (crosstab.iloc[:-1, :-1] == 0).sum().sum()
            min_cell = crosstab.iloc[:-1, :-1].min().min()

            if n_empty > 0:
                display(HTML(f"<div style='background:#f8d7da; padding:10px; margin-top:10px; border-radius:4px;'>"
                            f"⚠️ {n_empty} empty combinations - will be excluded</div>"))
            elif min_cell < 3:
                display(HTML(f"<div style='background:#fff3cd; padding:10px; margin-top:10px; border-radius:4px;'>"
                            f"⚠️ Min cell: {min_cell} - results may be unstable</div>"))
            elif min_cell < 5:
                display(HTML(f"<div style='background:#fff3cd; padding:10px; margin-top:10px; border-radius:4px;'>"
                            f"⚠️ Some combinations limited (min: {min_cell})</div>"))
            else:
                display(HTML("<div style='background:#d4edda; padding:10px; margin-top:10px; border-radius:4px;'>"
                            "✓ All combinations have ≥5 obs</div>"))

def update_thresholds_tab(change=None):
    with tab_thresholds:
        clear_output()

        if moderator1_widget is None:
            print("Initializing...")
            return

        display(HTML("<h3 style='margin-top:0;'>Quality Thresholds & Impact Analysis</h3>"))
        display(HTML("""<div style='background:#f8f9fa; padding:12px; border-radius:6px; margin-bottom:15px;'>
            <b>Purpose:</b> Ensure sufficient data for reliable estimation<br>
            <span style='font-size:13px; color:#555;'>Higher = more reliable but fewer subgroups</span></div>"""))

        display(min_papers_widget)
        display(min_obs_widget)
        display(HTML("<h4 style='margin-top:25px;'>Impact on Data Retention</h4>"))

        mod1 = moderator1_widget.value
        analysis_type = analysis_type_widget.value
        mod2 = moderator2_widget.value if analysis_type == 'two_way' and moderator2_widget.value != 'None' else None
        min_papers, min_obs = min_papers_widget.value, min_obs_widget.value

        groups_meeting, groups_failing = [], []

        if analysis_type == 'single':
            for cat in analysis_data[mod1].dropna().unique():
                group_data = analysis_data[analysis_data[mod1] == cat]
                n_papers, n_obs = group_data['id'].nunique(), len(group_data)

                if n_papers >= min_papers and n_obs >= min_obs:
                    groups_meeting.append((cat, n_obs, n_papers))
                else:
                    groups_failing.append((cat, n_obs, n_papers))
        else:
            if mod2:
                for cat1 in analysis_data[mod1].dropna().unique():
                    for cat2 in analysis_data[mod2].dropna().unique():
                        cell_data = analysis_data[(analysis_data[mod1] == cat1) & (analysis_data[mod2] == cat2)]
                        n_papers, n_obs = cell_data['id'].nunique(), len(cell_data)

                        if n_papers >= min_papers and n_obs >= min_obs:
                            groups_meeting.append((f"{cat1} × {cat2}", n_obs, n_papers))
                        elif n_obs > 0:
                            groups_failing.append((f"{cat1} × {cat2}", n_obs, n_papers))

        total_retained = sum(obs for _, obs, _ in groups_meeting)
        retention_pct = (total_retained / len(analysis_data)) * 100

        cards_html = f"""<div style='display:flex; gap:15px; margin-bottom:20px;'>
            <div style='flex:1; background:#d4edda; padding:15px; border-radius:6px; text-align:center;'>
                <div style='font-size:28px; font-weight:bold; color:#155724;'>{len(groups_meeting)}</div>
                <div style='font-size:13px; color:#155724;'>Groups Meeting Criteria</div></div>
            <div style='flex:1; background:#{'#f8d7da' if len(groups_failing) > 0 else '#e2e3e5'}; padding:15px; border-radius:6px; text-align:center;'>
                <div style='font-size:28px; font-weight:bold; color:#{'#721c24' if len(groups_failing) > 0 else '#6c757d'};'>{len(groups_failing)}</div>
                <div style='font-size:13px; color:#{'#721c24' if len(groups_failing) > 0 else '#6c757d'};'>Groups Excluded</div></div>
            <div style='flex:1; background:#{'#d4edda' if retention_pct >= 75 else '#fff3cd' if retention_pct >= 50 else '#f8d7da'}; padding:15px; border-radius:6px; text-align:center;'>
                <div style='font-size:28px; font-weight:bold;'>{retention_pct:.0f}%</div>
                <div style='font-size:13px;'>Data Retained</div></div></div>"""
        display(HTML(cards_html))

        if groups_meeting:
            display(HTML("<h4>✓ Groups Meeting Criteria:</h4>"))
            meet_html = "<ul style='margin-top:5px;'>"
            for cat, obs, papers in groups_meeting:
                meet_html += f"<li><b>{cat}:</b> {obs} obs, {papers} papers</li>"
            meet_html += "</ul>"
            display(HTML(meet_html))

        if groups_failing:
            display(HTML("<h4 style='margin-top:20px;'>✗ Groups Excluded:</h4>"))
            fail_html = "<ul style='margin-top:5px; color:#721c24;'>"
            for cat, obs, papers in groups_failing:
                reason = []
                if papers < min_papers:
                    reason.append(f"papers: {papers}<{min_papers}")
                if obs < min_obs:
                    reason.append(f"obs: {obs}<{min_obs}")
                fail_html += f"<li><b>{cat}:</b> {obs} obs, {papers} papers ({', '.join(reason)})</li>"
            fail_html += "</ul>"
            display(HTML(fail_html))

        if len(groups_meeting) < 2:
            display(HTML("<div style='background:#f8d7da; padding:12px; border-radius:6px; margin-top:15px;'>"
                        "🔴 <b>ERROR:</b> Need ≥2 groups. Lower thresholds.</div>"))
        elif retention_pct < 50:
            display(HTML("<div style='background:#fff3cd; padding:12px; border-radius:6px; margin-top:15px;'>"
                        "⚠️ <b>WARNING:</b> <50% data retained. Consider lowering thresholds.</div>"))

def update_all_tabs(change=None):
    update_config_tab()
    update_moderators_tab()
    update_thresholds_tab()

# --- 5. SAVE CONFIGURATION ---
def save_configuration(button):
    with status_output:
        clear_output()

        analysis_type = analysis_type_widget.value
        mod1 = moderator1_widget.value
        mod2 = moderator2_widget.value if analysis_type == 'two_way' and moderator2_widget.value != 'None' else None
        min_papers, min_obs = min_papers_widget.value, min_obs_widget.value

        validation_errors = []

        if analysis_type == 'two_way' and not mod2:
            validation_errors.append("Two-way requires Moderator 2")

        if mod1 == mod2:
            validation_errors.append("Moderators cannot be the same")

        valid_groups = []
        if analysis_type == 'single':
            for cat in analysis_data[mod1].dropna().unique():
                group_data = analysis_data[analysis_data[mod1] == cat]
                if group_data['id'].nunique() >= min_papers and len(group_data) >= min_obs:
                    valid_groups.append(cat)
        else:
            if mod2:
                for cat1 in analysis_data[mod1].dropna().unique():
                    for cat2 in analysis_data[mod2].dropna().unique():
                        cell_data = analysis_data[(analysis_data[mod1] == cat1) & (analysis_data[mod2] == cat2)]
                        if cell_data['id'].nunique() >= min_papers and len(cell_data) >= min_obs:
                            valid_groups.append((cat1, cat2))

        if len(valid_groups) < 2:
            validation_errors.append(f"Only {len(valid_groups)} group(s) meet criteria. Need ≥2.")

        if validation_errors:
            error_html = "<div style='background:#f8d7da; padding:15px; border-radius:6px; border-left:4px solid #dc3545;'>"
            error_html += "<h4 style='margin-top:0; color:#721c24;'>❌ Validation Failed</h4><ul style='margin-bottom:0; color:#721c24;'>"
            for err in validation_errors:
                error_html += f"<li>{err}</li>"
            error_html += "</ul></div>"
            display(HTML(error_html))
            return

        if analysis_type == 'single':
            retained_data = analysis_data[analysis_data[mod1].isin(valid_groups)]
        else:
            retained_data = analysis_data[analysis_data.apply(lambda row: (row[mod1], row[mod2]) in valid_groups, axis=1)]

        retention_pct = (len(retained_data) / len(analysis_data)) * 100

        ANALYSIS_CONFIG['subgroup_config'] = {
            'timestamp': datetime.datetime.now(),
            'analysis_type': analysis_type,
            'moderator1': mod1,
            'moderator2': mod2,
            'min_papers': min_papers,
            'min_obs': min_obs,
            'expected_groups': len(valid_groups),
            'valid_groups_list': valid_groups,
            'data_retained': len(retained_data),
            'retention_pct': retention_pct,
            'has_empty_cells': analysis_type == 'two_way' and mod2 and
                               (pd.crosstab(analysis_data[mod1], analysis_data[mod2]) == 0).sum().sum() > 0,
            'n_empty_cells': (pd.crosstab(analysis_data[mod1], analysis_data[mod2]) == 0).sum().sum()
                            if analysis_type == 'two_way' and mod2 else 0
        }

        ANALYSIS_CONFIG['subgroup_config']['moderator1_info'] = {
            'name': mod1,
            'n_categories': analysis_data[mod1].nunique(),
            'categories': sorted(analysis_data[mod1].dropna().unique().tolist())
        }

        if mod2:
            ANALYSIS_CONFIG['subgroup_config']['moderator2_info'] = {
                'name': mod2,
                'n_categories': analysis_data[mod2].nunique(),
                'categories': sorted(analysis_data[mod2].dropna().unique().tolist())
            }

        success_html = f"""<div style='background:#d4edda; padding:15px; border-radius:6px; border-left:4px solid #28a745;'>
            <h4 style='margin-top:0; color:#155724;'>✓ Configuration Saved Successfully</h4>
            <table style='width:100%; margin-top:10px;'>
                <tr><td><b>Analysis Type:</b></td><td>{analysis_type}</td></tr>
                <tr><td><b>Primary Moderator:</b></td><td>{mod1}</td></tr>
                {f'<tr><td><b>Secondary Moderator:</b></td><td>{mod2}</td></tr>' if mod2 else ''}
                <tr><td><b>Valid Groups:</b></td><td>{len(valid_groups)}</td></tr>
                <tr><td><b>Data Retained:</b></td><td>{len(retained_data)}/{len(analysis_data)} ({retention_pct:.1f}%)</td></tr>
            </table>
            <p style='margin:10px 0 0 0; color:#155724; font-size:14px;'><b>✅ Ready! Proceed to the next cell to run the subgroup analysis.</b></p></div>"""
        display(HTML(success_html))

        with tab_details:
            print("\n" + "="*70)
            print("✓ CONFIGURATION SAVED")
            print("="*70)
            print(f"Timestamp: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            print("\nConfiguration stored in ANALYSIS_CONFIG['subgroup_config']")
            print("Ready to proceed to subgroup analysis execution.")

# --- 6. INITIALIZE AND DISPLAY ---
try:
    available_mods = initialize_configuration()
    update_all_tabs()
    display(tabs)
except Exception as e:
    print(f"❌ Initialization failed: {e}")
    print("\nPlease ensure:")
    print("  1. Step 2 (Overall Meta-Analysis) has been run")
    print("  2. ANALYSIS_CONFIG is properly configured")
    print("  3. analysis_data is available")
    raise


In [ ]:
#@title 🔬 Step 3b: Subgroup Analysis - Execution

# =============================================================================
# SUBGROUP ANALYSIS EXECUTION
#
# Purpose: Run three-level meta-analysis for each subgroup
#
# Prerequisites:
#   - Cell 'Step 2' (Overall Meta-Analysis) must be run
#   - Cell 'Step 3a' (Configuration) must be run and button clicked
#
# Instructions:
#   1. Ensure you've configured moderator in previous cell
#   2. Run this cell to execute the subgroup analysis
#   3. Results will appear below
#
# Output: ANALYSIS_CONFIG['subgroup_results']
# =============================================================================


# =============================================================================
# CELL 8 (ADVANCED REPLACEMENT): THREE-LEVEL SUBGROUP ANALYSIS
# Purpose: Calculate pooled effects for each subgroup using a robust
#          three-level model to account for within-study dependency.
# Method: Runs a separate three-level REML analysis for each subgroup.
#         Partitions heterogeneity using standard Q-statistics.
# Dependencies: Cell 4.5, Cell 6, Cell 7
# Outputs: 'subgroup_results' in ANALYSIS_CONFIG, compatible with Cell 9
# =============================================================================

import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.optimize import minimize, minimize_scalar
from scipy.stats import norm, chi2
import matplotlib.pyplot as plt
import datetime
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import sys
import traceback
import warnings

# --- 0. HELPER FUNCTIONS (COPIED FROM PREVIOUS CELLS) ---
# This cell must be self-contained to run the analyses.

# --- 0a. Copied from Cell 4.5 (Advanced Heterogeneity Estimators) ---
# Needed to get starting values for the 3-level model

def calculate_tau_squared_REML(df, effect_col, var_col, max_iter=100, tol=1e-8):
    """REML estimator for tau-squared"""
    k = len(df)
    if k < 2: return 0.0
    try:
        yi = df[effect_col].values
        vi = df[var_col].values
        valid_mask = np.isfinite(vi) & (vi > 0)
        if not valid_mask.all():
            yi = yi[valid_mask]
            vi = vi[valid_mask]
            k = len(yi)
        if k < 2: return 0.0
        def reml_objective(tau2):
            tau2 = max(0, tau2)
            wi = 1 / (vi + tau2)
            sum_wi = wi.sum()
            if sum_wi <= 0: return 1e10
            mu = (wi * yi).sum() / sum_wi
            Q = (wi * (yi - mu)**2).sum()
            log_lik = -0.5 * (np.sum(np.log(vi + tau2)) + np.log(sum_wi) + Q)
            return -log_lik
        var_yi = np.var(yi, ddof=1) if k > 2 else 1.0
        upper_bound = max(10 * var_yi, 100)
        result = minimize_scalar(reml_objective, bounds=(0, upper_bound), method='bounded', options={'maxiter': max_iter, 'xatol': tol})
        if result.success:
            return max(0.0, result.x)
        else:
            return calculate_tau_squared_DL(df, effect_col, var_col)
    except Exception:
        return calculate_tau_squared_DL(df, effect_col, var_col)

# --- 0b. Copied from Cell 6.5 (Three-Level Model) ---
# The core 3-level analysis engine

def _run_three_level_reml_for_subgroup(analysis_data, effect_col, var_col):
    """
    Main optimization function for a *single subgroup*.
    Returns estimates or None on failure.
    """
    grouped = analysis_data.groupby('id')
    y_all = [group[effect_col].values for _, group in grouped]
    v_all = [group[var_col].values for _, group in grouped]
    N_total = len(analysis_data)
    M_studies = len(y_all)
    if M_studies < 2:
        print("  ⚠️  Not enough studies (<=1) for 3-level model in this subgroup.")
        return None, None
    try:
        tau_sq_start, _ = calculate_tau_squared(analysis_data, effect_col, var_col, method='REML')
    except Exception:
        tau_sq_start = 0.01
    initial_params = [max(0, tau_sq_start), 0.01]
    bounds = [(0, None), (0, None)]

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        optimizer_result = minimize(
            _negative_log_likelihood_reml,
            x0=initial_params,
            args=(y_all, v_all, N_total, M_studies),
            method='L-BFGS-B',
            bounds=bounds,
            options={'ftol': 1e-10, 'gtol': 1e-6, 'maxiter': 500}
        )
    if not optimizer_result.success:
        print(f"  ❌ SUBGROUP OPTIMIZATION FAILED: {optimizer_result.message}")
        return None, None

    final_estimates = _get_three_level_estimates(
        optimizer_result.x, y_all, v_all, N_total, M_studies
    )
    return final_estimates, (y_all, v_all, N_total, M_studies)


# --- 1. SCRIPT START ---

analysis_output = widgets.Output()
display(analysis_output) # Display the output area ONCE.

# All analysis will be directed into this output area
with analysis_output:
    print("="*70)
    print("THREE-LEVEL SUBGROUP ANALYSIS")
    print("="*70)
    print(f"Timestamp: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

    try:
        # --- 2. LOAD CONFIGURATION ---
        print("STEP 1: LOADING CONFIGURATION")
        print("---------------------------------")

        if 'ANALYSIS_CONFIG' not in globals():
            raise NameError("ANALYSIS_CONFIG not found. Run previous cells first.")

        # Load data from global scope first
        if 'analysis_data' in globals():
            analysis_data = analysis_data.copy()
            print(f"  ✓ Found global 'analysis_data' (Shape: {analysis_data.shape})")
        elif 'data_filtered' in globals():
            analysis_data = data_filtered.copy()
            print(f"  ✓ Found global 'data_filtered' as fallback (Shape: {analysis_data.shape})")
        else:
            raise ValueError("Data not found. Run Cell 5/6 first.")

        # Now, check for config keys
        required_keys = ['effect_col', 'var_col', 'se_col', 'es_config',
                         'overall_results', 'three_level_results', 'subgroup_config']

        missing_keys = [k for k in required_keys if k not in ANALYSIS_CONFIG]
        if missing_keys:
            raise KeyError(f"Missing required config keys: {missing_keys}. Run Cells 6, 6.5, and 7.")

        # Load config items
        effect_col = ANALYSIS_CONFIG['effect_col']
        var_col = ANALYSIS_CONFIG['var_col']
        se_col = ANALYSIS_CONFIG['se_col']
        es_config = ANALYSIS_CONFIG['es_config']
        overall_results = ANALYSIS_CONFIG['overall_results']
        three_level_results = ANALYSIS_CONFIG['three_level_results']
        subgroup_config = ANALYSIS_CONFIG['subgroup_config']

        if three_level_results.get('status') != 'completed':
            raise ValueError("Cell 6.5 (Three-Level Analysis) must be run successfully first.")

        analysis_type = subgroup_config['analysis_type']
        moderator1 = subgroup_config['moderator1']
        moderator2 = subgroup_config['moderator2']
        valid_groups_list = subgroup_config['valid_groups_list']

        print(f"  ✓ Configuration loaded successfully")
        print(f"  ✓ Analysis Type: {analysis_type}")
        print(f"  ✓ Moderator 1: {moderator1}")
        if moderator2:
            print(f"  ✓ Moderator 2: {moderator2}")
        print(f"  ✓ Found {len(valid_groups_list)} valid subgroups to analyze.")

        # Clean the moderator column(s) in the main data
        analysis_data[moderator1] = analysis_data[moderator1].astype(str).str.strip()
        if moderator2:
             analysis_data[moderator2] = analysis_data[moderator2].astype(str).str.strip()
        print(f"  ✓ Cleaned moderator column(s) in analysis_data")

        # --- 3. ANALYZE EACH SUBGROUP ---
        print("\nSTEP 2: RUNNING 3-LEVEL ANALYSIS FOR EACH SUBGROUP")
        print("---------------------------------")

        subgroup_results_list = []
        total_Q_within_fe = 0.0 # We use the standard FE Q-stats for the Q_between test

        for group_item in valid_groups_list:
            # --- Get Group Data ---
            if analysis_type == 'single':
                group_name = str(group_item) # group_item is a string like 'Barley'
                group_data = analysis_data[analysis_data[moderator1] == group_name].copy()
            else: # two_way
                group_tuple = group_item # group_item is a tuple like ('Barley', 'High_N')
                group_name = f"{group_tuple[0]} x {group_tuple[1]}"
                group_data = analysis_data[
                    (analysis_data[moderator1] == group_tuple[0]) &
                    (analysis_data[moderator2] == group_tuple[1])
                ].copy()

            print(f"\nAnalyzing Subgroup: {group_name}")
            k_group = len(group_data)
            n_papers_group = group_data['id'].nunique()
            print(f"  k_obs = {k_group}, k_studies = {n_papers_group}")

            if k_group < 2 or n_papers_group < 2:
                print("  ⚠️  Skipping group (k < 2 or papers < 2).")
                continue

            # --- Run 3-Level Model on Subgroup ---
            estimates, _ = _run_three_level_reml_for_subgroup(group_data, effect_col, var_col)

            if estimates is None:
                print(f"  ❌ 3-Level model failed for this subgroup. Skipping.")
                continue

            # --- Extract 3-Level Results (for plotting) ---
            mu_re = estimates['mu']
            se_re = estimates['se_mu']
            var_re = estimates['var_mu']
            ci_lower_re = mu_re - 1.96 * se_re
            ci_upper_re = mu_re + 1.96 * se_re
            p_value_re = 2 * (1 - norm.cdf(abs(mu_re / se_re)))

            tau_sq_re = estimates['tau_sq']
            sigma_sq_re = estimates['sigma_sq']

            # Calculate 3-Level I-squared
            mean_v_i = np.mean(group_data[var_col])
            total_variance_est = tau_sq_re + sigma_sq_re + mean_v_i
            I_squared_re = ((tau_sq_re + sigma_sq_re) / total_variance_est) * 100 if total_variance_est > 0 else 0

            # --- Run Standard FE Model on Subgroup (for Q_between test) ---
            w_fe = 1 / group_data[var_col]
            sum_w_fe = w_fe.sum()
            pooled_effect_fe = (w_fe * group_data[effect_col]).sum() / sum_w_fe
            Q_within_group = (w_fe * (group_data[effect_col] - pooled_effect_fe)**2).sum()
            total_Q_within_fe += Q_within_group

            # --- *** FIX: ADD FOLD CHANGE PLACEHOLDER *** ---
            if es_config.get('has_fold_change', False):
                # Calculate real fold change for lnRR
                RR = np.exp(mu_re)
                fold_change_re = RR if mu_re >= 0 else -1/RR
            else:
                # Add NaN placeholder for Hedges' g / other
                fold_change_re = np.nan
            # --- *** END FIX *** ---

            # --- Store Results ---
            result_dict = {
                'group': group_name,
                'k': k_group,
                'n_papers': n_papers_group,
                # These are the 3-LEVEL results, named to match Cell 9's expectations
                'pooled_effect_re': mu_re,
                'pooled_se_re': se_re,
                'pooled_var_re': var_re,
                'ci_lower_re': ci_lower_re,
                'ci_upper_re': ci_upper_re,
                'p_value_re': p_value_re,
                'I_squared': I_squared_re,
                'tau_squared': tau_sq_re,
                'sigma_squared': sigma_sq_re,
                'fold_change_re': fold_change_re, # <-- Added this line
                # Store FE Q-stat for partitioning
                'Q_within': Q_within_group,
                'df_Q': k_group - 1
            }
            if analysis_type == 'two_way':
                group_tuple = group_item
                result_dict[moderator1] = group_tuple[0]
                result_dict[moderator2] = group_tuple[1]

            subgroup_results_list.append(result_dict)
            print(f"  ✓ Subgroup analysis complete.")

        # --- 4. HETEROGENEITY PARTITIONING ---
        print("\nSTEP 3: PARTITIONING HETEROGENEITY")
        print("---------------------------------")

        results_df = pd.DataFrame(subgroup_results_list)
        if results_df.empty:
            raise ValueError("No subgroups were successfully analyzed.")

        # Use Q-total from the *standard* fixed-effect model (Cell 6)
        Qt_overall = overall_results['Qt']
        k_overall = overall_results['k']

        Qe_sum = results_df['Q_within'].sum()
        df_Qe = results_df['df_Q'].sum()

        M_groups = len(results_df)
        df_QM = M_groups - 1

        QM = max(0, Qt_overall - Qe_sum)

        p_value_QM = 1 - chi2.cdf(QM, df_QM) if df_QM > 0 else np.nan
        R_squared = max(0, (QM / Qt_overall) * 100) if Qt_overall > 0 else 0

        print(f"\n  Heterogeneity Decomposition (based on standard FE Q-stats):")
        print(f"  {'Component':<25} {'Q':>12} {'df':>8} {'P-value':>10}")
        print(f"  {'-'*25} {'-'*12} {'-'*8} {'-'*10}")
        print(f"  {'Total (Q_T)':<25} {Qt_overall:>12.4f} {k_overall-1:>8} {'-':>10}")
        print(f"  {'Between-Groups (Q_M)':<25} {QM:>12.4f} {df_QM:>8} {p_value_QM:>10.4g}")
        print(f"  {'Within-Groups (Q_E)':<25} {Qe_sum:>12.4f} {df_Qe:>8} {'-':>10}")

        print(f"\n  Variance Explained (R²): {R_squared:.1f}%")
        print(f"  Interpretation: The moderator explains {R_squared:.1f}% of the *standard* heterogeneity.")

        # --- 5. DISPLAY RESULTS TABLE ---
        print("\n" + "="*70)
        print("THREE-LEVEL SUBGROUP ANALYSIS: RESULTS")
        print("="*70)
        print("\n  NOTE: Pooled effects below are from robust 3-level models for each subgroup.\n")

        print(f"  {'Group':<35} {'k':>5} {'Papers':>8} {'Effect (RE)':>12} {'95% CI':>22} {'P-value':>10}")
        print(f"  {'-'*35} {'-'*5} {'-'*8} {'-'*12} {'-'*22} {'-'*10}")

        for _, row in results_df.iterrows():
            group_name = str(row['group'])[:35]
            ci_str = f"[{row['ci_lower_re']:.3f}, {row['ci_upper_re']:.3f}]"
            sig_marker = "***" if row['p_value_re'] < 0.001 else "**" if row['p_value_re'] < 0.01 else "*" if row['p_value_re'] < 0.05 else "ns"

            print(f"  {group_name:<35} {row['k']:>5} {row['n_papers']:>8} {row['pooled_effect_re']:>12.4f} {ci_str:>22} {row['p_value_re']:>9.4g} {sig_marker}")

        print(f"\n  Test for Subgroup Differences (Q_M): p = {p_value_QM:.4g}")

        # --- 6. SAVE RESULTS FOR CELL 9 ---
        print("\nSTEP 4: SAVING RESULTS")
        print("---------------------------------")

        # Save results in the format Cell 9 expects
        ANALYSIS_CONFIG['subgroup_results'] = {
            'timestamp': datetime.datetime.now(),
            'results_df': results_df, # This is the key part for Cell 9
            'analysis_type': analysis_type,
            'moderator1': moderator1,
            'moderator2': moderator2,
            # Add the partitioning results
            'Qt_overall': Qt_overall,
            'QM': QM,
            'Qe': Qe_sum,
            'df_QM': df_QM,
            'df_Qe': df_Qe,
            'p_value_QM': p_value_QM,
            'R_squared': R_squared
        }

        print("  ✓ Results saved to ANALYSIS_CONFIG['subgroup_results']")
        print("  ✓ The next cell (Forest Plot) will now use these 3-level estimates.")

        print("\n" + "="*70)
        print("✅ THREE-LEVEL SUBGROUP ANALYSIS COMPLETE")
        print("="*70)
        print("  ▶️  Run Cell 9 (Dynamic Forest Plot) to visualize these results.")


    except Exception as e:
        print(f"\n❌ AN ERROR OCCURRED:\n")
        print(f"  Type: {type(e).__name__}")
        print(f"  Message: {e}")
        print("\n  Traceback:")
        traceback.print_exc(file=sys.stdout)
        print("\n" + "="*70)
        print("ANALYSIS FAILED. See error message above.")
        print("Please check your data and configuration.")
        print("="*70)

# --- 4. INITIAL CHECK (REMOVED) ---
# The logic is now all contained within the `with analysis_output:` block.
# We just need the pre-run check.
try:
    if ('ANALYSIS_CONFIG' not in globals() or
        'overall_results' not in ANALYSIS_CONFIG or
        'three_level_results' not in ANALYSIS_CONFIG or
        'subgroup_config' not in ANALYSIS_CONFIG):

        print("="*70)
        print("⚠️  PREREQUISITES NOT MET")
        print("="*70)
        print("Please run the following cells in order before this one:")
        print("  1. Cell 6 (Overall Meta-Analysis)")
        print("  2. Cell 6.5 (Three-Level Meta-Analysis)")
        print("  3. Cell 7 (Subgroup Analysis Configuration)")
    else:
        # Check for auto-detection
        if 'analysis_data' in globals():
            data_check = analysis_data
        elif 'data_filtered' in globals():
            data_check = data_filtered
        else:
            raise ValueError("Data not found for pre-check")

        k_obs_check = len(data_check)
        k_studies_check = data_check['id'].nunique()

        if k_obs_check == k_studies_check:
            print("="*70)
            print("✅ THREE-LEVEL ANALYSIS NOT REQUIRED")
            print("="*70)
            print("  Your dataset has only one effect size per study.")
            print("  The standard meta-analysis from Cell 8 is sufficient.")
        else:
            print("="*70)
            print("✅ READY FOR THREE-LEVEL SUBGROUP ANALYSIS")
            print("="*70)
            print("  This cell is ready to run.")
            print("  It will use the configurations from Cells 6, 6.5, and 7.")
            print("  This will replace the standard subgroup analysis with a more robust 3-level model.")
            # Display the output area, which will be populated when the cell runs.

except Exception as e:
     print(f"❌ An error occurred during initialization: {e}")

Output()

✅ READY FOR THREE-LEVEL SUBGROUP ANALYSIS
  This cell is ready to run.
  It will use the configurations from Cells 6, 6.5, and 7.
  This will replace the standard subgroup analysis with a more robust 3-level model.


In [ ]:
#@title ⚖️ Cell 8.5: R Validation for Subgroup Analysis (Robust)
# =============================================================================
# CELL: R VALIDATION FOR SUBGROUP ANALYSIS
# Purpose: Verify 3-Level Subgroup estimates against R's metafor package.
# Fix: Returns vectors from R instead of DataFrames to prevent conversion errors.
# =============================================================================

import pandas as pd
import numpy as np
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
pandas2ri.activate()

# --- 1. Prepare Data & Config ---
if 'ANALYSIS_CONFIG' not in globals() or 'subgroup_results' not in ANALYSIS_CONFIG:
    print("❌ Error: Subgroup results not found. Please run Cell 8 first.")
else:
    subgroup_config = ANALYSIS_CONFIG['subgroup_results']

    # Get moderator info
    moderator1 = subgroup_config['moderator1']
    moderator2 = subgroup_config['moderator2']
    analysis_type = subgroup_config['analysis_type']

    # Get columns
    eff_col = ANALYSIS_CONFIG.get('effect_col', 'hedges_g')
    var_col = ANALYSIS_CONFIG.get('var_col', 'Vg')

    # Get Data
    if 'analysis_data' in globals(): df_sub_check = analysis_data.copy()
    elif 'data_filtered' in globals(): df_sub_check = data_filtered.copy()
    else: df_sub_check = None

    if df_sub_check is not None:
        print(f"🚀 Running R Validation for Subgroup Analysis...")
        print(f"   Moderator: {moderator1}" + (f" x {moderator2}" if moderator2 else ""))
        print(f"   Effect: {eff_col}, Variance: {var_col}")

        # Create combined group column
        if analysis_type == 'two_way' and moderator2:
            df_sub_check['subgroup_id'] = df_sub_check[moderator1].astype(str) + " x " + df_sub_check[moderator2].astype(str)
            py_results = subgroup_config['results_df'].set_index('group')
        else:
            df_sub_check['subgroup_id'] = df_sub_check[moderator1].astype(str)
            py_results = subgroup_config['results_df'].set_index('group')

        # Clean data for R
        df_r = df_sub_check[['id', eff_col, var_col, 'subgroup_id']].dropna()
        df_r = df_r[df_r[var_col] > 0]

        # Filter to valid groups
        valid_groups = py_results.index.tolist()
        df_r = df_r[df_r['subgroup_id'].isin(valid_groups)]

        ro.globalenv['df_python'] = df_r
        ro.globalenv['eff_col_name'] = eff_col
        ro.globalenv['var_col_name'] = var_col

        # --- 2. R Script (Vectorized Return) ---
        r_script = """
        library(metafor)

        dat <- df_python
        dat$rows <- 1:nrow(dat)
        dat$study_id <- as.factor(dat$id)

        # Get unique subgroups
        groups <- unique(dat$subgroup_id)
        n_groups <- length(groups)

        # Pre-allocate vectors (safer than building dataframe row-by-row)
        out_groups <- character(n_groups)
        out_ests <- numeric(n_groups)
        out_tau2s <- numeric(n_groups)
        out_valid <- logical(n_groups)

        # Loop through subgroups
        for (i in 1:n_groups) {
            g <- groups[i]
            sub_dat <- dat[dat$subgroup_id == g, ]

            out_groups[i] <- g

            # Skip if too small
            if (nrow(sub_dat) < 2) {
                out_valid[i] <- FALSE
                next
            }

            # Run 3-Level Model
            skip <- FALSE
            tryCatch({
                res <- rma.mv(yi=sub_dat[[eff_col_name]], V=sub_dat[[var_col_name]],
                              random = ~ 1 | study_id/rows,
                              data=sub_dat,
                              control=list(optimizer="optim", optmethod="Nelder-Mead"))

                out_ests[i] <- res$b[1]
                out_tau2s[i] <- res$sigma2[1]
                out_valid[i] <- TRUE
            }, error=function(e) {
                out_valid[i] <<- FALSE
            })
        }

        # Return as a simple list of vectors
        list(
            groups = out_groups,
            ests = out_ests,
            tau2s = out_tau2s,
            valid = out_valid
        )
        """

        try:
            # Run R
            r_list = ro.r(r_script)

            # Extract vectors
            r_groups = list(r_list.rx2('groups'))
            r_ests = list(r_list.rx2('ests'))
            r_valid = list(r_list.rx2('valid'))

            print("\n" + "="*85)
            print(f"{'Subgroup':<35} {'Python Effect':<15} {'R Effect':<15} {'Diff':<15}")
            print("="*85)

            matches = 0
            warnings_count = 0

            for i, group_name in enumerate(r_groups):
                if not r_valid[i]:
                    continue

                r_est = r_ests[i]

                if group_name in py_results.index:
                    py_est = py_results.loc[group_name, 'pooled_effect_re']
                    diff = abs(py_est - r_est)

                    print(f"{group_name[:35]:<35} {py_est:<15.4f} {r_est:<15.4f} {diff:.2e}")

                    if diff < 1e-3:
                        matches += 1
                    else:
                        warnings_count += 1
                else:
                    print(f"{group_name[:35]:<35} {'N/A':<15} {r_est:<15.4f} {'(Not in Py)'}")

            print("-" * 85)
            if warnings_count == 0 and matches > 0:
                print("✅ PASSED: All subgroups match R results.")
            elif matches > 0:
                print(f"⚠️  CHECK: {warnings_count} subgroups differ > 0.001. (Likely optimizer tolerance differences).")
            else:
                print("❌ FAIL: No matching subgroups found.")

        except Exception as e:
            print(f"\n❌ R Execution Error: {e}")

In [ ]:
#@title 📊 Cell 9: Dynamic Forest Plot (Fixed)
# =============================================================================
# CELL 9: PUBLICATION-READY FOREST PLOT
# Purpose: Create customizable forest plots for meta-analysis results
# Fix: Updated result keys to match the robust Cell 6 output.
# =============================================================================

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import norm
import datetime
from matplotlib.patches import Patch, Rectangle
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# --- 1. LOAD CONFIGURATION ---
print("="*70)
print("FOREST PLOT CONFIGURATION")
print("="*70)

try:
    if 'ANALYSIS_CONFIG' not in locals() and 'ANALYSIS_CONFIG' not in globals():
        raise NameError("ANALYSIS_CONFIG not found.")

    subgroup_results = ANALYSIS_CONFIG.get('subgroup_results', {})
    overall_results = ANALYSIS_CONFIG['overall_results']
    es_config = ANALYSIS_CONFIG['es_config']

    # Determine if we have subgroup analysis
    has_subgroups = bool(subgroup_results) and 'results_df' in subgroup_results

    if has_subgroups:
        analysis_type = subgroup_results['analysis_type']
        moderator1 = subgroup_results['moderator1']
        moderator2 = subgroup_results.get('moderator2', None)
        results_df = subgroup_results['results_df']

        # Set dynamic defaults
        if analysis_type == 'two_way':
            default_title = f'Forest Plot: {moderator1} × {moderator2}'
            default_y_label = moderator2
        else:
            default_title = f'Forest Plot: {moderator1}'
            default_y_label = moderator1
    else:
        # Overall only (no subgroups)
        analysis_type = 'overall_only'
        default_title = 'Forest Plot: Overall Effect'
        default_y_label = 'Study'
        moderator1 = None
        moderator2 = None

    default_x_label = es_config.get('effect_label', "Effect Size")

    print(f"✓ Analysis type: {analysis_type}")
    print(f"✓ Has subgroups: {has_subgroups}")
    print(f"✓ Configuration loaded successfully")

except (KeyError, NameError) as e:
    print(f"❌ ERROR: Failed to load configuration: {e}")
    print("   Please run Cell 6 (overall analysis) first")
    raise

# --- 2. DEFINE CUSTOMIZATION WIDGETS ---

# ========== TAB 1: PLOT STYLE ==========
style_header = widgets.HTML("<h3 style='color: #2E86AB;'>Plot Style & Layout</h3>")

model_widget = widgets.Dropdown(
    options=[('Random-Effects', 'RE'), ('Fixed-Effects', 'FE')],
    value='RE',
    description='Model:',
    style={'description_width': '130px'},
    layout=widgets.Layout(width='450px')
)

width_widget = widgets.FloatSlider(
    value=8.0, min=6.0, max=14.0, step=0.5,
    description='Plot Width (in):',
    continuous_update=False,
    style={'description_width': '130px'},
    layout=widgets.Layout(width='450px')
)

height_widget = widgets.FloatSlider(
    value=0.4, min=0.2, max=1.0, step=0.05,
    description='Height per Row (in):',
    continuous_update=False,
    style={'description_width': '130px'},
    layout=widgets.Layout(width='450px')
)

title_fontsize_widget = widgets.IntSlider(
    value=12, min=8, max=18, step=1,
    description='Title Font Size:',
    continuous_update=False,
    style={'description_width': '130px'},
    layout=widgets.Layout(width='450px')
)

label_fontsize_widget = widgets.IntSlider(
    value=11, min=8, max=16, step=1,
    description='Axis Label Size:',
    continuous_update=False,
    style={'description_width': '130px'},
    layout=widgets.Layout(width='450px')
)

tick_fontsize_widget = widgets.IntSlider(
    value=9, min=6, max=14, step=1,
    description='Tick Label Size:',
    continuous_update=False,
    style={'description_width': '130px'},
    layout=widgets.Layout(width='450px')
)

annot_fontsize_widget = widgets.IntSlider(
    value=8, min=6, max=12, step=1,
    description='Annotation Size:',
    continuous_update=False,
    style={'description_width': '130px'},
    layout=widgets.Layout(width='450px')
)

color_scheme_widget = widgets.Dropdown(
    options=[
        ('Grayscale (Publication)', 'gray'),
        ('Color (Presentation)', 'color'),
        ('Black & White Only', 'bw')
    ],
    value='gray',
    description='Color Scheme:',
    style={'description_width': '130px'},
    layout=widgets.Layout(width='450px')
)

marker_style_widget = widgets.Dropdown(
    options=[
        ('Circle/Diamond (●/◆)', 'circle_diamond'),
        ('Square/Diamond (■/◆)', 'square_diamond'),
        ('Circle/Star (●/★)', 'circle_star')
    ],
    value='circle_diamond',
    description='Marker Style:',
    style={'description_width': '130px'},
    layout=widgets.Layout(width='450px')
)

ci_style_widget = widgets.Dropdown(
    options=[
        ('Solid Line', 'solid'),
        ('Dashed Line', 'dashed'),
        ('Solid with Caps', 'caps')
    ],
    value='solid',
    description='CI Line Style:',
    style={'description_width': '130px'},
    layout=widgets.Layout(width='450px')
)

style_tab = widgets.VBox([
    style_header,
    model_widget,
    widgets.HTML("<hr style='margin: 10px 0;'>"),
    widgets.HTML("<b>Dimensions:</b>"),
    width_widget,
    height_widget,
    widgets.HTML("<hr style='margin: 10px 0;'>"),
    widgets.HTML("<b>Typography:</b>"),
    title_fontsize_widget,
    label_fontsize_widget,
    tick_fontsize_widget,
    annot_fontsize_widget,
    widgets.HTML("<hr style='margin: 10px 0;'>"),
    widgets.HTML("<b>Visual Style:</b>"),
    color_scheme_widget,
    marker_style_widget,
    ci_style_widget
])

# ========== TAB 2: TEXT & LABELS ==========
text_header = widgets.HTML("<h3 style='color: #2E86AB;'>Text & Labels</h3>")

show_title_widget = widgets.Checkbox(
    value=True,
    description='Show Plot Title',
    indent=False,
    layout=widgets.Layout(width='450px')
)

title_widget = widgets.Text(
    value=default_title,
    description='Plot Title:',
    layout=widgets.Layout(width='450px'),
    style={'description_width': '130px'}
)

xlabel_widget = widgets.Text(
    value=default_x_label,
    description='X-Axis Label:',
    layout=widgets.Layout(width='450px'),
    style={'description_width': '130px'}
)

ylabel_widget = widgets.Text(
    value=default_y_label,
    description='Y-Axis Label:',
    layout=widgets.Layout(width='450px'),
    style={'description_width': '130px'}
)

show_ylabel_widget = widgets.Checkbox(
    value=True,
    description='Show Y-Axis Label',
    indent=False,
    layout=widgets.Layout(width='450px')
)

text_tab = widgets.VBox([
    text_header,
    show_title_widget,
    title_widget,
    widgets.HTML("<hr style='margin: 10px 0;'>"),
    xlabel_widget,
    show_ylabel_widget,
    ylabel_widget
])

# ========== TAB 3: ANNOTATIONS ==========
annot_header = widgets.HTML("<h3 style='color: #2E86AB;'>Annotations</h3>")

show_k_widget = widgets.Checkbox(
    value=True,
    description='Show k (observations)',
    indent=False,
    layout=widgets.Layout(width='450px')
)

show_papers_widget = widgets.Checkbox(
    value=True,
    description='Show paper count',
    indent=False,
    layout=widgets.Layout(width='450px')
)

show_fold_change_widget = widgets.Checkbox(
    value=es_config.get('has_fold_change', False),
    description='Show Fold-Change',
    indent=False,
    layout=widgets.Layout(width='450px')
)

annot_pos_widget = widgets.Dropdown(
    options=[
        ('Right of CI', 'right'),
        ('Above Marker', 'above'),
        ('Below Marker', 'below')
    ],
    value='right',
    description='Position:',
    style={'description_width': '130px'},
    layout=widgets.Layout(width='450px')
)

annot_offset_widget = widgets.FloatSlider(
    value=0.0, min=-1.0, max=1.0, step=0.05,
    description='H-Offset:',
    continuous_update=False,
    style={'description_width': '130px'},
    layout=widgets.Layout(width='450px'),
    readout_format='.2f'
)

group_label_box = widgets.VBox()
# Group label widgets (always defined, conditionally displayed)
group_label_h_offset_widget = widgets.FloatSlider(
    value=0.0, min=-2.0, max=2.0, step=0.1,
    description='Group H-Offset:',
    continuous_update=False,
    style={'description_width': '130px'},
    layout=widgets.Layout(width='450px')
)

group_label_v_offset_widget = widgets.FloatSlider(
    value=0.0, min=-5.0, max=5.0, step=0.5,
    description='Group V-Offset:',
    continuous_update=False,
    style={'description_width': '130px'},
    layout=widgets.Layout(width='450px')
)

group_label_fontsize_widget = widgets.IntSlider(
    value=10, min=6, max=20, step=1,
    description='Group Fontsize:',
    continuous_update=False,
    style={'description_width': '130px'},
    layout=widgets.Layout(width='450px')
)

# Only display group label widgets for two-way analysis
if has_subgroups and analysis_type == 'two_way':
    group_label_box = widgets.VBox([
        widgets.HTML("<h4>Group Label Positioning (Two-Way Only)</h4>"),
        group_label_h_offset_widget,
        group_label_v_offset_widget,
        group_label_fontsize_widget
    ])
else:
    group_label_box = widgets.VBox()


annot_tab = widgets.VBox([
    annot_header,
    widgets.HTML("<b>Show in Annotations:</b>"),
    show_k_widget,
    show_papers_widget,
    show_fold_change_widget,
    widgets.HTML("<hr style='margin: 10px 0;'>"),
    widgets.HTML("<b>Position:</b>"),
    annot_pos_widget,
    annot_offset_widget,
    group_label_box
])

# ========== TAB 4: AXES & SCALE ==========
axes_header = widgets.HTML("<h3 style='color: #2E86AB;'>Axes & Scaling</h3>")

auto_scale_widget = widgets.Checkbox(
    value=True,
    description='Auto-Scale X-Axis',
    indent=False,
    layout=widgets.Layout(width='450px')
)

x_min_widget = widgets.FloatText(
    value=-2.0,
    description='X-Min:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='220px', visibility='hidden')
)

x_max_widget = widgets.FloatText(
    value=2.0,
    description='X-Max:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='220px', visibility='hidden')
)

manual_scale_box = widgets.HBox([x_min_widget, x_max_widget])

def toggle_manual_scale(change):
    if change['new']:
        x_min_widget.layout.visibility = 'hidden'
        x_max_widget.layout.visibility = 'hidden'
    else:
        x_min_widget.layout.visibility = 'visible'
        x_max_widget.layout.visibility = 'visible'

auto_scale_widget.observe(toggle_manual_scale, names='value')

show_grid_widget = widgets.Checkbox(
    value=True,
    description='Show Grid',
    indent=False,
    layout=widgets.Layout(width='450px')
)

grid_style_widget = widgets.Dropdown(
    options=[
        ('Dashed (Light)', 'dashed_light'),
        ('Dotted (Light)', 'dotted_light'),
        ('Solid (Light)', 'solid_light')
    ],
    value='dashed_light',
    description='Grid Style:',
    style={'description_width': '130px'},
    layout=widgets.Layout(width='450px')
)

show_null_line_widget = widgets.Checkbox(
    value=True,
    description='Show Null Effect Line',
    indent=False,
    layout=widgets.Layout(width='450px')
)

show_fold_axis_widget = widgets.Checkbox(
    value=es_config.get('has_fold_change', False) and show_fold_change_widget.value,
    description='Show Fold-Change Axis (Top)',
    indent=False,
    layout=widgets.Layout(width='450px')
)

axes_tab = widgets.VBox([
    axes_header,
    auto_scale_widget,
    manual_scale_box,
    widgets.HTML("<hr style='margin: 10px 0;'>"),
    widgets.HTML("<b>Grid & Reference Lines:</b>"),
    show_grid_widget,
    grid_style_widget,
    show_null_line_widget,
    widgets.HTML("<hr style='margin: 10px 0;'>"),
    show_fold_axis_widget
])

# ========== TAB 5: EXPORT OPTIONS ==========
export_header = widgets.HTML("<h3 style='color: #2E86AB;'>Export Options</h3>")

save_pdf_widget = widgets.Checkbox(
    value=True,
    description='Save as PDF',
    indent=False,
    layout=widgets.Layout(width='450px')
)

save_png_widget = widgets.Checkbox(
    value=True,
    description='Save as PNG',
    indent=False,
    layout=widgets.Layout(width='450px')
)

png_dpi_widget = widgets.IntSlider(
    value=300, min=150, max=600, step=50,
    description='PNG DPI:',
    continuous_update=False,
    style={'description_width': '130px'},
    layout=widgets.Layout(width='450px')
)

filename_prefix_widget = widgets.Text(
    value='ForestPlot',
    description='Filename Prefix:',
    layout=widgets.Layout(width='450px'),
    style={'description_width': '130px'}
)

transparent_bg_widget = widgets.Checkbox(
    value=False,
    description='Transparent Background',
    indent=False,
    layout=widgets.Layout(width='450px')
)

export_tab = widgets.VBox([
    export_header,
    save_pdf_widget,
    save_png_widget,
    png_dpi_widget,
    widgets.HTML("<hr style='margin: 10px 0;'>"),
    filename_prefix_widget,
    transparent_bg_widget
])

# ========== TAB 6: LABEL EDITOR ==========
label_editor_header = widgets.HTML("<h3 style='color: #2E86AB;'>Label Editor</h3>")
label_editor_desc = widgets.HTML(
    "<p style='color: #666;'><i>Customize display names for all groups and subgroups in the plot</i></p>"
)

print(f"\n🔍 Identifying labels for editor...")

unique_labels = set()
label_widgets_dict = {}

try:
    if has_subgroups:
        if analysis_type == 'single':
            unique_labels.update(results_df['group'].astype(str).unique())
        else:  # two_way
            unique_labels.update(results_df[moderator1].astype(str).unique())
            unique_labels.update(results_df[moderator2].astype(str).unique())

    unique_labels.add('Overall')
    sorted_labels = sorted(list(unique_labels))

    print(f"  ✓ Found {len(sorted_labels)} unique labels")

    label_editor_widgets = []
    for label in sorted_labels:
        widget_label = f"Overall Effect:" if label == 'Overall' else f"{label}:"
        text_widget = widgets.Text(
            value=str(label),
            description=widget_label,
            layout=widgets.Layout(width='500px'),
            style={'description_width': '200px'}
        )
        label_editor_widgets.append(text_widget)
        label_widgets_dict[str(label)] = text_widget

    label_editor_tab = widgets.VBox([
        label_editor_header,
        label_editor_desc,
        widgets.HTML("<hr style='margin: 10px 0;'>"),
        widgets.HTML(
            "<p><b>Instructions:</b> Edit the text on the right to change how labels appear in the plot. "
            "The original coded names are shown on the left.</p>"
        ),
        widgets.HTML("<hr style='margin: 10px 0;'>"),
        *label_editor_widgets
    ])

    print(f"  ✓ Label editor created")

except Exception as e:
    print(f"  ⚠️  Error creating label editor: {e}")
    label_editor_tab = widgets.VBox([
        label_editor_header,
        widgets.HTML("<p style='color: red;'>Error creating label editor.</p>")
    ])
    label_widgets_dict = {}

# ========== CREATE TAB WIDGET ==========
tab_children = [style_tab, text_tab, annot_tab, axes_tab, export_tab, label_editor_tab]
tab = widgets.Tab(children=tab_children)
tab.set_title(0, '🎨 Style')
tab.set_title(1, '📝 Text')
tab.set_title(2, '🏷️ Annotations')
tab.set_title(3, '📏 Axes')
tab.set_title(4, '💾 Export')
tab.set_title(5, '✏️ Labels')

# --- 3. DEFINE PLOT GENERATION FUNCTION ---
plot_output = widgets.Output()

def generate_plot(b):
    with plot_output:
        clear_output(wait=True)

        print("\n" + "="*70)
        print("GENERATING FOREST PLOT")
        print("="*70)
        print(f"Timestamp: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

        try:
            # --- GET WIDGET VALUES ---
            plot_model = model_widget.value
            plot_width = width_widget.value
            height_per_row = height_widget.value
            title_fontsize = title_fontsize_widget.value
            label_fontsize = label_fontsize_widget.value
            tick_fontsize = tick_fontsize_widget.value
            annot_fontsize = annot_fontsize_widget.value
            color_scheme = color_scheme_widget.value
            marker_style = marker_style_widget.value
            ci_style = ci_style_widget.value

            show_title = show_title_widget.value
            graph_title = title_widget.value
            x_label = xlabel_widget.value
            show_ylabel = show_ylabel_widget.value
            y_label = ylabel_widget.value

            show_k = show_k_widget.value
            show_papers = show_papers_widget.value
            show_fold_change = show_fold_change_widget.value
            annot_pos = annot_pos_widget.value
            annot_offset = annot_offset_widget.value

            auto_scale = auto_scale_widget.value
            x_min_manual = x_min_widget.value
            x_max_manual = x_max_widget.value
            show_grid = show_grid_widget.value
            grid_style = grid_style_widget.value
            show_null_line = show_null_line_widget.value
            show_fold_axis = show_fold_axis_widget.value

            save_pdf = save_pdf_widget.value
            save_png = save_png_widget.value
            png_dpi = png_dpi_widget.value
            filename_prefix = filename_prefix_widget.value
            transparent_bg = transparent_bg_widget.value

            # Group label offsets (two-way only)
            if has_subgroups and analysis_type == 'two_way':
                group_label_h_offset = group_label_h_offset_widget.value
                group_label_v_offset = group_label_v_offset_widget.value
                group_label_fontsize = group_label_fontsize_widget.value
            else:
                group_label_h_offset = 0
                group_label_v_offset = 0
                group_label_fontsize = 10

            # --- BUILD LABEL MAPPING FROM EDITOR ---
            label_mapping = {}
            for original_label, widget in label_widgets_dict.items():
                custom_label = widget.value
                label_mapping[original_label] = custom_label
                label_mapping[str(original_label)] = custom_label

            print(f"📊 Configuration:")
            print(f"  Model: {plot_model}")
            print(f"  Dimensions: {plot_width}\" × auto")
            print(f"  Color scheme: {color_scheme}")
            print(f"  Has subgroups: {has_subgroups}")

            # Show custom labels if any were changed
            changed_labels = {k: v for k, v in label_mapping.items() if k != v}
            if changed_labels:
                print(f"\n📝 Custom labels ({len(changed_labels)} changed):")
                for orig, custom in list(changed_labels.items())[:5]:
                    print(f"  '{orig}' → '{custom}'")
                if len(changed_labels) > 5:
                    print(f"  ... and {len(changed_labels)-5} more")

            overall_label_text = label_mapping.get('Overall', 'Overall Effect')

            # --- DETERMINE COLUMN NAMES BASED ON MODEL ---
            if plot_model == 'FE':
                effect_col = 'pooled_effect_fe'
                se_col = 'pooled_se_fe'
                ci_lower_col = 'ci_lower_fe'
                ci_upper_col = 'ci_upper_fe'
                fold_col = 'fold_change_fe'

                overall_effect_key = 'pooled_effect_fixed'
                overall_se_key = 'pooled_SE_fixed'
                overall_ci_lower_key = 'ci_lower_fixed'
                overall_ci_upper_key = 'ci_upper_fixed'
                overall_fold_key = 'pooled_fold_fixed' # Assuming this exists
            else:  # RE
                effect_col = 'pooled_effect_re'
                se_col = 'pooled_se_re'
                ci_lower_col = 'ci_lower_re'
                ci_upper_col = 'ci_upper_re'
                fold_col = 'fold_change_re'

                overall_effect_key = 'pooled_effect_random'
                # FIX: Use keys that exist in Cell 6 output
                overall_se_key = 'pooled_SE_random_reported'
                overall_ci_lower_key = 'ci_lower_random_reported'
                overall_ci_upper_key = 'ci_upper_random_reported'
                overall_fold_key = 'pooled_fold_random'

            # --- PREPARE DATA ---
            if has_subgroups:
                plot_df_subgroups = results_df.copy()

                plot_df_subgroups = plot_df_subgroups.rename(columns={
                    effect_col: 'EffectSize',
                    se_col: 'SE',
                    ci_lower_col: 'CI_Lower',
                    ci_upper_col: 'CI_Upper',
                    fold_col: 'FoldChange',
                    'k': 'k',
                    'n_papers': 'nPapers'
                })

                if analysis_type == 'two_way':
                    plot_df_subgroups['GroupVar'] = plot_df_subgroups[moderator1].astype(str)
                    plot_df_subgroups['LabelVar'] = plot_df_subgroups[moderator2].astype(str)
                else:  # single
                    plot_df_subgroups['GroupVar'] = 'Subgroup'
                    plot_df_subgroups['LabelVar'] = plot_df_subgroups['group'].astype(str)

                required_cols = ['GroupVar', 'LabelVar', 'k', 'nPapers',
                               'EffectSize', 'SE', 'CI_Lower', 'CI_Upper', 'FoldChange']
                plot_df_subgroups = plot_df_subgroups[required_cols]
                plot_df_subgroups.dropna(subset=['EffectSize', 'SE'], inplace=True)

                print(f"  Subgroups: {len(plot_df_subgroups)}")
            else:
                plot_df_subgroups = pd.DataFrame(columns=[
                    'GroupVar', 'LabelVar', 'k', 'nPapers',
                    'EffectSize', 'SE', 'CI_Lower', 'CI_Upper', 'FoldChange'
                ])

            # --- ADD OVERALL EFFECT ---
            overall_effect_val = overall_results[overall_effect_key]
            # FIX: Safely get values or default to Z-test version if reported missing
            overall_se_val = overall_results.get(overall_se_key, overall_results.get('pooled_SE_random_Z'))
            overall_ci_lower_val = overall_results.get(overall_ci_lower_key, overall_results.get('ci_lower_random_Z'))
            overall_ci_upper_val = overall_results.get(overall_ci_upper_key, overall_results.get('ci_upper_random_Z'))

            overall_k_val = overall_results['k']
            overall_papers_val = overall_results['k_papers']
            overall_fold_val = overall_results.get(overall_fold_key, np.nan)

            overall_row = pd.DataFrame([{
                'GroupVar': 'Overall',
                'LabelVar': 'Overall',
                'k': overall_k_val,
                'nPapers': overall_papers_val,
                'EffectSize': overall_effect_val,
                'SE': overall_se_val,
                'CI_Lower': overall_ci_lower_val,
                'CI_Upper': overall_ci_upper_val,
                'FoldChange': overall_fold_val
            }])

            print(f"  Overall: k={overall_k_val}, papers={overall_papers_val}")

            # --- COMBINE DATA (OVERALL ON TOP) ---
            plot_df = pd.concat([overall_row, plot_df_subgroups], ignore_index=True)

            plot_df['SortKey_Group'] = plot_df['GroupVar'].apply(
                lambda x: 'AAAAA' if x == 'Overall' else str(x)
            )
            plot_df['SortKey_Label'] = plot_df['LabelVar'].apply(
                lambda x: 'AAAAA' if x == 'Overall' else str(x)
            )
            plot_df.sort_values(by=['SortKey_Group', 'SortKey_Label'], inplace=True)
            plot_df.reset_index(drop=True, inplace=True)

            if plot_df.empty:
                print("❌ ERROR: No data to plot")
                return

            print(f"  Total rows: {len(plot_df)}")

            # --- CALCULATE PLOT DIMENSIONS ---
            num_rows = len(plot_df)
            y_positions = np.arange(num_rows)

            base_height = 2.5
            plot_height = max(base_height, num_rows * height_per_row + 1.5)

            y_margin_top = 0.75
            y_margin_bottom = 0.75
            y_lim_bottom = y_positions[0] - y_margin_bottom
            y_lim_top = y_positions[-1] + y_margin_top

            # --- Y-TICK LABELS (USE CUSTOM MAPPING) ---
            y_tick_labels = []
            for i, row in plot_df.iterrows():
                if row['GroupVar'] == 'Overall':
                    y_tick_labels.append(overall_label_text)
                else:
                    original_label = str(row['LabelVar'])
                    display_label = label_mapping.get(original_label, original_label)
                    y_tick_labels.append(display_label)

            # --- CALCULATE X-AXIS LIMITS (FIXED - USE ALL DATA) ---
            min_ci = plot_df['CI_Lower'].min()
            max_ci = plot_df['CI_Upper'].max()
            min_effect = plot_df['EffectSize'].min()
            max_effect = plot_df['EffectSize'].max()

            plot_min = min(min_ci, 0)
            plot_max = max(max_ci, 0)
            x_range = plot_max - plot_min

            if x_range == 0:
                x_range = 1

            print(f"\n📏 Data range:")
            print(f"  Effect sizes: [{min_effect:.3f}, {max_effect:.3f}]")
            print(f"  CI range: [{min_ci:.3f}, {max_ci:.3f}]")
            print(f"  Plot range: [{plot_min:.3f}, {plot_max:.3f}]")

            # --- ESTIMATE ANNOTATION SPACE NEEDED ---
            max_k = int(plot_df['k'].max())
            max_np = int(plot_df['nPapers'].max()) if 'nPapers' in plot_df.columns else 0

            annot_parts = []
            if show_k:
                annot_parts.append(f"k={max_k}")
            if show_papers:
                annot_parts.append(f"({max_np})")
            if show_fold_change and es_config.get('has_fold_change', False):
                max_fold = plot_df['FoldChange'].abs().max() if 'FoldChange' in plot_df.columns else 10
                annot_parts.append(f"[-{max_fold:.2f}×]")

            example_annot = " ".join(annot_parts) if annot_parts else "k=100 (10)"

            char_width_fraction = (annot_fontsize / 8.0) * 0.006
            annot_space_fraction = len(example_annot) * char_width_fraction

            print(f"  Annotation example: '{example_annot}' ({len(example_annot)} chars)")

            # --- CALCULATE SPACE FOR GROUP LABELS (TWO-WAY) ---
            group_label_space = 0
            if has_subgroups and analysis_type == 'two_way':
                max_group_len = 0
                for group_val in plot_df[plot_df['GroupVar'] != 'Overall']['GroupVar'].unique():
                    custom_label = label_mapping.get(str(group_val), str(group_val))
                    max_group_len = max(max_group_len, len(custom_label))

                char_width_group = (group_label_fontsize / 8.0) * 0.006
                group_label_space = max_group_len * char_width_group

                print(f"  Group label max: {max_group_len} chars")

            # --- AUTO-SCALE CALCULATION ---
            if auto_scale:
                left_padding = 0.05
                annot_distance = 0.015
                right_padding = 0.03

                total_right_fraction = (annot_distance +
                                       annot_space_fraction +
                                       group_label_space +
                                       right_padding)

                x_min_auto = plot_min - x_range * left_padding
                x_max_auto = plot_max + x_range * (total_right_fraction / (1 - total_right_fraction))

                x_limits = (x_min_auto, x_max_auto)
                print(f"  X-axis (auto): [{x_min_auto:.3f}, {x_max_auto:.3f}]")
            else:
                x_limits = (x_min_manual, x_max_manual)
                print(f"  X-axis (manual): [{x_min_manual:.3f}, {x_max_manual:.3f}]")

            # --- DETERMINE COLORS AND MARKERS ---
            if color_scheme == 'gray':
                subgroup_color = 'dimgray'
                overall_color = 'black'
                ci_color_subgroup = 'gray'
                ci_color_overall = 'black'
            elif color_scheme == 'color':
                subgroup_color = '#4A90E2'
                overall_color = '#E74C3C'
                ci_color_subgroup = '#4A90E2'
                ci_color_overall = '#E74C3C'
            else:  # bw
                subgroup_color = 'black'
                overall_color = 'black'
                ci_color_subgroup = 'black'
                ci_color_overall = 'black'

            if marker_style == 'circle_diamond':
                subgroup_marker = 'o'
                overall_marker = 'D'
            elif marker_style == 'square_diamond':
                subgroup_marker = 's'
                overall_marker = 'D'
            else:  # circle_star
                subgroup_marker = 'o'
                overall_marker = '*'

            subgroup_marker_size = 6
            overall_marker_size = 8
            subgroup_ci_width = 1.5
            overall_ci_width = 2.0

            if ci_style == 'solid':
                capsize = 0
            elif ci_style == 'dashed':
                capsize = 0
            else:  # caps
                capsize = 4

            # --- CREATE FIGURE ---
            fig, ax = plt.subplots(figsize=(plot_width, plot_height))

            if transparent_bg:
                fig.patch.set_alpha(0)
                ax.patch.set_alpha(0)

            print(f"\n🎨 Plotting {num_rows} rows...")

            # --- PLOT DATA POINTS AND ERROR BARS ---
            for i, row in plot_df.iterrows():
                is_overall = (row['GroupVar'] == 'Overall')

                marker = overall_marker if is_overall else subgroup_marker
                msize = overall_marker_size if is_overall else subgroup_marker_size
                color = overall_color if is_overall else subgroup_color
                ci_color = ci_color_overall if is_overall else ci_color_subgroup
                ci_width = overall_ci_width if is_overall else subgroup_ci_width
                zorder = 5 if is_overall else 3

                linestyle = '-' if ci_style != 'dashed' else '--'

                ax.errorbar(
                    x=row['EffectSize'],
                    y=y_positions[i],
                    xerr=[[row['EffectSize'] - row['CI_Lower']],
                          [row['CI_Upper'] - row['EffectSize']]],
                    fmt='none',
                    capsize=capsize,
                    color=ci_color,
                    linewidth=ci_width,
                    linestyle=linestyle,
                    alpha=0.9,
                    zorder=zorder-1
                )

                ax.plot(
                    row['EffectSize'],
                    y_positions[i],
                    marker=marker,
                    markersize=msize,
                    markerfacecolor=color,
                    markeredgecolor='black' if color_scheme != 'bw' else 'black',
                    markeredgewidth=1.0,
                    linestyle='none',
                    zorder=zorder
                )

            # --- SET AXIS LIMITS FIRST ---
            ax.set_xlim(x_limits[0], x_limits[1])
            ax.set_ylim(y_lim_top, y_lim_bottom)  # Inverted

            final_xlims = ax.get_xlim()
            final_xrange = final_xlims[1] - final_xlims[0]

            print(f"  Final X-axis: [{final_xlims[0]:.3f}, {final_xlims[1]:.3f}]")

            # --- ADD ANNOTATIONS ---
            print(f"  Adding annotations...")

            annot_x_offset = annot_distance * final_xrange

            for i, row in plot_df.iterrows():
                is_overall = (row['GroupVar'] == 'Overall')
                font_weight = 'bold' if is_overall else 'normal'

                annot_parts = []
                if show_k:
                    annot_parts.append(f"k={int(row['k'])}")
                if show_papers and pd.notna(row['nPapers']):
                    annot_parts.append(f"({int(row['nPapers'])})")
                if show_fold_change and pd.notna(row['FoldChange']) and es_config.get('has_fold_change', False):
                    fold_sign = "+" if row['FoldChange'] > 0 else ""
                    annot_parts.append(f"[{fold_sign}{row['FoldChange']:.2f}×]")

                annotation_text = " ".join(annot_parts) if annot_parts else ""

                if annotation_text:
                    if annot_pos == 'right':
                        x_pos = row['CI_Upper'] + annot_x_offset + (annot_offset * final_xrange * 0.1)
                        y_pos = y_positions[i]
                        va = 'center'
                        ha = 'left'
                    elif annot_pos == 'above':
                        x_pos = row['EffectSize'] + (annot_offset * final_xrange * 0.1)
                        y_pos = y_positions[i] - 0.2
                        va = 'bottom'
                        ha = 'center'
                    else:  # below
                        x_pos = row['EffectSize'] + (annot_offset * final_xrange * 0.1)
                        y_pos = y_positions[i] + 0.2
                        va = 'top'
                        ha = 'center'

                    ax.text(
                        x_pos, y_pos,
                        annotation_text,
                        va=va, ha=ha,
                        fontsize=annot_fontsize,
                        fontweight=font_weight,
                        clip_on=False
                    )

            # --- ADD GROUP LABELS (TWO-WAY) ---
            if has_subgroups and analysis_type == 'two_way':
                print(f"  Adding group labels...")

                current_group = None
                first_subgroup_idx = 1 if 'Overall' in plot_df['GroupVar'].values else 0
                group_label_x_base = final_xlims[1] - (right_padding * final_xrange)

                for i, row in plot_df.iterrows():
                    group_val = str(row['GroupVar'])

                    if group_val != 'Overall' and group_val != current_group:
                        if i > first_subgroup_idx:
                            ax.axhline(
                                y=y_positions[i] - 0.5,
                                color='darkgray',
                                linewidth=0.8,
                                linestyle='-',
                                xmin=0.01,
                                xmax=0.99,
                                zorder=1
                            )

                        group_indices = plot_df[plot_df['GroupVar'] == group_val].index
                        label_y = (y_positions[group_indices[0]] + y_positions[group_indices[-1]]) / 2.0

                        label_x = group_label_x_base + (group_label_h_offset * final_xrange * 0.05)
                        label_y = label_y + group_label_v_offset

                        display_group_label = label_mapping.get(group_val, group_val)

                        ax.text(
                            label_x, label_y,
                            display_group_label,
                            va='center',
                            ha='right',
                            fontweight='bold',
                            fontsize=group_label_fontsize,
                            color='black',
                            clip_on=False
                        )

                        current_group = group_val

            # --- ADD SEPARATOR LINE BELOW OVERALL ---
            if len(plot_df) > 1:
                separator_y = y_positions[0] + 0.5
                ax.axhline(
                    y=separator_y,
                    color='black',
                    linewidth=1.5,
                    linestyle='-'
                )

            # --- CUSTOMIZE AXES ---
            print(f"  Customizing axes...")

            if show_null_line:
                ax.axvline(
                    x=0,
                    color='black',
                    linestyle='-',
                    linewidth=1.5,
                    alpha=0.8,
                    zorder=1
                )

            ax.set_xlabel(x_label, fontsize=label_fontsize, fontweight='bold')
            if show_ylabel:
                ax.set_ylabel(y_label, fontsize=label_fontsize, fontweight='bold')

            if show_title:
                ax.set_title(graph_title, fontweight='bold', fontsize=title_fontsize, pad=15)

            ax.set_yticks(y_positions)
            ax.set_yticklabels(y_tick_labels, fontsize=tick_fontsize)
            ax.tick_params(axis='x', labelsize=tick_fontsize)

            if show_grid:
                if grid_style == 'dashed_light':
                    ax.grid(axis='x', alpha=0.3, linestyle='--', linewidth=0.5)
                elif grid_style == 'dotted_light':
                    ax.grid(axis='x', alpha=0.3, linestyle=':', linewidth=0.5)
                else:  # solid_light
                    ax.grid(axis='x', alpha=0.2, linestyle='-', linewidth=0.5)

            # --- ADD FOLD-CHANGE AXIS (TOP) ---
            if show_fold_axis and es_config.get('has_fold_change', False):
                print(f"  Adding fold-change axis...")

                ax2 = ax.twiny()

                fold_ticks_lnRR = np.array([-2, -1.5, -1, -0.5, 0, 0.5, 1, 1.5, 2])
                fold_ticks_RR = np.exp(fold_ticks_lnRR)

                valid_mask = ((fold_ticks_lnRR >= final_xlims[0]) &
                             (fold_ticks_lnRR <= final_xlims[1]))
                fold_ticks_lnRR = fold_ticks_lnRR[valid_mask]
                fold_ticks_RR = fold_ticks_RR[valid_mask]

                ax2.set_xlim(final_xlims[0], final_xlims[1])
                ax2.set_xticks(fold_ticks_lnRR)

                fold_labels = []
                for rr in fold_ticks_RR:
                    if rr < 1:
                        fold_labels.append(f"{1/rr:.1f}× ↓")
                    elif rr > 1:
                        fold_labels.append(f"{rr:.1f}× ↑")
                    else:
                        fold_labels.append("1×")

                ax2.set_xticklabels(fold_labels, fontsize=tick_fontsize)
                ax2.set_xlabel("Fold-Change", fontsize=label_fontsize, fontweight='bold')

            # --- FINALIZE PLOT ---
            fig.tight_layout()

            # --- SAVE FILES ---
            print(f"\n💾 Saving files...")

            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            base_filename = f"{filename_prefix}_{plot_model}_{timestamp}"

            saved_files = []

            if save_pdf:
                pdf_filename = f"{base_filename}.pdf"
                fig.savefig(pdf_filename, bbox_inches='tight', transparent=transparent_bg)
                saved_files.append(pdf_filename)
                print(f"  ✓ {pdf_filename}")

            if save_png:
                png_filename = f"{base_filename}.png"
                fig.savefig(png_filename, dpi=png_dpi, bbox_inches='tight', transparent=transparent_bg)
                saved_files.append(png_filename)
                print(f"  ✓ {png_filename} (DPI: {png_dpi})")

            plt.show()

            print(f"\n" + "="*70)
            print("✅ FOREST PLOT COMPLETE")
            print("="*70)
            print(f"Files: {', '.join(saved_files)}")

        except Exception as e:
            print(f"\n❌ ERROR: {e}")
            import traceback
            traceback.print_exc()

# --- 4. CREATE BUTTON AND DISPLAY ---
plot_button = widgets.Button(
    description='📊 Generate Forest Plot',
    button_style='success',
    layout=widgets.Layout(width='450px', height='50px'),
    style={'font_weight': 'bold', 'font_size': '14px'}
)

plot_button.on_click(generate_plot)

print("\n" + "="*70)
print("✅ FOREST PLOT INTERFACE READY")
print("="*70)
print("👆 Customize your plot using the tabs above, then click Generate")
print("\n📝 Tips:")
print("  • Use the 'Labels' tab to rename coded variables")
print("  • Auto-scale considers ALL data points for proper spacing")
print("  • Annotations and group labels will fit within the plot")
print("="*70 + "\n")

display(widgets.VBox([
    widgets.HTML("<h3 style='color: #2E86AB;'>📊 Forest Plot Generator</h3>"),
    widgets.HTML("<p style='color: #666;'>Create publication-ready forest plots with full customization</p>"),
    widgets.HTML("<hr style='margin: 15px 0;'>"),
    tab,
    widgets.HTML("<hr style='margin: 15px 0;'>"),
    plot_button,
    plot_output
]))

In [ ]:
#@title ⚙️ Cell 9.5: High-Precision Regression Engine (Final Robust)
# =============================================================================
# CELL: REGRESSION ENGINE (Stability + Range Fix)
# Purpose: Core math for 3-Level Meta-Regression
# Fix: Added large-variance start points and matrix jitter for stability.
# =============================================================================

import numpy as np
import scipy.stats as stats
from scipy.optimize import minimize
import statsmodels.api as sm

def _run_three_level_reml_regression_v2(analysis_data, moderator_col, effect_col, var_col):
    """Main optimizer with Expanded Search Range."""
    grouped = analysis_data.groupby('id')
    y_all, v_all, X_all = [], [], []

    for _, group in grouped:
        y_all.append(group[effect_col].values)
        v_all.append(group[var_col].values)
        X_i = sm.add_constant(group[moderator_col].values, prepend=True)
        X_all.append(X_i)

    N_total = len(analysis_data)
    M_studies = len(y_all)
    p_params = 2

    # --- STRATEGY: Broad Global Search ---
    # We include 'large' variance start points (5.0, 10.0) to catch cases
    # like yours where Tau^2 is ~4.25
    start_points = [
        [0.1, 0.1],   # Standard small
        [1.0, 0.1],   # Medium Between
        [5.0, 0.1],   # Large Between (Targeting your data)
        [10.0, 0.5],  # Very Large
        [0.01, 1.0]   # Large Within
    ]

    best_res = None
    best_fun = np.inf

    for start in start_points:
        res = minimize(
            _neg_log_lik_reml_reg, x0=start,
            args=(y_all, v_all, X_all, N_total, M_studies, p_params),
            method='L-BFGS-B', bounds=[(1e-8, None), (1e-8, None)],
            options={'ftol': 1e-10}
        )
        if res.success and res.fun < best_fun:
            best_fun = res.fun
            best_res = res

    if not best_res:
        # If all fail, try one last desperate run with Nelder-Mead from a safe point
        best_res = minimize(
            _neg_log_lik_reml_reg, x0=[1.0, 1.0],
            args=(y_all, v_all, X_all, N_total, M_studies, p_params),
            method='Nelder-Mead', bounds=[(1e-8, None), (1e-8, None)]
        )

    if not best_res.success and not best_res.message:
         return None, None, None

    # 2. Polishing (Nelder-Mead)
    final_res = minimize(
        _neg_log_lik_reml_reg, x0=best_res.x,
        args=(y_all, v_all, X_all, N_total, M_studies, p_params),
        method='Nelder-Mead', bounds=[(1e-8, None), (1e-8, None)],
        options={'xatol': 1e-10, 'fatol': 1e-10}
    )

    final_est = _get_three_level_regression_estimates_v2(
        final_res.x, y_all, v_all, X_all, N_total, M_studies, p_params
    )

    return final_est, (N_total, M_studies, p_params), final_res

print("✅ High-Precision Regression Engine Ready (Robust Mode).")

In [ ]:
#@title 📈 Cell 10: Meta-Regression (Robust Aggregation Fix)
# =============================================================================
# CELL 10: META-REGRESSION UI
# Purpose: Run regression with automatic fallback for constant moderators.
# Fix: Solved DataFrameGroupBy.apply deprecation warning.
# =============================================================================

import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import numpy as np
import datetime
from scipy.stats import t, norm
from scipy.optimize import minimize_scalar
import statsmodels.api as sm

# --- 1. HELPER: Standard Random-Effects Regression (2-Level) ---
def _run_aggregated_re_regression(agg_df, moderator_col, effect_col, var_col):
    """
    Runs a standard Random-Effects Meta-Regression (2-Level).
    Used when the moderator is constant within studies.
    """
    # 1. Define REML Objective for 2-Level Model
    y = agg_df[effect_col].values
    v = agg_df[var_col].values
    X = sm.add_constant(agg_df[moderator_col].values)

    def re_nll(tau2):
        if tau2 < 0: tau2 = 0
        weights = 1.0 / (v + tau2)

        # WLS to get betas for this tau2
        try:
            wls = sm.WLS(y, X, weights=weights).fit()
            betas = wls.params
            resid = y - wls.fittedvalues

            # REML Log-Likelihood
            ll = -0.5 * (np.sum(np.log(v + tau2)) +
                         np.log(np.linalg.det(X.T @ np.diag(weights) @ X)) +
                         np.sum((resid**2) * weights))
            return -ll
        except:
            return np.inf

    # 2. Optimize Tau2
    res = minimize_scalar(re_nll, bounds=(0, 100), method='bounded')
    tau2_est = res.x

    # 3. Final Fit
    weights_final = 1.0 / (v + tau2_est)
    final_model = sm.WLS(y, X, weights=weights_final).fit()

    return {
        'betas': final_model.params,
        'se_betas': final_model.bse,
        'p_values': final_model.pvalues,
        'tau_sq': tau2_est,
        'model_type': 'Aggregated Random-Effects (2-Level)',
        'n_obs': len(agg_df),
        'resid_df': final_model.df_resid
    }

# --- 2. DATA LOADING & PREP ---
def get_potential_moderators(df):
    valid_mods = []
    exclude = ['id', 'w_fixed', 'w_random']
    if 'ANALYSIS_CONFIG' in globals():
        exclude.extend([
            ANALYSIS_CONFIG.get('effect_col'),
            ANALYSIS_CONFIG.get('var_col'),
            ANALYSIS_CONFIG.get('se_col')
        ])

    for col in df.columns:
        if col in exclude or col is None: continue
        if pd.api.types.is_numeric_dtype(df[col]):
            if df[col].nunique() > 1: valid_mods.append(col)
        elif df[col].dtype == 'object':
            try:
                nums = pd.to_numeric(df[col], errors='coerce')
                if nums.notna().sum() >= 3 and nums.nunique() > 1:
                    valid_mods.append(col)
            except: pass
    return sorted(list(set(valid_mods)))

def get_analysis_data():
    if 'analysis_data' in globals(): return analysis_data
    elif 'data_filtered' in globals(): return data_filtered
    else: return None

# --- 3. WIDGET SETUP ---
df_reg = get_analysis_data()
reg_options = get_potential_moderators(df_reg) if df_reg is not None else ['Data not loaded']
if not reg_options: reg_options = ['No numeric moderators found']

moderator_widget = widgets.Dropdown(
    options=reg_options, description='Moderator:',
    style={'description_width': 'initial'}, layout=widgets.Layout(width='400px')
)

run_reg_btn = widgets.Button(description="▶ Run Meta-Regression", button_style='success')
reg_output = widgets.Output()

def run_regression(b):
    global ANALYSIS_CONFIG
    with reg_output:
        clear_output()
        mod_col = moderator_widget.value
        df_working = get_analysis_data()

        if df_working is None: print("❌ Error: Data not found."); return
        if mod_col in ['No numeric moderators found', 'Data not loaded']: print("❌ Error: No valid moderator."); return

        if 'ANALYSIS_CONFIG' in globals():
            effect_col = ANALYSIS_CONFIG.get('effect_col', 'hedges_g')
            var_col = ANALYSIS_CONFIG.get('var_col', 'Vg')
        else:
            effect_col = 'hedges_g'; var_col = 'Vg'

        print(f"🚀 Running Meta-Regression on '{mod_col}'...")

        # Data Prep
        reg_df = df_working.copy()
        reg_df[mod_col] = pd.to_numeric(reg_df[mod_col], errors='coerce')
        reg_df = reg_df.dropna(subset=[mod_col, effect_col, var_col]).copy()
        reg_df = reg_df[reg_df[var_col] > 0]

        if len(reg_df) < 3: print(f"❌ Error: Not enough data (n={len(reg_df)})."); return

        # --- CHECK FOR CONSTANT MODERATOR ---
        studies_with_variation = reg_df.groupby('id')[mod_col].nunique()
        varying_studies = (studies_with_variation > 1).sum()

        # LOGIC BRANCH
        if varying_studies == 0:
            print(f"\n⚠️  WARNING: '{mod_col}' is constant within every study.")
            print(f"   🔄 SWITCHING STRATEGY: Aggregating data to study level...")

            # Aggregate Data
            reg_df['wi'] = 1 / reg_df[var_col]

            # --- FIX FOR PANDAS DEPRECATION WARNING ---
            def agg_func(x):
                return pd.Series({
                    effect_col: np.average(x[effect_col], weights=x['wi']),
                    var_col: 1 / np.sum(x['wi']),
                    mod_col: x[mod_col].iloc[0]
                })

            try:
                # New pandas (>2.2) requires include_groups=False
                agg_df = reg_df.groupby('id').apply(agg_func, include_groups=False).reset_index()
            except TypeError:
                # Older pandas compatibility
                agg_df = reg_df.groupby('id').apply(agg_func).reset_index()
            # ------------------------------------------

            print(f"   ✓ Aggregated {len(reg_df)} observations into {len(agg_df)} studies.")

            # Run Simplified 2-Level Regression
            res = _run_aggregated_re_regression(agg_df, mod_col, effect_col, var_col)

            beta0, beta1 = res['betas']
            se0, se1 = res['se_betas']
            p0, p1 = res['p_values']
            tau_sq = res['tau_sq']
            sigma_sq = 0.0 # Not applicable in 2-level aggregation
            df_resid = res['resid_df']
            t_stat = beta1 / se1

            # Create fake covariance matrix for plotting downstream (Cell 11)
            var_betas_robust = np.array([[se0**2, 0], [0, se1**2]])

            # Update reg_df for plotting to be the AGGREGATED data
            reg_df_for_plot = agg_df

        else:
            # Run Full 3-Level Regression
            if '_run_three_level_reml_regression_v2' not in globals():
                 print("❌ Error: Run Cell 9.5 first.")
                 return

            est, _, _ = _run_three_level_reml_regression_v2(reg_df, mod_col, effect_col, var_col)

            if not est: print("❌ Optimization Failed."); return

            beta0, beta1 = est['betas']
            se0, se1 = est['se_betas']
            m_studies = reg_df['id'].nunique()
            df_resid = max(1, m_studies - 2)
            t_stat = beta1 / se1
            p1 = 2 * (1 - t.cdf(abs(t_stat), df_resid))
            p0 = 2 * (1 - t.cdf(abs(beta0/se0), df_resid)) # Approx
            tau_sq = est['tau_sq']
            sigma_sq = est['sigma_sq']
            var_betas_robust = est['var_betas']
            reg_df_for_plot = reg_df

        # --- REPORTING ---
        print("\n" + "="*60)
        print(f"META-REGRESSION RESULTS (Moderator: {mod_col})")
        print("="*60)
        print(f"\nModel Type: {res.get('model_type', '3-Level Cluster-Robust') if 'res' in locals() else '3-Level Cluster-Robust'}")
        print(f"  • Studies (k): {reg_df['id'].nunique()}")
        print(f"  • Observations used: {len(reg_df_for_plot)}")
        print(f"  • Tau² (Between-Study): {tau_sq:.5f}")
        if sigma_sq > 0: print(f"  • Sigma² (Within-Study): {sigma_sq:.5f}")

        print(f"\nCoefficients:")
        print(f"  {'Term':<15} {'Estimate':<10} {'SE':<10} {'t-value':<10} {'p-value':<10}")
        print("-" * 60)
        print(f"  {'Intercept':<15} {beta0:<10.4f} {se0:<10.4f} {beta0/se0:<10.3f} {p0:<10.4f}")
        print(f"  {mod_col[:15]:<15} {beta1:<10.4f} {se1:<10.4f} {t_stat:<10.3f} {p1:<10.4f}")

        if p1 < 0.05: print(f"\n✅ Significant relationship detected (p < 0.05).")
        else: print(f"\nChecking for relationship... Not significant (p >= 0.05).")

        if 'ANALYSIS_CONFIG' not in globals(): ANALYSIS_CONFIG = {}
        ANALYSIS_CONFIG['meta_regression_RVE_results'] = {
            'reg_df': reg_df_for_plot, 'moderator_col_name': mod_col, 'effect_col': effect_col,
            'betas': [beta0, beta1], 'var_betas_robust': var_betas_robust,
            'std_errors_robust': [se0, se1], 'p_slope': p1,
            'R_squared_adj': 0, 'df_robust': df_resid
        }
        ANALYSIS_CONFIG['var_col'] = var_col

run_reg_btn.on_click(run_regression)

display(widgets.VBox([
    widgets.HTML("<h3>📊 Meta-Regression</h3>"),
    moderator_widget,
    run_reg_btn,
    reg_output
]))

In [ ]:
#@title 📈 META-REGRESSION PLOT (Cluster-Robust)

# =============================================================================
# CELL 11 (REPLACEMENT): META-REGRESSION PLOT
# Purpose: Visualize the meta-regression results from Cell 10
# Method: Creates a bubble plot with cluster-robust confidence bands
# Dependencies: Cell 10 (meta_regression_RVE_results)
# Outputs: Publication-ready plot (PDF/PNG)
# =============================================================================

import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.stats import t
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import datetime
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import sys
import traceback
import warnings

# --- 1. WIDGET DEFINITIONS ---
# Initialize lists
available_color_moderators = ['None']
analysis_data_init = None
default_x_label = "Moderator"
default_y_label = "Effect Size"
default_title = "Meta-Regression Plot"
label_widgets_dict = {} # Dictionary to store label widgets

try:
    if 'ANALYSIS_CONFIG' not in globals():
        raise NameError("ANALYSIS_CONFIG not found")

    if 'analysis_data' in globals():
        analysis_data_init = analysis_data.copy()
    elif 'data_filtered' in globals():
        analysis_data_init = data_filtered.copy()
    else:
        raise ValueError("No data found")

    if 'meta_regression_RVE_results' in ANALYSIS_CONFIG:
        reg_results = ANALYSIS_CONFIG['meta_regression_RVE_results']
        es_config = ANALYSIS_CONFIG['es_config']
        default_x_label = reg_results['moderator_col_name']
        default_y_label = es_config['effect_label']
        default_title = f"Meta-Regression: {default_y_label} vs. {default_x_label}"

    # Find categorical moderators for color AND labels
    excluded_cols = [
        ANALYSIS_CONFIG.get('effect_col'), ANALYSIS_CONFIG.get('var_col'),
        ANALYSIS_CONFIG.get('se_col'), 'w_fixed', 'w_random', 'id',
        'xe', 'sde', 'ne', 'xc', 'sdc', 'nc',
        ANALYSIS_CONFIG.get('ci_lower_col'), ANALYSIS_CONFIG.get('ci_upper_col')
    ]
    excluded_cols = [col for col in excluded_cols if col is not None]

    categorical_cols = analysis_data_init.select_dtypes(include=['object', 'category']).columns
    available_color_moderators.extend([
        col for col in categorical_cols
        if col not in excluded_cols and analysis_data_init[col].nunique() <= 10
    ])

    # *** NEW: Find all unique labels for the Label Editor ***
    all_categorical_labels = set()
    for col in available_color_moderators:
        if col != 'None' and col in analysis_data_init.columns:
            # Add the column name itself (e.g., "Crop")
            all_categorical_labels.add(col)
            # Add all unique values in that column (e.g., "B", "C", "R", "W")
            all_categorical_labels.update(analysis_data_init[col].astype(str).str.strip().unique())

    # Remove any empty strings
    all_categorical_labels.discard('')
    all_categorical_labels.discard('nan')

except Exception as e:
    print(f"⚠️  Initialization Error: {e}. Please run previous cells.")


# --- Widget Interface ---
header = widgets.HTML(
    "<h3 style='color: #2E86AB;'>Meta-Regression Plot Setup</h3>"
    "<p style='color: #666;'><i>Visualize the relationship between moderator and effect size</i></p>"
)

# ========== TAB 1: PLOT STYLE ==========
show_title_widget = widgets.Checkbox(value=True, description='Show Plot Title', indent=False)
title_widget = widgets.Text(value=default_title, description='Plot Title:',
                            layout=widgets.Layout(width='450px'), style={'description_width': '120px'})
xlabel_widget = widgets.Text(value=default_x_label, description='X-Axis Label:',
                             layout=widgets.Layout(width='450px'), style={'description_width': '120px'})
ylabel_widget = widgets.Text(value=default_y_label, description='Y-Axis Label:',
                             layout=widgets.Layout(width='450px'), style={'description_width': '120px'})
width_widget = widgets.FloatSlider(value=8.0, min=5.0, max=14.0, step=0.5, description='Plot Width (in):',
                                   continuous_update=False, style={'description_width': '120px'},
                                   layout=widgets.Layout(width='450px'))
height_widget = widgets.FloatSlider(value=6.0, min=4.0, max=12.0, step=0.5, description='Plot Height (in):',
                                    continuous_update=False, style={'description_width': '120px'},
                                    layout=widgets.Layout(width='450px'))

style_tab = widgets.VBox([
    widgets.HTML("<h4 style='color: #2E86AB;'>Labels & Size</h4>"),
    show_title_widget, title_widget, xlabel_widget, ylabel_widget, width_widget, height_widget
])

# ========== TAB 2: DATA POINTS ==========
color_mod_widget = widgets.Dropdown(options=available_color_moderators, value='None', description='Color By:',
                                    style={'description_width': '120px'}, layout=widgets.Layout(width='450px'))
point_color_widget = widgets.Dropdown(options=['gray', 'blue', 'red', 'green', 'purple', 'orange'], value='gray',
                                      description='Point Color:', style={'description_width': '120px'},
                                      layout=widgets.Layout(width='450px'))
bubble_base_widget = widgets.IntSlider(value=20, min=0, max=200, step=10, description='Min Bubble Size:',
                                       continuous_update=False, style={'description_width': '120px'},
                                       layout=widgets.Layout(width='450px'))
bubble_range_widget = widgets.IntSlider(value=800, min=100, max=2000, step=100, description='Max Bubble Size:',
                                        continuous_update=False, style={'description_width': '120px'},
                                        layout=widgets.Layout(width='450px'))
bubble_alpha_widget = widgets.FloatSlider(value=0.6, min=0.1, max=1.0, step=0.1, description='Transparency:',
                                          continuous_update=False, style={'description_width': '120px'},
                                          layout=widgets.Layout(width='450px'))

points_tab = widgets.VBox([
    widgets.HTML("<h4 style='color: #2E86AB;'>Data Points</h4>"),
    color_mod_widget, point_color_widget,
    widgets.HTML("<hr style='margin: 10px 0;'>"),
    widgets.HTML("<b>Bubble Size (by precision):</b>"),
    bubble_base_widget, bubble_range_widget, bubble_alpha_widget
])

# ========== TAB 3: REGRESSION LINE ==========
show_ci_widget = widgets.Checkbox(value=True, description='Show 95% Confidence Band', indent=False)
line_color_widget = widgets.Dropdown(options=['red', 'blue', 'black', 'green', 'purple'], value='red',
                                     description='Line Color:', style={'description_width': '120px'},
                                     layout=widgets.Layout(width='450px'))
line_width_widget = widgets.FloatSlider(value=2.0, min=0.5, max=5.0, step=0.5, description='Line Width:',
                                        continuous_update=False, style={'description_width': '120px'},
                                        layout=widgets.Layout(width='450px'))
ci_alpha_widget = widgets.FloatSlider(value=0.3, min=0.1, max=0.8, step=0.1, description='CI Transparency:',
                                      continuous_update=False, style={'description_width': '120px'},
                                      layout=widgets.Layout(width='450px'))
show_equation_widget = widgets.Checkbox(value=True, description='Show Regression Equation & P-value', indent=False)
show_r2_widget = widgets.Checkbox(value=True, description='Show R² Value', indent=False)

regline_tab = widgets.VBox([
    widgets.HTML("<h4 style='color: #2E86AB;'>Regression Line</h4>"),
    line_color_widget, line_width_widget, show_ci_widget, ci_alpha_widget,
    widgets.HTML("<hr style='margin: 10px 0;'>"),
    show_equation_widget, show_r2_widget
])

# ========== TAB 4: LAYOUT & EXPORT ==========
show_grid_widget = widgets.Checkbox(value=True, description='Show Grid', indent=False)
show_null_line_widget = widgets.Checkbox(value=True, description='Show Null Effect Line (y=0)', indent=False)
legend_loc_widget = widgets.Dropdown(options=['best', 'upper right', 'upper left', 'lower left', 'lower right'],
                                     value='best', description='Legend Position:',
                                     style={'description_width': '120px'}, layout=widgets.Layout(width='450px'))
legend_fontsize_widget = widgets.IntSlider(value=10, min=6, max=14, step=1, description='Legend Font:',
                                           continuous_update=False, style={'description_width': '120px'},
                                           layout=widgets.Layout(width='450px'))
save_pdf_widget = widgets.Checkbox(value=True, description='Save as PDF', indent=False)
save_png_widget = widgets.Checkbox(value=True, description='Save as PNG', indent=False)
png_dpi_widget = widgets.IntSlider(value=300, min=150, max=600, step=50, description='PNG DPI:',
                                   continuous_update=False, style={'description_width': '120px'},
                                   layout=widgets.Layout(width='450px'))
filename_prefix_widget = widgets.Text(value='MetaRegression_Plot', description='Filename Prefix:',
                                      layout=widgets.Layout(width='450px'), style={'description_width': '120px'})
transparent_bg_widget = widgets.Checkbox(value=False, description='Transparent Background', indent=False)

layout_tab = widgets.VBox([
    widgets.HTML("<h4 style='color: #2E86AB;'>Layout & Legend</h4>"),
    show_grid_widget, show_null_line_widget, legend_loc_widget, legend_fontsize_widget,
    widgets.HTML("<hr style='margin: 10px 0;'>"),
    widgets.HTML("<h4 style='color: #2E86AB;'>Export</h4>"),
    save_pdf_widget, save_png_widget, png_dpi_widget, filename_prefix_widget, transparent_bg_widget
])

# ========== TAB 5: LABELS (NEW) ==========
label_editor_widgets = []
for label in sorted(list(all_categorical_labels)):
    text_widget = widgets.Text(
        value=str(label),
        description=f"{label}:",
        layout=widgets.Layout(width='500px'),
        style={'description_width': '200px'}
    )
    label_editor_widgets.append(text_widget)
    label_widgets_dict[str(label)] = text_widget # Store widget by its original name

label_tab = widgets.VBox([
    widgets.HTML("<h4 style='color: #2E86AB;'>Edit Plot Labels</h4>"),
    widgets.HTML("<p style='color: #666;'><i>Rename raw data values (e.g., 'W') to publication-ready labels (e.g., 'Wheat').</i></p>"),
    *label_editor_widgets
])


# --- Assemble Tabs ---
tab = widgets.Tab(children=[style_tab, points_tab, regline_tab, layout_tab, label_tab])
tab.set_title(0, '🎨 Style'); tab.set_title(1, '⚫ Points'); tab.set_title(2, '📈 Regression')
tab.set_title(3, '💾 Layout/Export'); tab.set_title(4, '✏️ Labels')

run_plot_button = widgets.Button(
    description='📊 Generate Regression Plot',
    button_style='success',
    layout=widgets.Layout(width='450px', height='50px'),
    style={'font_weight': 'bold'}
)
plot_output = widgets.Output()

# --- 2. PLOTTING FUNCTION ---
@run_plot_button.on_click
def generate_regression_plot(b):
    """Generate meta-regression scatter plot with regression line"""
    with plot_output:
        clear_output(wait=True)

        print("="*70)
        print("GENERATING CLUSTER-ROBUST META-REGRESSION PLOT")
        print("="*70)
        print(f"Timestamp: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

        try:
            # --- 1. Load Data & Config ---
            print("STEP 1: LOADING RESULTS FROM CELL 10")
            print("---------------------------------")
            if 'meta_regression_RVE_results' not in ANALYSIS_CONFIG:
                raise ValueError("No meta-regression results found. Please re-run Cell 10.")

            reg_results = ANALYSIS_CONFIG['meta_regression_RVE_results']
            es_config = ANALYSIS_CONFIG['es_config']

            plot_data = reg_results['reg_df'].copy()
            moderator_col = reg_results['moderator_col_name']
            effect_col = reg_results['effect_col']
            var_col = ANALYSIS_CONFIG['var_col']

            b0, b1 = reg_results['betas']
            var_betas_robust = reg_results['var_betas_robust']
            R_sq = reg_results['R_squared_adj']
            p_slope = reg_results['p_slope']
            df_robust = reg_results['df_robust']

            print(f"  ✓ Loaded results for moderator: {moderator_col}")
            print(f"  ✓ Found {len(plot_data)} data points to plot.")

            # --- 2. Get Widget Values (*** FIX: ADDED .value TO ALL ***) ---
            show_title = show_title_widget.value
            graph_title = title_widget.value
            x_label = xlabel_widget.value
            y_label = ylabel_widget.value
            plot_width = width_widget.value
            plot_height = height_widget.value

            color_mod_name = color_mod_widget.value
            point_color = point_color_widget.value
            bubble_base = bubble_base_widget.value
            bubble_range = bubble_range_widget.value
            bubble_alpha = bubble_alpha_widget.value

            show_ci = show_ci_widget.value
            line_color = line_color_widget.value
            line_width = line_width_widget.value
            ci_alpha = ci_alpha_widget.value
            show_equation = show_equation_widget.value
            show_r2 = show_r2_widget.value

            show_grid = show_grid_widget.value
            show_null_line = show_null_line_widget.value
            legend_loc = legend_loc_widget.value
            legend_fontsize = legend_fontsize_widget.value

            save_pdf = save_pdf_widget.value
            save_png = save_png_widget.value
            png_dpi = png_dpi_widget.value
            filename_prefix = filename_prefix_widget.value
            transparent_bg = transparent_bg_widget.value
            # *** END FIX ***

            print(f"\n📊 Configuration:")
            print(f"  Plot size: {plot_width}\\\" × {plot_height}\\\"")
            print(f"  Color by: {color_mod_name}")

            # --- 2b. Build Label Mapping ---
            label_mapping = {orig: w.value for orig, w in label_widgets_dict.items()}

            # --- 3. Prepare Data for Plotting ---
            print("\nSTEP 2: PREPARING PLOT DATA")
            print("---------------------------------")

            if 'weights' not in plot_data.columns:
                tau_sq_overall = ANALYSIS_CONFIG['overall_results']['tau_squared']
                plot_data['weights'] = 1 / (plot_data[var_col] + tau_sq_overall)

            min_w = plot_data['weights'].min()
            max_w = plot_data['weights'].max()

            if max_w > min_w:
                plot_data['BubbleSize'] = bubble_base + (
                    ((plot_data['weights'] - min_w) / (max_w - min_w)) * bubble_range
                )
            else:
                plot_data['BubbleSize'] = bubble_base + bubble_range / 2

            print(f"  ✓ Bubble sizes calculated (Range: {plot_data['BubbleSize'].min():.0f} to {plot_data['BubbleSize'].max():.0f})")

            # --- Handle Color Coding (*** FIX: Corrected logic ***) ---
            c_values = point_color
            cmap = None
            norm = None
            unique_cats = []

            if color_mod_name != 'None':
                if color_mod_name in analysis_data_init.columns:
                    # Merge color data from the original dataframe based on index
                    color_data = analysis_data_init[[color_mod_name]].copy()
                    plot_data = plot_data.merge(color_data, left_index=True, right_index=True, how='left',
                                                suffixes=('', '_color'))

                    # Use the merged column
                    color_col_merged = f"{color_mod_name}"
                    plot_data[color_col_merged] = plot_data[color_col_merged].fillna('N/A').astype(str).str.strip()
                    plot_data['color_codes'], unique_cats = pd.factorize(plot_data[color_col_merged])
                    c_values = plot_data['color_codes']
                    cmap = 'tab10' # A good categorical colormap
                    norm = plt.Normalize(vmin=0, vmax=len(unique_cats)-1)
                    print(f"  ✓ Applying color based on '{color_mod_name}' ({len(unique_cats)} categories)")
                else:
                    print(f"  ⚠️  Color moderator '{color_mod_name}' not found, using default.")
                    color_mod_name = 'None'
            # *** END COLOR FIX ***

            # --- 4. Create Figure ---
            print("\nSTEP 3: GENERATING PLOT")
            print("---------------------------------")
            fig, ax = plt.subplots(figsize=(plot_width, plot_height))
            if transparent_bg:
                fig.patch.set_alpha(0)
                ax.patch.set_alpha(0)

            # --- Plot Data Points ---
            ax.scatter(
                x=plot_data[moderator_col],
                y=plot_data[effect_col],
                s=plot_data['BubbleSize'],
                c=c_values,
                cmap=cmap,
                norm=norm,
                alpha=bubble_alpha,
                edgecolors='black',
                linewidths=0.5,
                zorder=3
            )

            # --- Plot Regression Line & Confidence Band ---
            x_min = plot_data[moderator_col].min()
            x_max = plot_data[moderator_col].max()
            x_range_val = x_max - x_min
            x_padding = x_range_val * 0.05 if x_range_val > 0 else 1

            x_line = np.linspace(x_min - x_padding, x_max + x_padding, 100)
            y_line = b0 + b1 * x_line

            ax.plot(x_line, y_line, color=line_color, linewidth=line_width, zorder=2, label="Regression Line")

            if show_ci:
                X_line_pred = sm.add_constant(x_line, prepend=True)
                se_line = np.array([
                    np.sqrt(np.array([1, x]) @ var_betas_robust @ np.array([1, x]).T)
                    for x in x_line
                ])
                t_crit = t.ppf(0.975, df=df_robust)
                y_ci_upper = y_line + t_crit * se_line
                y_ci_lower = y_line - t_crit * se_line
                ax.fill_between(x_line, y_ci_lower, y_ci_upper,
                                color=line_color, alpha=ci_alpha, zorder=1, label=f"95% CI (Robust, df={df_robust})")
                print("  ✓ Plotted regression line and robust confidence band.")

            # --- Customize Axes ---
            if show_null_line:
                ax.axhline(es_config.get('null_value', 0), color='gray', linestyle='--', linewidth=1.0, zorder=0)

            ax.set_xlabel(x_label, fontsize=12, fontweight='bold')
            ax.set_ylabel(y_label, fontsize=12, fontweight='bold')
            if show_title:
                ax.set_title(graph_title, fontsize=14, fontweight='bold', pad=15)
            if show_grid:
                ax.grid(True, linestyle=':', alpha=0.4, zorder=0)

            # --- Add Equation and R² ---
            if show_equation or show_r2:
                text_lines = []
                if show_equation:
                    sign = "+" if b1 >= 0 else ""
                    sig_marker = "***" if p_slope < 0.001 else "**" if p_slope < 0.01 else "*" if p_slope < 0.05 else "ns"
                    eq_text = f"y = {b0:.3f} {sign} {b1:.3f}x"
                    p_text = f"p (slope) = {p_slope:.3g} {sig_marker}"
                    text_lines.append(eq_text)
                    text_lines.append(p_text)
                if show_r2:
                    r2_text = f"R² (adj) ≈ {R_sq:.1f}%"
                    text_lines.append(r2_text)

                ax.text(
                    0.05, 0.95, "\n".join(text_lines),
                    transform=ax.transAxes, fontsize=10, verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8, edgecolor='gray'),
                    zorder=10
                )

            # --- Create Legend ---
            handles, labels = ax.get_legend_handles_labels()

            # *** FIX: Use Label Mapping ***
            if color_mod_name != 'None':
                for i, cat in enumerate(unique_cats):
                    display_label = label_mapping.get(cat, cat) # Get new label
                    color_val = plt.get_cmap(cmap)(norm(i))
                    handles.append(mpatches.Patch(color=color_val, label=display_label, alpha=bubble_alpha, ec='black', lw=0.5))
                    labels.append(display_label)

            handles.append(plt.scatter([], [], s=bubble_base + bubble_range/2, c='gray' if color_mod_name == 'None' else 'lightgray',
                                       alpha=bubble_alpha, ec='black', lw=0.5))
            labels.append("Weight (1 / (vᵢ + τ²))")

            display_legend_title = label_mapping.get(color_mod_name, color_mod_name)

            ax.legend(handles=handles, labels=labels, loc=legend_loc,
                      fontsize=legend_fontsize, framealpha=0.9,
                      title=display_legend_title if color_mod_name != 'None' else None)
            # *** END FIX ***

            fig.tight_layout()
            plt.show()

            # --- 5. Save Files ---
            print(f"\nSTEP 4: SAVING FILES")
            print("---------------------------------")

            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            base_filename = f"{filename_prefix}_{moderator_col.replace(' ','_')}_{timestamp}"

            saved_files = []
            if save_pdf:
                pdf_filename = f"{base_filename}.pdf"
                fig.savefig(pdf_filename, bbox_inches='tight', transparent=transparent_bg)
                saved_files.append(pdf_filename)
                print(f"  ✓ {pdf_filename}")
            if save_png:
                png_filename = f"{base_filename}.png"
                fig.savefig(png_filename, dpi=png_dpi, bbox_inches='tight', transparent=transparent_bg)
                saved_files.append(png_filename)
                print(f"  ✓ {png_filename} (DPI: {png_dpi})")

            print(f"\n" + "="*70)
            print("✅ PLOT GENERATION COMPLETE")
            print("="*70)

        except Exception as e:
            print(f"\n❌ AN ERROR OCCURRED:\n")
            print(f"  Type: {type(e).__name__}")
            print(f"  Message: {e}")
            print("\n  Traceback:")
            traceback.print_exc(file=sys.stdout)
            print("\n" + "="*70)
            print("ANALYSIS FAILED. See error message above.")
            print("Please check your data and configuration.")
            print("="*70)


# --- 6. DISPLAY WIDGETS ---
try:
    if 'ANALYSIS_CONFIG' not in globals() or 'meta_regression_RVE_results' not in ANALYSIS_CONFIG:
        print("="*70)
        print("⚠️  PREREQUISITE NOT MET")
        print("="*70)
        print("Please run Cell 10 (Meta-Regression) successfully before running this cell.")
    else:
        print("="*70)
        print("✅ ROBUST META-REGRESSION PLOTTER READY")
        print("="*70)
        print("  ✓ Results from Cell 10 are loaded.")
        print("  ✓ Customize your plot using the tabs below and click 'Generate'.")

        # Hook up widget events
        def on_color_mod_change(change):
            point_color_widget.layout.display = 'none' if change['new'] != 'None' else 'flex'
        color_mod_widget.observe(on_color_mod_change, names='value')

        display(widgets.VBox([
            header,
            widgets.HTML("<hr style='margin: 15px 0;'>"),
            widgets.HTML("<b>Plot Options:</b>"),
            tab,
            widgets.HTML("<hr style='margin: 15px 0;'>"),
            run_plot_button,
            plot_output
        ]))

except Exception as e:
    print(f"❌ An error occurred during initialization: {e}")
    print("Please ensure the notebook has been run in order.")

In [ ]:
#@title ⚖️ Cell 10.5: R Validation for Meta-Regression (Fixed)
# =============================================================================
# CELL: R DIAGNOSTIC
# Purpose: Check how metafor handles the 'constant-within-study' moderator.
# Fix: Automatically detects correct variance column (Vg vs vg)
# =============================================================================

import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- 1. Setup & Data Prep ---
if 'data_filtered' not in globals():
    print("❌ Error: 'data_filtered' not found. Please run previous cells.")
else:
    # Select the problematic moderator
    moderator = 'kgPot'  # Hardcoded for this test based on your request

    print(f"🚀 Sending data to R to test moderator: '{moderator}'...")

    # --- FIX: Robust Column Detection ---
    # 1. Identify Effect Size Column
    if 'hedges_g' in data_filtered.columns:
        eff_col = 'hedges_g'
    elif 'ANALYSIS_CONFIG' in globals():
        eff_col = ANALYSIS_CONFIG.get('effect_col', 'hedges_g')
    else:
        eff_col = 'hedges_g'

    # 2. Identify Variance Column (The source of your error)
    if 'Vg' in data_filtered.columns:
        var_col = 'Vg'
    elif 'vg' in data_filtered.columns:
        var_col = 'vg'
    elif 'var_col' in globals().get('ANALYSIS_CONFIG', {}):
        var_col = ANALYSIS_CONFIG['var_col']
    else:
        print("❌ Error: Could not find variance column (checked 'Vg' and 'vg')")
        var_col = None

    if var_col:
        print(f"   Using Effect: '{eff_col}', Variance: '{var_col}'")

        # Create clean subset
        cols_to_keep = ['id', eff_col, var_col, moderator]

        # Check if we have the raw data columns (optional, just for context)
        raw_cols = ['xe', 'xc', 'ne', 'nc', 'sde', 'sdc']
        existing_raw = [c for c in raw_cols if c in data_filtered.columns]
        cols_to_keep.extend(existing_raw)

        df_r_test = data_filtered[cols_to_keep].copy()

        # Ensure moderator is numeric
        df_r_test[moderator] = pd.to_numeric(df_r_test[moderator], errors='coerce')
        df_r_test = df_r_test.dropna(subset=[eff_col, var_col, moderator])

        print(f"   Data shape: {len(df_r_test)} observations, {df_r_test['id'].nunique()} studies")

        # --- 2. Run R Code ---
        try:
            import rpy2.robjects as ro
            from rpy2.robjects import pandas2ri
            pandas2ri.activate()

            # Pass data to R
            ro.globalenv['df_python'] = df_r_test

            r_script = f"""
            library(metafor)

            # Ensure clean data inside R
            dat <- df_python
            dat$rows <- 1:nrow(dat)
            dat$study_id <- as.factor(dat$id)

            # Run 3-Level Meta-Regression
            # mods = ~ kgPot
            res <- rma.mv(yi={eff_col}, V={var_col},
                          mods = ~ {moderator},
                          random = ~ 1 | study_id/rows,
                          data=dat,
                          control=list(optimizer="optim", optmethod="Nelder-Mead"))

            print(summary(res))

            # Extract key metrics for Python display
            list(
                beta0 = res$b[1],
                beta1 = res$b[2],
                se1 = res$se[2],
                pval = res$pval[2],
                tau2 = res$sigma2[1],   # Level 3 (Between-Study)
                sigma2 = res$sigma2[2]  # Level 2 (Within-Study)
            )
            """

            print("\n" + "="*60)
            print("R (METAFOR) OUTPUT LOG")
            print("="*60)

            # Run and capture output
            r_result = ro.r(r_script)

            # Extract values
            r_beta1 = r_result.rx2('beta1')[0]
            r_pval = r_result.rx2('pval')[0]
            r_tau2 = r_result.rx2('tau2')[0]
            r_sigma2 = r_result.rx2('sigma2')[0]

            print("\n" + "="*60)
            print("DIAGNOSIS")
            print("="*60)
            print(f"Moderator: {moderator}")
            print(f"Slope (Beta): {r_beta1:.5f} (p={r_pval:.4f})")
            print("-" * 30)
            print(f"Level 3 Variance (Between-Study): {r_tau2:.8f}")
            print(f"Level 2 Variance (Within-Study):  {r_sigma2:.8f}")

            if r_tau2 < 0.0001:
                print("\n✅ DIAGNOSIS CONFIRMED:")
                print("   The Level 3 variance (Tau²) collapsed to ZERO.")
                print("   This caused the Python optimizer to crash (Singular Matrix).")
                print("   The moderator explains nearly all the between-study variation.")
            else:
                print("\nℹ️  Tau² is not zero. The Python crash might be due to starting parameters.")

        except Exception as e:
            print(f"\n❌ R Interface Error: {e}")

In [ ]:
#@title 🌊 Cell 11: 3-Level Spline Analysis (Plug-in Stability)
# =============================================================================
# CELL 11: ROBUST SPLINE ANALYSIS (PLUG-IN ESTIMATOR)
# Purpose: Non-linear meta-regression.
# Fix: Uses Tau² from the stable Linear Model (Cell 10) to prevent overfitting.
# =============================================================================

import numpy as np
import pandas as pd
from scipy.stats import t, chi2, norm
import statsmodels.api as sm
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings

# Check for patsy
try:
    import patsy
    PATSY_AVAILABLE = True
except ImportError:
    PATSY_AVAILABLE = False

# --- 1. HELPER: Aggregated Spline Engine (Plug-in Tau2) ---
# --- 1. HELPER: Aggregated Spline Engine (Fixed Tau2) ---
def _run_aggregated_spline_re(agg_df, moderator_col, effect_col, var_col, df_spline, mod_mean, mod_std, fixed_tau2):
    """
    Runs a Random-Effects Spline Model using a FIXED Tau^2.
    This prevents the optimizer from crashing on flat likelihood surfaces.
    """
    # Reset index to ensure alignment
    agg_df = agg_df.reset_index(drop=True)

    # Generate Basis
    mod_z = (agg_df[moderator_col] - mod_mean) / mod_std
    formula = f"cr(x, df={df_spline}) - 1"

    try:
        basis_matrix = patsy.dmatrix(formula, {"x": mod_z}, return_type='dataframe')
    except Exception as e:
        return None, f"Basis Error: {e}"

    y = agg_df[effect_col].values
    v = agg_df[var_col].values

    # Ensure basis aligns with y
    basis_matrix.index = agg_df.index
    X = sm.add_constant(basis_matrix) # Add intercept

    # FIT MODEL (No optimization needed - Tau2 is known!)
    # We use the passed 'fixed_tau2' directly
    weights = 1.0 / (v + fixed_tau2 + 1e-8)

    try:
        final_model = sm.WLS(y, X, weights=weights).fit()

        # Calculate Log-Likelihood manually for validation
        # REML LogLik = -0.5 * (sum(log(w^-1)) + log(det(X'WX)) + r'Wr)
        resid = y - final_model.fittedvalues

        XTWX = X.T @ np.diag(weights) @ X
        sign, logdet = np.linalg.slogdet(XTWX)
        if sign <= 0: logdet = 0

        ll = -0.5 * (np.sum(np.log(v + fixed_tau2 + 1e-8)) +
                     logdet +
                     np.sum((resid**2) * weights))

        return {
            'betas': final_model.params.values,
            'var_betas': final_model.cov_params().values,
            'tau_sq': fixed_tau2,
            'sigma_sq': 0.0, # Not applicable for aggregated model
            'log_lik_reml': ll,
            'mod_mean': mod_mean,
            'mod_std': mod_std,
            'formula': formula,
            'model_type': 'Aggregated Spline (Plug-in Tau²)',
            'X_design': X
        }, None
    except Exception as e:
        return None, f"Final Fit Error: {e}"

def _run_fixed_tau_spline(agg_df, moderator_col, effect_col, var_col, df_spline, mod_mean, mod_std, fixed_tau2):
    """
    Runs spline regression using a FIXED Tau^2 from the linear model.
    This prevents the optimizer from drifting into unrealistic variance estimates.
    """
    agg_df = agg_df.reset_index(drop=True)
    mod_z = (agg_df[moderator_col] - mod_mean) / mod_std
    formula = f"cr(x, df={df_spline}) - 1"

    try:
        basis_matrix = patsy.dmatrix(formula, {"x": mod_z}, return_type='dataframe')
    except Exception as e: return None, f"Basis Error: {e}"

    y = agg_df[effect_col].values
    v = agg_df[var_col].values

    # Align and create design matrix
    basis_matrix.index = agg_df.index
    X = sm.add_constant(basis_matrix)

    # FIT MODEL (No optimization needed - Tau2 is known!)
    weights = 1.0 / (v + fixed_tau2)

    try:
        final_model = sm.WLS(y, X, weights=weights).fit()

        # Calculate Log-Likelihood manually for validation
        resid = y - final_model.fittedvalues
        sign, logdet = np.linalg.slogdet(X.T @ np.diag(weights) @ X)
        if sign <= 0: logdet = 0
        ll = -0.5 * (np.sum(np.log(v + fixed_tau2)) + logdet + np.sum((resid**2) * weights))

        return {
            'betas': final_model.params.values,
            'var_betas': final_model.cov_params().values,
            'tau_sq': fixed_tau2,
            'sigma_sq': 0.0,
            'log_lik_reml': ll,
            'mod_mean': mod_mean, 'mod_std': mod_std,
            'formula': formula,
            'model_type': 'Aggregated Spline (Plug-in Tau²)',
            'X_design': X
        }, None
    except Exception as e:
        return None, f"Fit Error: {e}"

# --- 2. WIDGETS & LOGIC ---
header = widgets.HTML("<h3 style='color: #2E86AB;'>🌊 3-Level Spline Analysis</h3>")

def get_numeric_mods_robust(df):
    if df is None: return []
    valid_mods = []
    technical_cols = ['id', 'xe', 'xc', 'ne', 'nc', 'sde', 'sdc', 'w_fixed', 'w_random', 'df', 'sp', 'sp_squared', 'hedges_j', 'weights', 'wi']
    if 'ANALYSIS_CONFIG' in globals():
        technical_cols.extend([ANALYSIS_CONFIG.get('effect_col'), ANALYSIS_CONFIG.get('var_col'), ANALYSIS_CONFIG.get('se_col')])
    for col in df.columns:
        if col in technical_cols or col is None: continue
        if pd.api.types.is_numeric_dtype(df[col]): valid_mods.append(col)
        elif df[col].dtype == 'object':
            try:
                if pd.to_numeric(df[col], errors='coerce').notna().sum() >= 3: valid_mods.append(col)
            except: pass
    return sorted(list(set(valid_mods)))

def get_analysis_data():
    if 'analysis_data' in globals(): return analysis_data
    elif 'data_filtered' in globals(): return data_filtered
    else: return None

df_spline_in = get_analysis_data()
opts = get_numeric_mods_robust(df_spline_in) if df_spline_in is not None else ['Data not loaded']
mod_widget = widgets.Dropdown(options=opts, description='Moderator:', layout=widgets.Layout(width='400px'))
df_widget = widgets.IntSlider(value=3, min=3, max=6, description='df:', style={'description_width': 'initial'})
run_spline_btn = widgets.Button(description='▶ Run Spline Model', button_style='success', layout=widgets.Layout(width='400px'))
spline_output = widgets.Output()

def run_spline(b):
    global ANALYSIS_CONFIG
    with spline_output:
        clear_output(wait=True)

        if not PATSY_AVAILABLE: print("❌ Error: 'patsy' not installed."); return
        mod_col = mod_widget.value
        df_k = df_widget.value

        df_working = get_analysis_data()
        if df_working is None: print("❌ Data not found."); return
        if 'ANALYSIS_CONFIG' not in globals(): print("❌ Config not found."); return

        eff_col = ANALYSIS_CONFIG.get('effect_col', 'hedges_g')
        var_col = ANALYSIS_CONFIG.get('var_col', 'Vg')

        # --- GET TAU^2 FROM LINEAR MODEL (CELL 10) ---
        # This is the "Plug-in" magic
        fixed_tau2 = 0.1 # Default fallback

        if 'meta_regression_RVE_results' in ANALYSIS_CONFIG:
            reg_res = ANALYSIS_CONFIG['meta_regression_RVE_results']
            # Check if this is the same moderator
            if reg_res.get('moderator_col_name') == mod_col:
                # We calculate Tau2 from the betas/se of the linear model to be safe
                # Or ideally, we saved it.
                # Let's assume user ran Cell 10 on this moderator.
                # Cell 10 unfortunately didn't save 'tau_sq' explicitly in the config dict
                # BUT, we can re-run the linear aggregation quickly here to get it.
                pass

        # --- PREP DATA ---
        df = df_working.copy()
        df[mod_col] = pd.to_numeric(df[mod_col], errors='coerce')
        df = df.dropna(subset=[mod_col, eff_col, var_col])
        df = df[df[var_col] > 0]

        # --- AGGREGATION ---
        # We aggregate regardless to get a stable Tau^2
        df['wi'] = 1 / df[var_col]
        def agg_func(x):
            return pd.Series({
                eff_col: np.average(x[eff_col], weights=x['wi']),
                var_col: 1 / np.sum(x['wi']),
                mod_col: x[mod_col].iloc[0]
            })
        try: agg_df = df.groupby('id').apply(agg_func, include_groups=False).reset_index()
        except TypeError: agg_df = df.groupby('id').apply(agg_func).reset_index()

        # --- ESTIMATE STABLE TAU^2 (LINEAR) ---
        print(f"⚙️  Estimating stable baseline variance (Linear Model)...")
        # Simple REML on linear model to get a sane Tau^2
        X_lin = sm.add_constant(agg_df[mod_col])
        y_agg = agg_df[eff_col].values
        v_agg = agg_df[var_col].values

        def lin_nll(t2):
            if t2 < 0: t2 = 0
            w = 1/(v_agg + t2)
            try:
                res = sm.WLS(y_agg, X_lin, weights=w).fit()
                return -(-0.5*(np.sum(np.log(v_agg+t2)) + np.log(np.linalg.det(X_lin.T@np.diag(w)@X_lin)) + np.sum(res.resid**2 * w)))
            except: return np.inf

        opt_lin = minimize_scalar(lin_nll, bounds=(0, 100), method='bounded')
        fixed_tau2 = opt_lin.x
        print(f"   ✓ Using fixed Tau² = {fixed_tau2:.4f} (from Linear Meta-Regression)")

        # --- RUN SPLINE WITH FIXED TAU^2 ---
        print(f"🚀 Fitting Spline (df={df_k}) using fixed variance...")
        est, err = _run_aggregated_spline_re(agg_df, mod_col, eff_col, var_col, df_k, df[mod_col].mean(), df[mod_col].std(), fixed_tau2)

        if err: print(f"❌ {err}"); return

        # --- REPORTING ---
        print("\n" + "="*60)
        print("SPLINE MODEL RESULTS")
        print("="*60)
        print(f"Model Type: {est['model_type']}")
        print(f"  • Studies: {len(agg_df)}")
        print(f"  • Tau² (Fixed): {est['tau_sq']:.5f}")

        # Omnibus Test
        betas = est['betas']
        cov = est['var_betas']
        if len(betas) > 1:
            b_spline = betas[1:]
            cov_spline = cov[1:, 1:]
            try:
                chi2_stat = b_spline.T @ np.linalg.inv(cov_spline) @ b_spline
                df_test = len(b_spline)
                p_val = 1 - chi2.cdf(chi2_stat, df_test)
                print(f"\nOmnibus Test for Non-Linearity:")
                print(f"  • Chi2({df_test}) = {chi2_stat:.3f}")
                print(f"  • P-value = {p_val:.5f}")
                if p_val < 0.05: print("  ✅ Significant non-linear relationship.")
                else: print("  ℹ️  Not significant.")
            except: pass


        # Save Results
        ANALYSIS_CONFIG['spline_model_results'] = {
            'reg_df': agg_df, 'betas': betas, 'var_betas': cov,
            'tau_sq': est['tau_sq'], 'log_lik': est['log_lik_reml'],
            'mod_mean': est['mod_mean'], 'mod_std': est['mod_std'],
            'df_spline': df_k, 'moderator_col': mod_col, 'sigma_sq': 0.0,
            'formula': est['formula'], 'model_type': est['model_type']
        }

run_spline_btn.on_click(run_spline)
display(widgets.VBox([header, mod_widget, df_widget, run_spline_btn, spline_output]))

In [ ]:
#@title 📊 Cell 11b: Publication-Ready Spline Plot (Full Feature)
# =============================================================================
# CELL 11b: ADVANCED SPLINE PLOTTER
# Purpose: Visualize results from Cell 11 with full customization.
# Features: Tabs for Style, Points, Curve, Layout, and Label Editing.
# Compatibility: Works with the new Robust/Aggregated Spline results.
# =============================================================================

import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.stats import t
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output
import traceback
import patsy

# --- 1. INITIALIZATION & CONFIG LOADING ---
available_color_moderators = ['None']
analysis_data_init = None
default_x_label = "Moderator"
default_y_label = "Effect Size"
default_title = "Natural Cubic Spline Analysis"
label_widgets_dict = {}

try:
    if 'ANALYSIS_CONFIG' not in globals():
        raise NameError("ANALYSIS_CONFIG not found")

    # Get data for dropdowns
    if 'analysis_data' in globals():
        analysis_data_init = analysis_data.copy()
    elif 'data_filtered' in globals():
        analysis_data_init = data_filtered.copy()
    else:
        # Fallback to reg_df if main data missing
        if 'spline_model_results' in ANALYSIS_CONFIG:
            analysis_data_init = ANALYSIS_CONFIG['spline_model_results']['reg_df'].copy()

    # Load Defaults from Results
    if 'spline_model_results' in ANALYSIS_CONFIG:
        spline_results = ANALYSIS_CONFIG['spline_model_results']
        es_config = ANALYSIS_CONFIG.get('es_config', {})
        default_x_label = spline_results.get('moderator_col', 'Moderator')
        default_y_label = es_config.get('effect_label', 'Effect Size')
        default_title = f"Spline Regression: {default_y_label} vs. {default_x_label}"

    # Identify Categorical Moderators for Coloring
    if analysis_data_init is not None:
        excluded_cols = [
            ANALYSIS_CONFIG.get('effect_col'), ANALYSIS_CONFIG.get('var_col'),
            ANALYSIS_CONFIG.get('se_col'), 'w_fixed', 'w_random', 'id',
            'xe', 'sde', 'ne', 'xc', 'sdc', 'nc'
        ]

        for col in analysis_data_init.columns:
            if col in excluded_cols or col is None: continue
            # Check if categorical (object or category) and reasonable size
            if analysis_data_init[col].dtype == 'object' or isinstance(analysis_data_init[col].dtype, pd.CategoricalDtype):
                if analysis_data_init[col].nunique() <= 15: # Limit to reasonable number of colors
                    available_color_moderators.append(col)

    # Find unique labels for Editor
    all_categorical_labels = set()
    for col in available_color_moderators:
        if col != 'None' and col in analysis_data_init.columns:
            all_categorical_labels.add(col)
            unique_vals = analysis_data_init[col].astype(str).str.strip().unique()
            all_categorical_labels.update(unique_vals)

    all_categorical_labels.discard('')
    all_categorical_labels.discard('nan')

except Exception as e:
    print(f"⚠️  Initialization Warning: {e}")

# --- 2. WIDGET DEFINITIONS ---

# === TAB 1: STYLE ===
show_title_widget = widgets.Checkbox(value=True, description='Show Plot Title', indent=False)
title_widget = widgets.Text(value=default_title, description='Title:', layout=widgets.Layout(width='450px'))
xlabel_widget = widgets.Text(value=default_x_label, description='X Label:', layout=widgets.Layout(width='450px'))
ylabel_widget = widgets.Text(value=default_y_label, description='Y Label:', layout=widgets.Layout(width='450px'))
width_widget = widgets.FloatSlider(value=10.0, min=5.0, max=16.0, step=0.5, description='Width (in):', continuous_update=False)
height_widget = widgets.FloatSlider(value=6.0, min=4.0, max=12.0, step=0.5, description='Height (in):', continuous_update=False)

style_tab = widgets.VBox([
    widgets.HTML("<h4 style='color: #2E86AB;'>Labels & Dimensions</h4>"),
    show_title_widget, title_widget,
    xlabel_widget, ylabel_widget,
    widgets.HTML("<hr style='margin: 5px 0;'>"),
    width_widget, height_widget
])

# === TAB 2: POINTS ===
show_points_widget = widgets.Checkbox(value=True, description='Show Data Points', indent=False)
color_mod_widget = widgets.Dropdown(options=available_color_moderators, value='None', description='Color By:', layout=widgets.Layout(width='400px'))
point_color_widget = widgets.Dropdown(options=['gray', 'steelblue', 'black', 'red', 'green', 'purple'], value='gray', description='Color:')
point_size_widget = widgets.IntSlider(value=40, min=10, max=150, step=5, description='Size:')
point_alpha_widget = widgets.FloatSlider(value=0.5, min=0.1, max=1.0, step=0.1, description='Opacity:')

points_tab = widgets.VBox([
    widgets.HTML("<h4 style='color: #2E86AB;'>Data Points</h4>"),
    show_points_widget,
    color_mod_widget,
    point_color_widget,
    point_size_widget,
    point_alpha_widget
])

# === TAB 3: CURVE ===
curve_color_widget = widgets.Dropdown(options=['blue', 'red', 'black', 'green', 'purple'], value='blue', description='Line Color:')
curve_width_widget = widgets.FloatSlider(value=2.5, min=0.5, max=6.0, step=0.5, description='Line Width:')
show_ci_widget = widgets.Checkbox(value=True, description='Show 95% Confidence Band', indent=False)
ci_alpha_widget = widgets.FloatSlider(value=0.15, min=0.05, max=0.5, step=0.05, description='CI Opacity:')
show_stats_widget = widgets.Checkbox(value=True, description='Show Stats (P-value/R²)', indent=False)

curve_tab = widgets.VBox([
    widgets.HTML("<h4 style='color: #2E86AB;'>Spline Curve</h4>"),
    curve_color_widget, curve_width_widget,
    widgets.HTML("<hr style='margin: 5px 0;'>"),
    show_ci_widget, ci_alpha_widget,
    show_stats_widget
])

# === TAB 4: LAYOUT ===
show_grid_widget = widgets.Checkbox(value=True, description='Show Grid', indent=False)
show_null_line_widget = widgets.Checkbox(value=True, description='Show Null Line (y=0)', indent=False)
legend_loc_widget = widgets.Dropdown(options=['best', 'upper right', 'upper left', 'lower right', 'lower left', 'none'], value='best', description='Legend:')
save_pdf_widget = widgets.Checkbox(value=True, description='Save as PDF', indent=False)
save_png_widget = widgets.Checkbox(value=True, description='Save as PNG', indent=False)
filename_prefix_widget = widgets.Text(value='Spline_Plot', description='Filename:', layout=widgets.Layout(width='300px'))

layout_tab = widgets.VBox([
    widgets.HTML("<h4 style='color: #2E86AB;'>Layout & Export</h4>"),
    show_grid_widget, show_null_line_widget, legend_loc_widget,
    widgets.HTML("<hr style='margin: 5px 0;'>"),
    save_pdf_widget, save_png_widget, filename_prefix_widget
])

# === TAB 5: LABELS (Dynamic) ===
label_editor_widgets = []
label_widgets_dict = {}

if all_categorical_labels:
    for label in sorted(list(all_categorical_labels)):
        w = widgets.Text(value=str(label), description=f"{label}:", layout=widgets.Layout(width='400px'))
        label_editor_widgets.append(w)
        label_widgets_dict[str(label)] = w
else:
    label_editor_widgets.append(widgets.Label("No categorical labels found to edit."))

labels_tab = widgets.VBox([
    widgets.HTML("<h4 style='color: #2E86AB;'>Label Editor</h4>"),
    widgets.HTML("<i>Rename data categories for the legend:</i>"),
    *label_editor_widgets
])

# Assemble Tabs
tabs = widgets.Tab(children=[style_tab, points_tab, curve_tab, layout_tab, labels_tab])
tabs.set_title(0, '🎨 Style')
tabs.set_title(1, '⚫ Points')
tabs.set_title(2, '🌊 Curve')
tabs.set_title(3, '💾 Layout')
tabs.set_title(4, '✏️ Labels')

run_plot_btn = widgets.Button(
    description='📊 Generate Spline Plot',
    button_style='success',
    layout=widgets.Layout(width='450px', height='50px'),
    style={'font_weight': 'bold'}
)
plot_output = widgets.Output()

# --- 3. PLOTTING LOGIC ---
def generate_spline_plot(b):
    with plot_output:
        clear_output(wait=True)

        try:
            # 1. Load Results
            if 'ANALYSIS_CONFIG' not in globals() or 'spline_model_results' not in ANALYSIS_CONFIG:
                print("❌ Error: Please run the Spline Analysis (Cell 11) first.")
                return

            res = ANALYSIS_CONFIG['spline_model_results']
            df = res['reg_df'].copy() # Dataframe used in model

            # Extract Model info
            betas = res['betas']
            cov = res['var_betas']
            formula = res['formula']
            mod_mean = res['mod_mean']
            mod_std = res['mod_std']
            mod_col = res['moderator_col']
            eff_col = ANALYSIS_CONFIG['effect_col']

            # 2. Re-calculate Curve (High Resolution)
            x_min, x_max = df[mod_col].min(), df[mod_col].max()
            padding = (x_max - x_min) * 0.05
            x_grid = np.linspace(x_min - padding, x_max + padding, 200)
            x_grid_z = (x_grid - mod_mean) / mod_std

            # Generate Basis for Grid
            try:
                # We need to match the column structure of the model
                # The model might have dropped columns (collinearity), so we need to be careful.
                pred_matrix = patsy.dmatrix(formula, {"x": x_grid_z}, return_type='dataframe')
                X_pred_full = sm.add_constant(pred_matrix)

                # Filter columns to match what the model used
                # If 'kept_cols' or similar isn't saved, we assume simple match by length or name if possible
                # But simpler: matrix multiplication handles it if shapes match
                # Check shape
                if X_pred_full.shape[1] != len(betas):
                    # Try to align by column names if available, otherwise simple slice
                    if hasattr(res, 'get') and res.get('X_design_cols') is not None:
                        # Robust matching using saved column names
                        cols = res['X_design_cols']
                        # Make sure X_pred_full has these columns (it should if formula is same)
                        # Note: patsy names might differ slightly if not careful, but usually stable
                        X_pred = X_pred_full.values[:, :len(betas)] # Fallback
                    else:
                         # Fallback: Assume the first K columns are the ones kept
                         X_pred = X_pred_full.values[:, :len(betas)]
                else:
                    X_pred = X_pred_full.values

                # Calculate
                y_pred = X_pred @ betas
                pred_var = np.sum((X_pred @ cov) * X_pred, axis=1)
                pred_se = np.sqrt(pred_var)

                ci_lower = y_pred - 1.96 * pred_se
                ci_upper = y_pred + 1.96 * pred_se

            except Exception as e:
                print(f"❌ Error calculating curve: {e}")
                print("   (Did the model structure change?)")
                return

            # 3. Prepare Plot
            fig, ax = plt.subplots(figsize=(width_widget.value, height_widget.value))

            # Handle Colors & Labels
            color_col = color_mod_widget.value
            label_map = {k: v.value for k, v in label_widgets_dict.items()}

            # --- Plot Points ---
            if show_points_widget.value:
                if color_col != 'None' and color_col in analysis_data_init.columns:
                    # Merge color data back if not in reg_df (reg_df might be aggregated)
                    # If aggregated, we might lose the categorical info unless we merge back by ID
                    # For simplicity, we try to use what's in df

                    # Check if color_col exists in df, if not, try merge
                    plot_df = df
                    if color_col not in plot_df.columns:
                        # Try to recover color info from initial data
                        # This assumes 1-to-1 mapping if aggregated
                        temp_merge = analysis_data_init[['id', color_col]].drop_duplicates()
                        plot_df = plot_df.merge(temp_merge, on='id', how='left')

                    # Get unique categories
                    categories = plot_df[color_col].dropna().unique()
                    cmap = plt.get_cmap('tab10')

                    for i, cat in enumerate(categories):
                        cat_str = str(cat)
                        display_label = label_map.get(cat_str, cat_str)
                        mask = plot_df[color_col] == cat

                        ax.scatter(plot_df.loc[mask, mod_col], plot_df.loc[mask, eff_col],
                                  color=cmap(i % 10), alpha=point_alpha_widget.value,
                                  s=point_size_widget.value, label=display_label,
                                  edgecolors='k', linewidth=0.5)

                    # Legend title
                    legend_title = label_map.get(color_col, color_col)

                else:
                    # Single color
                    ax.scatter(df[mod_col], df[eff_col],
                              color=point_color_widget.value, alpha=point_alpha_widget.value,
                              s=point_size_widget.value, label='Observations',
                              edgecolors='k', linewidth=0.5)
                    legend_title = None

            # --- Plot Curve ---
            ax.plot(x_grid, y_pred, color=curve_color_widget.value,
                   linewidth=curve_width_widget.value, label='Spline Fit')

            if show_ci_widget.value:
                ax.fill_between(x_grid, ci_lower, ci_upper,
                               color=curve_color_widget.value, alpha=ci_alpha_widget.value,
                               label='95% CI')

            # --- Decoration ---
            if show_null_line_widget.value:
                ax.axhline(0, color='black', linestyle=':', linewidth=1.5, alpha=0.6)

            if show_grid_widget.value:
                ax.grid(True, linestyle=':', alpha=0.4)

            if show_title_widget.value:
                ax.set_title(title_widget.value, fontsize=14, fontweight='bold', pad=15)

            ax.set_xlabel(xlabel_widget.value, fontsize=12, fontweight='bold')
            ax.set_ylabel(ylabel_widget.value, fontsize=12, fontweight='bold')

            if legend_loc_widget.value != 'none':
                ax.legend(loc=legend_loc_widget.value, title=legend_title, frameon=True, fancybox=True)

            # Stats annotation
            if show_stats_widget.value:
                # Try to get stats from results
                p_val = res.get('f_pvalue', None)
                tau2 = res.get('tau_sq', None)

                stats_text = []
                if p_val is not None:
                    sig = "***" if p_val < 0.001 else "**" if p_val < 0.01 else "*" if p_val < 0.05 else "ns"
                    stats_text.append(f"P-value: {p_val:.4g} {sig}")
                if tau2 is not None:
                    stats_text.append(f"τ²: {tau2:.3f}")

                if stats_text:
                    ax.text(0.05, 0.95, "\n".join(stats_text), transform=ax.transAxes,
                           verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

            plt.tight_layout()

            # --- Export ---
            ts = datetime.datetime.now().strftime("%H%M%S")
            fn = filename_prefix_widget.value

            if save_pdf_widget.value:
                plt.savefig(f"{fn}_{ts}.pdf", bbox_inches='tight')
                print(f"💾 Saved: {fn}_{ts}.pdf")

            if save_png_widget.value:
                plt.savefig(f"{fn}_{ts}.png", dpi=png_dpi_widget.value, bbox_inches='tight')
                print(f"💾 Saved: {fn}_{ts}.png")

            plt.show()
            print(f"✅ Plot Generated (n={len(df)})")

        except Exception as e:
            print(f"❌ Plotting Error: {e}")
            traceback.print_exc()

run_plot_btn.on_click(generate_spline_plot)

# --- 4. DISPLAY ---
display(widgets.VBox([
    header,
    tabs,
    widgets.HTML("<hr style='margin: 10px 0;'>"),
    run_plot_btn,
    plot_output
]))

In [ ]:
#@title 🧪 Cell 11.5: Spline Validation (Robust Model Comparison)
# =============================================================================
# CELL: VALIDATION (Linear vs Spline)
# Purpose: Check if the complex Spline model is better than a simple Line.
# Fix: Now handles collinearity (dropping columns) to match Cell 11.
# =============================================================================

import statsmodels.api as sm
import numpy as np
import pandas as pd
import patsy
from IPython.display import display, HTML
from scipy.optimize import minimize_scalar
from scipy import linalg

# --- HELPER: Collinearity Remover (Must match Cell 11) ---
def remove_collinear_cols(X_df):
    try:
        X_np = X_df.values.astype(float)
        Q, R, P = linalg.qr(X_np, pivot=True)
        tol = np.finfo(float).eps * max(X_np.shape) * np.abs(R[0,0])
        rank = np.sum(np.abs(np.diag(R)) > tol)
        keep_idx = sorted(P[:rank])
        return X_df.iloc[:, keep_idx]
    except:
        return X_df

# 1. Get Data from Spline Result
if 'ANALYSIS_CONFIG' not in globals() or 'spline_model_results' not in ANALYSIS_CONFIG:
    print("❌ Error: Run Cell 11 (Spline Analysis) first.")
else:
    res_spline = ANALYSIS_CONFIG['spline_model_results']

    # Use the EXACT same dataframe
    df_agg = res_spline['reg_df'].copy()
    mod_col = res_spline['moderator_col']
    eff_col = ANALYSIS_CONFIG['effect_col']
    var_col = ANALYSIS_CONFIG['var_col']

    print(f"🚀 Running Validation on {len(df_agg)} studies...")
    print(f"   Comparing: Linear Model vs. Natural Cubic Spline (df={res_spline['df_spline']})")

    # --- MODEL 1: SPLINE (Re-verify Likelihood) ---
    tau_sq_spline = res_spline['tau_sq']

    # Re-create Spline Basis
    mod_z = (df_agg[mod_col] - res_spline['mod_mean']) / res_spline['mod_std']

    # Robust formula retrieval
    formula = res_spline.get('formula', f"cr(x, df={res_spline['df_spline']}) - 1")

    try:
        X_spline_basis = patsy.dmatrix(formula, {"x": mod_z}, return_type='dataframe')
        X_spline_full = sm.add_constant(X_spline_basis)

        # *** CRITICAL FIX: Remove collinear columns to match Cell 11 ***
        X_spline = remove_collinear_cols(X_spline_full)

    except Exception as e:
        print(f"⚠️ Warning: Could not recreate basis ({e}).")
        X_spline = None

    if X_spline is not None:
        # Calculate Spline LL
        y = df_agg[eff_col].values
        v = df_agg[var_col].values
        weights_spline = 1.0 / (v + tau_sq_spline + 1e-8)

        try:
            # Calculate using the Reduced Matrix (X_spline)
            model_spline = sm.WLS(y, X_spline, weights=weights_spline).fit()
            resid_spline = y - model_spline.fittedvalues

            # REML LogLik
            sign, logdet = np.linalg.slogdet(X_spline.T @ np.diag(weights_spline) @ X_spline)
            if sign <= 0: logdet = 0

            ll_spline = -0.5 * (np.sum(np.log(v + tau_sq_spline + 1e-8)) +
                                logdet +
                                np.sum(resid_spline**2 * weights_spline))

            # Recalculate k (parameters) based on KEPT columns
            k_spline = X_spline.shape[1] + 1 # betas + tau2

        except Exception as e:
            print(f"⚠️ Error calculating Spline LL: {e}")
            ll_spline = res_spline.get('log_lik', np.nan)
            k_spline = res_spline['df_spline'] + 2
    else:
        ll_spline = res_spline.get('log_lik', np.nan)
        k_spline = res_spline['df_spline'] + 2

    # Calculate AIC
    aic_spline = 2*k_spline - 2*ll_spline

    # --- MODEL 2: LINEAR (Benchmark) ---
    # Fit standard weighted least squares (Linear)
    X_lin = sm.add_constant(df_agg[mod_col])
    y = df_agg[eff_col].values
    v = df_agg[var_col].values

    # Function to find optimal Tau2 for Linear Model
    def linear_nll(tau2):
        if tau2 < 0: tau2 = 0
        w = 1.0 / (v + tau2 + 1e-8)
        try:
            res = sm.WLS(y, X_lin, weights=w).fit()
            sign, logdet = np.linalg.slogdet(X_lin.T @ np.diag(w) @ X_lin)
            if sign <= 0: return np.inf

            ll = -0.5 * (np.sum(np.log(v + tau2 + 1e-8)) +
                         logdet +
                         np.sum(res.resid**2 * w))
            return -ll
        except: return np.inf

    # Optimize Linear Model
    opt = minimize_scalar(linear_nll, bounds=(0, 100), method='bounded')
    ll_linear = -opt.fun
    k_linear = 2 + 1 # Intercept + Slope + Tau2
    aic_linear = 2*k_linear - 2*ll_linear

    # --- REPORT ---
    print("\n" + "="*60)
    print("MODEL COMPARISON REPORT")
    print("="*60)
    print(f"{'Model':<20} {'Log-Likelihood':<15} {'Params (k)':<12} {'AIC':<10}")
    print("-" * 60)
    print(f"{'Linear (Baseline)':<20} {ll_linear:<15.4f} {k_linear:<12} {aic_linear:<10.4f}")
    print(f"{'Spline (Your Model)':<20} {ll_spline:<15.4f} {k_spline:<12} {aic_spline:<10.4f}")

    print("-" * 60)
    diff = aic_linear - aic_spline

    if diff > 2:
        print(f"✅ VALIDATED: Spline model is better (AIC reduced by {diff:.2f})")
        print("   The non-linear pattern is strong enough to justify the extra complexity.")
    elif diff > -2:
        print(f"⚠️  INCONCLUSIVE: Models are statistically similar (AIC diff {abs(diff):.2f} < 2)")
        print("   The relationship might be linear. Check the plot visually.")
    else:
        print(f"❌ CHECK: Linear model fits better (AIC lower by {abs(diff):.2f})")
        print("   You might be overfitting. Consider reporting the linear regression instead.")

In [ ]:
#@title ⚖️ Cell 11.6: R Validation for Spline Analysis (Journal Proof)
# =============================================================================
# CELL: JOURNAL-GRADE VALIDATION
# Purpose: Prove that Python's Spline Optimizer matches R's metafor exactly.
# =============================================================================

import pandas as pd
import numpy as np
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
import patsy
pandas2ri.activate()

if 'ANALYSIS_CONFIG' not in globals() or 'spline_model_results' not in ANALYSIS_CONFIG:
    print("❌ Error: Run Spline Analysis (Cell 11) first.")
else:
    res_py = ANALYSIS_CONFIG['spline_model_results']
    df_orig = res_py['reg_df']
    eff_col = ANALYSIS_CONFIG['effect_col']
    var_col = ANALYSIS_CONFIG['var_col']

    print("🚀 Running R Validation for Spline Model...")
    print(f"   Model Type: {res_py.get('model_type', 'Unknown')}")

    # 1. Reconstruct Basis
    mod_z = (df_orig[res_py['moderator_col']] - res_py['mod_mean']) / res_py['mod_std']
    formula = res_py['formula']
    basis_matrix = patsy.dmatrix(formula, {"x": mod_z}, return_type='dataframe')

    # 2. Prepare Data for R
    df_r = df_orig[['id', eff_col, var_col]].copy()
    spline_cols = []
    for i in range(basis_matrix.shape[1]):
        col_name = f'spline_basis_{i+1}'
        df_r[col_name] = basis_matrix.iloc[:, i].values
        spline_cols.append(col_name)

    ro.globalenv['df_python'] = df_r
    ro.globalenv['eff_col_name'] = eff_col
    ro.globalenv['var_col_name'] = var_col
    mods_formula = " + ".join(spline_cols)

    # 3. R Script
    r_script = f"""
    library(metafor)
    dat <- df_python
    is_aggregated <- nrow(dat) == length(unique(dat$id))

    if (is_aggregated) {{
        res <- rma(yi={eff_col}, vi={var_col}, mods = ~ {mods_formula},
                   data=dat, method="REML",
                   control=list(optimizer="optim", optmethod="Nelder-Mead"))
        tau2 <- res$tau2
        sigma2 <- 0
    }} else {{
        dat$rows <- 1:nrow(dat)
        res <- rma.mv(yi={eff_col}, V={var_col}, mods = ~ {mods_formula},
                      random = ~ 1 | id/rows, data=dat, method="REML",
                      control=list(optimizer="optim", optmethod="Nelder-Mead"))
        tau2 <- res$sigma2[1]
        sigma2 <- res$sigma2[2]
    }}
    list(ll = as.numeric(logLik(res)), tau2 = tau2, sigma2 = sigma2)
    """

    try:
        r_res = ro.r(r_script)
        r_ll = r_res.rx2('ll')[0]
        r_tau2 = r_res.rx2('tau2')[0]
        r_sigma2 = r_res.rx2('sigma2')[0]

        py_ll = res_py['log_lik']
        py_tau2 = res_py['tau_sq']
        py_sigma2 = res_py.get('sigma_sq', 0.0)

        print("\n" + "="*60)
        print("VALIDATION REPORT (JOURNAL PROOF)")
        print("="*60)
        print(f"{'Metric':<20} {'Python':<12} {'R (metafor)':<12} {'Diff':<12}")
        print("-" * 60)

        def diff(a, b): return f"{abs(a-b):.2e}"
        print(f"{'Log-Likelihood':<20} {py_ll:<12.4f} {r_ll:<12.4f} {diff(py_ll, r_ll):<12}")
        print(f"{'Tau² (L3)':<20} {py_tau2:<12.4f} {r_tau2:<12.4f} {diff(py_tau2, r_tau2):<12}")

        if res_py.get('model_type', '').startswith('3-Level'):
             print(f"{'Sigma² (L2)':<20} {py_sigma2:<12.4f} {r_sigma2:<12.4f} {diff(py_sigma2, r_sigma2):<12}")

        if abs(py_ll - r_ll) < 0.1:
            print("\n✅ PERFECT MATCH: Python results are validated against R.")
        else:
            print("\n⚠️  CHECK: Minor differences found.")

    except Exception as e:
        print(f"❌ R Error: {e}")

In [ ]:
#@title 📉 Step 5: Publication Bias Diagnostics (Egger's Test & Trim-and-Fill)

# =============================================================================
# PUBLICATION BIAS DIAGNOSTICS - ANALYSIS ONLY
#
# This cell runs two complementary tests for publication bias:
#   1. Egger's Regression Test - Tests for funnel plot asymmetry
#   2. Trim-and-Fill Analysis - Estimates number of missing studies
#
# Note: Use Cells 12b and 14b for visualizations (funnel plots)
#
# Outputs:
#   - ANALYSIS_CONFIG['funnel_results']: Egger's test results
#   - ANALYSIS_CONFIG['trimfill_results']: Trim-and-fill results
# =============================================================================

import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.stats import norm, t, rankdata
import statsmodels.api as sm
import datetime
from scipy.optimize import minimize
import warnings

# =============================================================================
# SECTION 1: EGGER'S REGRESSION TEST
# =============================================================================

def _neg_log_lik_reml_reg(params, y_all, v_all, X_all, N_total, M_studies, p_params):
    est = _get_three_level_regression_estimates_v2(params, y_all, v_all, X_all, N_total, M_studies, p_params)
    return -est['log_lik_reml']

def _run_robust_eggers_test(analysis_data, effect_col, var_col, se_col):
    """Runs Egger's Test using 3-Level Meta-Regression on SE."""
    grouped = analysis_data.groupby('id')
    y_all, v_all, X_all = [], [], []

    for _, group in grouped:
        y_all.append(group[effect_col].values)
        v_all.append(group[var_col].values)
        # Egger's Predictor: Standard Error
        X_i = sm.add_constant(group[se_col].values, prepend=True)
        X_all.append(X_i)

    N_total = len(analysis_data)
    M_studies = len(y_all)
    p_params = 2

    # Optimization (Global + Polishing)
    best_res = None
    best_fun = np.inf
    start_points = [[0.1, 0.1], [1.0, 0.1], [5.0, 0.1]] # Broad search

    for start in start_points:
        res = minimize(_neg_log_lik_reml_reg, x0=start, args=(y_all, v_all, X_all, N_total, M_studies, p_params),
                       method='L-BFGS-B', bounds=[(1e-8, None), (1e-8, None)], options={'ftol': 1e-10})
        if res.success and res.fun < best_fun:
            best_fun = res.fun; best_res = res

    if not best_res: return None

    final_res = minimize(_neg_log_lik_reml_reg, x0=best_res.x, args=(y_all, v_all, X_all, N_total, M_studies, p_params),
                         method='Nelder-Mead', bounds=[(1e-8, None), (1e-8, None)], options={'xatol': 1e-10, 'fatol': 1e-10})

    return _get_three_level_regression_estimates_v2(final_res.x, y_all, v_all, X_all, N_total, M_studies, p_params)


# Run Egger's Test
print("\n" + "="*70)
print("EGGER'S REGRESSION TEST")
print("="*70)

try:
    if 'ANALYSIS_CONFIG' not in globals():
        raise NameError("ANALYSIS_CONFIG not found. Run previous cells first.")

    if 'three_level_results' not in ANALYSIS_CONFIG:
        print("⚠️  Three-level results not found. Run Step 2 first.")
    else:
        effect_col = ANALYSIS_CONFIG.get('effect_col', 'hedges_g')
        var_col = ANALYSIS_CONFIG.get('var_col', 'Vg')
        se_col = ANALYSIS_CONFIG.get('se_col', 'SE_g')

        if 'analysis_data' in ANALYSIS_CONFIG:
            df_plot = ANALYSIS_CONFIG['analysis_data']
        elif 'data_filtered' in globals():
            df_plot = data_filtered
        else:
            raise ValueError("No analysis data found")

        print(f"\n📊 Running Egger's test on {len(df_plot)} observations...")

        est = _run_robust_eggers_test(df_plot, effect_col, var_col, se_col)

        if est:
            # Extract Egger's test results
            intercept = est['betas'][0]
            se_intercept = est['se_betas'][0]
            # Calculate p-value from t-statistic
            t_stat = intercept / se_intercept
            from scipy.stats import t as t_dist
            df = est.get('df', 100)  # degrees of freedom
            p_value = 2 * (1 - t_dist.cdf(abs(t_stat), df))

            print(f"\n✅ Egger's Test Results:")
            print(f"  Intercept: {intercept:.4f}")
            print(f"  SE: {se_intercept:.4f}")
            print(f"  p-value: {p_value:.4g}")

            if p_value < 0.05:
                print(f"\n  ⚠️  Significant asymmetry detected (p < 0.05)")
                print(f"  Interpretation: Evidence of potential publication bias")
            elif p_value < 0.10:
                print(f"\n  ⚡ Moderate evidence of asymmetry (p < 0.10)")
                print(f"  Interpretation: Some evidence of potential publication bias")
            else:
                print(f"\n  ✓ No significant asymmetry detected (p ≥ 0.10)")
                print(f"  Interpretation: No strong evidence of publication bias")

            # Save results
            ANALYSIS_CONFIG['funnel_results'] = {
                'timestamp': datetime.datetime.now(),
                'intercept': intercept,
                'se': se_intercept,
                'p_value': p_value,
                'estimates': est
            }
            print(f"\n  ✓ Results saved to ANALYSIS_CONFIG['funnel_results']")
        else:
            print("\n❌ Egger's test failed")
except Exception as e:
    print(f"\n❌ Error running Egger's test: {e}")
    import traceback
    traceback.print_exc()

# =============================================================================
# SECTION 2: TRIM-AND-FILL ANALYSIS
# =============================================================================


def trimfill_analysis(data, effect_col, var_col, estimator='L0', side='auto', max_iter=100):
    """Duval & Tweedie Trim-and-Fill Method (Matching metafor)."""

    # Prepare data
    yi = data[effect_col].values
    vi = data[var_col].values
    ni = len(yi)

    # Sort data by effect size (crucial for correct trimming)
    sort_indices = np.argsort(yi)
    yi = yi[sort_indices]
    vi = vi[sort_indices]

    # 1. Determine Side (if auto)
    if side == 'auto':
        # Simple skewness check of the funnel
        wi = 1/vi
        pooled_fe = np.sum(wi * yi) / np.sum(wi)
        skew = np.sum(wi * (yi - pooled_fe)**3)
        side = 'left' if skew > 0 else 'right'

    # 2. Iterative Estimator for k0 (Number of missing studies)
    k0 = 0
    iter_safe = 0

    while iter_safe < max_iter:
        # a. Estimate center (Fixed Effect) using remaining data
        n_curr = ni - k0

        if side == 'left':
            # Missing left -> Trim right (largest values)
            yi_curr = yi[:n_curr]
            vi_curr = vi[:n_curr]
        else: # right
            # Missing right -> Trim left (smallest values)
            yi_curr = yi[k0:]
            vi_curr = vi[k0:]

        wi_curr = 1 / vi_curr
        pooled_fe = np.sum(wi_curr * yi_curr) / np.sum(wi_curr)

        # b. Calculate ranks of absolute residuals
        residuals = yi - pooled_fe

        if side == 'left':
            signed_res = residuals
        else:
            signed_res = -residuals

        abs_res = np.abs(signed_res)
        ranks = rankdata(abs_res, method='average')

        # Sum of ranks for the "positive" (excess) side
        pos_ranks = np.where(signed_res > 0, ranks, 0)
        Sn = np.sum(pos_ranks)

        # c. Calculate k0 (L0 estimator)
        k0_new = int(round((4 * Sn - ni * (ni + 1)) / (2 * ni - 1)))
        k0_new = max(0, k0_new)

        if k0_new == k0:
            break

        k0 = k0_new
        k0 = min(k0, ni - 2)
        iter_safe += 1

    # 3. Fill Data
    if k0 > 0:
        if side == 'left':
            # Excess is on the right (largest values)
            idx_fill = slice(ni - k0, ni)
        else:
            # Excess is on the left (smallest values)
            idx_fill = slice(0, k0)

        yi_excess = yi[idx_fill]
        vi_excess = vi[idx_fill]

        # Mirror them across the pooled estimate
        yi_filled = 2 * pooled_fe - yi_excess
        vi_filled = vi_excess

        # Combine
        yi_final = np.concatenate([yi, yi_filled])
        vi_final = np.concatenate([vi, vi_filled])
    else:
        yi_final = yi
        vi_final = vi
        yi_filled = []
        vi_filled = []

    # 4. Final Pooled Estimate (Fixed Effect on filled data)
    wi_final = 1 / vi_final
    pooled_final = np.sum(wi_final * yi_final) / np.sum(wi_final)
    var_final = 1 / np.sum(wi_final)
    se_final = np.sqrt(var_final)

    # Original Estimate
    wi_orig = 1 / vi
    pooled_orig = np.sum(wi_orig * yi) / np.sum(wi_orig)
    se_orig = np.sqrt(1 / np.sum(wi_orig))

    return {
        'k0': k0,
        'side': side,
        'pooled_original': pooled_orig,
        'se_original': se_orig,
        'ci_lower_original': pooled_orig - 1.96*se_orig,
        'ci_upper_original': pooled_orig + 1.96*se_orig,
        'pooled_filled': pooled_final,
        'se_filled': se_final,
        'ci_lower_filled': pooled_final - 1.96*se_final,
        'ci_upper_filled': pooled_final + 1.96*se_final,
        'yi_filled': yi_filled,
        'vi_filled': vi_filled if k0 > 0 else [],
        'yi_combined': yi_final,  # <--- ADDED THIS
        'vi_combined': vi_final   # <--- ADDED THIS
    }


# Run Trim-and-Fill
print("\n" + "="*70)
print("TRIM-AND-FILL ANALYSIS")
print("="*70)

try:
    if 'ANALYSIS_CONFIG' not in globals():
        raise NameError("ANALYSIS_CONFIG not found")

    eff_col = ANALYSIS_CONFIG['effect_col']
    var_col = ANALYSIS_CONFIG['var_col']

    if 'analysis_data' in ANALYSIS_CONFIG:
        df = ANALYSIS_CONFIG['analysis_data']
    elif 'data_filtered' in globals():
        df = data_filtered
    else:
        raise ValueError("No analysis data found")

    print(f"\n📊 Running Trim-and-Fill on {len(df)} observations...")

    res = trimfill_analysis(df, eff_col, var_col, side='auto')

    print(f"\n✅ Trim-and-Fill Results:")
    print(f"  Missing studies (k₀): {res['k0']}")
    print(f"  Side: {res['side']}")
    print(f"  Original effect: {res['pooled_original']:.4f}")
    print(f"  Adjusted effect: {res['pooled_filled']:.4f}")
    print(f"  Difference: {res['pooled_filled'] - res['pooled_original']:.4f}")

    if res['k0'] == 0:
        print(f"\n  ✓ No missing studies detected")
        print(f"  Interpretation: No evidence of publication bias via trim-and-fill")
    else:
        pct_change = abs((res['pooled_filled'] - res['pooled_original']) / res['pooled_original']) * 100
        print(f"\n  ⚠️  {res['k0']} missing studies estimated")
        print(f"  Effect changes by {pct_change:.1f}% after adjustment")
        if pct_change > 10:
            print(f"  Interpretation: Substantial evidence of publication bias")
        else:
            print(f"  Interpretation: Mild evidence of publication bias")

    # Save results
    ANALYSIS_CONFIG['trimfill_results'] = res
    print(f"\n  ✓ Results saved to ANALYSIS_CONFIG['trimfill_results']")

except Exception as e:
    print(f"\n❌ Error running Trim-and-Fill: {e}")
    import traceback
    traceback.print_exc()

# =============================================================================
# COMBINED INTERPRETATION
# =============================================================================

print("\n" + "="*70)
print("COMBINED INTERPRETATION")
print("="*70)

if 'funnel_results' in ANALYSIS_CONFIG and 'trimfill_results' in ANALYSIS_CONFIG:
    egger_p = ANALYSIS_CONFIG['funnel_results']['p_value']
    tf_k0 = ANALYSIS_CONFIG['trimfill_results']['k0']

    egger_bias = egger_p < 0.10
    tf_bias = tf_k0 > 0

    if egger_bias and tf_bias:
        print("⚠️  BOTH tests suggest publication bias")
        print("   Strong evidence for publication bias")
        print("   → Interpret main results with caution")
        print("   → Consider using adjusted estimates")
    elif egger_bias or tf_bias:
        print("⚡ ONE test suggests publication bias")
        print("   Moderate evidence for publication bias")
        print("   → Results should be interpreted carefully")
    else:
        print("✓ NEITHER test suggests publication bias")
        print("   Low risk of publication bias")
        print("   → Main results appear robust")

    print(f"\n💡 Note: Use Cells 12b and 14b to generate funnel plots for visualization")
else:
    print("⚠️  Results incomplete. Check errors above.")

print("\n" + "="*70)


In [ ]:
#@title 📊 Cell 12b: Publication-Ready Funnel Plot
# =============================================================================
# CELL 12b: ADVANCED FUNNEL PLOTTER
# Purpose: Visualize publication bias with full customization.
# Features: Tabs for Style, Points, Contours, and Export.
# =============================================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import ipywidgets as widgets
from IPython.display import display, clear_output
import datetime
import traceback

# --- 1. INITIALIZATION ---
default_title = "Funnel Plot"
default_xlabel = "Effect Size"
default_ylabel = "Standard Error"

try:
    if 'ANALYSIS_CONFIG' in globals():
        es_config = ANALYSIS_CONFIG.get('es_config', {})
        default_xlabel = es_config.get('effect_label', 'Effect Size')
        if 'funnel_results' in ANALYSIS_CONFIG:
            default_title = "Funnel Plot with Pseudo-95% CI"
except: pass

# --- 2. WIDGET DEFINITIONS ---

# === TAB 1: STYLE ===
show_title_widget = widgets.Checkbox(value=True, description='Show Plot Title', indent=False)
title_widget = widgets.Text(value=default_title, description='Title:', layout=widgets.Layout(width='450px'))
xlabel_widget = widgets.Text(value=default_xlabel, description='X Label:', layout=widgets.Layout(width='450px'))
ylabel_widget = widgets.Text(value=default_ylabel, description='Y Label:', layout=widgets.Layout(width='450px'))
width_widget = widgets.FloatSlider(value=10.0, min=5.0, max=16.0, step=0.5, description='Width (in):', continuous_update=False)
height_widget = widgets.FloatSlider(value=7.0, min=4.0, max=12.0, step=0.5, description='Height (in):', continuous_update=False)

style_tab = widgets.VBox([
    widgets.HTML("<h4 style='color: #2E86AB;'>Labels & Dimensions</h4>"),
    show_title_widget, title_widget,
    xlabel_widget, ylabel_widget,
    widgets.HTML("<hr style='margin: 5px 0;'>"),
    width_widget, height_widget
])

# === TAB 2: POINTS ===
point_color_widget = widgets.Dropdown(options=['gray', 'steelblue', 'black', 'red', 'purple'], value='gray', description='Color:')
point_size_widget = widgets.IntSlider(value=40, min=10, max=150, step=5, description='Size:')
point_alpha_widget = widgets.FloatSlider(value=0.6, min=0.1, max=1.0, step=0.1, description='Opacity:')
point_shape_widget = widgets.Dropdown(options=[('Circle', 'o'), ('Diamond', 'D'), ('Square', 's')], value='o', description='Shape:')

points_tab = widgets.VBox([
    widgets.HTML("<h4 style='color: #2E86AB;'>Data Points</h4>"),
    point_color_widget,
    point_size_widget,
    point_alpha_widget,
    point_shape_widget
])

# === TAB 3: LINES & CONTOURS ===
show_center_widget = widgets.Checkbox(value=True, description='Show Pooled Effect Line', indent=False)
center_color_widget = widgets.Dropdown(options=['red', 'black', 'blue'], value='red', description='Center Color:')
show_ci_widget = widgets.Checkbox(value=True, description='Show 95% CI Funnel', indent=False)
ci_fill_widget = widgets.Checkbox(value=True, description='Fill CI Region', indent=False)
show_contours_widget = widgets.Checkbox(value=False, description='Show Significance Contours (p<0.05/0.01)', indent=False)

lines_tab = widgets.VBox([
    widgets.HTML("<h4 style='color: #2E86AB;'>Reference Lines</h4>"),
    show_center_widget, center_color_widget,
    widgets.HTML("<hr style='margin: 5px 0;'>"),
    show_ci_widget, ci_fill_widget,
    show_contours_widget
])

# === TAB 4: LAYOUT & EXPORT ===
show_grid_widget = widgets.Checkbox(value=True, description='Show Grid', indent=False)
show_stats_widget = widgets.Checkbox(value=True, description="Show Egger's Test Result", indent=False)
legend_loc_widget = widgets.Dropdown(options=['best', 'upper right', 'upper left', 'lower right', 'lower left', 'none'], value='upper right', description='Legend:')
save_pdf_widget = widgets.Checkbox(value=True, description='Save as PDF', indent=False)
save_png_widget = widgets.Checkbox(value=True, description='Save as PNG', indent=False)
filename_prefix_widget = widgets.Text(value='Funnel_Plot', description='Filename:', layout=widgets.Layout(width='300px'))

layout_tab = widgets.VBox([
    widgets.HTML("<h4 style='color: #2E86AB;'>Layout & Export</h4>"),
    show_grid_widget, show_stats_widget, legend_loc_widget,
    widgets.HTML("<hr style='margin: 5px 0;'>"),
    save_pdf_widget, save_png_widget, filename_prefix_widget
])

# Assemble Tabs
tabs = widgets.Tab(children=[style_tab, points_tab, lines_tab, layout_tab])
tabs.set_title(0, '🎨 Style')
tabs.set_title(1, '⚫ Points')
tabs.set_title(2, '📐 Lines')
tabs.set_title(3, '💾 Export')

run_plot_btn = widgets.Button(
    description='📊 Generate Funnel Plot',
    button_style='success',
    layout=widgets.Layout(width='450px', height='50px'),
    style={'font_weight': 'bold'}
)
plot_output = widgets.Output()

# Header
header = widgets.HTML(
    "<h3 style='color: #2E86AB; margin-bottom: 10px;'>📊 Funnel Plot - Interactive Plotter</h3>"
    "<p style='color: #555; margin-top: 0;'>Customize the plot using the tabs below, then click Generate to create the visualization.</p>"
)

# --- 3. PLOTTING LOGIC ---
def generate_funnel_plot(b):
    with plot_output:
        clear_output(wait=True)

        try:
            # 1. Load Data & Config
            if 'ANALYSIS_CONFIG' not in globals():
                print("❌ Error: Config not found.")
                return

            if 'analysis_data' in globals(): df = analysis_data.copy()
            elif 'data_filtered' in globals(): df = data_filtered.copy()
            else: print("❌ Data not found."); return

            if 'three_level_results' not in ANALYSIS_CONFIG:
                print("❌ Error: Run Cell 6.5 (Three-Level Analysis) first.")
                return

            eff_col = ANALYSIS_CONFIG.get('effect_col', 'hedges_g')
            se_col = ANALYSIS_CONFIG.get('se_col', 'SE_g')

            # Get Pooled Effect (Center Line)
            pooled_effect = ANALYSIS_CONFIG['three_level_results']['pooled_effect']

            # Clean Data
            df = df.dropna(subset=[eff_col, se_col])
            df = df[df[se_col] > 0]

            # 2. Prepare Plot
            fig, ax = plt.subplots(figsize=(width_widget.value, height_widget.value))

            # Max SE for Y-axis limit
            max_se = df[se_col].max() * 1.1
            y_range = np.linspace(0, max_se, 100)

            # --- Plot Funnel Lines ---
            # 95% CI: +/- 1.96 * SE
            x_left = pooled_effect - 1.96 * y_range
            x_right = pooled_effect + 1.96 * y_range

            if show_ci_widget.value:
                ax.plot(x_left, y_range, color='gray', linestyle='--', linewidth=1.5, alpha=0.7)
                ax.plot(x_right, y_range, color='gray', linestyle='--', linewidth=1.5, alpha=0.7)

                if ci_fill_widget.value:
                    ax.fill_betweenx(y_range, x_left, x_right, color='gray', alpha=0.1, label='95% CI Region')

            # --- Plot Significance Contours ---
            if show_contours_widget.value:
                # 99% CI: +/- 2.58 * SE
                x_99_left = pooled_effect - 2.58 * y_range
                x_99_right = pooled_effect + 2.58 * y_range

                ax.plot(x_99_left, y_range, color='gray', linestyle=':', linewidth=1, alpha=0.5)
                ax.plot(x_99_right, y_range, color='gray', linestyle=':', linewidth=1, alpha=0.5, label='99% CI Region')

            # --- Plot Points ---
            ax.scatter(df[eff_col], df[se_col],
                      c=point_color_widget.value,
                      s=point_size_widget.value,
                      alpha=point_alpha_widget.value,
                      marker=point_shape_widget.value,
                      edgecolors='black', linewidth=0.5,
                      label='Studies', zorder=3)

            # --- Plot Center Line ---
            if show_center_widget.value:
                ax.axvline(pooled_effect, color=center_color_widget.value, linestyle='-', linewidth=2,
                          label=f'Pooled Effect ({pooled_effect:.3f})', zorder=2)

            # --- Axis Customization ---
            ax.set_ylim(max_se, 0) # Invert Y-axis (standard for funnel plots)

            if show_title_widget.value:
                ax.set_title(title_widget.value, fontsize=14, fontweight='bold', pad=15)
            ax.set_xlabel(xlabel_widget.value, fontsize=12, fontweight='bold')
            ax.set_ylabel(ylabel_widget.value, fontsize=12, fontweight='bold')

            if show_grid_widget.value:
                ax.grid(True, linestyle=':', alpha=0.4)

            if legend_loc_widget.value != 'none':
                ax.legend(loc=legend_loc_widget.value, frameon=True, fancybox=True)

            # --- Show Egger's Test Stats ---
            if show_stats_widget.value and 'funnel_results' in ANALYSIS_CONFIG:
                res_funnel = ANALYSIS_CONFIG['funnel_results']
                if res_funnel.get('egger_p') is not None:
                    p_val = res_funnel['egger_p']
                    beta = res_funnel.get('beta_slope', 0)
                    sig = "***" if p_val < 0.001 else "**" if p_val < 0.01 else "*" if p_val < 0.05 else "ns"

                    stats_text = f"Egger's Test:\nSlope = {beta:.3f}\np = {p_val:.4f} {sig}"

                    # Place text in bottom right (since Y is inverted, bottom is large SE)
                    # We use axes coordinates: (0.95, 0.05) is bottom-right
                    ax.text(0.95, 0.05, stats_text, transform=ax.transAxes,
                           ha='right', va='bottom', fontsize=10,
                           bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

            plt.tight_layout()

            # --- Export ---
            ts = datetime.datetime.now().strftime("%H%M%S")
            fn = filename_prefix_widget.value

            if save_pdf_widget.value:
                plt.savefig(f"{fn}_{ts}.pdf", bbox_inches='tight')
                print(f"💾 Saved: {fn}_{ts}.pdf")

            if save_png_widget.value:
                plt.savefig(f"{fn}_{ts}.png", dpi=png_dpi_widget.value, bbox_inches='tight')
                print(f"💾 Saved: {fn}_{ts}.png")

            plt.show()

        except Exception as e:
            print(f"❌ Plotting Error: {e}")
            traceback.print_exc()

run_plot_btn.on_click(generate_funnel_plot)

display(widgets.VBox([
    header,
    tabs,
    widgets.HTML("<hr style='margin: 10px 0;'>"),
    run_plot_btn,
    plot_output
]))

In [ ]:
#@title ⚖️ Cell 12.5: R Validation for Egger's Test
# =============================================================================
# CELL: R VALIDATION FOR EGGER'S TEST
# Purpose: Verify the robust bias assessment against R's metafor package.
# =============================================================================

import pandas as pd
import numpy as np
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
pandas2ri.activate()

# --- 1. Prepare Data ---
if 'analysis_data' in globals():
    df_bias_check = analysis_data.copy()
elif 'data_filtered' in globals():
    df_bias_check = data_filtered.copy()
else:
    print("❌ Error: Data not found.")
    df_bias_check = None

if df_bias_check is not None:
    # Get columns from config or defaults
    if 'ANALYSIS_CONFIG' in globals():
        eff_col = ANALYSIS_CONFIG.get('effect_col', 'hedges_g')
        var_col = ANALYSIS_CONFIG.get('var_col', 'Vg')
        se_col = ANALYSIS_CONFIG.get('se_col', 'SE_g')
    else:
        eff_col = 'hedges_g'; var_col = 'Vg'; se_col = 'SE_g'

    print(f"🚀 Running R Validation for Robust Egger's Test...")
    print(f"   Model: {eff_col} ~ {se_col} (random = ~1|study/id)")

    # Clean data for R
    df_r = df_bias_check[['id', eff_col, var_col, se_col]].dropna()
    ro.globalenv['df_python'] = df_r

    # Pass column names to R
    ro.globalenv['eff_col_name'] = eff_col
    ro.globalenv['var_col_name'] = var_col
    ro.globalenv['se_col_name'] = se_col

    # --- 2. R Script ---
    r_script = f"""
    library(metafor)

    dat <- df_python
    dat$rows <- 1:nrow(dat)
    dat$study_id <- as.factor(dat$id)

    # Run Egger's Test (Meta-Regression on SE)
    # Note: Standard Egger's is often weighted by 1/SE^2 (which is V)
    # We use rma.mv to replicate the 3-level structure

    res <- rma.mv(yi=dat[[eff_col_name]], V=dat[[var_col_name]],
                  mods = ~ dat[[se_col_name]],
                  random = ~ 1 | study_id/rows,
                  data=dat,
                  control=list(optimizer="optim", optmethod="Nelder-Mead"))

    list(
        intercept = res$b[1],
        slope = res$b[2],
        se_slope = res$se[2],
        pval_slope = res$pval[2]
    )
    """

    try:
        # Run R
        r_res = ro.r(r_script)

        r_int = r_res.rx2('intercept')[0]
        r_slope = r_res.rx2('slope')[0]
        r_se = r_res.rx2('se_slope')[0]
        r_pval = r_res.rx2('pval_slope')[0]

        print("\n" + "="*60)
        print("VALIDATION REPORT (EGGER'S TEST)")
        print("="*60)

        # Retrieve Python Results if available
        py_slope = "N/A"
        py_pval = "N/A"

        if 'ANALYSIS_CONFIG' in globals() and 'funnel_results' in ANALYSIS_CONFIG:
            fr = ANALYSIS_CONFIG['funnel_results']
            if fr.get('beta_slope') is not None:
                py_slope = fr['beta_slope']
                py_pval = fr['egger_p']

        print(f"{'Metric':<20} {'Python':<12} {'R (metafor)':<12} {'Diff':<12}")
        print("-" * 60)

        # Format helpers
        def fmt(x): return f"{x:.4f}" if isinstance(x, (float, int)) else str(x)
        def diff(p, r): return f"{abs(p-r):.2e}" if isinstance(p, (float, int)) else "-"

        print(f"{'Slope (Asymmetry)':<20} {fmt(py_slope):<12} {fmt(r_slope):<12} {diff(py_slope, r_slope):<12}")
        print(f"{'P-value':<20} {fmt(py_pval):<12} {fmt(r_pval):<12} {diff(py_pval, r_pval):<12}")

        print("-" * 60)
        print(f"R Intercept (Bias): {r_int:.4f}")

        if isinstance(py_pval, float) and abs(py_pval - r_pval) < 1e-3:
            print("\n✅ PASSED: Robust Egger's test matches R.")
        elif py_slope == "N/A":
            print("\n⚠️  NOTE: Run Cell 12 first to generate Python results for comparison.")
        else:
            print("\n⚠️  CHECK: Minor differences in optimization (acceptable if < 0.01).")

    except Exception as e:
        print(f"\n❌ R Error: {e}")

In [ ]:
#@title 📊 Cell 14b: Publication-Ready Trim-and-Fill Plot
# =============================================================================
# CELL 14b: ADVANCED TRIM-AND-FILL PLOTTER
# Purpose: Visualize publication bias sensitivity with full customization.
# Features: Highlight imputed studies, compare original vs. adjusted effects.
# =============================================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import ipywidgets as widgets
from IPython.display import display, clear_output
import datetime
import traceback

# --- 1. INITIALIZATION ---
default_title = "Trim-and-Fill Funnel Plot"
default_xlabel = "Effect Size"
default_ylabel = "Standard Error"

try:
    if 'ANALYSIS_CONFIG' in globals():
        es_config = ANALYSIS_CONFIG.get('es_config', {})
        default_xlabel = es_config.get('effect_label', 'Effect Size')
except: pass

# --- 2. WIDGET DEFINITIONS ---

# === TAB 1: STYLE ===
show_title_widget = widgets.Checkbox(value=True, description='Show Plot Title', indent=False)
title_widget = widgets.Text(value=default_title, description='Title:', layout=widgets.Layout(width='450px'))
xlabel_widget = widgets.Text(value=default_xlabel, description='X Label:', layout=widgets.Layout(width='450px'))
ylabel_widget = widgets.Text(value=default_ylabel, description='Y Label:', layout=widgets.Layout(width='450px'))
width_widget = widgets.FloatSlider(value=10.0, min=5.0, max=16.0, step=0.5, description='Width (in):', continuous_update=False)
height_widget = widgets.FloatSlider(value=7.0, min=4.0, max=12.0, step=0.5, description='Height (in):', continuous_update=False)

style_tab = widgets.VBox([
    widgets.HTML("<h4 style='color: #2E86AB;'>Labels & Dimensions</h4>"),
    show_title_widget, title_widget,
    xlabel_widget, ylabel_widget,
    widgets.HTML("<hr style='margin: 5px 0;'>"),
    width_widget, height_widget
])

# === TAB 2: POINTS ===
obs_color_widget = widgets.Dropdown(options=['black', 'gray', 'steelblue', 'blue'], value='black', description='Observed:')
imp_color_widget = widgets.Dropdown(options=['white', 'red', 'orange', 'none'], value='white', description='Imputed:')
imp_edge_widget = widgets.Dropdown(options=['red', 'black', 'orange'], value='red', description='Imp Edge:')
point_size_widget = widgets.IntSlider(value=50, min=10, max=150, step=5, description='Size:')
point_alpha_widget = widgets.FloatSlider(value=0.7, min=0.1, max=1.0, step=0.1, description='Opacity:')

points_tab = widgets.VBox([
    widgets.HTML("<h4 style='color: #2E86AB;'>Data Points</h4>"),
    obs_color_widget,
    imp_color_widget,
    imp_edge_widget,
    widgets.HTML("<hr style='margin: 5px 0;'>"),
    point_size_widget,
    point_alpha_widget
])

# === TAB 3: LINES ===
show_orig_widget = widgets.Checkbox(value=True, description='Show Original Mean', indent=False)
orig_color_widget = widgets.Dropdown(options=['black', 'gray', 'blue'], value='black', description='Orig Color:')
show_adj_widget = widgets.Checkbox(value=True, description='Show Adjusted Mean', indent=False)
adj_color_widget = widgets.Dropdown(options=['red', 'orange', 'magenta'], value='red', description='Adj Color:')
show_funnel_widget = widgets.Checkbox(value=True, description='Show Funnel Guidelines', indent=False)

lines_tab = widgets.VBox([
    widgets.HTML("<h4 style='color: #2E86AB;'>Reference Lines</h4>"),
    show_orig_widget, orig_color_widget,
    widgets.HTML("<hr style='margin: 5px 0;'>"),
    show_adj_widget, adj_color_widget,
    widgets.HTML("<hr style='margin: 5px 0;'>"),
    show_funnel_widget
])

# === TAB 4: LAYOUT & EXPORT ===
show_grid_widget = widgets.Checkbox(value=True, description='Show Grid', indent=False)
legend_loc_widget = widgets.Dropdown(options=['best', 'upper right', 'upper left', 'lower right', 'lower left', 'none'], value='upper right', description='Legend:')
save_pdf_widget = widgets.Checkbox(value=True, description='Save as PDF', indent=False)
save_png_widget = widgets.Checkbox(value=True, description='Save as PNG', indent=False)
filename_prefix_widget = widgets.Text(value='TrimFill_Plot', description='Filename:', layout=widgets.Layout(width='300px'))

layout_tab = widgets.VBox([
    widgets.HTML("<h4 style='color: #2E86AB;'>Layout & Export</h4>"),
    show_grid_widget, legend_loc_widget,
    widgets.HTML("<hr style='margin: 5px 0;'>"),
    save_pdf_widget, save_png_widget, filename_prefix_widget
])

# Assemble Tabs
tabs = widgets.Tab(children=[style_tab, points_tab, lines_tab, layout_tab])
tabs.set_title(0, '🎨 Style')
tabs.set_title(1, '⚫ Points')
tabs.set_title(2, 'zk Lines')
tabs.set_title(3, '💾 Export')

run_plot_btn = widgets.Button(
    description='📊 Generate Plot',
    button_style='success',
    layout=widgets.Layout(width='450px', height='50px'),
    style={'font_weight': 'bold'}
)
plot_output = widgets.Output()

# Header
header = widgets.HTML(
    "<h3 style='color: #2E86AB; margin-bottom: 10px;'>📊 Trim-and-Fill Plot - Interactive Plotter</h3>"
    "<p style='color: #555; margin-top: 0;'>Customize the plot using the tabs below, then click Generate to create the visualization.</p>"
)

# --- 3. PLOTTING LOGIC ---
def generate_tf_plot(b):
    with plot_output:
        clear_output(wait=True)

        try:
            # 1. Load Data & Results
            if 'ANALYSIS_CONFIG' not in globals() or 'trimfill_results' not in ANALYSIS_CONFIG:
                print("❌ Error: Run Cell 14 (Trim-and-Fill) first.")
                return

            tf_res = ANALYSIS_CONFIG['trimfill_results']

            # Reconstruct original data from stored results or global data
            # We need the original points.
            # The tf_res has 'yi_combined' and 'vi_combined' which includes imputed.
            # We can split them using k0.

            yi_all = tf_res['yi_combined']
            vi_all = tf_res['vi_combined']
            se_all = np.sqrt(vi_all)

            k0 = tf_res['k0']
            n_orig = len(yi_all) - k0

            yi_orig = yi_all[:n_orig]
            se_orig = se_all[:n_orig]

            yi_fill = yi_all[n_orig:]
            se_fill = se_all[n_orig:]

            orig_mean = tf_res['pooled_original']
            fill_mean = tf_res['pooled_filled']

            # 2. Prepare Plot
            fig, ax = plt.subplots(figsize=(width_widget.value, height_widget.value))

            # Max SE for Y-axis limit
            max_se = np.max(se_all) * 1.1 if len(se_all) > 0 else 1.0
            y_range = np.linspace(0, max_se, 100)

            # --- Funnel Lines (Centered on Adjusted Mean) ---
            if show_funnel_widget.value:
                # 95% CI: +/- 1.96 * SE
                # x = mean +/- 1.96 * y
                x_left = fill_mean - 1.96 * y_range
                x_right = fill_mean + 1.96 * y_range

                ax.plot(x_left, y_range, color='gray', linestyle='--', linewidth=1, alpha=0.5)
                ax.plot(x_right, y_range, color='gray', linestyle='--', linewidth=1, alpha=0.5)
                ax.fill_betweenx(y_range, x_left, x_right, color='lightgray', alpha=0.1)

            # --- Plot Points ---
            # Original Studies
            ax.scatter(yi_orig, se_orig,
                      c=obs_color_widget.value,
                      s=point_size_widget.value,
                      alpha=point_alpha_widget.value,
                      edgecolors='black', linewidth=0.5,
                      label='Observed Studies', zorder=3)

            # Imputed Studies
            if k0 > 0:
                ax.scatter(yi_fill, se_fill,
                          c=imp_color_widget.value,
                          s=point_size_widget.value,
                          alpha=point_alpha_widget.value,
                          edgecolors=imp_edge_widget.value, linewidth=1.5,
                          marker='o',
                          label=f'Imputed Studies (k={k0})', zorder=3)

            # --- Plot Center Lines ---
            if show_orig_widget.value:
                ax.axvline(orig_mean, color=orig_color_widget.value, linestyle='--', linewidth=2,
                          label=f'Original: {orig_mean:.3f}', zorder=2)

            if show_adj_widget.value:
                ax.axvline(fill_mean, color=adj_color_widget.value, linestyle='-', linewidth=2,
                          label=f'Adjusted: {fill_mean:.3f}', zorder=2)

            # --- Axis Customization ---
            ax.set_ylim(max_se, 0) # Invert Y-axis

            if show_title_widget.value:
                ax.set_title(title_widget.value, fontsize=14, fontweight='bold', pad=15)
            ax.set_xlabel(xlabel_widget.value, fontsize=12, fontweight='bold')
            ax.set_ylabel(ylabel_widget.value, fontsize=12, fontweight='bold')

            if show_grid_widget.value:
                ax.grid(True, linestyle=':', alpha=0.4)

            if legend_loc_widget.value != 'none':
                ax.legend(loc=legend_loc_widget.value, frameon=True, fancybox=True)

            plt.tight_layout()

            # --- Export ---
            ts = datetime.datetime.now().strftime("%H%M%S")
            fn = filename_prefix_widget.value

            if save_pdf_widget.value:
                plt.savefig(f"{fn}_{ts}.pdf", bbox_inches='tight')
                print(f"💾 Saved: {fn}_{ts}.pdf")

            if save_png_widget.value:
                plt.savefig(f"{fn}_{ts}.png", dpi=png_dpi_widget.value, bbox_inches='tight')
                print(f"💾 Saved: {fn}_{ts}.png")

            plt.show()

        except Exception as e:
            print(f"❌ Plotting Error: {e}")
            traceback.print_exc()

run_plot_btn.on_click(generate_tf_plot)

display(widgets.VBox([
    header,
    tabs,
    widgets.HTML("<hr style='margin: 10px 0;'>"),
    run_plot_btn,
    plot_output
]))

In [ ]:
#@title ⚖️ Cell 14.5: R Validation for Trim-and-Fill (Debug Mode)
# =============================================================================
# CELL: R VALIDATION (DEBUG MODE)
# Purpose: Robustly validate Trim-and-Fill results against R.
# Fix: Added extensive error checking and raw object inspection.
# =============================================================================

import pandas as pd
import numpy as np
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
pandas2ri.activate()

# --- 1. Robust Data Prep ---
print("🔍 Checking data...")
if 'analysis_data' in globals():
    df_tf_check = analysis_data.copy()
elif 'data_filtered' in globals():
    df_tf_check = data_filtered.copy()
else:
    print("❌ Error: No data found. Run Cell 5/6 first.")
    df_tf_check = None

if df_tf_check is not None:
    # Configuration
    if 'ANALYSIS_CONFIG' in globals():
        eff_col = ANALYSIS_CONFIG.get('effect_col', 'hedges_g')
        var_col = ANALYSIS_CONFIG.get('var_col', 'Vg')
    else:
        eff_col = 'hedges_g'; var_col = 'Vg'

    # --- SIDE DETECTION ---
    r_side_arg = "left" # Safe default
    if 'ANALYSIS_CONFIG' in globals() and 'trimfill_results' in ANALYSIS_CONFIG:
        py_res = ANALYSIS_CONFIG['trimfill_results']
        if isinstance(py_res, dict):
            py_side = py_res.get('side')
            if py_side in ['left', 'right']:
                r_side_arg = py_side

    print(f"   Effect: '{eff_col}', Variance: '{var_col}'")
    print(f"   Side: '{r_side_arg}'")

    # Clean Data
    # Ensure columns exist
    if eff_col not in df_tf_check.columns or var_col not in df_tf_check.columns:
        print(f"❌ Error: Columns {eff_col}/{var_col} missing from dataframe.")
    else:
        df_r = df_tf_check[[eff_col, var_col]].dropna()
        df_r = df_r[df_r[var_col] > 0]

        print(f"   Rows sent to R: {len(df_r)}")

        if len(df_r) < 3:
            print("❌ Error: Not enough valid rows for R (need >= 3).")
        else:
            # Transfer to R
            ro.globalenv['df_python'] = df_r
            ro.globalenv['eff_col_name'] = eff_col
            ro.globalenv['var_col_name'] = var_col
            ro.globalenv['side_val'] = r_side_arg

            # --- 2. Defensive R Script ---
            print("🚀 Running R script...")
            r_script = """
            library(metafor)

            # Wrap in tryCatch to guarantee a return list
            result <- tryCatch({
                # 1. Fixed Effect Model
                res <- rma(yi=df_python[[eff_col_name]], vi=df_python[[var_col_name]], method="FE")

                # 2. Trim and Fill
                tf <- trimfill(res, estimator="L0", side=side_val)

                # 3. Extract Values safely
                list(
                    status = "success",
                    k0 = as.integer(tf$k0),
                    side = as.character(tf$side),
                    fill_est = as.numeric(tf$beta[1]),
                    fill_se = as.numeric(tf$se[1]),
                    orig_est = as.numeric(res$b[1])
                )
            }, error = function(e) {
                list(status = "error", message = conditionMessage(e))
            })

            result
            """

            try:
                r_res = ro.r(r_script)

                # --- 3. Inspect Raw Result ---
                # This prevents the NULLType error by checking before accessing
                if r_res == ro.r("NULL"):
                    print("❌ CRITICAL ERROR: R returned NULL.")
                else:
                    # Extract Status safely
                    try:
                        # Use 0-based index for .rx2() result if it's a vector/list
                        status_vec = r_res.rx2('status')
                        status = status_vec[0]
                    except Exception as e:
                        print(f"❌ Error extracting status: {e}")
                        status = "unknown"

                    if status == "error":
                        msg = r_res.rx2('message')[0]
                        print(f"\n❌ R Execution Failed: {msg}")
                    elif status == "success":
                        r_k0 = r_res.rx2('k0')[0]
                        r_side = r_res.rx2('side')[0]
                        r_fill = r_res.rx2('fill_est')[0]

                        # Get Python values for comparison
                        py_fill = "N/A"
                        if 'ANALYSIS_CONFIG' in globals() and 'trimfill_results' in ANALYSIS_CONFIG:
                            py_fill = ANALYSIS_CONFIG['trimfill_results'].get('pooled_filled', "N/A")
                            py_k0 = ANALYSIS_CONFIG['trimfill_results'].get('k0', "N/A")

                        print("\n" + "="*60)
                        print("VALIDATION REPORT (TRIM-AND-FILL)")
                        print("="*60)
                        print(f"{'Metric':<20} {'Python':<12} {'R (metafor)':<12} {'Diff':<12}")
                        print("-" * 60)

                        def fmt(x): return f"{x:.4f}" if isinstance(x, (float, int)) else str(x)
                        def diff(p, r): return f"{abs(p-r):.2e}" if isinstance(p, (float, int)) and isinstance(r, (float, int)) else "-"

                        print(f"{'Missing Studies':<20} {py_k0:<12} {r_k0:<12} {'-'}")
                        print(f"{'Filled Estimate':<20} {fmt(py_fill):<12} {fmt(r_fill):<12} {diff(py_fill, r_fill):<12}")

                        if isinstance(py_fill, float) and abs(py_fill - r_fill) < 1e-4:
                             print("\n✅ PASSED: Trim-and-Fill matches R.")
                        elif py_fill == "N/A":
                             print("\n⚠️  NOTE: Run Cell 14 first to generate Python results.")
                        else:
                             print("\n⚠️  CHECK: Results differ. Check 'side' or estimator settings.")

            except Exception as e:
                print(f"\n❌ Python Interface Error: {e}")

In [ ]:
#@title 🔄 Cell 13: Leave-One-Out Sensitivity (Calculation Only)
# =============================================================================
# CELL 13: ROBUST LEAVE-ONE-OUT ANALYSIS (Math Only)
# Purpose: Calculate influence of each study on the 3-level pooled effect.
# Note: Plots have been moved to Cell 13b.
# =============================================================================

import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.optimize import minimize
from scipy.stats import norm
import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings

# --- 1. ROBUST ENGINE (Same as Cell 6.5) ---
def _run_three_level_reml_loo(analysis_data, effect_col, var_col):
    """Optimization with Two-Pass High Precision Strategy."""
    grouped = analysis_data.groupby('id')
    y_all = [group[effect_col].values for _, group in grouped]
    v_all = [group[var_col].values for _, group in grouped]
    N_total = len(analysis_data)
    M_studies = len(y_all)

    if M_studies < 2: return None

    # 1. Global Search (L-BFGS-B)
    start_points = [[0.01, 0.01], [0.5, 0.1], [0.1, 0.5]]
    best_res = None
    best_fun = np.inf

    for start in start_points:
        res = minimize(
            _neg_log_lik_reml_loo, x0=start,
            args=(y_all, v_all, N_total, M_studies),
            method='L-BFGS-B', bounds=[(1e-8, None), (1e-8, None)],
            options={'ftol': 1e-10}
        )
        if res.success and res.fun < best_fun:
            best_fun = res.fun
            best_res = res

    if not best_res: return None

    # 2. Polishing (Nelder-Mead)
    final_res = minimize(
        _neg_log_lik_reml_loo, x0=best_res.x,
        args=(y_all, v_all, N_total, M_studies),
        method='Nelder-Mead', bounds=[(1e-8, None), (1e-8, None)],
        options={'xatol': 1e-10, 'fatol': 1e-10}
    )

    return _get_three_level_estimates_loo(
        final_res.x, y_all, v_all, N_total, M_studies
    )

# --- 2. WIDGETS ---
header = widgets.HTML(
    "<h3 style='color: #2E86AB;'>Three-Level Leave-One-Out Sensitivity Analysis</h3>"
    "<p style='color: #666;'><i>Calculates the influence of each study. (Math Only - Plotting in Cell 13b)</i></p>"
    "<p style='color: red;'>⚠️ This is computationally intensive.</p>"
)

run_loo_btn = widgets.Button(description='▶ Run LOO Calculation', button_style='success', layout=widgets.Layout(width='400px'))
loo_output = widgets.Output()

# --- 3. MAIN LOGIC ---
def run_loo_analysis(b):
    global ANALYSIS_CONFIG
    with loo_output:
        clear_output(wait=True)
        print("="*70)
        print("RUNNING HIGH-PRECISION LEAVE-ONE-OUT ANALYSIS")
        print("="*70)

        try:
            # Load Data
            if 'analysis_data' in globals(): df_loo = analysis_data.copy()
            elif 'data_filtered' in globals(): df_loo = data_filtered.copy()
            else: print("❌ Data not found."); return

            if 'three_level_results' not in ANALYSIS_CONFIG:
                print("❌ Run Cell 6.5 first.")
                return

            # Get Config
            effect_col = ANALYSIS_CONFIG.get('effect_col', 'hedges_g')
            var_col = ANALYSIS_CONFIG.get('var_col', 'Vg')
            es_config = ANALYSIS_CONFIG.get('es_config', {})
            orig_res = ANALYSIS_CONFIG['three_level_results']
            orig_eff = orig_res['pooled_effect']
            orig_ci_lower = orig_res['ci_lower']
            orig_ci_upper = orig_res['ci_upper']

            # Run Loop
            studies = df_loo['id'].unique()
            results = []
            print(f"  Processing {len(studies)} studies...")

            for i, study in enumerate(studies):
                if i % 5 == 0: print(f"  ... {i}/{len(studies)}", end='\r')

                # Remove study
                subset = df_loo[df_loo['id'] != study]

                # Run Robust Optimizer
                est = _run_three_level_reml_loo(subset, effect_col, var_col)

                if est:
                    mu = est['mu']
                    se = est['se_mu']
                    # Check significance change
                    null_val = es_config.get('null_value', 0)
                    orig_sig = not (orig_ci_lower <= null_val <= orig_ci_upper)
                    loo_sig = not (mu - 1.96*se <= null_val <= mu + 1.96*se)

                    results.append({
                        'unit_removed': str(study),
                        'k_studies': subset['id'].nunique(),
                        'k_obs': len(subset),
                        'pooled_effect': mu,
                        'se': se,
                        'ci_lower': mu - 1.96*se,
                        'ci_upper': mu + 1.96*se,
                        'effect_diff': mu - orig_eff,
                        'abs_diff': abs(mu - orig_eff),
                        'changes_sig': (orig_sig != loo_sig),
                        'tau_squared': est['tau_sq'],
                        'sigma_squared': est['sigma_sq']
                    })

            print(f"  ✓ Completed {len(results)} iterations.\n")

            if len(results) == 0:
                print("❌ Error: No iterations succeeded.")
                return

            results_df = pd.DataFrame(results)

            # Check for Significance Changes
            sig_changers = results_df[results_df['changes_sig'] == True]

            print("\n" + "="*70)
            print("RESULTS SUMMARY")
            print("="*70)
            print(f"  Original Effect: {orig_eff:.4f}")
            print(f"  Range of LOO Effects: {results_df['pooled_effect'].min():.4f} to {results_df['pooled_effect'].max():.4f}")

            if not sig_changers.empty:
                print(f"\n⚠️  WARNING: Removing these studies changed statistical significance:")
                print(f"    {', '.join(sig_changers['unit_removed'].tolist())}")
            else:
                print("\n✅ ROBUST: No single study removal changed the statistical significance.")

            # --- SAVE RESULTS ---
            ANALYSIS_CONFIG['loo_3level_results'] = {
                'timestamp': datetime.datetime.now(),
                'results_df': results_df,
                'removal_unit': 'study',
                'original_effect': orig_eff,
                'n_sig_changers': len(sig_changers)
            }
            print("\n✅ DONE: Results saved to 'loo_3level_results'")
            print("   👉 NOW RUN CELL 13b TO SEE THE PLOT")

        except Exception as e:
            print(f"❌ Error: {e}")
            import traceback
            traceback.print_exc()

run_loo_btn.on_click(run_loo_analysis)

display(widgets.VBox([
    header,
    run_loo_btn,
    loo_output
]))

In [ ]:
#@title 📊 Cell 13b: Publication-Ready Leave-One-Out Plot (Fixed)
# =============================================================================
# CELL 13b: ADVANCED LEAVE-ONE-OUT PLOTTER
# Purpose: Visualize sensitivity analysis with full customization.
# Fix: Corrected 'ecolor' error by splitting plots for normal/highlighted studies.
# =============================================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import ipywidgets as widgets
from IPython.display import display, clear_output
import datetime

# --- 1. INITIALIZATION ---
default_title = "Leave-One-Out Sensitivity Analysis"
default_xlabel = "Pooled Effect Size"
default_ylabel = "Study Removed"

try:
    if 'ANALYSIS_CONFIG' in globals():
        es_config = ANALYSIS_CONFIG.get('es_config', {})
        default_xlabel = f"Pooled {es_config.get('effect_label', 'Effect Size')}"
except: pass

# --- 2. WIDGET DEFINITIONS ---

# === TAB 1: STYLE ===
show_title_widget = widgets.Checkbox(value=True, description='Show Plot Title', indent=False)
title_widget = widgets.Text(value=default_title, description='Title:', layout=widgets.Layout(width='450px'))
xlabel_widget = widgets.Text(value=default_xlabel, description='X Label:', layout=widgets.Layout(width='450px'))
ylabel_widget = widgets.Text(value=default_ylabel, description='Y Label:', layout=widgets.Layout(width='450px'))
width_widget = widgets.FloatSlider(value=10.0, min=5.0, max=16.0, step=0.5, description='Width (in):', continuous_update=False)
height_auto_widget = widgets.Checkbox(value=True, description='Auto-Height (based on # studies)', indent=False)
height_widget = widgets.FloatSlider(value=8.0, min=4.0, max=20.0, step=0.5, description='Manual Height:', continuous_update=False)

style_tab = widgets.VBox([
    widgets.HTML("<h4 style='color: #2E86AB;'>Labels & Dimensions</h4>"),
    show_title_widget, title_widget,
    xlabel_widget, ylabel_widget,
    widgets.HTML("<hr style='margin: 5px 0;'>"),
    width_widget, height_auto_widget, height_widget
])

# === TAB 2: DATA & SORTING ===
sort_by_widget = widgets.Dropdown(
    options=[('Effect Size (Low to High)', 'effect'),
             ('Influence (Diff from Original)', 'influence'),
             ('Study ID (Alphabetical)', 'id')],
    value='effect', description='Sort By:', layout=widgets.Layout(width='400px')
)

highlight_sig_widget = widgets.Checkbox(value=True, description='Highlight Significance Changers (Red)', indent=False)
point_color_widget = widgets.Dropdown(options=['blue', 'black', 'gray', 'steelblue'], value='blue', description='Point Color:')
point_size_widget = widgets.IntSlider(value=6, min=2, max=20, description='Point Size:')

data_tab = widgets.VBox([
    widgets.HTML("<h4 style='color: #2E86AB;'>Data Presentation</h4>"),
    sort_by_widget,
    widgets.HTML("<hr style='margin: 5px 0;'>"),
    highlight_sig_widget,
    point_color_widget,
    point_size_widget
])

# === TAB 3: REFERENCE LINES ===
show_orig_line_widget = widgets.Checkbox(value=True, description='Show Original Effect Line', indent=False)
orig_color_widget = widgets.Dropdown(options=['red', 'black', 'green'], value='red', description='Line Color:')
show_orig_ci_widget = widgets.Checkbox(value=True, description='Show Original 95% CI Band', indent=False)
ci_band_alpha_widget = widgets.FloatSlider(value=0.1, min=0.05, max=0.5, step=0.05, description='Band Alpha:')
show_null_line_widget = widgets.Checkbox(value=True, description='Show Null Effect Line', indent=False)

lines_tab = widgets.VBox([
    widgets.HTML("<h4 style='color: #2E86AB;'>Reference Lines</h4>"),
    show_orig_line_widget, orig_color_widget,
    show_orig_ci_widget, ci_band_alpha_widget,
    widgets.HTML("<hr style='margin: 5px 0;'>"),
    show_null_line_widget
])

# === TAB 4: EXPORT ===
save_pdf_widget = widgets.Checkbox(value=True, description='Save as PDF', indent=False)
save_png_widget = widgets.Checkbox(value=True, description='Save as PNG', indent=False)
filename_prefix_widget = widgets.Text(value='LOO_Plot', description='Filename:', layout=widgets.Layout(width='300px'))

export_tab = widgets.VBox([
    widgets.HTML("<h4 style='color: #2E86AB;'>Export</h4>"),
    save_pdf_widget, save_png_widget, filename_prefix_widget
])

# Assemble Tabs
tabs = widgets.Tab(children=[style_tab, data_tab, lines_tab, export_tab])
tabs.set_title(0, '🎨 Style')
tabs.set_title(1, '📊 Data')
tabs.set_title(2, '📐 Lines')
tabs.set_title(3, '💾 Export')

run_plot_btn = widgets.Button(
    description='📊 Generate LOO Plot',
    button_style='success',
    layout=widgets.Layout(width='450px', height='50px'),
    style={'font_weight': 'bold'}
)
plot_output = widgets.Output()

# --- 3. PLOTTING LOGIC ---
def generate_loo_plot(b):
    with plot_output:
        clear_output(wait=True)

        try:
            # 1. Load Results
            if 'ANALYSIS_CONFIG' not in globals() or 'loo_3level_results' not in ANALYSIS_CONFIG:
                print("❌ Error: Run Cell 13 (Leave-One-Out Analysis) first.")
                return

            loo_res = ANALYSIS_CONFIG['loo_3level_results']
            df = loo_res['results_df'].copy()

            # Get original results for reference
            if 'three_level_results' in ANALYSIS_CONFIG:
                orig_res = ANALYSIS_CONFIG['three_level_results']
                orig_eff = orig_res['pooled_effect']
                orig_ci_lower = orig_res['ci_lower']
                orig_ci_upper = orig_res['ci_upper']
            else:
                # Fallback if cell 13 was run but cell 6.5 missing (unlikely)
                orig_eff = loo_res['original_effect']
                orig_ci_lower = df['ci_lower'].mean() # Approx
                orig_ci_upper = df['ci_upper'].mean() # Approx

            # 2. Sorting
            sort_mode = sort_by_widget.value
            if sort_mode == 'influence':
                df = df.sort_values('abs_diff', ascending=True) # Small diff at bottom
            elif sort_mode == 'id':
                df = df.sort_values('unit_removed', ascending=False) # Z-A (so A is at top)
            else: # effect
                df = df.sort_values('pooled_effect', ascending=True)

            df = df.reset_index(drop=True)

            # 3. Prepare Plot
            n_studies = len(df)

            # Auto-height calculation: Base + (studies * factor)
            if height_auto_widget.value:
                plot_height = max(5, 1 + n_studies * 0.25)
            else:
                plot_height = height_widget.value

            fig, ax = plt.subplots(figsize=(width_widget.value, plot_height))

            y_pos = np.arange(n_studies)

            # --- Create Splitted Dataframes for Error Bars ---
            # ax.errorbar doesn't accept a list of colors in all matplotlib versions.
            # Solution: Plot normal and highlighted bars separately.

            if highlight_sig_widget.value:
                # Identify rows that changed significance
                mask_sig = df['changes_sig'] == True
                mask_norm = ~mask_sig
            else:
                # Treat all as normal
                mask_sig = pd.Series([False] * n_studies)
                mask_norm = pd.Series([True] * n_studies)

            # Plot Normal Error Bars
            if mask_norm.any():
                ax.errorbar(df.loc[mask_norm, 'pooled_effect'], y_pos[mask_norm],
                           xerr=[df.loc[mask_norm, 'pooled_effect'] - df.loc[mask_norm, 'ci_lower'],
                                 df.loc[mask_norm, 'ci_upper'] - df.loc[mask_norm, 'pooled_effect']],
                           fmt='none', ecolor=point_color_widget.value, alpha=0.5, capsize=3)

            # Plot Highlighted Error Bars (Red)
            if mask_sig.any():
                ax.errorbar(df.loc[mask_sig, 'pooled_effect'], y_pos[mask_sig],
                           xerr=[df.loc[mask_sig, 'pooled_effect'] - df.loc[mask_sig, 'ci_lower'],
                                 df.loc[mask_sig, 'ci_upper'] - df.loc[mask_sig, 'pooled_effect']],
                           fmt='none', ecolor='red', alpha=0.8, capsize=3)

            # Plot Points (Scatter accepts list of colors)
            colors = ['red' if (x and highlight_sig_widget.value) else point_color_widget.value for x in df['changes_sig']]
            ax.scatter(df['pooled_effect'], y_pos, c=colors, s=point_size_widget.value*5, zorder=3)

            # --- Reference Lines ---
            # Null Line
            null_val = ANALYSIS_CONFIG.get('es_config', {}).get('null_value', 0)
            if show_null_line_widget.value:
                ax.axvline(null_val, color='black', linestyle='-', linewidth=1, alpha=0.5, zorder=1)

            # Original CI Band
            if show_orig_ci_widget.value:
                ax.axvspan(orig_ci_lower, orig_ci_upper, color=orig_color_widget.value,
                          alpha=ci_band_alpha_widget.value, label='Original 95% CI', zorder=0)

            # Original Mean Line
            if show_orig_line_widget.value:
                ax.axvline(orig_eff, color=orig_color_widget.value, linestyle='--', linewidth=2,
                          label=f'Original Effect ({orig_eff:.3f})', zorder=2)

            # --- Layout ---
            ax.set_yticks(y_pos)
            ax.set_yticklabels(df['unit_removed'], fontsize=9)

            if show_title_widget.value:
                ax.set_title(title_widget.value, fontsize=14, fontweight='bold', pad=15)
            ax.set_xlabel(xlabel_widget.value, fontsize=12, fontweight='bold')
            ax.set_ylabel(ylabel_widget.value, fontsize=12, fontweight='bold')

            # Add grid for easier reading
            ax.grid(axis='y', linestyle=':', alpha=0.3)
            ax.grid(axis='x', linestyle=':', alpha=0.3)

            # Legend
            handles, labels = ax.get_legend_handles_labels()
            # Add custom handle for "Changed Significance" if needed
            if highlight_sig_widget.value and df['changes_sig'].any():
                handles.append(mpatches.Patch(color='red', label='Changed Significance'))

            ax.legend(handles=handles, loc='best', frameon=True, fancybox=True)

            plt.tight_layout()

            # --- Export ---
            ts = datetime.datetime.now().strftime("%H%M%S")
            fn = filename_prefix_widget.value

            if save_pdf_widget.value:
                plt.savefig(f"{fn}_{ts}.pdf", bbox_inches='tight')
                print(f"💾 Saved: {fn}_{ts}.pdf")

            if save_png_widget.value:
                plt.savefig(f"{fn}_{ts}.png", dpi=png_dpi_widget.value, bbox_inches='tight')
                print(f"💾 Saved: {fn}_{ts}.png")

            plt.show()

        except Exception as e:
            print(f"❌ Plotting Error: {e}")
            traceback.print_exc()

run_plot_btn.on_click(generate_loo_plot)

display(widgets.VBox([
    header,
    tabs,
    widgets.HTML("<hr style='margin: 10px 0;'>"),
    run_plot_btn,
    plot_output
]))

In [ ]:
#@title ⚖️ Cell 13.5: R Validation for LOO (Study-Level)
# =============================================================================
# CELL: R VALIDATION FOR LOO
# Purpose: Run Cluster-Level Leave-One-Out in R to verify Python results.
# =============================================================================

import pandas as pd
import numpy as np
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
pandas2ri.activate()

# --- 1. Prepare Data ---
if 'analysis_data' in globals():
    df_loo_check = analysis_data.copy()
elif 'data_filtered' in globals():
    df_loo_check = data_filtered.copy()
else:
    print("❌ Error: Data not found.")
    df_loo_check = None

if df_loo_check is not None:
    # Get columns from config or defaults
    if 'ANALYSIS_CONFIG' in globals():
        eff_col = ANALYSIS_CONFIG.get('effect_col', 'hedges_g')
        var_col = ANALYSIS_CONFIG.get('var_col', 'Vg')
    else:
        eff_col = 'hedges_g'; var_col = 'Vg'

    print(f"🚀 Running R Validation for Study-Level LOO...")
    print(f"   Effect: {eff_col}, Variance: {var_col}")

    # Clean data for R
    df_r = df_loo_check[['id', eff_col, var_col]].dropna()
    ro.globalenv['df_python'] = df_r

    # --- 2. R Script (Manual Study-Level Loop) ---
    r_script = f"""
    library(metafor)

    dat <- df_python
    dat$rows <- 1:nrow(dat)
    dat$study_id <- as.factor(dat$id)

    # Get list of unique studies
    study_list <- unique(dat$study_id)
    n_studies <- length(study_list)

    # Storage
    loo_estimates <- numeric(n_studies)

    # Loop: Remove one study at a time
    for (i in 1:n_studies) {{
        # Subset: Remove study i
        subset_dat <- dat[dat$study_id != study_list[i], ]

        # Refit 3-Level Model
        # We use 'try' to skip if a subset fails (rare)
        tryCatch({{
            res <- rma.mv(yi={eff_col}, V={var_col},
                          random = ~ 1 | study_id/rows,
                          data=subset_dat,
                          control=list(optimizer="optim", optmethod="Nelder-Mead"))
            loo_estimates[i] <- res$b[1]
        }}, error=function(e) {{ loo_estimates[i] <- NA }})
    }}

    # Original Full Model
    res_full <- rma.mv(yi={eff_col}, V={var_col},
                       random = ~ 1 | study_id/rows,
                       data=dat)

    list(
        orig = res_full$b[1],
        min_loo = min(loo_estimates, na.rm=TRUE),
        max_loo = max(loo_estimates, na.rm=TRUE)
    )
    """

    try:
        # Run R
        r_res = ro.r(r_script)

        r_orig = r_res.rx2('orig')[0]
        r_min = r_res.rx2('min_loo')[0]
        r_max = r_res.rx2('max_loo')[0]

        print("\n" + "="*60)
        print("VALIDATION REPORT")
        print("="*60)
        print(f"{'Metric':<20} {'Python':<12} {'R (metafor)':<12} {'Diff':<12}")
        print("-" * 60)

        # Compare Original
        # Note: You mentioned 1.3598 as your result. I'll use a placeholder var for Python
        # You can visually compare the printed R result to your Python output above.
        print(f"{'Original Effect':<20} {'(See Above)':<12} {r_orig:.4f}")

        print(f"{'LOO Min':<20} {'(See Above)':<12} {r_min:.4f}")
        print(f"{'LOO Max':<20} {'(See Above)':<12} {r_max:.4f}")

        print("\n✅ Interpretation:")
        print(f"   R Range: [{r_min:.4f}, {r_max:.4f}]")
        print("   If your Python range is [1.2989, 1.3817], that is extremely close.")
        print("   (Differences < 0.01 are usually just optimizer tolerance differences).")

    except Exception as e:
        print(f"\n❌ R Error: {e}")

In [ ]:
#@title 📈 CUMULATIVE META-ANALYSIS

# =============================================================================
# CELL 14: CUMULATIVE META-ANALYSIS
# Purpose: Show how effect sizes evolve chronologically as studies accumulate.
# Method:  "Two-Step" Approach for clustered data:
#          1. Aggregate effects within each study (if 'By Study' selected)
#          2. Perform cumulative Random-Effects meta-analysis over time
# Dependencies: Cell 6 (overall_results), Cell 5 (data)
# Outputs: Cumulative forest plot and stability metrics
# =============================================================================

import numpy as np
import pandas as pd
from scipy.stats import norm, chi2
import matplotlib.pyplot as plt
import datetime
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

print("="*70)
print("CUMULATIVE META-ANALYSIS")
print("="*70)

# --- 1. HELPER FUNCTIONS ---

# --- 2. LOAD CONFIGURATION ---
try:
    if 'ANALYSIS_CONFIG' not in locals() and 'ANALYSIS_CONFIG' not in globals():
        raise NameError("ANALYSIS_CONFIG not found.")

    if 'analysis_data' in ANALYSIS_CONFIG:
        analysis_data = ANALYSIS_CONFIG['analysis_data']
    elif 'data_filtered' in globals():
        analysis_data = data_filtered
    else:
        raise ValueError("Cannot find analysis data")

    if analysis_data.empty:
        raise ValueError("Analysis data is empty")

    effect_col = ANALYSIS_CONFIG['effect_col']
    var_col = ANALYSIS_CONFIG['var_col']
    es_config = ANALYSIS_CONFIG['es_config']
    overall_results = ANALYSIS_CONFIG['overall_results']

    if 'year' not in analysis_data.columns:
        raise ValueError("'year' column not found. Ensure data has publication years.")

    # Clean year data
    analysis_data_with_year = analysis_data.copy()
    analysis_data_with_year['year'] = pd.to_numeric(analysis_data_with_year['year'], errors='coerce')
    analysis_data_with_year = analysis_data_with_year.dropna(subset=['year'])

    if len(analysis_data_with_year) < 2:
        raise ValueError(f"Insufficient data with valid years. Need at least 2.")

    n_studies = analysis_data_with_year['id'].nunique()
    n_obs = len(analysis_data_with_year)
    year_range = (int(analysis_data_with_year['year'].min()), int(analysis_data_with_year['year'].max()))

    print(f"✓ Configuration loaded")
    print(f"  Effect size: {es_config['effect_label']}")
    print(f"  Data: {n_obs} observations from {n_studies} studies")
    print(f"  Year range: {year_range[0]} - {year_range[1]}")

except (NameError, KeyError, ValueError) as e:
    print(f"❌ ERROR: {e}")
    print("  Please ensure Cells 1-6 have been run.")
    raise

# --- 3. CREATE WIDGETS ---

header = widgets.HTML(
    "<h3 style='color: #2E86AB;'>Cumulative Meta-Analysis Setup</h3>"
    "<p style='color: #666;'><i>Visualize how pooled effect sizes change as evidence accumulates over time</i></p>"
)

sort_order_widget = widgets.RadioButtons(
    options=[('Chronological (oldest first)', 'ascending'), ('Reverse Chronological (newest first)', 'descending')],
    value='ascending', description='Sort Order:', style={'description_width': '120px'}, layout=widgets.Layout(width='500px')
)

unit_widget = widgets.RadioButtons(
    options=[('By Study (aggregate first - Recommended)', 'study'), ('By Observation (ignore clustering)', 'observation')],
    value='study', description='Aggregation:', style={'description_width': '120px'}, layout=widgets.Layout(width='500px')
)

show_title_widget = widgets.Checkbox(value=True, description='Show Plot Title', indent=False, layout=widgets.Layout(width='450px'))
title_widget = widgets.Text(value=f'Cumulative Meta-Analysis: {es_config["effect_label"]} Over Time', description='Title:', layout=widgets.Layout(width='500px'), style={'description_width': '120px'})
xlabel_widget = widgets.Text(value='Year', description='X-Axis Label:', layout=widgets.Layout(width='500px'), style={'description_width': '120px'})
ylabel_widget = widgets.Text(value=es_config['effect_label'], description='Y-Axis Label:', layout=widgets.Layout(width='500px'), style={'description_width': '120px'})
plot_width_widget = widgets.FloatSlider(value=12.0, min=8.0, max=16.0, step=0.5, description='Plot Width:', continuous_update=False, style={'description_width': '120px'}, layout=widgets.Layout(width='450px'))
plot_height_widget = widgets.FloatSlider(value=8.0, min=4.0, max=12.0, step=0.5, description='Plot Height:', continuous_update=False, style={'description_width': '120px'}, layout=widgets.Layout(width='450px'))

show_ci_widget = widgets.Checkbox(value=True, description='Show 95% Confidence Intervals', indent=False, layout=widgets.Layout(width='450px'))
show_null_widget = widgets.Checkbox(value=True, description='Show Null Effect Line', indent=False, layout=widgets.Layout(width='450px'))
show_final_widget = widgets.Checkbox(value=True, description='Highlight Final Effect (dashed line)', indent=False, layout=widgets.Layout(width='450px'))
show_i2_widget = widgets.Checkbox(value=False, description='Show I² Trajectory (secondary axis)', indent=False, layout=widgets.Layout(width='450px'))
line_color_widget = widgets.Dropdown(options=['blue', 'red', 'black', 'green', 'purple', 'orange'], value='blue', description='Line Color:', style={'description_width': '120px'}, layout=widgets.Layout(width='450px'))
line_width_widget = widgets.FloatSlider(value=2.0, min=0.5, max=4.0, step=0.5, description='Line Width:', continuous_update=False, style={'description_width': '120px'}, layout=widgets.Layout(width='450px'))
ci_alpha_widget = widgets.FloatSlider(value=0.3, min=0.1, max=0.8, step=0.1, description='CI Transparency:', continuous_update=False, style={'description_width': '120px'}, layout=widgets.Layout(width='450px'))
marker_size_widget = widgets.IntSlider(value=50, min=20, max=200, step=10, description='Marker Size:', continuous_update=False, style={'description_width': '120px'}, layout=widgets.Layout(width='450px'))

save_pdf_widget = widgets.Checkbox(value=True, description='Save as PDF', indent=False, layout=widgets.Layout(width='450px'))
save_png_widget = widgets.Checkbox(value=True, description='Save as PNG', indent=False, layout=widgets.Layout(width='450px'))
png_dpi_widget = widgets.IntSlider(value=300, min=150, max=600, step=50, description='PNG DPI:', continuous_update=False, style={'description_width': '120px'}, layout=widgets.Layout(width='450px'))
show_table_widget = widgets.Checkbox(value=True, description='Show detailed results table', indent=False, layout=widgets.Layout(width='450px'))

tab1 = widgets.VBox([widgets.HTML("<h4 style='color: #2E86AB;'>Analysis Options</h4>"), sort_order_widget, widgets.HTML("<hr style='margin: 10px 0;'>"), unit_widget])
tab2 = widgets.VBox([widgets.HTML("<h4 style='color: #2E86AB;'>Labels & Dimensions</h4>"), show_title_widget, title_widget, widgets.HTML("<hr style='margin: 10px 0;'>"), xlabel_widget, ylabel_widget, widgets.HTML("<hr style='margin: 10px 0;'>"), plot_width_widget, plot_height_widget])
tab3 = widgets.VBox([widgets.HTML("<h4 style='color: #2E86AB;'>Visual Elements</h4>"), show_ci_widget, show_null_widget, show_final_widget, show_i2_widget, widgets.HTML("<hr style='margin: 10px 0;'>"), line_color_widget, line_width_widget, ci_alpha_widget, marker_size_widget])
tab4 = widgets.VBox([widgets.HTML("<h4 style='color: #2E86AB;'>Export Options</h4>"), save_pdf_widget, save_png_widget, png_dpi_widget, widgets.HTML("<hr style='margin: 10px 0;'>"), show_table_widget])

tabs = widgets.Tab(children=[tab1, tab2, tab3, tab4])
tabs.set_title(0, '⚙️ Analysis'); tabs.set_title(1, '📝 Labels'); tabs.set_title(2, '🎨 Visuals'); tabs.set_title(3, '💾 Export')

run_button = widgets.Button(description='▶ Run Cumulative Meta-Analysis', button_style='success', layout=widgets.Layout(width='500px', height='50px'), style={'font_weight': 'bold'})
analysis_output = widgets.Output()

# --- 4. DEFINE ANALYSIS FUNCTION ---
def run_cumulative_analysis(b):
    with analysis_output:
        clear_output(wait=True)
        print("\n" + "="*70)
        print("CUMULATIVE META-ANALYSIS")
        print("="*70)
        print(f"Timestamp: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

        try:
            # Prepare data
            data = analysis_data_with_year.copy()
            unit = unit_widget.value
            sort_order = sort_order_widget.value

            # --- Step 1: Aggregation (Handle Clustering) ---
            if unit == 'study':
                print(f"⚙️  Aggregating observations by study (Two-Step Approach)...")
                # For each study, take the earliest year
                study_years = data.groupby('id')['year'].min().reset_index()
                study_years.columns = ['id', 'study_year']
                data = data.merge(study_years, on='id', how='left')

                study_data = []
                for study_id in data['id'].unique():
                    study_obs = data[data['id'] == study_id]
                    study_year = study_obs['study_year'].iloc[0]

                    # Pool observations within study using fixed-effects (standard practice)
                    if len(study_obs) > 1:
                        w_study = 1 / study_obs[var_col]
                        sum_w_study = w_study.sum()
                        pooled_es = (w_study * study_obs[effect_col]).sum() / sum_w_study
                        pooled_var = 1 / sum_w_study
                    else:
                        pooled_es = study_obs[effect_col].iloc[0]
                        pooled_var = study_obs[var_col].iloc[0]

                    study_data.append({
                        'id': study_id,
                        'year': study_year,
                        effect_col: pooled_es,
                        var_col: pooled_var,
                        'n_obs': len(study_obs)
                    })

                data_sorted = pd.DataFrame(study_data)
                print(f"  ✓ Aggregated {len(data)} observations into {len(data_sorted)} studies")
            else:
                # Use observations directly (less robust)
                data_sorted = data[[effect_col, var_col, 'year', 'id']].copy()
                data_sorted['n_obs'] = 1

            # --- Step 2: Cumulative Analysis ---
            data_sorted = data_sorted.sort_values('year', ascending=(sort_order == 'ascending'))
            data_sorted = data_sorted.reset_index(drop=True)

            n_units = len(data_sorted)
            print(f"\n⚙️  Running cumulative analysis on {n_units} {unit}s...")

            cumulative_results = []
            for i in range(1, n_units + 1):
                df_cum = data_sorted.iloc[:i].copy()
                tau2_cum = calculate_tau_squared_dl(df_cum, effect_col, var_col)
                effect_cum, se_cum, ci_lower_cum, ci_upper_cum, I2_cum = calculate_re_pooled(
                    df_cum, tau2_cum, effect_col, var_col
                )

                cumulative_results.append({
                    'step': i,
                    'year': df_cum['year'].iloc[-1],
                    'id_added': df_cum['id'].iloc[-1],
                    'n_studies': df_cum['id'].nunique(),
                    'pooled_effect': effect_cum,
                    'ci_lower': ci_lower_cum,
                    'ci_upper': ci_upper_cum,
                    'I_squared': I2_cum
                })

                if i % 10 == 0 or i == n_units: print(f"  Progress: {i}/{n_units}", end='\r')

            print(f"\n  ✓ Analysis complete")
            results_df = pd.DataFrame(cumulative_results)

            # --- Step 3: Display Table ---
            if show_table_widget.value:
                print(f"\n" + "="*70)
                print("CUMULATIVE RESULTS TABLE")
                print("="*70)
                print(f"\n{'Step':<5} {'Year':<6} {'N':<4} {'Effect':<10} {'95% CI':<25} {'I²%':<8}")
                print("-" * 70)

                indices_to_show = (list(range(5)) + list(range(len(results_df)-5, len(results_df)))) if len(results_df) > 10 else range(len(results_df))
                last_shown = -1
                for idx in indices_to_show:
                    if idx >= len(results_df): continue
                    if idx - last_shown > 1: print("  ...")
                    row = results_df.iloc[idx]
                    ci_str = f"[{row['ci_lower']:.4f}, {row['ci_upper']:.4f}]"
                    print(f"{int(row['step']):<5} {int(row['year']):<6} {int(row['n_studies']):<4} {row['pooled_effect']:<10.4f} {ci_str:<25} {row['I_squared']:<8.1f}")
                    last_shown = idx

            # --- Step 4: Create Plot ---
            fig, ax1 = plt.subplots(figsize=(plot_width_widget.value, plot_height_widget.value))
            ax1.plot(results_df['year'], results_df['pooled_effect'],
                     color=line_color_widget.value, linewidth=line_width_widget.value, marker='o',
                     markersize=marker_size_widget.value/10, label='Cumulative Effect', zorder=3)

            if show_ci_widget.value:
                ax1.fill_between(results_df['year'], results_df['ci_lower'], results_df['ci_upper'],
                                 color=line_color_widget.value, alpha=ci_alpha_widget.value, label='95% CI', zorder=2)

            if show_null_widget.value:
                ax1.axhline(y=es_config['null_value'], color='gray', linestyle='--', linewidth=1.5, label='Null Effect', zorder=1)

            if show_final_widget.value:
                ax1.axhline(y=results_df.iloc[-1]['pooled_effect'], color=line_color_widget.value, linestyle=':',
                           linewidth=2, alpha=0.7, label='Final Effect', zorder=1)

            ax1.set_xlabel(xlabel_widget.value, fontsize=12, fontweight='bold')
            ax1.set_ylabel(ylabel_widget.value, fontsize=12, fontweight='bold')
            ax1.grid(True, alpha=0.3)
            ax1.legend(loc='upper left', frameon=True)

            if show_i2_widget.value:
                ax2 = ax1.twinx()
                ax2.plot(results_df['year'], results_df['I_squared'], color='orange', linestyle='--', alpha=0.7, label='I² (%)')
                ax2.set_ylabel('Heterogeneity (I²%)', color='orange', fontweight='bold')
                ax2.set_ylim(0, 100)
                ax2.legend(loc='upper right')

            if show_title_widget.value:
                plt.title(title_widget.value, fontsize=14, fontweight='bold', pad=20)

            plt.tight_layout()

            # --- Step 5: Save ---
            timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
            if save_pdf_widget.value:
                plt.savefig(f'Cumulative_Meta_{timestamp}.pdf', bbox_inches='tight')
                print(f"  ✓ Saved PDF")
            if save_png_widget.value:
                plt.savefig(f'Cumulative_Meta_{timestamp}.png', dpi=png_dpi_widget.value, bbox_inches='tight')
                print(f"  ✓ Saved PNG")

            plt.show()
            ANALYSIS_CONFIG['cumulative_results'] = results_df

        except Exception as e:
            print(f"\n❌ ERROR: {e}")
            traceback.print_exc()

run_button.on_click(run_cumulative_analysis)

display(header)
display(tabs)
display(run_button)
display(analysis_output)
print("\n✅ Widget interface ready.")

In [ ]:
#@title ⚖️ Cell 14.5: R Validation for Cumulative Meta-Analysis
# =============================================================================
# CELL: R VALIDATION FOR CUMULATIVE ANALYSIS
# Purpose: Verify cumulative meta-analysis trends against R's metafor::cumul()
# Method:  Aggregates by study (to match Python default), sorts by year,
#          and runs cumulative REML.
# =============================================================================

import pandas as pd
import numpy as np
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
pandas2ri.activate()

# --- 1. Prepare Data ---
if 'analysis_data' in globals():
    df_cum_check = analysis_data.copy()
elif 'data_filtered' in globals():
    df_cum_check = data_filtered.copy()
else:
    print("❌ Error: Data not found.")
    df_cum_check = None

if df_cum_check is not None:
    # Configuration
    if 'ANALYSIS_CONFIG' in globals():
        eff_col = ANALYSIS_CONFIG.get('effect_col', 'hedges_g')
        var_col = ANALYSIS_CONFIG.get('var_col', 'Vg')
    else:
        eff_col = 'hedges_g'; var_col = 'Vg'

    print(f"🚀 Running R Validation for Cumulative Meta-Analysis...")
    print(f"   Effect: {eff_col}, Variance: {var_col}")

    # --- 2. Python Data Prep (Match the Pipeline) ---
    # We must replicate the 'By Study' aggregation to ensure fair comparison

    # Clean and ensure year is numeric
    df_clean = df_cum_check.dropna(subset=[eff_col, var_col, 'year']).copy()
    df_clean['year'] = pd.to_numeric(df_clean['year'], errors='coerce')
    df_clean = df_clean.dropna(subset=['year'])

    # Aggregate by Study (Fixed-Effect mean within study)
    # This matches the default "By Study" behavior of your Python cell
    df_clean['wi'] = 1 / df_clean[var_col]

    def agg_func(x):
        return pd.Series({
            'year': x['year'].min(), # Earliest year for the study
            'effect': np.average(x[eff_col], weights=x['wi']),
            'var': 1 / np.sum(x['wi'])
        })

    # Group and Sort
    df_agg = df_clean.groupby('id').apply(agg_func).reset_index()
    df_agg = df_agg.sort_values(by=['year', 'id']) # Sort by year, then ID for consistency

    print(f"   Aggregated Data: {len(df_agg)} studies (from {len(df_clean)} observations)")

    # Pass to R
    ro.globalenv['df_python'] = df_agg

    # --- 3. R Script ---
    r_script = """
    library(metafor)

    # Load data
    dat <- df_python

    # 1. Run Full Random-Effects Model (REML)
    # We sort inside R just to be absolutely sure
    dat <- dat[order(dat$year, dat$id), ]

    res <- rma(yi=effect, vi=var, data=dat, method="REML")

    # 2. Run Cumulative Meta-Analysis
    cum <- cumul(res, order=order(dat$year, dat$id))

    # Extract Results for the FINAL step (all studies included)
    n <- length(cum$est)

    list(
        final_est = cum$est[n],
        final_ci_lb = cum$ci.lb[n],
        final_ci_ub = cum$ci.ub[n],
        final_tau2 = cum$tau2[n],

        # Also get the first step for checking sort order
        first_est = cum$est[1],
        first_year = dat$year[1],
        last_year = dat$year[n]
    )
    """

    try:
        r_res = ro.r(r_script)

        r_est = r_res.rx2('final_est')[0]
        r_lb = r_res.rx2('final_ci_lb')[0]
        r_ub = r_res.rx2('final_ci_ub')[0]
        r_tau2 = r_res.rx2('final_tau2')[0]

        # Get Python Results from Config
        py_est, py_lb, py_ub = "N/A", "N/A", "N/A"

        if 'ANALYSIS_CONFIG' in globals() and 'cumulative_results' in ANALYSIS_CONFIG:
            # Get the last row of the cumulative results dataframe
            cum_df = ANALYSIS_CONFIG['cumulative_results']
            if not cum_df.empty:
                last_row = cum_df.iloc[-1]
                py_est = last_row['pooled_effect']
                py_lb = last_row['ci_lower']
                py_ub = last_row['ci_upper']
                # Check if tau2 is available in the df
                py_tau2 = last_row['tau_squared'] if 'tau_squared' in last_row else "N/A"

        print("\n" + "="*60)
        print("VALIDATION REPORT (FINAL CUMULATIVE STEP)")
        print("="*60)
        print(f"{'Metric':<20} {'Python':<12} {'R (metafor)':<12} {'Diff':<12}")
        print("-" * 60)

        def fmt(x): return f"{x:.4f}" if isinstance(x, (float, int)) else str(x)
        def diff(p, r): return f"{abs(p-r):.2e}" if isinstance(p, (float, int)) and isinstance(r, (float, int)) else "-"

        print(f"{'Pooled Estimate':<20} {fmt(py_est):<12} {fmt(r_est):<12} {diff(py_est, r_est):<12}")
        print(f"{'95% CI Lower':<20} {fmt(py_lb):<12} {fmt(r_lb):<12} {diff(py_lb, r_lb):<12}")
        print(f"{'95% CI Upper':<20} {fmt(py_ub):<12} {fmt(r_ub):<12} {diff(py_ub, r_ub):<12}")

        if isinstance(py_tau2, (int, float)):
             print(f"{'Tau²':<20} {fmt(py_tau2):<12} {fmt(r_tau2):<12} {diff(py_tau2, r_tau2):<12}")

        print("-" * 60)
        print(f"Time Range Checked: {int(r_res.rx2('first_year')[0])} - {int(r_res.rx2('last_year')[0])}")

        if isinstance(py_est, float) and abs(py_est - r_est) < 0.01:
            print("\n✅ PASSED: Cumulative analysis trends match R.")
        elif py_est == "N/A":
             print("\n⚠️  NOTE: Run the Cumulative Analysis cell (Cell 14) first to generate Python results.")
        else:
             print("\n⚠️  CHECK: Differences detected. This is often due to:")
             print("    1. Different aggregation methods (Python uses Fixed-Effect pool within study).")
             print("    2. Sorting order (if multiple studies have the same year).")
             print("    3. Tau² estimator differences (DL vs REML).")

    except Exception as e:
        print(f"\n❌ R Error: {e}")

In [ ]:
#@title 📚 Cell 1: Imports & Setup
# =============================================================================
# VALIDATION NOTEBOOK - SETUP
# Purpose: Install R, rpy2, and Python stats libraries.
# =============================================================================

import sys
import subprocess
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.optimize import minimize
from scipy.stats import norm, t
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# --- 1. Install/Check R Interface ---
try:
    import rpy2.robjects as ro
    from rpy2.robjects import pandas2ri
    from rpy2.robjects.packages import importr
    pandas2ri.activate()
    print("✅ rpy2 detected.")
except ImportError:
    print("⚙️ Installing rpy2...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "rpy2"])
    import rpy2.robjects as ro
    from rpy2.robjects import pandas2ri
    from rpy2.robjects.packages import importr
    pandas2ri.activate()

# --- 2. Install R 'metafor' Package ---
print("⚙️ Checking R 'metafor' package...")
ro.r("""
if (!require("metafor")) {
    install.packages("metafor", repos="https://cloud.r-project.org", quiet=TRUE)
}
library(metafor)
""")

print("\n✅ Environment Ready: Python & R (metafor) are linked.")

In [ ]:
#@title 📁 Cell 2: Load Raw Data
# =============================================================================
# CELL: DATA LOADING
# Purpose: Load raw data (Mean, SD, N) for parallel processing.
# =============================================================================

# --- Widgets ---
load_demo_btn = widgets.Button(description="🧪 Load Demo Data", button_style='success')
output_data = widgets.Output()

def load_demo(b):
    global raw_df
    with output_data:
        clear_output()
        # Create synthetic dataset (Nested Structure)
        np.random.seed(42)
        n_studies = 20
        obs_per_study = 3

        ids = np.repeat([f"Study_{i}" for i in range(1, n_studies+1)], obs_per_study)
        ne = np.random.randint(20, 100, len(ids))
        nc = np.random.randint(20, 100, len(ids))

        # True effect = 0.5, Tau2 = 0.1, Sigma2 = 0.05
        true_eff = 0.5 + np.random.normal(0, np.sqrt(0.1), len(ids)) # Random study effect

        raw_df = pd.DataFrame({
            'id': ids,
            'xe': true_eff + np.random.normal(0.5, 0.1, len(ids)), # Mean Exp
            'xc': np.random.normal(0, 0.1, len(ids)),              # Mean Ctrl
            'ne': ne,
            'nc': nc,
            'sde': np.random.uniform(0.8, 1.2, len(ids)),
            'sdc': np.random.uniform(0.8, 1.2, len(ids))
        })

        print(f"✅ Loaded {len(raw_df)} rows of raw data.")
        print("   Columns: id, xe, xc, ne, nc, sde, sdc")
        display(raw_df.head())

load_demo_btn.on_click(load_demo)
display(load_demo_btn, output_data)

In [ ]:
#@title 🧮 Cell 3: Python Pipeline Functions (Fixed)
# =============================================================================
# CELL: PYTHON ENGINE
# Purpose: Replicate Co-Met's calculations (Effect Size + 3-Level Model).
# =============================================================================

import numpy as np
from scipy.special import gamma

# --- 1. Effect Size Calculator (Hedges' g) ---
# --- 2. Three-Level Model (High Precision Optimizer) ---
print("✅ Python Pipeline Ready (High Precision Mode).")

In [ ]:
#@title ⚖️ Cell 4: Run Comparison (Fixed)
# =============================================================================
# CELL: EXECUTION & REPORTING
# Purpose: Run Python and R pipelines side-by-side and verify accuracy.
# =============================================================================

# 1. Check Data
if 'raw_df' not in globals():
    print("❌ Error: No data found. Please run Cell 2 first.")
else:
    print("🚀 Starting Parallel Analysis...")

    # --- A. PYTHON PIPELINE ---
    print("   🔵 Running Python (Co-Met)...")

    # A1. Effect Sizes
    py_g, py_v = calculate_hedges_g_python(raw_df)

    # A2. Model
    py_tau2, py_sigma2 = run_python_3level(py_g.values, py_v.values, raw_df['id'].values)

    # Compute Pooled Effect (Mu) from Python params (Simplified Weighted Mean for Verification)
    print(f"      Python Tau²: {py_tau2:.5f}")
    print(f"      Python Sigma²: {py_sigma2:.5f}")


    # --- B. R PIPELINE ---
    print("   🟠 Running R (Metafor)...")

    # Push raw data to R
    ro.globalenv['df_r'] = raw_df

    # Run R Script (Escalc + rma.mv)
    r_script = """
    library(metafor)

    # 1. Calculate Effects
    dat <- escalc(measure="SMD",
                  m1i=xe, m2i=xc, sd1i=sde, sd2i=sdc, n1i=ne, n2i=nc,
                  data=df_r)

    # --- FIX: Create the 'rows' column for observation-level random effect ---
    dat$rows <- 1:nrow(dat)

    # 2. Run Model
    #    random = ~ 1 | study_id / observation_id
    res <- rma.mv(yi, vi, random = ~ 1 | id/rows, data=dat)

    list(
        yi = dat$yi,       # Effect sizes
        vi = dat$vi,       # Variances
        tau2 = res$sigma2[1],
        sigma2 = res$sigma2[2],
        mu = res$b[1]
    )
    """
    r_res = ro.r(r_script)

    # Extract R Results
    r_yi = np.array(r_res.rx2('yi'))
    r_vi = np.array(r_res.rx2('vi'))
    r_tau2 = r_res.rx2('tau2')[0]
    r_sigma2 = r_res.rx2('sigma2')[0]
    r_mu = r_res.rx2('mu')[0] # Added mu extraction

    print(f"      R Tau²: {r_tau2:.5f}")
    print(f"      R Sigma²: {r_sigma2:.5f}")

    # --- C. COMPARISON REPORT ---
    print("\n" + "="*60)
    print("🔍 VALIDATION REPORT")
    print("="*60)

    # 1. Effect Size Calc Comparison
    diff_es = np.abs(py_g.values - r_yi).max()
    diff_var = np.abs(py_v.values - r_vi).max()

    print(f"1. Effect Size Calculation (Hedges' g):")
    if diff_es < 1e-5:
        print(f"   ✅ PERFECT MATCH (Max diff: {diff_es:.2e})")
    else:
        print(f"   ⚠️ DISCREPANCY (Max diff: {diff_es:.2e}) - Check 'J' correction.")

    # 2. Variance Components Comparison
    diff_tau = abs(py_tau2 - r_tau2)
    diff_sigma = abs(py_sigma2 - r_sigma2)

    print(f"\n2. Three-Level Variance Estimates:")
    print(f"   {'Parameter':<15} {'Python':<12} {'R':<12} {'Diff':<12}")
    print("-" * 55)
    print(f"   {'Tau² (L3)':<15} {py_tau2:.5f}     {r_tau2:.5f}     {diff_tau:.2e}")
    print(f"   {'Sigma² (L2)':<15} {py_sigma2:.5f}     {r_sigma2:.5f}     {diff_sigma:.2e}")

    if diff_tau < 1e-4 and diff_sigma < 1e-4:
        print("\n✅ PASSED: Python optimizer matches R/metafor within tolerance.")
    else:
        print("\n⚠️ CHECK OPTIMIZER: Variance components differ slightly.")

In [ ]:
#@title 📈 Cell 12: Meta-Regression Engine (Three-Level)
# =============================================================================
# CELL 12: ROBUST META-REGRESSION ENGINE
# Purpose: Core functions to estimate slopes (betas) in a 3-level model.
# =============================================================================

import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.optimize import minimize
from scipy.stats import norm, t
import statsmodels.api as sm

# --- 1. Core Regression Math (GLS + REML) ---
def _run_three_level_reml_regression_v2(analysis_data, moderator_col, effect_col, var_col):
    """
    Main function to run 3-level meta-regression.
    Returns: final_estimates dictionary.
    """
    # 1. Prepare Data
    grouped = analysis_data.groupby('id')
    y_all = []
    v_all = []
    X_all = []

    for _, group in grouped:
        y_all.append(group[effect_col].values)
        v_all.append(group[var_col].values)
        # Create Design Matrix X (Intercept + Moderator)
        mod_data = group[moderator_col].values
        # Add constant column
        X_i = np.column_stack([np.ones(len(mod_data)), mod_data])
        X_all.append(X_i)

    N_total = len(analysis_data)
    M_studies = len(y_all)
    p_params = 2 # Intercept + 1 Moderator

    # 2. Optimize (Two-pass for precision)
    best_res = None
    best_fun = np.inf
    start_points = [[0.01, 0.01], [0.5, 0.1], [0.1, 0.5]]

    for start in start_points:
        res = minimize(
            _negative_log_likelihood_reml_reg_v2,
            x0=start,
            args=(y_all, v_all, X_all, N_total, M_studies, p_params),
            method='L-BFGS-B',
            bounds=[(1e-8, None), (1e-8, None)],
            options={'ftol': 1e-10}
        )
        if res.success and res.fun < best_fun:
            best_fun = res.fun
            best_res = res

    if not best_res: return None, None, None

    # 3. Final Estimates
    tau_sq, sigma_sq = best_res.x
    final_est = _get_three_level_regression_estimates_v2(
        [tau_sq, sigma_sq], y_all, v_all, X_all, N_total, M_studies, p_params
    )

    return final_est, (N_total, M_studies, p_params), best_res

print("✅ Meta-Regression Engine Ready.")

In [ ]:
#@title 🧪 Cell 5: Advanced Validation (Inference & Regression)
# =============================================================================
# CELL: FULL VALIDATION
# Purpose: Compare Pooled Effect (Mu), SE, and Meta-Regression Slopes.
# =============================================================================

import numpy as np
import pandas as pd
import statsmodels.api as sm

# 1. Prepare Data with a Mock Moderator
if 'raw_df' not in globals():
    print("❌ Error: Run Cell 2 first to load data.")
else:
    print("🚀 Starting Advanced Validation...")

    # Add a random continuous moderator for regression testing
    np.random.seed(999)
    raw_df['dose'] = np.random.uniform(0, 10, len(raw_df))

    # --- A. VALIDATION 1: POOLED EFFECT (Intercept-Only) ---
    print("\n📊 TEST 1: POOLED EFFECT (Intercept-Only Model)")

    # 1. Python Calculation
    py_g, py_v = calculate_hedges_g_python(raw_df)
    # Get variance components from your optimized run
    py_tau2, py_sigma2 = run_python_3level(py_g.values, py_v.values, raw_df['id'].values)

    # Get Inference (Mu, SE) using your internal helper
    # We perform the matrix math using the optimal variances
    y_all = [group['hedges_g'].values for _, group in pd.DataFrame({'id': raw_df['id'], 'hedges_g': py_g}).groupby('id')]
    v_all = [group['vg'].values for _, group in pd.DataFrame({'id': raw_df['id'], 'vg': py_v}).groupby('id')]

    # Re-use the helper from Cell 6.5 to get estimates
    est = _get_three_level_estimates([py_tau2, py_sigma2], y_all, v_all, len(raw_df), len(y_all))
    py_mu, py_se = est['mu'], est['se_mu']

    # 2. R Calculation
    ro.globalenv['df_r'] = raw_df
    r_script_pooled = """
    library(metafor)
    dat <- escalc(measure="SMD", m1i=xe, m2i=xc, sd1i=sde, sd2i=sdc, n1i=ne, n2i=nc, data=df_r)
    dat$rows <- 1:nrow(dat)

    # Intercept-only model
    res <- rma.mv(yi, vi, random = ~ 1 | id/rows, data=dat, control=list(optimizer="optim", optmethod="BFGS"))

    list(mu = res$b[1], se = res$se[1])
    """
    r_res_pooled = ro.r(r_script_pooled)
    r_mu, r_se = r_res_pooled.rx2('mu')[0], r_res_pooled.rx2('se')[0]

    # Report 1
    print(f"   {'Parameter':<15} {'Python':<12} {'R':<12} {'Diff':<12}")
    print("-" * 55)
    print(f"   {'Effect (μ)':<15} {py_mu:.6f}     {r_mu:.6f}     {abs(py_mu-r_mu):.2e}")
    print(f"   {'Std Error':<15} {py_se:.6f}     {r_se:.6f}     {abs(py_se-r_se):.2e}")


    # --- B. VALIDATION 2: META-REGRESSION (Slope) ---
    print("\n📈 TEST 2: META-REGRESSION (Slope for 'Dose')")

    # 1. Python Calculation (using your regression helper from Cell 10/12)
    # We need to construct the matrices manually since we aren't running the full widget UI here
    grouped = pd.DataFrame({'id': raw_df['id'], 'y': py_g, 'v': py_v, 'dose': raw_df['dose']}).groupby('id')
    X_all = []
    y_reg_all = []
    v_reg_all = []

    for _, grp in grouped:
        y_reg_all.append(grp['y'].values)
        v_reg_all.append(grp['v'].values)
        # Add intercept column to X
        X_i = sm.add_constant(grp['dose'].values, prepend=True)
        X_all.append(X_i)

    # Run optimizer for regression (estimates Tau2/Sigma2 specifically for this model)
    # Note: We use the helper function _run_three_level_reml_regression_v2 you defined in Cell 12
    # But since that might not be in scope of this cell, we'll rely on the core math if possible.
    # For simplicity, let's assume the helper _get_three_level_regression_estimates_v2 exists (from Cell 12)

    if '_run_three_level_reml_regression_v2' in globals():
        # Run your python regression engine
        est_reg, _, _ = _run_three_level_reml_regression_v2(
            pd.DataFrame({'id': raw_df['id'], 'dose': raw_df['dose'], 'y': py_g, 'v': py_v}),
            'dose', 'y', 'v'
        )
        py_beta_dose = est_reg['betas'][1] # Slope
        py_se_dose = est_reg['se_betas'][1]
    else:
        print("   ⚠️ Python regression function not found. Did you run Cell 12?")
        py_beta_dose, py_se_dose = np.nan, np.nan

    # 2. R Calculation
    r_script_reg = """
    res_reg <- rma.mv(yi, vi, mods = ~ dose, random = ~ 1 | id/rows, data=dat,
                      control=list(optimizer="optim", optmethod="BFGS"))
    list(beta = res_reg$b[2], se = res_reg$se[2])
    """
    if not np.isnan(py_beta_dose):
        r_res_reg = ro.r(r_script_reg)
        r_beta, r_se_reg = r_res_reg.rx2('beta')[0], r_res_reg.rx2('se')[0]

        # Report 2
        print(f"   {'Parameter':<15} {'Python':<12} {'R':<12} {'Diff':<12}")
        print("-" * 55)
        print(f"   {'Slope (β1)':<15} {py_beta_dose:.6f}     {r_beta:.6f}     {abs(py_beta_dose-r_beta):.2e}")
        print(f"   {'SE (Slope)':<15} {py_se_dose:.6f}     {r_se_reg:.6f}     {abs(py_se_dose-r_se_reg):.2e}")

        if abs(py_beta_dose - r_beta) < 1e-5:
             print("\n✅ PASSED: Meta-regression engine is accurate.")
        else:
             print("\n⚠️ CHECK REGRESSION: Slopes differ.")